# Analysis of Trajectories

## Import Libraries

In [ ]:
!git clone https://github.com/clinicalml/gumbel-max-scm.git

Cloning into 'gumbel-max-scm'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 113 (delta 0), reused 0 (delta 0), pack-reused 110
Receiving objects: 100% (113/113), 1.48 MiB | 4.01 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [ ]:
#Enable importing code from parent directory
import os, sys
simulator_path = os.path.abspath('./gumbel-max-scm')
sys.path.insert(1, simulator_path)

In [ ]:
!pip install pymdptoolbox

  Preparing metadata (setup.py) ... done
  Created wheel for pymdptoolbox: filename=pymdptoolbox-4.0b3-py3-none-any.whl size=25657 sha256=8548156dd5e60749a1377150004e415ee6146a711567f941c4a3e0fc15a3455c
  Stored in directory: /root/.cache/pip/wheels/2b/e7/c7/d7abf9e309f3573a934fed2750c70bd75d9e9d901f7f16e183
Successfully built pymdptoolbox


**IMPORTANT NOTE:** At this stage, to reproduce our experiments, one must modify line 38 of `gumbel-max-scm/sepsisSimDiabetes/DataGenerator.py` so that it reads:

```
mdp = MDP(init_state_idx=%state%,
          policy_array=policy, policy_idx_type=policy_idx_type,
          p_diabetes=p_diabetes)

```

We have essentially set the initial state to a fixed value so that we may estimate the Q-function from that state. Additionally, line 58 of the same file must be modified to:

```
mdp.state = mdp.get_new_state(state_idx = %state%)
```

In [ ]:
import numpy as np
import cf.counterfactual as cf
import cf.utils as utils
import pandas as pd
import pickle
import itertools as it
from tqdm import tqdm_notebook as tqdm
from scipy.linalg import block_diag

# Sepsis Simulator code
from sepsisSimDiabetes.State import State
from sepsisSimDiabetes.Action import Action
from sepsisSimDiabetes.DataGenerator import DataGenerator
import sepsisSimDiabetes.MDP as simulator

import mdptoolboxSrc.mdp as mdptools

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Set up Variables for Policy and Data

In [ ]:
import numpy as np
SEED = 1
np.random.seed(SEED)
NSIMSAMPS = 100000  # Samples to draw from the simulator
NSTEPS = 20  # Max length of each trajectory
NCFSAMPS = 5  # Counterfactual Samples per observed sample
DISCOUNT_Pol = 0.99 # Used for computing optimal policies
DISCOUNT = 1 # Used for computing actual reward
PHYS_EPSILON = 0.05 # Used for sampling using physician pol as eps greedy

PROB_DIAB = 1.0

# Option 1: Use bootstrapping w/replacement on the original NSIMSAMPS to estimate errors
USE_BOOSTRAP=True
N_BOOTSTRAP = 100

# Option 2: Use repeated sampling (i.e., NSIMSAMPS fresh simulations each time) to get error bars;
# This is done in the appendix of the paper, but not in the main paper
N_REPEAT_SAMPLING = 1

# These are properties of the simulator, do not change
n_actions = Action.NUM_ACTIONS_TOTAL
n_components = 2

# These are added as absorbing states
n_states_abs = State.NUM_OBS_STATES + 2
discStateIdx = n_states_abs - 1
deadStateIdx = n_states_abs - 2

In [ ]:
import zipfile
with zipfile.ZipFile("gumbel-max-scm/data/diab_txr_mats-replication.zip", 'r') as zip_ref:
    zip_ref.extractall("gumbel-max-scm/data")

In [ ]:
# Get the transition and reward matrix from file
with open("gumbel-max-scm/data/diab_txr_mats-replication.pkl", "rb") as f:
    mdict = pickle.load(f)

tx_mat = mdict["tx_mat"]
r_mat = mdict["r_mat"]
p_mixture = np.array([1 - PROB_DIAB, PROB_DIAB])

In [ ]:
from scipy.linalg import block_diag

tx_mat_full = np.zeros((n_actions, State.NUM_FULL_STATES, State.NUM_FULL_STATES))
r_mat_full = np.zeros((n_actions, State.NUM_FULL_STATES, State.NUM_FULL_STATES))

for a in range(n_actions):
    tx_mat_full[a, ...] = block_diag(tx_mat[0, a, ...], tx_mat[1, a,...])
    r_mat_full[a, ...] = block_diag(r_mat[0, a, ...], r_mat[1, a, ...])

In [ ]:
fullMDP = cf.MatrixMDP(tx_mat_full, r_mat_full)
fullPol = fullMDP.policyIteration(discount=DISCOUNT_Pol, eval_type=1)

Functions to convert data and policies into factored format

In [ ]:
def factorise_dataset(data):
  #With two action spaces: (ABX, VASO) and (VENT), we can perfectly satisfy theorem 1
  f_transitions_anva_ve = np.zeros((NSIMSAMPS, NSTEPS, 2, 5))
  #We also investigate the factorisations that would not satisfy theorem 1:
  # (ABX, VENT), (VASO)
  f_transitions_anve_va = np.zeros((NSIMSAMPS, NSTEPS, 2, 5))
  # (VENT, VASO), (ABX)
  f_transitions_veva_an = np.zeros((NSIMSAMPS, NSTEPS, 2, 5))
  #With three action spaces, we can reduce the variance but may increase bias
  f_transitions_an_va_ve = np.zeros((NSIMSAMPS, NSTEPS, 3, 5))

  #Transform unfactored data sample by sample
  for step in range(NSTEPS):
    for trajectory in range(NSIMSAMPS):
      #Extract necessary features
      seg = data[trajectory, step, :]
      time = seg[0]
      state = seg[1]
      action = seg[2]
      reward = seg[3]
      next_state = seg[4]

      #Set times
      f_transitions_anva_ve[trajectory, step, :, 0] = [time]*2
      f_transitions_anve_va[trajectory, step, :, 0] = [time]*2
      f_transitions_veva_an[trajectory, step, :, 0] = [time]*2
      f_transitions_an_va_ve[trajectory, step, :, 0] = [time]*3

      #Set factored states (state abstractions)
      f_transitions_anva_ve[trajectory, step, :, 1] = [state]*2
      f_transitions_anve_va[trajectory, step, :, 1] = [state]*2
      f_transitions_veva_an[trajectory, step, :, 1] = [state]*2
      f_transitions_an_va_ve[trajectory, step, :, 1] = [state]*3

      #Set factored actions
      actionObj = Action(action_idx=action)
      action_vec = actionObj.get_action_vec()

      f_transitions_anva_ve[trajectory, step, 0, 2] = action_vec[1,0]
      f_transitions_anva_ve[trajectory, step, 1, 2] = 2*action_vec[0,0] + action_vec[2,0]

      f_transitions_anve_va[trajectory, step, 0, 2] = action_vec[2,0]
      f_transitions_anve_va[trajectory, step, 1, 2] = 2*action_vec[0,0] + action_vec[1,0]

      f_transitions_veva_an[trajectory, step, 0, 2] = action_vec[0,0]
      f_transitions_veva_an[trajectory, step, 1, 2] = 2*action_vec[1,0] + action_vec[2,0]

      f_transitions_an_va_ve[trajectory, step, 0, 2] = action_vec[0,0]
      f_transitions_an_va_ve[trajectory, step, 1, 2] = action_vec[1,0]
      f_transitions_an_va_ve[trajectory, step, 2, 2] = action_vec[2,0]

      #Set factored rewards
      f_transitions_anva_ve[trajectory, step, :, 3] = [reward/2.0]*2
      f_transitions_anve_va[trajectory, step, :, 3] = [reward/2.0]*2
      f_transitions_veva_an[trajectory, step, :, 3] = [reward/2.0]*2
      f_transitions_an_va_ve[trajectory, step, :, 3] = [reward/3.0]*3

      #Set factored next states (state abstractions)
      f_transitions_anva_ve[trajectory, step, :, 4] = [next_state]*2
      f_transitions_anve_va[trajectory, step, :, 4] = [next_state]*2
      f_transitions_veva_an[trajectory, step, :, 4] = [next_state]*2
      f_transitions_an_va_ve[trajectory, step, :, 4] = [next_state]*3

  return f_transitions_anva_ve,  f_transitions_anve_va, f_transitions_veva_an, f_transitions_an_va_ve

In [ ]:
def bin_to_num(bitlist):
  out = 0
  for bit in bitlist:
    out = (out << 1) | bit
  return out

#Number codes:
#1 = (ABX, VAS), (VENT)
#2 = (ABX, VENT), (VAS)
#3 = (VENT, VAS), (ABX)
#4 = (ABX), (VENT), (VAS)
def factor_policy_1(policy):

  factored_policy_1 = np.zeros((policy.shape[0], 2))
  for ind in range(policy.shape[0]):
    max_pair = None
    max_sum = 0.0
    for index_pair in [(0,0), (0,1), (1,0), (1,1)]:
      new_sum = policy[ind, bin_to_num([index_pair[0], 0, index_pair[1]])] + policy[ind, bin_to_num([index_pair[0], 1, index_pair[1]])]
      if new_sum > max_sum:
        max_pair = index_pair
        max_sum = new_sum
    factored_policy_1[ind, 0] = policy[ind, bin_to_num([max_pair[0], 0, max_pair[1]])]/max_sum
    factored_policy_1[ind, 1] = policy[ind, bin_to_num([max_pair[0], 1, max_pair[1]])]/max_sum

  factored_policy_2 = np.zeros((policy.shape[0], 4))
  for ind in range(policy.shape[0]):
    max_id = None
    max_sum = 0.0
    for index in [0, 1]:
      new_sum = policy[ind, bin_to_num([0, index, 0])] + policy[ind, bin_to_num([0, index, 1])] + policy[ind, bin_to_num([1, index, 0])] + + policy[ind, bin_to_num([1, index, 1])]
      if new_sum > max_sum:
        max_id = index
        max_sum = new_sum
    factored_policy_2[ind, 0] = policy[ind, bin_to_num([0, max_id, 0])]/max_sum
    factored_policy_2[ind, 1] = policy[ind, bin_to_num([0, max_id, 1])]/max_sum
    factored_policy_2[ind, 2] = policy[ind, bin_to_num([1, max_id, 0])]/max_sum
    factored_policy_2[ind, 3] = policy[ind, bin_to_num([1, max_id, 1])]/max_sum

  reconstruction_error = np.zeros(policy.shape)
  for ind in range(policy.shape[0]):
    for index in [[x,y,z] for x in [0,1] for y in [0,1] for z in [0,1]]:
      action_id = bin_to_num(index)
      true = policy[ind, action_id]
      reconstruct = factored_policy_1[ind, index[0]]*factored_policy_2[ind, bin_to_num(index[1:])]
      reconstruction_error[ind, action_id] = abs(true - reconstruct)

  return [factored_policy_1, factored_policy_2], reconstruction_error

In [ ]:
#Number codes:
#1 = (ABX, VAS), (VENT)
#2 = (ABX, VENT), (VAS)
#3 = (VENT, VAS), (ABX)
#4 = (ABX), (VENT), (VAS)
def factor_policy_2(policy):

  factored_policy_1 = np.zeros((policy.shape[0], 2))
  for ind in range(policy.shape[0]):
    max_pair = None
    max_sum = 0.0
    for index_pair in [(0,0), (0,1), (1,0), (1,1)]:
      new_sum = policy[ind, bin_to_num([index_pair[0], index_pair[1], 0])] + policy[ind, bin_to_num([index_pair[0], index_pair[1], 1])]
      if new_sum > max_sum:
        max_pair = index_pair
        max_sum = new_sum
    factored_policy_1[ind, 0] = policy[ind, bin_to_num([max_pair[0], max_pair[1], 0])]/max_sum
    factored_policy_1[ind, 1] = policy[ind, bin_to_num([max_pair[0], max_pair[1], 1])]/max_sum

  factored_policy_2 = np.zeros((policy.shape[0], 4))
  for ind in range(policy.shape[0]):
    max_id = None
    max_sum = 0.0
    for index in [0, 1]:
      new_sum = policy[ind, bin_to_num([0, 0, index])] + policy[ind, bin_to_num([0, 1, index])] + policy[ind, bin_to_num([1, 0, index])] + + policy[ind, bin_to_num([1, 1, index])]
      if new_sum > max_sum:
        max_id = index
        max_sum = new_sum
    factored_policy_2[ind, 0] = policy[ind, bin_to_num([0, 0, max_id])]/max_sum
    factored_policy_2[ind, 1] = policy[ind, bin_to_num([0, 1, max_id])]/max_sum
    factored_policy_2[ind, 2] = policy[ind, bin_to_num([1, 0, max_id])]/max_sum
    factored_policy_2[ind, 3] = policy[ind, bin_to_num([1, 1, max_id])]/max_sum

  reconstruction_error = np.zeros(policy.shape)
  for ind in range(policy.shape[0]):
    for index in [[x,y,z] for x in [0,1] for y in [0,1] for z in [0,1]]:
      action_id = bin_to_num(index)
      true = policy[ind, action_id]
      reconstruct = factored_policy_1[ind, index[0]]*factored_policy_2[ind, bin_to_num(index[1:])]
      reconstruction_error[ind, action_id] = abs(true - reconstruct)

  return [factored_policy_1, factored_policy_2], reconstruction_error

In [ ]:
#Number codes:
#1 = (ABX, VAS), (VENT)
#2 = (ABX, VENT), (VAS)
#3 = (VENT, VAS), (ABX)
#4 = (ABX), (VENT), (VAS)
def factor_policy_3(policy):

  factored_policy_1 = np.zeros((policy.shape[0], 2))
  for ind in range(policy.shape[0]):
    max_pair = None
    max_sum = 0.0
    for index_pair in [(0,0), (0,1), (1,0), (1,1)]:
      new_sum = policy[ind, bin_to_num([0, index_pair[0], index_pair[1]])] + policy[ind, bin_to_num([1, index_pair[0], index_pair[1]])]
      if new_sum > max_sum:
        max_pair = index_pair
        max_sum = new_sum
    factored_policy_1[ind, 0] = policy[ind, bin_to_num([0, max_pair[0], max_pair[1]])]/max_sum
    factored_policy_1[ind, 1] = policy[ind, bin_to_num([1, max_pair[0], max_pair[1]])]/max_sum

  factored_policy_2 = np.zeros((policy.shape[0], 4))
  for ind in range(policy.shape[0]):
    max_id = None
    max_sum = 0.0
    for index in [0, 1]:
      new_sum = policy[ind, bin_to_num([index, 0, 0])] + policy[ind, bin_to_num([index, 0, 1])] + policy[ind, bin_to_num([index, 1, 0])] + + policy[ind, bin_to_num([index, 1, 1])]
      if new_sum > max_sum:
        max_id = index
        max_sum = new_sum
    factored_policy_2[ind, 0] = policy[ind, bin_to_num([max_id, 0, 0])]/max_sum
    factored_policy_2[ind, 1] = policy[ind, bin_to_num([max_id, 0, 1])]/max_sum
    factored_policy_2[ind, 2] = policy[ind, bin_to_num([max_id, 1, 0])]/max_sum
    factored_policy_2[ind, 3] = policy[ind, bin_to_num([max_id, 1, 1])]/max_sum

  reconstruction_error = np.zeros(policy.shape)
  for ind in range(policy.shape[0]):
    for index in [[x,y,z] for x in [0,1] for y in [0,1] for z in [0,1]]:
      action_id = bin_to_num(index)
      true = policy[ind, action_id]
      reconstruct = factored_policy_1[ind, index[0]]*factored_policy_2[ind, bin_to_num(index[1:])]
      reconstruction_error[ind, action_id] = abs(true - reconstruct)

  return [factored_policy_1, factored_policy_2], reconstruction_error

In [ ]:
#Number codes:
#1 = (ABX, VAS), (VENT)
#2 = (ABX, VENT), (VAS)
#3 = (VENT, VAS), (ABX)
#4 = (ABX), (VENT), (VAS)
def factor_policy_4(policy):

  factored_policy_1 = np.zeros((policy.shape[0], 2))
  for ind in range(policy.shape[0]):
    max_pair = None
    max_sum = 0.0
    for index_pair in [(0,0), (0,1), (1,0), (1,1)]:
      new_sum = policy[ind, bin_to_num([0, index_pair[0], index_pair[1]])] + policy[ind, bin_to_num([1, index_pair[0], index_pair[1]])]
      if new_sum > max_sum:
        max_pair = index_pair
        max_sum = new_sum
    factored_policy_1[ind, 0] = policy[ind, bin_to_num([0, max_pair[0], max_pair[1]])]/max_sum
    factored_policy_1[ind, 1] = policy[ind, bin_to_num([1, max_pair[0], max_pair[1]])]/max_sum

  factored_policy_2 = np.zeros((policy.shape[0], 2))
  for ind in range(policy.shape[0]):
    max_pair = None
    max_sum = 0.0
    for index_pair in [(0,0), (0,1), (1,0), (1,1)]:
      new_sum = policy[ind, bin_to_num([index_pair[0], 0, index_pair[1]])] + policy[ind, bin_to_num([index_pair[0], 1, index_pair[1]])]
      if new_sum > max_sum:
        max_pair = index_pair
        max_sum = new_sum
    factored_policy_2[ind, 0] = policy[ind, bin_to_num([max_pair[0], 0, max_pair[1]])]/max_sum
    factored_policy_2[ind, 1] = policy[ind, bin_to_num([max_pair[0], 1, max_pair[1]])]/max_sum

  factored_policy_3 = np.zeros((policy.shape[0], 2))
  for ind in range(policy.shape[0]):
    max_pair = None
    max_sum = 0.0
    for index_pair in [(0,0), (0,1), (1,0), (1,1)]:
      new_sum = policy[ind, bin_to_num([index_pair[0], index_pair[1], 0])] + policy[ind, bin_to_num([index_pair[0], index_pair[1], 1])]
      if new_sum > max_sum:
        max_pair = index_pair
        max_sum = new_sum
    factored_policy_3[ind, 0] = policy[ind, bin_to_num([max_pair[0], max_pair[1], 0])]/max_sum
    factored_policy_3[ind, 1] = policy[ind, bin_to_num([max_pair[0], max_pair[1], 1])]/max_sum

  reconstruction_error = np.zeros(policy.shape)
  for ind in range(policy.shape[0]):
    for index in [[x,y,z] for x in [0,1] for y in [0,1] for z in [0,1]]:
      action_id = bin_to_num(index)
      true = policy[ind, action_id]
      reconstruct = factored_policy_1[ind, index[0]]*factored_policy_2[ind, index[1]]*factored_policy_3[ind, index[2]]
      reconstruction_error[ind, action_id] = abs(true - reconstruct)

  return [factored_policy_1, factored_policy_2, factored_policy_3], reconstruction_error

## Perform Analysis on Physician Policy

Generate Behaviour Policy Data

In [ ]:
dgen = DataGenerator()
states, actions, lengths, rewards, diab, emp_tx_totals, emp_r_totals = dgen.simulate(
    NSIMSAMPS, NSTEPS, policy=fullPol, policy_idx_type='full',
    p_diabetes=PROB_DIAB, use_tqdm=False) #True, tqdm_desc='Behaviour Policy Simulation')

obs_samps = utils.format_dgen_samps(
    states, actions, rewards, diab, NSTEPS, NSIMSAMPS)

Convert data into array format

In [ ]:
time = np.arange(NSTEPS)
times = np.stack(axis=0, arrays=[time]*NSIMSAMPS)
times = times[..., np.newaxis]

nf_transitions_b = np.concatenate((times, states[:, 0:NSTEPS, :], actions, rewards, states[:, 1:, :]), axis=2)
nf_transitions_b.shape

(100000, 20, 5)

In [ ]:
f_transitions_b1, f_transitions_b2, f_transitions_b3, f_transitions_b4 = factorise_dataset(nf_transitions_b)

In [ ]:
factored_fullPol1, error1 = factor_policy_1(fullPol)
factored_fullPol2, error2 = factor_policy_2(fullPol)
factored_fullPol3, error3 = factor_policy_3(fullPol)
factored_fullPol4, error4 = factor_policy_4(fullPol)

In [ ]:
import numpy as np
import gc

def hashgen(arr):
  return np.array2string(arr[:, 2], precision=2, separator=',', suppress_small=True)

def analyse_trajectories_nf(nf, NUM_EPISODES, EPISODE_LENGTH=20):

  #Obtain dimensions
  N, T = nf.shape[0], nf.shape[1]

  counts = {}
  last_actions = {}

  for n in range(NUM_EPISODES):
      nf_tr = nf[n, :EPISODE_LENGTH, :]

      last_action_id = EPISODE_LENGTH - 1
      try:
        last_action_id = np.where(nf_tr[:, 2] == -1)[0][0] - 1
      except:
        pass

      last_action = nf_tr[last_action_id, 2]

      hash = hashgen(nf_tr[:last_action_id+1, :])
      if hash not in counts.keys():
        counts[hash] = 1
      else:
        counts[hash] += 1
        last_actions[hash] = last_action

  return counts, last_actions, len(counts.keys())

In [ ]:
def hashgen(arr):
  return np.array2string(arr[:, 2], precision=2, separator=',', suppress_small=True)

def analyse_trajectories_f(f, NUM_EPISODES, EPISODE_LENGTH=20):

  #Obtain dimensions
  N, T, D = f.shape[0], f.shape[1], f.shape[2]

  counts = []
  last_actions = []
  varieties = []

  for d in range(D):
    count_d = {}
    last_actions_d = {}
    for n in range(NUM_EPISODES):

      f_tr = f[n, :EPISODE_LENGTH, d, :]

      last_action_id = EPISODE_LENGTH - 1
      try:
        last_action_id = np.where(f_tr[:, 2] == -1)[0][0] - 1
      except:
        pass

      last_action = f_tr[last_action_id, 2]

      hash = hashgen(f_tr[:last_action_id+1, :])

      if hash not in count_d.keys():
        count_d[hash] = 1
      else:
        count_d[hash] += 1
        last_actions_d[hash] = last_action

    counts.append(count_d)
    last_actions.append(last_actions_d)
    varieties.append(len(count_d.keys()))

  return counts, last_actions, varieties

In [ ]:
nf_count, nf_last_actions, nf_num = analyse_trajectories_nf(nf_transitions_b, 100000)
f_count1, f_last_actions1, f_num1 = analyse_trajectories_f(f_transitions_b1, 100000)
f_count2, f_last_actions2, f_num2 = analyse_trajectories_f(f_transitions_b2, 100000)
f_count3, f_last_actions3, f_num3 = analyse_trajectories_f(f_transitions_b3, 100000)
f_count4, f_last_actions4, f_num4 = analyse_trajectories_f(f_transitions_b4, 100000)

f_counts = [f_count1, f_count2, f_count3, f_count4]
f_last_actions = [f_last_actions1, f_last_actions2, f_last_actions3, f_last_actions4]
f_num = [f_num1, f_num2, f_num3, f_num4]
Ds = [2,2,2,3]

def print_seq(traj_count, last_actions):
  counts = list(traj_count.values())
  trajs = list(traj_count.keys())
  for (seq, id) in enumerate(np.flip(np.argsort(counts)[-50:])):
    print(f'{seq+1}. Occurrences: {counts[id]}. Action Sequence: {trajs[id]}')

print('NON FACTORED SPACE')
print(f'{nf_num} different trajectories')
print_seq(nf_count, nf_last_actions)
print()
for f in [1,2,3,4]:
  for d in range(Ds[f-1]):
    print(f'FACTORED SPACE {d}, factorisation {f}')
    print(f'{f_num[f-1][d]} different trajectories')
    print_seq(f_counts[f-1][d], f_last_actions[f-1][d])
    print()
  print('--------------------------------------------------------')

NON FACTORED SPACE
4452 different trajectories
1. Occurrences: 14945. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
2. Occurrences: 1647. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.]
3. Occurrences: 1640. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.]
4. Occurrences: 1512. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.]
5. Occurrences: 1427. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.]
6. Occurrences: 1311. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.,7.]
7. Occurrences: 1266. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.,7.,7.]
8. Occurrences: 1177. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.,7.,7.,7.]
9. Occurrences: 1099. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.,7.,7.,7.,7.]
10. Occurrences: 1011. Action Sequence: [3.

In [ ]:
int(PROB_DIAB)

0

## Perform Analysis on Random Policy

In [ ]:
#Set up fully random policy
randPol = np.ones(fullPol.shape)/(fullPol.shape[1])

Generate Behaviour Policy Data

In [ ]:
dgen = DataGenerator()
states, actions, lengths, rewards, diab, emp_tx_totals, emp_r_totals = dgen.simulate(
    NSIMSAMPS, NSTEPS, policy=randPol, policy_idx_type='full',
    p_diabetes=PROB_DIAB, use_tqdm=False) #True, tqdm_desc='Behaviour Policy Simulation')

obs_samps = utils.format_dgen_samps(
    states, actions, rewards, diab, NSTEPS, NSIMSAMPS)

Convert data into array format

In [ ]:
time = np.arange(NSTEPS)
times = np.stack(axis=0, arrays=[time]*NSIMSAMPS)
times = times[..., np.newaxis]

nf_transitions_b = np.concatenate((times, states[:, 0:NSTEPS, :], actions, rewards, states[:, 1:, :]), axis=2)
nf_transitions_b.shape

(100000, 20, 5)

In [ ]:
f_transitions_b1, f_transitions_b2, f_transitions_b3, f_transitions_b4 = factorise_dataset(nf_transitions_b)

In [ ]:
factored_fullPol1, error1 = factor_policy_1(fullPol)
factored_fullPol2, error2 = factor_policy_2(fullPol)
factored_fullPol3, error3 = factor_policy_3(fullPol)
factored_fullPol4, error4 = factor_policy_4(fullPol)

In [ ]:
factored_randPol1, error1 = factor_policy_1(randPol)
factored_randPol2, error2 = factor_policy_2(randPol)
factored_randPol3, error3 = factor_policy_3(randPol)
factored_randPol4, error4 = factor_policy_4(randPol)

In [ ]:
import numpy as np
import gc

def hashgen(arr):
  return np.array2string(arr[:, 2], precision=2, separator=',', suppress_small=True)

def analyse_trajectories_nf(nf, NUM_EPISODES, EPISODE_LENGTH=20):

  #Obtain dimensions
  N, T = nf.shape[0], nf.shape[1]

  counts = {}
  last_actions = {}

  for n in range(NUM_EPISODES):
      nf_tr = nf[n, :EPISODE_LENGTH, :]

      last_action_id = EPISODE_LENGTH - 1
      try:
        last_action_id = np.where(nf_tr[:, 2] == -1)[0][0] - 1
      except:
        pass

      last_action = nf_tr[last_action_id, 2]

      hash = hashgen(nf_tr[:last_action_id+1, :])
      if hash not in counts.keys():
        counts[hash] = 1
      else:
        counts[hash] += 1
        last_actions[hash] = last_action

  return counts, last_actions, len(counts.keys())

In [ ]:
def hashgen(arr):
  return np.array2string(arr[:, 2], precision=2, separator=',', suppress_small=True)

def analyse_trajectories_f(f, factored_randPol, factored_fullPol, NUM_EPISODES, EPISODE_LENGTH=20):

  #Obtain dimensions
  N, T, D = f.shape[0], f.shape[1], f.shape[2]

  counts = []
  last_actions = []
  varieties = []
  polVals = []

  for d in range(D):
    count_d = {}
    last_actions_d = {}
    polVals_d = {}
    for n in range(NUM_EPISODES):

      f_tr = f[n, :EPISODE_LENGTH, d, :]
      randPol_d = factored_randPol[d]
      fullPol_d = factored_fullPol[d]

      last_action_id = EPISODE_LENGTH - 1
      try:
        last_action_id = np.where(f_tr[:, 2] == -1)[0][0] - 1
      except:
        pass

      last_action = f_tr[last_action_id, 2]

      hash = hashgen(f_tr[:last_action_id+1, :])

      randPolVals = randPol_d[f_tr[:last_action_id+1, 1].astype(int), f_tr[:last_action_id+1, 2].astype(int)]
      physPolVals = fullPol_d[f_tr[:last_action_id+1, 1].astype(int), f_tr[:last_action_id+1, 2].astype(int)]

      if hash not in count_d.keys():
        count_d[hash] = 1
      else:
        count_d[hash] += 1
        last_actions_d[hash] = last_action
        polVals_d[hash] = zip(randPolVals, physPolVals)

    counts.append(count_d)
    last_actions.append(last_actions_d)
    varieties.append(len(count_d.keys()))
    polVals.append(polVals_d)

  return counts, last_actions, varieties, polVals

In [ ]:
nf_count, nf_last_actions, nf_num = analyse_trajectories_nf(nf_transitions_b, 100000)
f_count1, f_last_actions1, f_num1, pv1 = analyse_trajectories_f(f_transitions_b1, factored_randPol1, factored_fullPol1, 100000)
f_count2, f_last_actions2, f_num2, pv2 = analyse_trajectories_f(f_transitions_b2, factored_randPol2, factored_fullPol2, 100000)
f_count3, f_last_actions3, f_num3, pv3 = analyse_trajectories_f(f_transitions_b3, factored_randPol3, factored_fullPol3, 100000)
f_count4, f_last_actions4, f_num4, pv4 = analyse_trajectories_f(f_transitions_b4, factored_randPol4, factored_fullPol4, 100000)

f_counts = [f_count1, f_count2, f_count3, f_count4]
f_last_actions = [f_last_actions1, f_last_actions2, f_last_actions3, f_last_actions4]
f_num = [f_num1, f_num2, f_num3, f_num4]
pv = [pv1, pv2, pv3, pv4]
Ds = [2,2,2,3]

def print_seq(traj_count, last_actions, pv=None):
  counts = list(traj_count.values())
  trajs = list(traj_count.keys())
  for (seq, id) in enumerate(np.flip(np.argsort(counts)[-50:])):
    print(f'{seq+1}. Occurrences: {counts[id]}. Action Sequence: {trajs[id]}')
    if not (pv is None):
      print(f'(pi_b, pi_e): {list(pv[trajs[id]])}')

print('NON FACTORED SPACE')
print(f'{nf_num} different trajectories')
print_seq(nf_count, nf_last_actions)
print()
for f in [1,2,3,4]:
  for d in range(Ds[f-1]):
    print(f'FACTORED SPACE {d}, factorisation {f}')
    print(f'{f_num[f-1][d]} different trajectories')
    print_seq(f_counts[f-1][d], f_last_actions[f-1][d], pv[f-1][d])
    print()
  print('--------------------------------------------------------')

NON FACTORED SPACE
18144 different trajectories
1. Occurrences: 7992. Action Sequence: [4.]
2. Occurrences: 7483. Action Sequence: [6.]
3. Occurrences: 6657. Action Sequence: [0.]
4. Occurrences: 6052. Action Sequence: [2.]
5. Occurrences: 3771. Action Sequence: [5.]
6. Occurrences: 3367. Action Sequence: [1.]
7. Occurrences: 3057. Action Sequence: [7.]
8. Occurrences: 2903. Action Sequence: [3.]
9. Occurrences: 736. Action Sequence: [7.,1.]
10. Occurrences: 736. Action Sequence: [5.,5.]
11. Occurrences: 729. Action Sequence: [7.,5.]
12. Occurrences: 692. Action Sequence: [5.,1.]
13. Occurrences: 687. Action Sequence: [5.,7.]
14. Occurrences: 672. Action Sequence: [7.,3.]
15. Occurrences: 668. Action Sequence: [5.,3.]
16. Occurrences: 659. Action Sequence: [7.,7.]
17. Occurrences: 651. Action Sequence: [1.,5.]
18. Occurrences: 650. Action Sequence: [3.,1.]
19. Occurrences: 621. Action Sequence: [3.,5.]
20. Occurrences: 601. Action Sequence: [1.,1.]
21. Occurrences: 582. Action Sequence

In [ ]:
int(PROB_DIAB)

1

## Record Data for Physician Policy (top 50 Trajectories over 100000 episodes)

### State 136 With Diabetes

In [ ]:
'''
NON FACTORED SPACE
18051 different trajectories
1. Occurrences: 10927. Action Sequence: [2.]
2. Occurrences: 8510. Action Sequence: [2.,2.]
3. Occurrences: 6623. Action Sequence: [2.,2.,2.]
4. Occurrences: 5066. Action Sequence: [2.,2.,2.,2.]
5. Occurrences: 3722. Action Sequence: [2.,2.,2.,2.,2.]
6. Occurrences: 2746. Action Sequence: [2.,2.,2.,2.,2.,2.]
7. Occurrences: 2035. Action Sequence: [2.,2.,2.,2.,2.,2.,2.]
8. Occurrences: 1916. Action Sequence: [2.,6.,0.]
9. Occurrences: 1449. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.]
10. Occurrences: 1171. Action Sequence: [2.,2.,6.,0.]
11. Occurrences: 1069. Action Sequence: [2.,6.]
12. Occurrences: 1020. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.]
13. Occurrences: 830. Action Sequence: [2.,3.]
14. Occurrences: 781. Action Sequence: [2.,2.,2.,6.,0.]
15. Occurrences: 767. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
16. Occurrences: 740. Action Sequence: [2.,6.,0.,6.,0.]
17. Occurrences: 682. Action Sequence: [2.,2.,6.]
18. Occurrences: 552. Action Sequence: [2.,2.,2.,2.,6.,0.]
19. Occurrences: 528. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
20. Occurrences: 500. Action Sequence: [2.,2.,3.]
21. Occurrences: 435. Action Sequence: [2.,2.,6.,0.,6.,0.]
22. Occurrences: 369. Action Sequence: [2.,2.,2.,6.]
23. Occurrences: 365. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
24. Occurrences: 355. Action Sequence: [2.,2.,2.,2.,2.,6.,0.]
25. Occurrences: 311. Action Sequence: [2.,2.,2.,6.,0.,6.,0.]
26. Occurrences: 296. Action Sequence: [2.,2.,2.,2.,2.,2.,6.,0.]
27. Occurrences: 286. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
28. Occurrences: 275. Action Sequence: [2.,2.,2.,3.]
29. Occurrences: 248. Action Sequence: [2.,2.,2.,2.,6.]
30. Occurrences: 241. Action Sequence: [2.,6.,0.,6.,0.,6.,0.]
31. Occurrences: 217. Action Sequence: [2.,6.,6.]
32. Occurrences: 199. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
33. Occurrences: 193. Action Sequence: [2.,6.,2.,2.]
34. Occurrences: 190. Action Sequence: [2.,2.,2.,2.,6.,0.,6.,0.]
35. Occurrences: 186. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,6.,0.]
36. Occurrences: 178. Action Sequence: [2.,3.,6.]
37. Occurrences: 178. Action Sequence: [2.,6.,0.,6.,6.,0.]
38. Occurrences: 175. Action Sequence: [2.,2.,2.,2.,3.]
39. Occurrences: 173. Action Sequence: [2.,2.,6.,0.,6.,0.,6.,0.]
40. Occurrences: 171. Action Sequence: [2.,2.,2.,2.,2.,6.]
41. Occurrences: 170. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
42. Occurrences: 163. Action Sequence: [2.,3.,2.,2.]
43. Occurrences: 159. Action Sequence: [2.,2.,6.,6.,0.]
44. Occurrences: 157. Action Sequence: [2.,6.,6.,0.]
45. Occurrences: 154. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,6.,0.]
46. Occurrences: 148. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
47. Occurrences: 146. Action Sequence: [2.,2.,6.,2.,2.]
48. Occurrences: 145. Action Sequence: [2.,6.,2.,2.,2.]
49. Occurrences: 141. Action Sequence: [2.,2.,6.,6.]
50. Occurrences: 132. Action Sequence: [2.,2.,2.,2.,2.,6.,0.,6.,0.]

FACTORED SPACE 0, factorisation 1
4341 different trajectories
1. Occurrences: 59183. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 2061. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 1531. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 1247. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1081. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 892. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 799. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 744. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 696. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 607. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 605. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 499. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 489. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 406. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 403. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 354. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 346. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 337. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 286. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 274. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
21. Occurrences: 269. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 267. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 261. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
24. Occurrences: 254. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
25. Occurrences: 246. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
26. Occurrences: 244. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 240. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
28. Occurrences: 230. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
29. Occurrences: 225. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 204. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 197. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 184. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 180. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 173. Action Sequence: [1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 172. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.]
36. Occurrences: 161. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.]
37. Occurrences: 158. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 149. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 149. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 140. Action Sequence: [1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 139. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.]
42. Occurrences: 134. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 130. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 127. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.]
45. Occurrences: 123. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 123. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
47. Occurrences: 122. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.]
48. Occurrences: 115. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 113. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 113. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 1
17965 different trajectories
1. Occurrences: 10927. Action Sequence: [0.]
2. Occurrences: 8510. Action Sequence: [0.,0.]
3. Occurrences: 6623. Action Sequence: [0.,0.,0.]
4. Occurrences: 5066. Action Sequence: [0.,0.,0.,0.]
5. Occurrences: 3722. Action Sequence: [0.,0.,0.,0.,0.]
6. Occurrences: 2746. Action Sequence: [0.,0.,0.,0.,0.,0.]
7. Occurrences: 2035. Action Sequence: [0.,0.,0.,0.,0.,0.,0.]
8. Occurrences: 2032. Action Sequence: [0.,2.,0.]
9. Occurrences: 1449. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.]
10. Occurrences: 1247. Action Sequence: [0.,0.,2.,0.]
11. Occurrences: 1069. Action Sequence: [0.,2.]
12. Occurrences: 1020. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.]
13. Occurrences: 830. Action Sequence: [0.,1.]
14. Occurrences: 810. Action Sequence: [0.,0.,0.,2.,0.]
15. Occurrences: 767. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
16. Occurrences: 742. Action Sequence: [0.,2.,0.,2.,0.]
17. Occurrences: 682. Action Sequence: [0.,0.,2.]
18. Occurrences: 573. Action Sequence: [0.,0.,0.,0.,2.,0.]
19. Occurrences: 528. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
20. Occurrences: 500. Action Sequence: [0.,0.,1.]
21. Occurrences: 437. Action Sequence: [0.,0.,2.,0.,2.,0.]
22. Occurrences: 370. Action Sequence: [0.,0.,0.,0.,0.,2.,0.]
23. Occurrences: 369. Action Sequence: [0.,0.,0.,2.]
24. Occurrences: 365. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
25. Occurrences: 312. Action Sequence: [0.,0.,0.,2.,0.,2.,0.]
26. Occurrences: 309. Action Sequence: [0.,0.,0.,0.,0.,0.,2.,0.]
27. Occurrences: 286. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
28. Occurrences: 275. Action Sequence: [0.,0.,0.,1.]
29. Occurrences: 248. Action Sequence: [0.,0.,0.,0.,2.]
30. Occurrences: 241. Action Sequence: [0.,2.,0.,2.,0.,2.,0.]
31. Occurrences: 217. Action Sequence: [0.,2.,2.]
32. Occurrences: 199. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
33. Occurrences: 196. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,2.,0.]
34. Occurrences: 193. Action Sequence: [0.,2.,0.,0.]
35. Occurrences: 191. Action Sequence: [0.,0.,0.,0.,2.,0.,2.,0.]
36. Occurrences: 178. Action Sequence: [0.,2.,0.,2.,2.,0.]
37. Occurrences: 178. Action Sequence: [0.,1.,2.]
38. Occurrences: 176. Action Sequence: [0.,0.,2.,2.,0.]
39. Occurrences: 175. Action Sequence: [0.,0.,0.,0.,1.]
40. Occurrences: 173. Action Sequence: [0.,2.,2.,0.]
41. Occurrences: 173. Action Sequence: [0.,0.,2.,0.,2.,0.,2.,0.]
42. Occurrences: 171. Action Sequence: [0.,0.,0.,0.,0.,2.]
43. Occurrences: 170. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
44. Occurrences: 163. Action Sequence: [0.,1.,0.,0.]
45. Occurrences: 156. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,2.,0.]
46. Occurrences: 148. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
47. Occurrences: 146. Action Sequence: [0.,0.,2.,0.,0.]
48. Occurrences: 145. Action Sequence: [0.,2.,0.,0.,0.]
49. Occurrences: 141. Action Sequence: [0.,0.,2.,2.]
50. Occurrences: 132. Action Sequence: [0.,0.,0.,0.,0.,2.,0.,2.,0.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 2
794 different trajectories
1. Occurrences: 11801. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 11272. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
3. Occurrences: 10107. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 9849. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 7468. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 6437. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 5068. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 4276. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 3566. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 2952. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 2652. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 2230. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 1899. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 1690. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 1495. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 1395. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
17. Occurrences: 1252. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.]
18. Occurrences: 1079. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.]
19. Occurrences: 1049. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
20. Occurrences: 898. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.]
21. Occurrences: 623. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 424. Action Sequence: [0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 355. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 354. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 320. Action Sequence: [0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 301. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 265. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 237. Action Sequence: [0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 215. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 214. Action Sequence: [0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
31. Occurrences: 203. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 196. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 190. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 181. Action Sequence: [0.,0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 159. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 152. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 142. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 138. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 134. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 131. Action Sequence: [0.,0.,1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 129. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 128. Action Sequence: [0.,0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 122. Action Sequence: [0.,0.,1.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 122. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 120. Action Sequence: [0.,0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 116. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 111. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 106. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
49. Occurrences: 99. Action Sequence: [0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 95. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]

FACTORED SPACE 1, factorisation 2
13543 different trajectories
1. Occurrences: 10927. Action Sequence: [1.]
2. Occurrences: 9340. Action Sequence: [1.,1.]
3. Occurrences: 7208. Action Sequence: [1.,1.,1.]
4. Occurrences: 5573. Action Sequence: [1.,1.,1.,1.]
5. Occurrences: 4192. Action Sequence: [1.,1.,1.,1.,1.]
6. Occurrences: 3161. Action Sequence: [1.,1.,1.,1.,1.,1.]
7. Occurrences: 2389. Action Sequence: [1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 2038. Action Sequence: [1.,3.,0.]
9. Occurrences: 1719. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 1306. Action Sequence: [1.,1.,3.,0.]
11. Occurrences: 1270. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 1069. Action Sequence: [1.,3.]
13. Occurrences: 949. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 886. Action Sequence: [1.,1.,1.,3.,0.]
15. Occurrences: 860. Action Sequence: [1.,1.,3.]
16. Occurrences: 783. Action Sequence: [1.,3.,0.,3.,0.]
17. Occurrences: 652. Action Sequence: [1.,1.,1.,1.,3.,0.]
18. Occurrences: 640. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 499. Action Sequence: [1.,1.,1.,3.]
20. Occurrences: 493. Action Sequence: [1.,1.,3.,0.,3.,0.]
21. Occurrences: 489. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 421. Action Sequence: [1.,1.,1.,1.,1.,3.,0.]
23. Occurrences: 376. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 364. Action Sequence: [1.,1.,1.,3.,0.,3.,0.]
25. Occurrences: 352. Action Sequence: [1.,1.,1.,1.,1.,1.,3.,0.]
26. Occurrences: 343. Action Sequence: [1.,1.,1.,1.,3.]
27. Occurrences: 317. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 262. Action Sequence: [1.,3.,0.,3.,0.,3.,0.]
29. Occurrences: 250. Action Sequence: [1.,1.,1.,1.,1.,3.]
30. Occurrences: 244. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 233. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,3.,0.]
32. Occurrences: 217. Action Sequence: [1.,3.,3.]
33. Occurrences: 214. Action Sequence: [1.,1.,1.,1.,3.,0.,3.,0.]
34. Occurrences: 208. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 207. Action Sequence: [1.,3.,3.,0.]
36. Occurrences: 201. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,3.,0.]
37. Occurrences: 200. Action Sequence: [1.,1.,3.,3.,0.]
38. Occurrences: 193. Action Sequence: [1.,3.,1.,1.]
39. Occurrences: 191. Action Sequence: [1.,3.,0.,3.,3.,0.]
40. Occurrences: 187. Action Sequence: [1.,1.,3.,1.,1.]
41. Occurrences: 186. Action Sequence: [1.,1.,3.,0.,3.,0.,3.,0.]
42. Occurrences: 184. Action Sequence: [1.,1.,3.,3.]
43. Occurrences: 169. Action Sequence: [1.,1.,1.,1.,1.,1.,3.]
44. Occurrences: 161. Action Sequence: [1.,1.,1.,1.,1.,3.,0.,3.,0.]
45. Occurrences: 157. Action Sequence: [1.,3.,1.,1.,1.]
46. Occurrences: 155. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 146. Action Sequence: [1.,1.,3.,3.,3.,0.]
48. Occurrences: 144. Action Sequence: [1.,3.,3.,3.,0.]
49. Occurrences: 143. Action Sequence: [1.,1.,1.,3.,3.,0.]
50. Occurrences: 140. Action Sequence: [1.,1.,3.,0.,3.,3.,0.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 3
13464 different trajectories
1. Occurrences: 10927. Action Sequence: [0.]
2. Occurrences: 9340. Action Sequence: [0.,0.]
3. Occurrences: 7208. Action Sequence: [0.,0.,0.]
4. Occurrences: 5573. Action Sequence: [0.,0.,0.,0.]
5. Occurrences: 4192. Action Sequence: [0.,0.,0.,0.,0.]
6. Occurrences: 3161. Action Sequence: [0.,0.,0.,0.,0.,0.]
7. Occurrences: 2389. Action Sequence: [0.,0.,0.,0.,0.,0.,0.]
8. Occurrences: 2154. Action Sequence: [0.,1.,0.]
9. Occurrences: 1719. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.]
10. Occurrences: 1405. Action Sequence: [0.,0.,1.,0.]
11. Occurrences: 1270. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.]
12. Occurrences: 1069. Action Sequence: [0.,1.]
13. Occurrences: 949. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
14. Occurrences: 929. Action Sequence: [0.,0.,0.,1.,0.]
15. Occurrences: 860. Action Sequence: [0.,0.,1.]
16. Occurrences: 785. Action Sequence: [0.,1.,0.,1.,0.]
17. Occurrences: 681. Action Sequence: [0.,0.,0.,0.,1.,0.]
18. Occurrences: 640. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
19. Occurrences: 499. Action Sequence: [0.,0.,0.,1.]
20. Occurrences: 495. Action Sequence: [0.,0.,1.,0.,1.,0.]
21. Occurrences: 489. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 444. Action Sequence: [0.,0.,0.,0.,0.,1.,0.]
23. Occurrences: 378. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,0.]
24. Occurrences: 376. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
25. Occurrences: 365. Action Sequence: [0.,0.,0.,1.,0.,1.,0.]
26. Occurrences: 343. Action Sequence: [0.,0.,0.,0.,1.]
27. Occurrences: 317. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
28. Occurrences: 262. Action Sequence: [0.,1.,0.,1.,0.,1.,0.]
29. Occurrences: 250. Action Sequence: [0.,0.,0.,0.,0.,1.]
30. Occurrences: 247. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,0.]
31. Occurrences: 244. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 223. Action Sequence: [0.,1.,1.,0.]
33. Occurrences: 222. Action Sequence: [0.,0.,1.,1.,0.]
34. Occurrences: 217. Action Sequence: [0.,1.,1.]
35. Occurrences: 215. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,0.]
36. Occurrences: 208. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
37. Occurrences: 206. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,0.]
38. Occurrences: 193. Action Sequence: [0.,1.,0.,0.]
39. Occurrences: 191. Action Sequence: [0.,1.,0.,1.,1.,0.]
40. Occurrences: 187. Action Sequence: [0.,0.,1.,0.,0.]
41. Occurrences: 186. Action Sequence: [0.,0.,1.,0.,1.,0.,1.,0.]
42. Occurrences: 184. Action Sequence: [0.,0.,1.,1.]
43. Occurrences: 169. Action Sequence: [0.,0.,0.,0.,0.,0.,1.]
44. Occurrences: 161. Action Sequence: [0.,0.,0.,0.,0.,1.,0.,1.,0.]
45. Occurrences: 157. Action Sequence: [0.,1.,0.,0.,0.]
46. Occurrences: 155. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
47. Occurrences: 153. Action Sequence: [0.,0.,0.,1.,1.,0.]
48. Occurrences: 152. Action Sequence: [0.,0.,1.,1.,1.,0.]
49. Occurrences: 150. Action Sequence: [0.,1.,1.,1.,0.]
50. Occurrences: 143. Action Sequence: [0.,0.,0.,0.,1.,1.,0.]

FACTORED SPACE 1, factorisation 3
10160 different trajectories
1. Occurrences: 11801. Action Sequence: [2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
2. Occurrences: 10107. Action Sequence: [2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
3. Occurrences: 7933. Action Sequence: [2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
4. Occurrences: 6126. Action Sequence: [2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
5. Occurrences: 4638. Action Sequence: [2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
6. Occurrences: 3521. Action Sequence: [2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
7. Occurrences: 2658. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
8. Occurrences: 1975. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
9. Occurrences: 1916. Action Sequence: [2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
10. Occurrences: 1418. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
11. Occurrences: 1328. Action Sequence: [2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
12. Occurrences: 1086. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
13. Occurrences: 1050. Action Sequence: [2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
14. Occurrences: 848. Action Sequence: [2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
15. Occurrences: 794. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
16. Occurrences: 740. Action Sequence: [2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
17. Occurrences: 702. Action Sequence: [2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
18. Occurrences: 668. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
19. Occurrences: 570. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
20. Occurrences: 553. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.]
21. Occurrences: 514. Action Sequence: [2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
22. Occurrences: 502. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
23. Occurrences: 449. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.]
24. Occurrences: 413. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
25. Occurrences: 402. Action Sequence: [2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
26. Occurrences: 338. Action Sequence: [2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
27. Occurrences: 329. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
28. Occurrences: 292. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.]
29. Occurrences: 275. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
30. Occurrences: 266. Action Sequence: [2.,2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
31. Occurrences: 258. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.]
32. Occurrences: 254. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.]
33. Occurrences: 241. Action Sequence: [2.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
34. Occurrences: 238. Action Sequence: [2.,3.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
35. Occurrences: 232. Action Sequence: [2.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
36. Occurrences: 204. Action Sequence: [2.,3.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
37. Occurrences: 198. Action Sequence: [2.,2.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
38. Occurrences: 193. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
39. Occurrences: 189. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.]
40. Occurrences: 185. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.]
41. Occurrences: 178. Action Sequence: [2.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
42. Occurrences: 165. Action Sequence: [2.,3.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
43. Occurrences: 157. Action Sequence: [2.,2.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
44. Occurrences: 153. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.]
45. Occurrences: 151. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.]
46. Occurrences: 149. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.]
47. Occurrences: 146. Action Sequence: [2.,2.,2.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
48. Occurrences: 143. Action Sequence: [2.,2.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
49. Occurrences: 139. Action Sequence: [2.,2.,3.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
50. Occurrences: 128. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 4
13464 different trajectories
1. Occurrences: 10927. Action Sequence: [0.]
2. Occurrences: 9340. Action Sequence: [0.,0.]
3. Occurrences: 7208. Action Sequence: [0.,0.,0.]
4. Occurrences: 5573. Action Sequence: [0.,0.,0.,0.]
5. Occurrences: 4192. Action Sequence: [0.,0.,0.,0.,0.]
6. Occurrences: 3161. Action Sequence: [0.,0.,0.,0.,0.,0.]
7. Occurrences: 2389. Action Sequence: [0.,0.,0.,0.,0.,0.,0.]
8. Occurrences: 2154. Action Sequence: [0.,1.,0.]
9. Occurrences: 1719. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.]
10. Occurrences: 1405. Action Sequence: [0.,0.,1.,0.]
11. Occurrences: 1270. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.]
12. Occurrences: 1069. Action Sequence: [0.,1.]
13. Occurrences: 949. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
14. Occurrences: 929. Action Sequence: [0.,0.,0.,1.,0.]
15. Occurrences: 860. Action Sequence: [0.,0.,1.]
16. Occurrences: 785. Action Sequence: [0.,1.,0.,1.,0.]
17. Occurrences: 681. Action Sequence: [0.,0.,0.,0.,1.,0.]
18. Occurrences: 640. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
19. Occurrences: 499. Action Sequence: [0.,0.,0.,1.]
20. Occurrences: 495. Action Sequence: [0.,0.,1.,0.,1.,0.]
21. Occurrences: 489. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 444. Action Sequence: [0.,0.,0.,0.,0.,1.,0.]
23. Occurrences: 378. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,0.]
24. Occurrences: 376. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
25. Occurrences: 365. Action Sequence: [0.,0.,0.,1.,0.,1.,0.]
26. Occurrences: 343. Action Sequence: [0.,0.,0.,0.,1.]
27. Occurrences: 317. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
28. Occurrences: 262. Action Sequence: [0.,1.,0.,1.,0.,1.,0.]
29. Occurrences: 250. Action Sequence: [0.,0.,0.,0.,0.,1.]
30. Occurrences: 247. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,0.]
31. Occurrences: 244. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 223. Action Sequence: [0.,1.,1.,0.]
33. Occurrences: 222. Action Sequence: [0.,0.,1.,1.,0.]
34. Occurrences: 217. Action Sequence: [0.,1.,1.]
35. Occurrences: 215. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,0.]
36. Occurrences: 208. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
37. Occurrences: 206. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,0.]
38. Occurrences: 193. Action Sequence: [0.,1.,0.,0.]
39. Occurrences: 191. Action Sequence: [0.,1.,0.,1.,1.,0.]
40. Occurrences: 187. Action Sequence: [0.,0.,1.,0.,0.]
41. Occurrences: 186. Action Sequence: [0.,0.,1.,0.,1.,0.,1.,0.]
42. Occurrences: 184. Action Sequence: [0.,0.,1.,1.]
43. Occurrences: 169. Action Sequence: [0.,0.,0.,0.,0.,0.,1.]
44. Occurrences: 161. Action Sequence: [0.,0.,0.,0.,0.,1.,0.,1.,0.]
45. Occurrences: 157. Action Sequence: [0.,1.,0.,0.,0.]
46. Occurrences: 155. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
47. Occurrences: 153. Action Sequence: [0.,0.,0.,1.,1.,0.]
48. Occurrences: 152. Action Sequence: [0.,0.,1.,1.,1.,0.]
49. Occurrences: 150. Action Sequence: [0.,1.,1.,1.,0.]
50. Occurrences: 143. Action Sequence: [0.,0.,0.,0.,1.,1.,0.]

FACTORED SPACE 1, factorisation 4
4341 different trajectories
1. Occurrences: 59183. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 2061. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 1531. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 1247. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1081. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 892. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 799. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 744. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 696. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 607. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 605. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 499. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 489. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 406. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 403. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 354. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 346. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 337. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 286. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 274. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
21. Occurrences: 269. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 267. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 261. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
24. Occurrences: 254. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
25. Occurrences: 246. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
26. Occurrences: 244. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 240. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
28. Occurrences: 230. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
29. Occurrences: 225. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 204. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 197. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 184. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 180. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 173. Action Sequence: [1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 172. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.]
36. Occurrences: 161. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.]
37. Occurrences: 158. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 149. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 149. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 140. Action Sequence: [1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 139. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.]
42. Occurrences: 134. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 130. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 127. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.]
45. Occurrences: 123. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 123. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
47. Occurrences: 122. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.]
48. Occurrences: 115. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 113. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 113. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 2, factorisation 4
794 different trajectories
1. Occurrences: 11801. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 11272. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
3. Occurrences: 10107. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 9849. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 7468. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 6437. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 5068. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 4276. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 3566. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 2952. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 2652. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 2230. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 1899. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 1690. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 1495. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 1395. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
17. Occurrences: 1252. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.]
18. Occurrences: 1079. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.]
19. Occurrences: 1049. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
20. Occurrences: 898. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.]
21. Occurrences: 623. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 424. Action Sequence: [0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 355. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 354. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 320. Action Sequence: [0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 301. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 265. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 237. Action Sequence: [0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 215. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 214. Action Sequence: [0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
31. Occurrences: 203. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 196. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 190. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 181. Action Sequence: [0.,0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 159. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 152. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 142. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 138. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 134. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 131. Action Sequence: [0.,0.,1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 129. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 128. Action Sequence: [0.,0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 122. Action Sequence: [0.,0.,1.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 122. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 120. Action Sequence: [0.,0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 116. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 111. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 106. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
49. Occurrences: 99. Action Sequence: [0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 95. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]

--------------------------------------------------------
'''

### State 136 Without Diabetes

In [ ]:
'''
NON FACTORED SPACE
5306 different trajectories
1. Occurrences: 13515. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
2. Occurrences: 1630. Action Sequence: [3.,0.]
3. Occurrences: 1497. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.]
4. Occurrences: 1451. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.]
5. Occurrences: 1341. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.]
6. Occurrences: 1272. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.]
7. Occurrences: 1194. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.,7.]
8. Occurrences: 1117. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.,7.,7.]
9. Occurrences: 1016. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.,7.,7.,7.,7.]
10. Occurrences: 1005. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.,7.,7.,7.]
11. Occurrences: 931. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.]
12. Occurrences: 931. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.,7.,7.,7.,7.,7.]
13. Occurrences: 866. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.]
14. Occurrences: 816. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.]
15. Occurrences: 808. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.]
16. Occurrences: 760. Action Sequence: [3.,3.,3.,3.,3.,3.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.]
17. Occurrences: 683. Action Sequence: [3.,3.,3.,3.,3.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.]
18. Occurrences: 673. Action Sequence: [3.,3.,7.,0.]
19. Occurrences: 646. Action Sequence: [3.,3.,3.,3.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.]
20. Occurrences: 606. Action Sequence: [3.,3.,3.,7.,0.]
21. Occurrences: 602. Action Sequence: [3.,3.,3.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.]
22. Occurrences: 583. Action Sequence: [3.,7.,0.]
23. Occurrences: 582. Action Sequence: [3.,3.,3.,3.,7.,0.]
24. Occurrences: 572. Action Sequence: [3.,3.,7.,7.,0.]
25. Occurrences: 519. Action Sequence: [3.,3.,3.,7.,7.,0.]
26. Occurrences: 513. Action Sequence: [3.,3.,3.,3.,3.,7.,0.]
27. Occurrences: 505. Action Sequence: [3.,3.,3.,3.,7.,7.,0.]
28. Occurrences: 492. Action Sequence: [3.,3.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.]
29. Occurrences: 477. Action Sequence: [3.,3.,3.,7.,7.,7.,0.]
30. Occurrences: 476. Action Sequence: [3.,7.,7.,0.]
31. Occurrences: 470. Action Sequence: [3.,3.,7.,7.,7.,0.]
32. Occurrences: 455. Action Sequence: [3.,3.,3.,3.,3.,3.,7.,0.]
33. Occurrences: 442. Action Sequence: [3.,3.,0.]
34. Occurrences: 441. Action Sequence: [3.,3.,3.,3.,7.,7.,7.,0.]
35. Occurrences: 431. Action Sequence: [3.,3.,3.,3.,3.,7.,7.,0.]
36. Occurrences: 419. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,7.,0.]
37. Occurrences: 412. Action Sequence: [3.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
38. Occurrences: 396. Action Sequence: [3.,7.,7.,7.,0.]
39. Occurrences: 384. Action Sequence: [3.,3.,3.,3.,3.,3.,7.,7.,0.]
40. Occurrences: 384. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,7.,0.]
41. Occurrences: 383. Action Sequence: [3.,3.,7.,7.,7.,7.,0.]
42. Occurrences: 381. Action Sequence: [3.,3.,3.,3.,7.,7.,7.,7.,0.]
43. Occurrences: 379. Action Sequence: [3.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.]
44. Occurrences: 378. Action Sequence: [3.,3.,3.,3.,3.,7.,7.,7.,0.]
45. Occurrences: 375. Action Sequence: [3.,3.,3.,7.,7.,7.,7.,0.]
46. Occurrences: 355. Action Sequence: [3.,3.,3.,3.,3.,7.,7.,7.,7.,0.]
47. Occurrences: 353. Action Sequence: [3.,3.,7.,7.,7.,7.,7.,0.]
48. Occurrences: 347. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,7.,7.,0.]
49. Occurrences: 337. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,0.]
50. Occurrences: 335. Action Sequence: [3.,3.,3.,3.,3.,3.,7.,7.,7.,0.]

FACTORED SPACE 0, factorisation 1
1243 different trajectories
1. Occurrences: 31322. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 2912. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
3. Occurrences: 2850. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2725. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 2697. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 2691. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 2676. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 2644. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 2601. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
10. Occurrences: 2597. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
11. Occurrences: 2531. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
12. Occurrences: 2516. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 2498. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
14. Occurrences: 2447. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
15. Occurrences: 2361. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 2038. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 1827. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 1717. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 1362. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 1115. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 681. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 662. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 648. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.]
24. Occurrences: 632. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
25. Occurrences: 632. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 607. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
27. Occurrences: 606. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
28. Occurrences: 604. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 560. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 529. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.]
31. Occurrences: 523. Action Sequence: [1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 520. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.]
33. Occurrences: 494. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.]
34. Occurrences: 434. Action Sequence: [1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
35. Occurrences: 427. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.]
36. Occurrences: 412. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
37. Occurrences: 349. Action Sequence: [1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 303. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
39. Occurrences: 268. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
40. Occurrences: 240. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.]
41. Occurrences: 236. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.]
42. Occurrences: 235. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 228. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.]
44. Occurrences: 213. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 212. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 205. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.]
47. Occurrences: 203. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.]
48. Occurrences: 191. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 189. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 176. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 1
4294 different trajectories
1. Occurrences: 13515. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 1630. Action Sequence: [1.,0.]
3. Occurrences: 1497. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.]
4. Occurrences: 1451. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.]
5. Occurrences: 1341. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.]
6. Occurrences: 1272. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.]
7. Occurrences: 1194. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.]
8. Occurrences: 1117. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.]
9. Occurrences: 1016. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.]
10. Occurrences: 1005. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.]
11. Occurrences: 931. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
12. Occurrences: 931. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
13. Occurrences: 866. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
14. Occurrences: 816. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
15. Occurrences: 808. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
16. Occurrences: 760. Action Sequence: [1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
17. Occurrences: 683. Action Sequence: [1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
18. Occurrences: 673. Action Sequence: [1.,1.,3.,0.]
19. Occurrences: 646. Action Sequence: [1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
20. Occurrences: 606. Action Sequence: [1.,1.,1.,3.,0.]
21. Occurrences: 602. Action Sequence: [1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
22. Occurrences: 583. Action Sequence: [1.,3.,0.]
23. Occurrences: 582. Action Sequence: [1.,1.,1.,1.,3.,0.]
24. Occurrences: 572. Action Sequence: [1.,1.,3.,3.,0.]
25. Occurrences: 546. Action Sequence: [1.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
26. Occurrences: 519. Action Sequence: [1.,1.,1.,3.,3.,0.]
27. Occurrences: 513. Action Sequence: [1.,1.,1.,1.,1.,3.,0.]
28. Occurrences: 505. Action Sequence: [1.,1.,1.,1.,3.,3.,0.]
29. Occurrences: 492. Action Sequence: [1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
30. Occurrences: 477. Action Sequence: [1.,1.,1.,3.,3.,3.,0.]
31. Occurrences: 476. Action Sequence: [1.,3.,3.,0.]
32. Occurrences: 470. Action Sequence: [1.,1.,3.,3.,3.,0.]
33. Occurrences: 455. Action Sequence: [1.,1.,1.,1.,1.,1.,3.,0.]
34. Occurrences: 442. Action Sequence: [1.,1.,0.]
35. Occurrences: 441. Action Sequence: [1.,1.,1.,1.,3.,3.,3.,0.]
36. Occurrences: 431. Action Sequence: [1.,1.,1.,1.,1.,3.,3.,0.]
37. Occurrences: 419. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,3.,0.]
38. Occurrences: 396. Action Sequence: [1.,3.,3.,3.,0.]
39. Occurrences: 384. Action Sequence: [1.,1.,1.,1.,1.,1.,3.,3.,0.]
40. Occurrences: 384. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,3.,0.]
41. Occurrences: 383. Action Sequence: [1.,1.,3.,3.,3.,3.,0.]
42. Occurrences: 381. Action Sequence: [1.,1.,1.,1.,3.,3.,3.,3.,0.]
43. Occurrences: 379. Action Sequence: [1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
44. Occurrences: 378. Action Sequence: [1.,1.,1.,1.,1.,3.,3.,3.,0.]
45. Occurrences: 375. Action Sequence: [1.,1.,1.,3.,3.,3.,3.,0.]
46. Occurrences: 355. Action Sequence: [1.,1.,1.,1.,1.,3.,3.,3.,3.,0.]
47. Occurrences: 353. Action Sequence: [1.,1.,3.,3.,3.,3.,3.,0.]
48. Occurrences: 347. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,3.,3.,0.]
49. Occurrences: 337. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,0.]
50. Occurrences: 335. Action Sequence: [1.,1.,1.,1.,1.,1.,3.,3.,3.,0.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 2
1165 different trajectories
1. Occurrences: 31322. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 2912. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
3. Occurrences: 2782. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2669. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 2635. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 2614. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 2606. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 2580. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 2547. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
10. Occurrences: 2471. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
11. Occurrences: 2468. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
12. Occurrences: 2447. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 2377. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
14. Occurrences: 2356. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
15. Occurrences: 2287. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 1993. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 1734. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 1676. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 1316. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 1074. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 867. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 850. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 829. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
24. Occurrences: 827. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.]
25. Occurrences: 816. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 815. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.]
27. Occurrences: 807. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
28. Occurrences: 801. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 716. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 679. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.]
31. Occurrences: 675. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.]
32. Occurrences: 664. Action Sequence: [1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
33. Occurrences: 662. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.]
34. Occurrences: 583. Action Sequence: [1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
35. Occurrences: 557. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.]
36. Occurrences: 546. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
37. Occurrences: 474. Action Sequence: [1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 348. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
39. Occurrences: 256. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
40. Occurrences: 232. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 227. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.]
42. Occurrences: 223. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.]
43. Occurrences: 223. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.]
44. Occurrences: 209. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 203. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 200. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.]
47. Occurrences: 198. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.]
48. Occurrences: 186. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 183. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 171. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 2
4999 different trajectories
1. Occurrences: 13515. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 1630. Action Sequence: [1.,0.]
3. Occurrences: 1497. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.]
4. Occurrences: 1451. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.]
5. Occurrences: 1341. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.]
6. Occurrences: 1272. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.]
7. Occurrences: 1194. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.]
8. Occurrences: 1117. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.]
9. Occurrences: 1016. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.]
10. Occurrences: 1005. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.]
11. Occurrences: 931. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
12. Occurrences: 931. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
13. Occurrences: 866. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
14. Occurrences: 816. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
15. Occurrences: 808. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
16. Occurrences: 760. Action Sequence: [1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
17. Occurrences: 683. Action Sequence: [1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
18. Occurrences: 673. Action Sequence: [1.,1.,3.,0.]
19. Occurrences: 646. Action Sequence: [1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
20. Occurrences: 606. Action Sequence: [1.,1.,1.,3.,0.]
21. Occurrences: 602. Action Sequence: [1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
22. Occurrences: 583. Action Sequence: [1.,3.,0.]
23. Occurrences: 582. Action Sequence: [1.,1.,1.,1.,3.,0.]
24. Occurrences: 572. Action Sequence: [1.,1.,3.,3.,0.]
25. Occurrences: 519. Action Sequence: [1.,1.,1.,3.,3.,0.]
26. Occurrences: 513. Action Sequence: [1.,1.,1.,1.,1.,3.,0.]
27. Occurrences: 505. Action Sequence: [1.,1.,1.,1.,3.,3.,0.]
28. Occurrences: 492. Action Sequence: [1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
29. Occurrences: 477. Action Sequence: [1.,1.,1.,3.,3.,3.,0.]
30. Occurrences: 476. Action Sequence: [1.,3.,3.,0.]
31. Occurrences: 470. Action Sequence: [1.,1.,3.,3.,3.,0.]
32. Occurrences: 455. Action Sequence: [1.,1.,1.,1.,1.,1.,3.,0.]
33. Occurrences: 442. Action Sequence: [1.,1.,0.]
34. Occurrences: 441. Action Sequence: [1.,1.,1.,1.,3.,3.,3.,0.]
35. Occurrences: 431. Action Sequence: [1.,1.,1.,1.,1.,3.,3.,0.]
36. Occurrences: 419. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,3.,0.]
37. Occurrences: 412. Action Sequence: [1.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
38. Occurrences: 396. Action Sequence: [1.,3.,3.,3.,0.]
39. Occurrences: 384. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,3.,0.]
40. Occurrences: 384. Action Sequence: [1.,1.,1.,1.,1.,1.,3.,3.,0.]
41. Occurrences: 383. Action Sequence: [1.,1.,3.,3.,3.,3.,0.]
42. Occurrences: 381. Action Sequence: [1.,1.,1.,1.,3.,3.,3.,3.,0.]
43. Occurrences: 379. Action Sequence: [1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
44. Occurrences: 378. Action Sequence: [1.,1.,1.,1.,1.,3.,3.,3.,0.]
45. Occurrences: 375. Action Sequence: [1.,1.,1.,3.,3.,3.,3.,0.]
46. Occurrences: 355. Action Sequence: [1.,1.,1.,1.,1.,3.,3.,3.,3.,0.]
47. Occurrences: 353. Action Sequence: [1.,1.,3.,3.,3.,3.,3.,0.]
48. Occurrences: 347. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,3.,3.,0.]
49. Occurrences: 337. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,0.]
50. Occurrences: 335. Action Sequence: [1.,1.,1.,1.,1.,1.,3.,3.,3.,0.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 3
3239 different trajectories
1. Occurrences: 13563. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
2. Occurrences: 1630. Action Sequence: [0.,0.]
3. Occurrences: 1503. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.]
4. Occurrences: 1453. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
5. Occurrences: 1346. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.]
6. Occurrences: 1280. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.]
7. Occurrences: 1202. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
8. Occurrences: 1122. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 1059. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 1023. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 1012. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 935. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 934. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 868. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 819. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 815. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 782. Action Sequence: [0.,0.,1.,0.]
18. Occurrences: 769. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 761. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 701. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 696. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 640. Action Sequence: [0.,0.,0.,1.,0.]
23. Occurrences: 611. Action Sequence: [0.,0.,1.,1.,0.]
24. Occurrences: 589. Action Sequence: [0.,0.,0.,0.,1.,0.]
25. Occurrences: 583. Action Sequence: [0.,1.,0.]
26. Occurrences: 536. Action Sequence: [0.,0.,0.,1.,1.,0.]
27. Occurrences: 515. Action Sequence: [0.,0.,0.,0.,0.,1.,0.]
28. Occurrences: 513. Action Sequence: [0.,0.,0.,0.,1.,1.,0.]
29. Occurrences: 499. Action Sequence: [0.,0.,1.,1.,1.,0.]
30. Occurrences: 483. Action Sequence: [0.,0.,0.,1.,1.,1.,0.]
31. Occurrences: 476. Action Sequence: [0.,1.,1.,0.]
32. Occurrences: 458. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,0.]
33. Occurrences: 449. Action Sequence: [0.,0.,0.]
34. Occurrences: 442. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,0.]
35. Occurrences: 434. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,0.]
36. Occurrences: 419. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,0.]
37. Occurrences: 397. Action Sequence: [0.,0.,1.,1.,1.,1.,0.]
38. Occurrences: 396. Action Sequence: [0.,1.,1.,1.,0.]
39. Occurrences: 386. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,0.]
40. Occurrences: 384. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,0.]
41. Occurrences: 383. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,0.]
42. Occurrences: 381. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,0.]
43. Occurrences: 379. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 379. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,0.]
45. Occurrences: 369. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,0.]
46. Occurrences: 355. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,0.]
47. Occurrences: 348. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,0.]
48. Occurrences: 339. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.]
49. Occurrences: 337. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,0.]
50. Occurrences: 335. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,0.]

FACTORED SPACE 1, factorisation 3
1563 different trajectories
1. Occurrences: 31322. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
2. Occurrences: 2912. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
3. Occurrences: 2782. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.]
4. Occurrences: 2669. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
5. Occurrences: 2635. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
6. Occurrences: 2614. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.]
7. Occurrences: 2606. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
8. Occurrences: 2580. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.]
9. Occurrences: 2547. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.]
10. Occurrences: 2471. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.]
11. Occurrences: 2468. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.]
12. Occurrences: 2447. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
13. Occurrences: 2377. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.]
14. Occurrences: 2356. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.]
15. Occurrences: 2287. Action Sequence: [3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
16. Occurrences: 1993. Action Sequence: [3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
17. Occurrences: 1734. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
18. Occurrences: 1676. Action Sequence: [3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
19. Occurrences: 1316. Action Sequence: [3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
20. Occurrences: 1074. Action Sequence: [3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
21. Occurrences: 681. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 662. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 648. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.]
24. Occurrences: 632. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
25. Occurrences: 632. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 607. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
27. Occurrences: 606. Action Sequence: [3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
28. Occurrences: 604. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 560. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 529. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.]
31. Occurrences: 523. Action Sequence: [3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 520. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.]
33. Occurrences: 494. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.]
34. Occurrences: 434. Action Sequence: [3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
35. Occurrences: 427. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.]
36. Occurrences: 412. Action Sequence: [3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
37. Occurrences: 349. Action Sequence: [3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 268. Action Sequence: [3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
39. Occurrences: 256. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.]
40. Occurrences: 232. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.]
41. Occurrences: 227. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.,3.]
42. Occurrences: 223. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.]
43. Occurrences: 223. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.]
44. Occurrences: 209. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.]
45. Occurrences: 203. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
46. Occurrences: 200. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.]
47. Occurrences: 198. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.]
48. Occurrences: 186. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
49. Occurrences: 183. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.]
50. Occurrences: 171. Action Sequence: [3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 4
3239 different trajectories
1. Occurrences: 13563. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
2. Occurrences: 1630. Action Sequence: [0.,0.]
3. Occurrences: 1503. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.]
4. Occurrences: 1453. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
5. Occurrences: 1346. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.]
6. Occurrences: 1280. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.]
7. Occurrences: 1202. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
8. Occurrences: 1122. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 1059. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 1023. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 1012. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 935. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 934. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 868. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 819. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 815. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 782. Action Sequence: [0.,0.,1.,0.]
18. Occurrences: 769. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 761. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 701. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 696. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 640. Action Sequence: [0.,0.,0.,1.,0.]
23. Occurrences: 611. Action Sequence: [0.,0.,1.,1.,0.]
24. Occurrences: 589. Action Sequence: [0.,0.,0.,0.,1.,0.]
25. Occurrences: 583. Action Sequence: [0.,1.,0.]
26. Occurrences: 536. Action Sequence: [0.,0.,0.,1.,1.,0.]
27. Occurrences: 515. Action Sequence: [0.,0.,0.,0.,0.,1.,0.]
28. Occurrences: 513. Action Sequence: [0.,0.,0.,0.,1.,1.,0.]
29. Occurrences: 499. Action Sequence: [0.,0.,1.,1.,1.,0.]
30. Occurrences: 483. Action Sequence: [0.,0.,0.,1.,1.,1.,0.]
31. Occurrences: 476. Action Sequence: [0.,1.,1.,0.]
32. Occurrences: 458. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,0.]
33. Occurrences: 449. Action Sequence: [0.,0.,0.]
34. Occurrences: 442. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,0.]
35. Occurrences: 434. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,0.]
36. Occurrences: 419. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,0.]
37. Occurrences: 397. Action Sequence: [0.,0.,1.,1.,1.,1.,0.]
38. Occurrences: 396. Action Sequence: [0.,1.,1.,1.,0.]
39. Occurrences: 386. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,0.]
40. Occurrences: 384. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,0.]
41. Occurrences: 383. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,0.]
42. Occurrences: 381. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,0.]
43. Occurrences: 379. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 379. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,0.]
45. Occurrences: 369. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,0.]
46. Occurrences: 355. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,0.]
47. Occurrences: 348. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,0.]
48. Occurrences: 339. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.]
49. Occurrences: 337. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,0.]
50. Occurrences: 335. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,0.]

FACTORED SPACE 1, factorisation 4
1243 different trajectories
1. Occurrences: 31322. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 2912. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
3. Occurrences: 2850. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2725. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 2697. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 2691. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 2676. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 2644. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 2601. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
10. Occurrences: 2597. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
11. Occurrences: 2531. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
12. Occurrences: 2516. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 2498. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
14. Occurrences: 2447. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
15. Occurrences: 2361. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 2038. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 1827. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 1717. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 1362. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 1115. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 681. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 662. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 648. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.]
24. Occurrences: 632. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
25. Occurrences: 632. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 607. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
27. Occurrences: 606. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
28. Occurrences: 604. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 560. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 529. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.]
31. Occurrences: 523. Action Sequence: [1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 520. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.]
33. Occurrences: 494. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.]
34. Occurrences: 434. Action Sequence: [1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
35. Occurrences: 427. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.]
36. Occurrences: 412. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
37. Occurrences: 349. Action Sequence: [1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 303. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
39. Occurrences: 268. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
40. Occurrences: 240. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.]
41. Occurrences: 236. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.]
42. Occurrences: 235. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 228. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.]
44. Occurrences: 213. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 212. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 205. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.]
47. Occurrences: 203. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.]
48. Occurrences: 191. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 189. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 176. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 2, factorisation 4
1165 different trajectories
1. Occurrences: 31322. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 2912. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
3. Occurrences: 2782. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2669. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 2635. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 2614. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 2606. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 2580. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 2547. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
10. Occurrences: 2471. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
11. Occurrences: 2468. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
12. Occurrences: 2447. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 2377. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
14. Occurrences: 2356. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
15. Occurrences: 2287. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 1993. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 1734. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 1676. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 1316. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 1074. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 867. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 850. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 829. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
24. Occurrences: 827. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.]
25. Occurrences: 816. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 815. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.]
27. Occurrences: 807. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
28. Occurrences: 801. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 716. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 679. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.]
31. Occurrences: 675. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.]
32. Occurrences: 664. Action Sequence: [1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
33. Occurrences: 662. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.]
34. Occurrences: 583. Action Sequence: [1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
35. Occurrences: 557. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.]
36. Occurrences: 546. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
37. Occurrences: 474. Action Sequence: [1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 348. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
39. Occurrences: 256. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
40. Occurrences: 232. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 227. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.]
42. Occurrences: 223. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.]
43. Occurrences: 223. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.]
44. Occurrences: 209. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 203. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 200. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.]
47. Occurrences: 198. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.]
48. Occurrences: 186. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 183. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 171. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

--------------------------------------------------------
'''

### State 143, Diabetes

In [ ]:
'''
NON FACTORED SPACE
16283 different trajectories
1. Occurrences: 10186. Action Sequence: [2.,2.]
2. Occurrences: 8345. Action Sequence: [2.]
3. Occurrences: 7991. Action Sequence: [2.,2.,2.]
4. Occurrences: 6131. Action Sequence: [2.,2.,2.,2.]
5. Occurrences: 4421. Action Sequence: [2.,2.,2.,2.,2.]
6. Occurrences: 3286. Action Sequence: [2.,2.,2.,2.,2.,2.]
7. Occurrences: 2299. Action Sequence: [2.,2.,2.,2.,2.,2.,2.]
8. Occurrences: 1700. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.]
9. Occurrences: 1460. Action Sequence: [2.,2.,6.,0.]
10. Occurrences: 1183. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.]
11. Occurrences: 1162. Action Sequence: [2.,6.]
12. Occurrences: 898. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
13. Occurrences: 866. Action Sequence: [2.,2.,2.,6.,0.]
14. Occurrences: 818. Action Sequence: [2.,2.,6.]
15. Occurrences: 632. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
16. Occurrences: 595. Action Sequence: [2.,2.,2.,2.,6.,0.]
17. Occurrences: 573. Action Sequence: [2.,2.,3.]
18. Occurrences: 537. Action Sequence: [2.,2.,6.,0.,6.,0.]
19. Occurrences: 505. Action Sequence: [2.,2.,2.,6.]
20. Occurrences: 461. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
21. Occurrences: 440. Action Sequence: [2.,2.,2.,2.,2.,6.,0.]
22. Occurrences: 392. Action Sequence: [2.,2.,2.,6.,0.,6.,0.]
23. Occurrences: 364. Action Sequence: [2.,3.]
24. Occurrences: 349. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
25. Occurrences: 338. Action Sequence: [2.,2.,2.,2.,2.,2.,6.,0.]
26. Occurrences: 331. Action Sequence: [2.,2.,2.,3.]
27. Occurrences: 314. Action Sequence: [2.,2.,2.,2.,6.]
28. Occurrences: 250. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
29. Occurrences: 247. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
30. Occurrences: 235. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,6.,0.]
31. Occurrences: 231. Action Sequence: [2.,6.,6.]
32. Occurrences: 226. Action Sequence: [2.,2.,2.,2.,6.,0.,6.,0.]
33. Occurrences: 223. Action Sequence: [2.,6.,2.,2.]
34. Occurrences: 216. Action Sequence: [2.,2.,2.,2.,3.]
35. Occurrences: 215. Action Sequence: [2.,2.,6.,0.,6.,0.,6.,0.]
36. Occurrences: 213. Action Sequence: [2.,2.,2.,2.,2.,6.]
37. Occurrences: 179. Action Sequence: [2.,6.,2.,2.,2.]
38. Occurrences: 172. Action Sequence: [2.,2.,6.,6.,0.]
39. Occurrences: 171. Action Sequence: [2.,2.,2.,2.,2.,6.,0.,6.,0.]
40. Occurrences: 164. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
41. Occurrences: 152. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,6.,0.]
42. Occurrences: 151. Action Sequence: [2.,2.,6.,6.]
43. Occurrences: 148. Action Sequence: [2.,2.,2.,2.,2.,3.]
44. Occurrences: 145. Action Sequence: [2.,2.,6.,2.,2.]
45. Occurrences: 145. Action Sequence: [2.,2.,6.,0.,6.,6.,0.]
46. Occurrences: 134. Action Sequence: [2.,2.,3.,2.,2.]
47. Occurrences: 130. Action Sequence: [2.,2.,2.,6.,6.,0.]
48. Occurrences: 130. Action Sequence: [2.,2.,6.,6.,6.,0.]
49. Occurrences: 126. Action Sequence: [2.,2.,2.,2.,2.,2.,6.]
50. Occurrences: 126. Action Sequence: [2.,6.,2.,2.,2.,2.]

FACTORED SPACE 0, factorisation 1
3100 different trajectories
1. Occurrences: 64161. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 1583. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 1204. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 1092. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 971. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 881. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 742. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 668. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 599. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 548. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 535. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 457. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 410. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 388. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 349. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 344. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 328. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 322. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
19. Occurrences: 298. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
20. Occurrences: 293. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
21. Occurrences: 286. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 279. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
23. Occurrences: 253. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
24. Occurrences: 245. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 236. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
26. Occurrences: 235. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 218. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 196. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 194. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 182. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.]
31. Occurrences: 172. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 167. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.]
33. Occurrences: 161. Action Sequence: [1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 153. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.]
35. Occurrences: 152. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 150. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.]
37. Occurrences: 138. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
38. Occurrences: 137. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 136. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 135. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 135. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.]
42. Occurrences: 133. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.]
43. Occurrences: 133. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 120. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 114. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 107. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 105. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 102. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 102. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.]
50. Occurrences: 101. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.]

FACTORED SPACE 1, factorisation 1
16198 different trajectories
1. Occurrences: 10186. Action Sequence: [0.,0.]
2. Occurrences: 8345. Action Sequence: [0.]
3. Occurrences: 7991. Action Sequence: [0.,0.,0.]
4. Occurrences: 6131. Action Sequence: [0.,0.,0.,0.]
5. Occurrences: 4421. Action Sequence: [0.,0.,0.,0.,0.]
6. Occurrences: 3286. Action Sequence: [0.,0.,0.,0.,0.,0.]
7. Occurrences: 2299. Action Sequence: [0.,0.,0.,0.,0.,0.,0.]
8. Occurrences: 1700. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.]
9. Occurrences: 1536. Action Sequence: [0.,0.,2.,0.]
10. Occurrences: 1183. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.]
11. Occurrences: 1162. Action Sequence: [0.,2.]
12. Occurrences: 916. Action Sequence: [0.,0.,0.,2.,0.]
13. Occurrences: 898. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
14. Occurrences: 818. Action Sequence: [0.,0.,2.]
15. Occurrences: 632. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
16. Occurrences: 619. Action Sequence: [0.,0.,0.,0.,2.,0.]
17. Occurrences: 573. Action Sequence: [0.,0.,1.]
18. Occurrences: 539. Action Sequence: [0.,0.,2.,0.,2.,0.]
19. Occurrences: 505. Action Sequence: [0.,0.,0.,2.]
20. Occurrences: 461. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
21. Occurrences: 457. Action Sequence: [0.,0.,0.,0.,0.,2.,0.]
22. Occurrences: 393. Action Sequence: [0.,0.,0.,2.,0.,2.,0.]
23. Occurrences: 364. Action Sequence: [0.,1.]
24. Occurrences: 354. Action Sequence: [0.,0.,0.,0.,0.,0.,2.,0.]
25. Occurrences: 349. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 331. Action Sequence: [0.,0.,0.,1.]
27. Occurrences: 314. Action Sequence: [0.,0.,0.,0.,2.]
28. Occurrences: 250. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 247. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 244. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,2.,0.]
31. Occurrences: 231. Action Sequence: [0.,2.,2.]
32. Occurrences: 227. Action Sequence: [0.,0.,0.,0.,2.,0.,2.,0.]
33. Occurrences: 223. Action Sequence: [0.,2.,0.,0.]
34. Occurrences: 216. Action Sequence: [0.,0.,0.,0.,1.]
35. Occurrences: 215. Action Sequence: [0.,0.,2.,0.,2.,0.,2.,0.]
36. Occurrences: 213. Action Sequence: [0.,0.,0.,0.,0.,2.]
37. Occurrences: 200. Action Sequence: [0.,0.,2.,2.,0.]
38. Occurrences: 179. Action Sequence: [0.,2.,0.,0.,0.]
39. Occurrences: 171. Action Sequence: [0.,0.,0.,0.,0.,2.,0.,2.,0.]
40. Occurrences: 164. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
41. Occurrences: 157. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,2.,0.]
42. Occurrences: 151. Action Sequence: [0.,0.,2.,2.]
43. Occurrences: 148. Action Sequence: [0.,0.,0.,0.,0.,1.]
44. Occurrences: 145. Action Sequence: [0.,0.,2.,0.,0.]
45. Occurrences: 145. Action Sequence: [0.,0.,2.,0.,2.,2.,0.]
46. Occurrences: 140. Action Sequence: [0.,0.,2.,2.,2.,0.]
47. Occurrences: 135. Action Sequence: [0.,0.,0.,2.,2.,0.]
48. Occurrences: 134. Action Sequence: [0.,0.,1.,0.,0.]
49. Occurrences: 127. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,2.,0.]
50. Occurrences: 126. Action Sequence: [0.,2.,0.,0.,0.,0.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 2
783 different trajectories
1. Occurrences: 11948. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 11001. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
3. Occurrences: 9508. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 8871. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 8729. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 6562. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 5621. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 4396. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 3738. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 3044. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 2728. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 2195. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 1965. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 1671. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 1554. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 1325. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
17. Occurrences: 1202. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.]
18. Occurrences: 1094. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.]
19. Occurrences: 999. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
20. Occurrences: 917. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.]
21. Occurrences: 579. Action Sequence: [0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 366. Action Sequence: [0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 290. Action Sequence: [0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
24. Occurrences: 245. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 236. Action Sequence: [0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 223. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 212. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 208. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 202. Action Sequence: [0.,0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 174. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 163. Action Sequence: [0.,0.,1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 162. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
33. Occurrences: 160. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 153. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 150. Action Sequence: [0.,0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 141. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 139. Action Sequence: [0.,0.,1.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 138. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
39. Occurrences: 124. Action Sequence: [0.,0.,0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 122. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 118. Action Sequence: [0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 117. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
43. Occurrences: 114. Action Sequence: [0.,0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 110. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 109. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 100. Action Sequence: [0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 99. Action Sequence: [0.,0.,0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 98. Action Sequence: [0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 95. Action Sequence: [0.,0.,0.,1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 89. Action Sequence: [0.,0.,0.,0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 2
11682 different trajectories
1. Occurrences: 10550. Action Sequence: [1.,1.]
2. Occurrences: 8616. Action Sequence: [1.,1.,1.]
3. Occurrences: 8345. Action Sequence: [1.]
4. Occurrences: 6607. Action Sequence: [1.,1.,1.,1.]
5. Occurrences: 4871. Action Sequence: [1.,1.,1.,1.,1.]
6. Occurrences: 3729. Action Sequence: [1.,1.,1.,1.,1.,1.]
7. Occurrences: 2697. Action Sequence: [1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 2001. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 1561. Action Sequence: [1.,1.,3.,0.]
10. Occurrences: 1423. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 1162. Action Sequence: [1.,3.]
12. Occurrences: 1087. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 965. Action Sequence: [1.,1.,1.,3.,0.]
14. Occurrences: 932. Action Sequence: [1.,1.,3.]
15. Occurrences: 793. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 704. Action Sequence: [1.,1.,1.,1.,3.,0.]
17. Occurrences: 631. Action Sequence: [1.,1.,1.,3.]
18. Occurrences: 586. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 581. Action Sequence: [1.,1.,3.,0.,3.,0.]
20. Occurrences: 505. Action Sequence: [1.,1.,1.,1.,1.,3.,0.]
21. Occurrences: 471. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 437. Action Sequence: [1.,1.,1.,3.,0.,3.,0.]
23. Occurrences: 423. Action Sequence: [1.,1.,1.,1.,3.]
24. Occurrences: 412. Action Sequence: [1.,1.,1.,1.,1.,1.,3.,0.]
25. Occurrences: 365. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 341. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 296. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,3.,0.]
28. Occurrences: 284. Action Sequence: [1.,1.,1.,1.,1.,3.]
29. Occurrences: 262. Action Sequence: [1.,1.,1.,1.,3.,0.,3.,0.]
30. Occurrences: 233. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 231. Action Sequence: [1.,3.,3.]
32. Occurrences: 230. Action Sequence: [1.,1.,3.,0.,3.,0.,3.,0.]
33. Occurrences: 223. Action Sequence: [1.,3.,1.,1.]
34. Occurrences: 215. Action Sequence: [1.,1.,3.,3.,0.]
35. Occurrences: 195. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,3.,0.]
36. Occurrences: 195. Action Sequence: [1.,1.,1.,1.,1.,3.,0.,3.,0.]
37. Occurrences: 185. Action Sequence: [1.,3.,1.,1.,1.]
38. Occurrences: 183. Action Sequence: [1.,1.,1.,1.,1.,1.,3.]
39. Occurrences: 173. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 169. Action Sequence: [1.,1.,3.,3.]
41. Occurrences: 168. Action Sequence: [1.,1.,3.,3.,3.,0.]
42. Occurrences: 163. Action Sequence: [1.,1.,3.,1.,1.]
43. Occurrences: 163. Action Sequence: [1.,1.,1.,3.,3.,0.]
44. Occurrences: 156. Action Sequence: [1.,1.,3.,0.,3.,3.,0.]
45. Occurrences: 155. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,0.]
46. Occurrences: 144. Action Sequence: [1.,1.,1.,3.,0.,3.,0.,3.,0.]
47. Occurrences: 142. Action Sequence: [1.,1.,1.,1.,3.,3.,0.]
48. Occurrences: 137. Action Sequence: [1.,1.,1.,3.,3.,3.,0.]
49. Occurrences: 136. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,3.]
50. Occurrences: 132. Action Sequence: [1.,3.,1.,1.,1.,1.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 3
11601 different trajectories
1. Occurrences: 10550. Action Sequence: [0.,0.]
2. Occurrences: 8616. Action Sequence: [0.,0.,0.]
3. Occurrences: 8345. Action Sequence: [0.]
4. Occurrences: 6607. Action Sequence: [0.,0.,0.,0.]
5. Occurrences: 4871. Action Sequence: [0.,0.,0.,0.,0.]
6. Occurrences: 3729. Action Sequence: [0.,0.,0.,0.,0.,0.]
7. Occurrences: 2697. Action Sequence: [0.,0.,0.,0.,0.,0.,0.]
8. Occurrences: 2001. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.]
9. Occurrences: 1646. Action Sequence: [0.,0.,1.,0.]
10. Occurrences: 1423. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.]
11. Occurrences: 1162. Action Sequence: [0.,1.]
12. Occurrences: 1087. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
13. Occurrences: 1026. Action Sequence: [0.,0.,0.,1.,0.]
14. Occurrences: 932. Action Sequence: [0.,0.,1.]
15. Occurrences: 793. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
16. Occurrences: 736. Action Sequence: [0.,0.,0.,0.,1.,0.]
17. Occurrences: 631. Action Sequence: [0.,0.,0.,1.]
18. Occurrences: 586. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
19. Occurrences: 583. Action Sequence: [0.,0.,1.,0.,1.,0.]
20. Occurrences: 527. Action Sequence: [0.,0.,0.,0.,0.,1.,0.]
21. Occurrences: 471. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 439. Action Sequence: [0.,0.,0.,1.,0.,1.,0.]
23. Occurrences: 437. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,0.]
24. Occurrences: 423. Action Sequence: [0.,0.,0.,0.,1.]
25. Occurrences: 365. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 341. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
27. Occurrences: 308. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,0.]
28. Occurrences: 284. Action Sequence: [0.,0.,0.,0.,0.,1.]
29. Occurrences: 263. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,0.]
30. Occurrences: 246. Action Sequence: [0.,0.,1.,1.,0.]
31. Occurrences: 233. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 231. Action Sequence: [0.,1.,1.]
33. Occurrences: 230. Action Sequence: [0.,0.,1.,0.,1.,0.,1.,0.]
34. Occurrences: 223. Action Sequence: [0.,1.,0.,0.]
35. Occurrences: 206. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,0.]
36. Occurrences: 195. Action Sequence: [0.,0.,0.,0.,0.,1.,0.,1.,0.]
37. Occurrences: 185. Action Sequence: [0.,1.,0.,0.,0.]
38. Occurrences: 183. Action Sequence: [0.,0.,0.,0.,0.,0.,1.]
39. Occurrences: 178. Action Sequence: [0.,0.,1.,1.,1.,0.]
40. Occurrences: 173. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
41. Occurrences: 169. Action Sequence: [0.,0.,0.,1.,1.,0.]
42. Occurrences: 169. Action Sequence: [0.,0.,1.,1.]
43. Occurrences: 163. Action Sequence: [0.,0.,1.,0.,0.]
44. Occurrences: 162. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.]
45. Occurrences: 156. Action Sequence: [0.,0.,1.,0.,1.,1.,0.]
46. Occurrences: 147. Action Sequence: [0.,0.,0.,0.,1.,1.,0.]
47. Occurrences: 144. Action Sequence: [0.,0.,0.,1.,0.,1.,0.,1.,0.]
48. Occurrences: 140. Action Sequence: [0.,0.,0.,1.,1.,1.,0.]
49. Occurrences: 136. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.]
50. Occurrences: 132. Action Sequence: [0.,1.,0.,0.,0.,0.]

FACTORED SPACE 1, factorisation 3
8456 different trajectories
1. Occurrences: 11948. Action Sequence: [2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
2. Occurrences: 9508. Action Sequence: [2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
3. Occurrences: 8729. Action Sequence: [2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
4. Occurrences: 7411. Action Sequence: [2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
5. Occurrences: 5513. Action Sequence: [2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
6. Occurrences: 4197. Action Sequence: [2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
7. Occurrences: 3003. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
8. Occurrences: 2316. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
9. Occurrences: 1686. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
10. Occurrences: 1460. Action Sequence: [2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
11. Occurrences: 1262. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
12. Occurrences: 1038. Action Sequence: [2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
13. Occurrences: 927. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
14. Occurrences: 882. Action Sequence: [2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
15. Occurrences: 837. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
16. Occurrences: 786. Action Sequence: [2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
17. Occurrences: 699. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.]
18. Occurrences: 667. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
19. Occurrences: 546. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
20. Occurrences: 537. Action Sequence: [2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
21. Occurrences: 521. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.]
22. Occurrences: 478. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
23. Occurrences: 454. Action Sequence: [2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
24. Occurrences: 374. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.]
25. Occurrences: 363. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
26. Occurrences: 331. Action Sequence: [2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
27. Occurrences: 308. Action Sequence: [2.,2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
28. Occurrences: 291. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.]
29. Occurrences: 282. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.]
30. Occurrences: 257. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
31. Occurrences: 216. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.]
32. Occurrences: 215. Action Sequence: [2.,2.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
33. Occurrences: 197. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
34. Occurrences: 193. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.]
35. Occurrences: 180. Action Sequence: [2.,2.,3.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
36. Occurrences: 166. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.]
37. Occurrences: 165. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.]
38. Occurrences: 153. Action Sequence: [2.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
39. Occurrences: 153. Action Sequence: [2.,2.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
40. Occurrences: 152. Action Sequence: [2.,2.,2.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
41. Occurrences: 149. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.]
42. Occurrences: 145. Action Sequence: [2.,2.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
43. Occurrences: 144. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.]
44. Occurrences: 143. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.]
45. Occurrences: 139. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.]
46. Occurrences: 134. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,2.]
47. Occurrences: 134. Action Sequence: [2.,2.,2.,2.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
48. Occurrences: 133. Action Sequence: [2.,2.,3.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
49. Occurrences: 128. Action Sequence: [2.,2.,3.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
50. Occurrences: 124. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 4
11601 different trajectories
1. Occurrences: 10550. Action Sequence: [0.,0.]
2. Occurrences: 8616. Action Sequence: [0.,0.,0.]
3. Occurrences: 8345. Action Sequence: [0.]
4. Occurrences: 6607. Action Sequence: [0.,0.,0.,0.]
5. Occurrences: 4871. Action Sequence: [0.,0.,0.,0.,0.]
6. Occurrences: 3729. Action Sequence: [0.,0.,0.,0.,0.,0.]
7. Occurrences: 2697. Action Sequence: [0.,0.,0.,0.,0.,0.,0.]
8. Occurrences: 2001. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.]
9. Occurrences: 1646. Action Sequence: [0.,0.,1.,0.]
10. Occurrences: 1423. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.]
11. Occurrences: 1162. Action Sequence: [0.,1.]
12. Occurrences: 1087. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
13. Occurrences: 1026. Action Sequence: [0.,0.,0.,1.,0.]
14. Occurrences: 932. Action Sequence: [0.,0.,1.]
15. Occurrences: 793. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
16. Occurrences: 736. Action Sequence: [0.,0.,0.,0.,1.,0.]
17. Occurrences: 631. Action Sequence: [0.,0.,0.,1.]
18. Occurrences: 586. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
19. Occurrences: 583. Action Sequence: [0.,0.,1.,0.,1.,0.]
20. Occurrences: 527. Action Sequence: [0.,0.,0.,0.,0.,1.,0.]
21. Occurrences: 471. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 439. Action Sequence: [0.,0.,0.,1.,0.,1.,0.]
23. Occurrences: 437. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,0.]
24. Occurrences: 423. Action Sequence: [0.,0.,0.,0.,1.]
25. Occurrences: 365. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 341. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
27. Occurrences: 308. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,0.]
28. Occurrences: 284. Action Sequence: [0.,0.,0.,0.,0.,1.]
29. Occurrences: 263. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,0.]
30. Occurrences: 246. Action Sequence: [0.,0.,1.,1.,0.]
31. Occurrences: 233. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 231. Action Sequence: [0.,1.,1.]
33. Occurrences: 230. Action Sequence: [0.,0.,1.,0.,1.,0.,1.,0.]
34. Occurrences: 223. Action Sequence: [0.,1.,0.,0.]
35. Occurrences: 206. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,0.]
36. Occurrences: 195. Action Sequence: [0.,0.,0.,0.,0.,1.,0.,1.,0.]
37. Occurrences: 185. Action Sequence: [0.,1.,0.,0.,0.]
38. Occurrences: 183. Action Sequence: [0.,0.,0.,0.,0.,0.,1.]
39. Occurrences: 178. Action Sequence: [0.,0.,1.,1.,1.,0.]
40. Occurrences: 173. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
41. Occurrences: 169. Action Sequence: [0.,0.,0.,1.,1.,0.]
42. Occurrences: 169. Action Sequence: [0.,0.,1.,1.]
43. Occurrences: 163. Action Sequence: [0.,0.,1.,0.,0.]
44. Occurrences: 162. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.]
45. Occurrences: 156. Action Sequence: [0.,0.,1.,0.,1.,1.,0.]
46. Occurrences: 147. Action Sequence: [0.,0.,0.,0.,1.,1.,0.]
47. Occurrences: 144. Action Sequence: [0.,0.,0.,1.,0.,1.,0.,1.,0.]
48. Occurrences: 140. Action Sequence: [0.,0.,0.,1.,1.,1.,0.]
49. Occurrences: 136. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.]
50. Occurrences: 132. Action Sequence: [0.,1.,0.,0.,0.,0.]

FACTORED SPACE 1, factorisation 4
3100 different trajectories
1. Occurrences: 64161. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 1583. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 1204. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 1092. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 971. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 881. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 742. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 668. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 599. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 548. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 535. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 457. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 410. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 388. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 349. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 344. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 328. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 322. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
19. Occurrences: 298. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
20. Occurrences: 293. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
21. Occurrences: 286. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 279. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
23. Occurrences: 253. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
24. Occurrences: 245. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 236. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
26. Occurrences: 235. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 218. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 196. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 194. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 182. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.]
31. Occurrences: 172. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 167. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.]
33. Occurrences: 161. Action Sequence: [1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 153. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.]
35. Occurrences: 152. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 150. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.]
37. Occurrences: 138. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
38. Occurrences: 137. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 136. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 135. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 135. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.]
42. Occurrences: 133. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.]
43. Occurrences: 133. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 120. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 114. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 107. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 105. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 102. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 102. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.]
50. Occurrences: 101. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.]

FACTORED SPACE 2, factorisation 4
783 different trajectories
1. Occurrences: 11948. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 11001. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
3. Occurrences: 9508. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 8871. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 8729. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 6562. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 5621. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 4396. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 3738. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 3044. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 2728. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 2195. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 1965. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 1671. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 1554. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 1325. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
17. Occurrences: 1202. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.]
18. Occurrences: 1094. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.]
19. Occurrences: 999. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
20. Occurrences: 917. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.]
21. Occurrences: 579. Action Sequence: [0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 366. Action Sequence: [0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 290. Action Sequence: [0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
24. Occurrences: 245. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 236. Action Sequence: [0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 223. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 212. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 208. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 202. Action Sequence: [0.,0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 174. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 163. Action Sequence: [0.,0.,1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 162. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
33. Occurrences: 160. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 153. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 150. Action Sequence: [0.,0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 141. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 139. Action Sequence: [0.,0.,1.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 138. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
39. Occurrences: 124. Action Sequence: [0.,0.,0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 122. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 118. Action Sequence: [0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 117. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
43. Occurrences: 114. Action Sequence: [0.,0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 110. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 109. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 100. Action Sequence: [0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 99. Action Sequence: [0.,0.,0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 98. Action Sequence: [0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 95. Action Sequence: [0.,0.,0.,1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 89. Action Sequence: [0.,0.,0.,0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

--------------------------------------------------------
'''

### State 63, Diabetes

In [ ]:
'''
NON FACTORED SPACE
15265 different trajectories
1. Occurrences: 25038. Action Sequence: [3.]
2. Occurrences: 6069. Action Sequence: [3.,6.]
3. Occurrences: 5521. Action Sequence: [3.,2.,2.]
4. Occurrences: 4098. Action Sequence: [3.,2.,2.,2.]
5. Occurrences: 3187. Action Sequence: [3.,2.,2.,2.,2.]
6. Occurrences: 2281. Action Sequence: [3.,2.,2.,2.,2.,2.]
7. Occurrences: 1603. Action Sequence: [3.,2.,2.,2.,2.,2.,2.]
8. Occurrences: 1468. Action Sequence: [3.,2.]
9. Occurrences: 1224. Action Sequence: [3.,6.,2.,2.]
10. Occurrences: 1168. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.]
11. Occurrences: 1135. Action Sequence: [3.,6.,6.]
12. Occurrences: 1113. Action Sequence: [3.,3.]
13. Occurrences: 960. Action Sequence: [3.,2.,6.,0.]
14. Occurrences: 868. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.]
15. Occurrences: 841. Action Sequence: [3.,6.,2.,2.,2.]
16. Occurrences: 670. Action Sequence: [3.,6.,2.]
17. Occurrences: 642. Action Sequence: [3.,2.,2.,6.,0.]
18. Occurrences: 628. Action Sequence: [3.,6.,2.,2.,2.,2.]
19. Occurrences: 582. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
20. Occurrences: 467. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
21. Occurrences: 444. Action Sequence: [3.,6.,2.,2.,2.,2.,2.]
22. Occurrences: 434. Action Sequence: [3.,2.,2.,2.,6.,0.]
23. Occurrences: 373. Action Sequence: [3.,6.,2.,2.,2.,2.,2.,2.]
24. Occurrences: 347. Action Sequence: [3.,2.,6.,0.,6.,0.]
25. Occurrences: 334. Action Sequence: [3.,2.,2.,6.]
26. Occurrences: 313. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
27. Occurrences: 288. Action Sequence: [3.,2.,2.,2.,2.,6.,0.]
28. Occurrences: 263. Action Sequence: [3.,2.,2.,3.]
29. Occurrences: 260. Action Sequence: [3.,3.,6.]
30. Occurrences: 249. Action Sequence: [3.,2.,2.,6.,0.,6.,0.]
31. Occurrences: 248. Action Sequence: [3.,3.,2.,2.]
32. Occurrences: 239. Action Sequence: [3.,6.,2.,2.,2.,2.,2.,2.,2.]
33. Occurrences: 226. Action Sequence: [3.,2.,2.,2.,2.,2.,6.,0.]
34. Occurrences: 222. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
35. Occurrences: 208. Action Sequence: [3.,6.,6.,6.]
36. Occurrences: 206. Action Sequence: [3.,2.,2.,2.,6.]
37. Occurrences: 204. Action Sequence: [3.,6.,6.,2.,2.]
38. Occurrences: 204. Action Sequence: [3.,6.,2.,2.,2.,2.,2.,2.,2.,2.]
39. Occurrences: 183. Action Sequence: [3.,3.,2.,2.,2.]
40. Occurrences: 176. Action Sequence: [3.,2.,2.,2.,6.,0.,6.,0.]
41. Occurrences: 174. Action Sequence: [3.,2.,3.]
42. Occurrences: 172. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
43. Occurrences: 170. Action Sequence: [3.,2.,2.,2.,2.,6.]
44. Occurrences: 169. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,6.,0.]
45. Occurrences: 158. Action Sequence: [3.,2.,6.,0.,6.,0.,6.,0.]
46. Occurrences: 154. Action Sequence: [3.,6.,6.,2.,2.,2.]
47. Occurrences: 153. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
48. Occurrences: 149. Action Sequence: [3.,2.,2.,2.,3.]
49. Occurrences: 136. Action Sequence: [3.,6.,2.,2.,6.,0.]
50. Occurrences: 135. Action Sequence: [3.,3.,2.,2.,2.,2.]

FACTORED SPACE 0, factorisation 1
2822 different trajectories
1. Occurrences: 71869. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 1077. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 896. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 876. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 792. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 723. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 661. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 511. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 439. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 400. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 383. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 360. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 347. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 334. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 325. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 290. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 276. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
18. Occurrences: 275. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
19. Occurrences: 262. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 248. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
21. Occurrences: 239. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 223. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
23. Occurrences: 222. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
24. Occurrences: 216. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 203. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
26. Occurrences: 193. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 186. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 181. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 164. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.]
30. Occurrences: 144. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 140. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 138. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.]
33. Occurrences: 134. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.]
34. Occurrences: 134. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
35. Occurrences: 130. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.]
36. Occurrences: 121. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.]
37. Occurrences: 121. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 119. Action Sequence: [1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 117. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 113. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.]
41. Occurrences: 109. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 102. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.]
43. Occurrences: 100. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 92. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 89. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 89. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 87. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 87. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.]
49. Occurrences: 85. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 84. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 1
15175 different trajectories
1. Occurrences: 25038. Action Sequence: [1.]
2. Occurrences: 6069. Action Sequence: [1.,2.]
3. Occurrences: 5521. Action Sequence: [1.,0.,0.]
4. Occurrences: 4098. Action Sequence: [1.,0.,0.,0.]
5. Occurrences: 3187. Action Sequence: [1.,0.,0.,0.,0.]
6. Occurrences: 2281. Action Sequence: [1.,0.,0.,0.,0.,0.]
7. Occurrences: 1603. Action Sequence: [1.,0.,0.,0.,0.,0.,0.]
8. Occurrences: 1468. Action Sequence: [1.,0.]
9. Occurrences: 1224. Action Sequence: [1.,2.,0.,0.]
10. Occurrences: 1168. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.]
11. Occurrences: 1135. Action Sequence: [1.,2.,2.]
12. Occurrences: 1113. Action Sequence: [1.,1.]
13. Occurrences: 960. Action Sequence: [1.,0.,2.,0.]
14. Occurrences: 868. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.]
15. Occurrences: 841. Action Sequence: [1.,2.,0.,0.,0.]
16. Occurrences: 677. Action Sequence: [1.,0.,0.,2.,0.]
17. Occurrences: 670. Action Sequence: [1.,2.,0.]
18. Occurrences: 628. Action Sequence: [1.,2.,0.,0.,0.,0.]
19. Occurrences: 582. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
20. Occurrences: 467. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
21. Occurrences: 459. Action Sequence: [1.,0.,0.,0.,2.,0.]
22. Occurrences: 444. Action Sequence: [1.,2.,0.,0.,0.,0.,0.]
23. Occurrences: 373. Action Sequence: [1.,2.,0.,0.,0.,0.,0.,0.]
24. Occurrences: 347. Action Sequence: [1.,0.,2.,0.,2.,0.]
25. Occurrences: 334. Action Sequence: [1.,0.,0.,2.]
26. Occurrences: 313. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
27. Occurrences: 303. Action Sequence: [1.,0.,0.,0.,0.,2.,0.]
28. Occurrences: 263. Action Sequence: [1.,0.,0.,1.]
29. Occurrences: 260. Action Sequence: [1.,1.,2.]
30. Occurrences: 250. Action Sequence: [1.,0.,0.,2.,0.,2.,0.]
31. Occurrences: 248. Action Sequence: [1.,1.,0.,0.]
32. Occurrences: 239. Action Sequence: [1.,2.,0.,0.,0.,0.,0.,0.,0.]
33. Occurrences: 236. Action Sequence: [1.,0.,0.,0.,0.,0.,2.,0.]
34. Occurrences: 222. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
35. Occurrences: 208. Action Sequence: [1.,2.,2.,2.]
36. Occurrences: 206. Action Sequence: [1.,0.,0.,0.,2.]
37. Occurrences: 204. Action Sequence: [1.,2.,0.,0.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 204. Action Sequence: [1.,2.,2.,0.,0.]
39. Occurrences: 183. Action Sequence: [1.,1.,0.,0.,0.]
40. Occurrences: 178. Action Sequence: [1.,0.,0.,0.,2.,0.,2.,0.]
41. Occurrences: 175. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,2.,0.]
42. Occurrences: 174. Action Sequence: [1.,0.,1.]
43. Occurrences: 172. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
44. Occurrences: 170. Action Sequence: [1.,0.,0.,0.,0.,2.]
45. Occurrences: 158. Action Sequence: [1.,0.,2.,0.,2.,0.,2.,0.]
46. Occurrences: 154. Action Sequence: [1.,2.,2.,0.,0.,0.]
47. Occurrences: 153. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
48. Occurrences: 149. Action Sequence: [1.,0.,0.,0.,1.]
49. Occurrences: 141. Action Sequence: [1.,2.,0.,0.,2.,0.]
50. Occurrences: 135. Action Sequence: [1.,1.,0.,0.,0.,0.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 2
779 different trajectories
1. Occurrences: 26198. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 8605. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
3. Occurrences: 7773. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 7603. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 7248. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 5603. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 4742. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 3654. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 3206. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 2636. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 2105. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 1818. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 1560. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 1332. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 1217. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 1101. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
17. Occurrences: 1019. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.]
18. Occurrences: 922. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.]
19. Occurrences: 803. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
20. Occurrences: 713. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.]
21. Occurrences: 546. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 417. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 337. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 336. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 301. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 272. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 250. Action Sequence: [1.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
28. Occurrences: 247. Action Sequence: [1.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 198. Action Sequence: [1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 196. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 185. Action Sequence: [1.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 159. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
33. Occurrences: 143. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 137. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 127. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 121. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 102. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 101. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
39. Occurrences: 101. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 99. Action Sequence: [1.,0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 92. Action Sequence: [1.,0.,0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 91. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 90. Action Sequence: [1.,0.,0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 90. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 89. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 89. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 88. Action Sequence: [1.,0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 86. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 80. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 77. Action Sequence: [1.,0.,0.,0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 2
11342 different trajectories
1. Occurrences: 25038. Action Sequence: [1.]
2. Occurrences: 6069. Action Sequence: [1.,3.]
3. Occurrences: 5808. Action Sequence: [1.,1.,1.]
4. Occurrences: 4634. Action Sequence: [1.,1.,1.,1.]
5. Occurrences: 3615. Action Sequence: [1.,1.,1.,1.,1.]
6. Occurrences: 2618. Action Sequence: [1.,1.,1.,1.,1.,1.]
7. Occurrences: 2581. Action Sequence: [1.,1.]
8. Occurrences: 1920. Action Sequence: [1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 1421. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 1234. Action Sequence: [1.,3.,1.,1.]
11. Occurrences: 1135. Action Sequence: [1.,3.,3.]
12. Occurrences: 1065. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 1060. Action Sequence: [1.,1.,3.,0.]
14. Occurrences: 891. Action Sequence: [1.,3.,1.,1.,1.]
15. Occurrences: 744. Action Sequence: [1.,1.,1.,3.,0.]
16. Occurrences: 724. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 718. Action Sequence: [1.,3.,1.]
18. Occurrences: 677. Action Sequence: [1.,3.,1.,1.,1.,1.]
19. Occurrences: 601. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 511. Action Sequence: [1.,1.,1.,1.,3.,0.]
21. Occurrences: 484. Action Sequence: [1.,3.,1.,1.,1.,1.,1.]
22. Occurrences: 429. Action Sequence: [1.,3.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 426. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 378. Action Sequence: [1.,1.,1.,3.]
25. Occurrences: 378. Action Sequence: [1.,1.,3.,0.,3.,0.]
26. Occurrences: 359. Action Sequence: [1.,1.,1.,1.,1.,3.,0.]
27. Occurrences: 305. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 299. Action Sequence: [1.,1.,1.,1.,3.]
29. Occurrences: 280. Action Sequence: [1.,1.,1.,3.,0.,3.,0.]
30. Occurrences: 274. Action Sequence: [1.,1.,1.,1.,1.,1.,3.,0.]
31. Occurrences: 272. Action Sequence: [1.,3.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 260. Action Sequence: [1.,1.,3.]
33. Occurrences: 244. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 241. Action Sequence: [1.,3.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 227. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 227. Action Sequence: [1.,1.,1.,1.,1.,3.]
37. Occurrences: 208. Action Sequence: [1.,3.,3.,3.]
38. Occurrences: 207. Action Sequence: [1.,1.,1.,1.,3.,0.,3.,0.]
39. Occurrences: 207. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,3.,0.]
40. Occurrences: 204. Action Sequence: [1.,3.,3.,1.,1.]
41. Occurrences: 174. Action Sequence: [1.,1.,3.,0.,3.,0.,3.,0.]
42. Occurrences: 172. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 160. Action Sequence: [1.,3.,3.,1.,1.,1.]
44. Occurrences: 151. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,3.,0.]
45. Occurrences: 146. Action Sequence: [1.,3.,1.,1.,3.,0.]
46. Occurrences: 144. Action Sequence: [1.,1.,1.,1.,1.,3.,0.,3.,0.]
47. Occurrences: 139. Action Sequence: [1.,1.,1.,3.,3.,0.]
48. Occurrences: 138. Action Sequence: [1.,1.,1.,1.,1.,1.,3.]
49. Occurrences: 130. Action Sequence: [1.,3.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 124. Action Sequence: [1.,3.,3.,1.,1.,1.,1.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 3
11262 different trajectories
1. Occurrences: 25038. Action Sequence: [0.]
2. Occurrences: 6069. Action Sequence: [0.,1.]
3. Occurrences: 5808. Action Sequence: [0.,0.,0.]
4. Occurrences: 4634. Action Sequence: [0.,0.,0.,0.]
5. Occurrences: 3615. Action Sequence: [0.,0.,0.,0.,0.]
6. Occurrences: 2618. Action Sequence: [0.,0.,0.,0.,0.,0.]
7. Occurrences: 2581. Action Sequence: [0.,0.]
8. Occurrences: 1920. Action Sequence: [0.,0.,0.,0.,0.,0.,0.]
9. Occurrences: 1421. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.]
10. Occurrences: 1234. Action Sequence: [0.,1.,0.,0.]
11. Occurrences: 1135. Action Sequence: [0.,1.,1.]
12. Occurrences: 1088. Action Sequence: [0.,0.,1.,0.]
13. Occurrences: 1065. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.]
14. Occurrences: 891. Action Sequence: [0.,1.,0.,0.,0.]
15. Occurrences: 784. Action Sequence: [0.,0.,0.,1.,0.]
16. Occurrences: 724. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
17. Occurrences: 718. Action Sequence: [0.,1.,0.]
18. Occurrences: 677. Action Sequence: [0.,1.,0.,0.,0.,0.]
19. Occurrences: 601. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
20. Occurrences: 552. Action Sequence: [0.,0.,0.,0.,1.,0.]
21. Occurrences: 484. Action Sequence: [0.,1.,0.,0.,0.,0.,0.]
22. Occurrences: 429. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 426. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
24. Occurrences: 378. Action Sequence: [0.,0.,0.,1.]
25. Occurrences: 378. Action Sequence: [0.,0.,1.,0.,1.,0.]
26. Occurrences: 377. Action Sequence: [0.,0.,0.,0.,0.,1.,0.]
27. Occurrences: 305. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
28. Occurrences: 299. Action Sequence: [0.,0.,0.,0.,1.]
29. Occurrences: 289. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,0.]
30. Occurrences: 281. Action Sequence: [0.,0.,0.,1.,0.,1.,0.]
31. Occurrences: 272. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 260. Action Sequence: [0.,0.,1.]
33. Occurrences: 244. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
34. Occurrences: 241. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.]
35. Occurrences: 227. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
36. Occurrences: 227. Action Sequence: [0.,0.,0.,0.,0.,1.]
37. Occurrences: 217. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,0.]
38. Occurrences: 209. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,0.]
39. Occurrences: 208. Action Sequence: [0.,1.,1.,1.]
40. Occurrences: 204. Action Sequence: [0.,1.,1.,0.,0.]
41. Occurrences: 174. Action Sequence: [0.,0.,1.,0.,1.,0.,1.,0.]
42. Occurrences: 172. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
43. Occurrences: 160. Action Sequence: [0.,1.,1.,0.,0.,0.]
44. Occurrences: 159. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,0.]
45. Occurrences: 151. Action Sequence: [0.,1.,0.,0.,1.,0.]
46. Occurrences: 145. Action Sequence: [0.,0.,0.,1.,1.,0.]
47. Occurrences: 144. Action Sequence: [0.,0.,0.,0.,0.,1.,0.,1.,0.]
48. Occurrences: 138. Action Sequence: [0.,0.,0.,0.,0.,0.,1.]
49. Occurrences: 136. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.]
50. Occurrences: 130. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]

FACTORED SPACE 1, factorisation 3
7476 different trajectories
1. Occurrences: 26198. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
2. Occurrences: 7773. Action Sequence: [3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
3. Occurrences: 7603. Action Sequence: [3.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
4. Occurrences: 6288. Action Sequence: [3.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
5. Occurrences: 4875. Action Sequence: [3.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
6. Occurrences: 3654. Action Sequence: [3.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
7. Occurrences: 2659. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
8. Occurrences: 2061. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
9. Occurrences: 1539. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
10. Occurrences: 1119. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
11. Occurrences: 960. Action Sequence: [3.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
12. Occurrences: 821. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
13. Occurrences: 738. Action Sequence: [3.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
14. Occurrences: 727. Action Sequence: [3.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
15. Occurrences: 642. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
16. Occurrences: 614. Action Sequence: [3.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
17. Occurrences: 605. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.]
18. Occurrences: 560. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
19. Occurrences: 488. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
20. Occurrences: 446. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.]
21. Occurrences: 350. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
22. Occurrences: 347. Action Sequence: [3.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
23. Occurrences: 342. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.]
24. Occurrences: 337. Action Sequence: [3.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
25. Occurrences: 337. Action Sequence: [3.,3.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
26. Occurrences: 284. Action Sequence: [3.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
27. Occurrences: 278. Action Sequence: [3.,3.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
28. Occurrences: 276. Action Sequence: [3.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
29. Occurrences: 274. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
30. Occurrences: 259. Action Sequence: [3.,2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
31. Occurrences: 254. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.]
32. Occurrences: 246. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.]
33. Occurrences: 227. Action Sequence: [3.,3.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
34. Occurrences: 203. Action Sequence: [3.,3.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
35. Occurrences: 202. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.]
36. Occurrences: 196. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
37. Occurrences: 194. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.]
38. Occurrences: 167. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
39. Occurrences: 159. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.]
40. Occurrences: 158. Action Sequence: [3.,2.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
41. Occurrences: 145. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,2.]
42. Occurrences: 143. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.]
43. Occurrences: 143. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.]
44. Occurrences: 142. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.,3.]
45. Occurrences: 139. Action Sequence: [3.,3.,2.,2.,2.,2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
46. Occurrences: 129. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.]
47. Occurrences: 114. Action Sequence: [3.,2.,2.,2.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
48. Occurrences: 112. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.]
49. Occurrences: 106. Action Sequence: [3.,2.,2.,2.,2.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
50. Occurrences: 101. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 4
11262 different trajectories
1. Occurrences: 25038. Action Sequence: [0.]
2. Occurrences: 6069. Action Sequence: [0.,1.]
3. Occurrences: 5808. Action Sequence: [0.,0.,0.]
4. Occurrences: 4634. Action Sequence: [0.,0.,0.,0.]
5. Occurrences: 3615. Action Sequence: [0.,0.,0.,0.,0.]
6. Occurrences: 2618. Action Sequence: [0.,0.,0.,0.,0.,0.]
7. Occurrences: 2581. Action Sequence: [0.,0.]
8. Occurrences: 1920. Action Sequence: [0.,0.,0.,0.,0.,0.,0.]
9. Occurrences: 1421. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.]
10. Occurrences: 1234. Action Sequence: [0.,1.,0.,0.]
11. Occurrences: 1135. Action Sequence: [0.,1.,1.]
12. Occurrences: 1088. Action Sequence: [0.,0.,1.,0.]
13. Occurrences: 1065. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.]
14. Occurrences: 891. Action Sequence: [0.,1.,0.,0.,0.]
15. Occurrences: 784. Action Sequence: [0.,0.,0.,1.,0.]
16. Occurrences: 724. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
17. Occurrences: 718. Action Sequence: [0.,1.,0.]
18. Occurrences: 677. Action Sequence: [0.,1.,0.,0.,0.,0.]
19. Occurrences: 601. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
20. Occurrences: 552. Action Sequence: [0.,0.,0.,0.,1.,0.]
21. Occurrences: 484. Action Sequence: [0.,1.,0.,0.,0.,0.,0.]
22. Occurrences: 429. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 426. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
24. Occurrences: 378. Action Sequence: [0.,0.,0.,1.]
25. Occurrences: 378. Action Sequence: [0.,0.,1.,0.,1.,0.]
26. Occurrences: 377. Action Sequence: [0.,0.,0.,0.,0.,1.,0.]
27. Occurrences: 305. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
28. Occurrences: 299. Action Sequence: [0.,0.,0.,0.,1.]
29. Occurrences: 289. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,0.]
30. Occurrences: 281. Action Sequence: [0.,0.,0.,1.,0.,1.,0.]
31. Occurrences: 272. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 260. Action Sequence: [0.,0.,1.]
33. Occurrences: 244. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
34. Occurrences: 241. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.]
35. Occurrences: 227. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
36. Occurrences: 227. Action Sequence: [0.,0.,0.,0.,0.,1.]
37. Occurrences: 217. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,0.]
38. Occurrences: 209. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,0.]
39. Occurrences: 208. Action Sequence: [0.,1.,1.,1.]
40. Occurrences: 204. Action Sequence: [0.,1.,1.,0.,0.]
41. Occurrences: 174. Action Sequence: [0.,0.,1.,0.,1.,0.,1.,0.]
42. Occurrences: 172. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
43. Occurrences: 160. Action Sequence: [0.,1.,1.,0.,0.,0.]
44. Occurrences: 159. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,0.]
45. Occurrences: 151. Action Sequence: [0.,1.,0.,0.,1.,0.]
46. Occurrences: 145. Action Sequence: [0.,0.,0.,1.,1.,0.]
47. Occurrences: 144. Action Sequence: [0.,0.,0.,0.,0.,1.,0.,1.,0.]
48. Occurrences: 138. Action Sequence: [0.,0.,0.,0.,0.,0.,1.]
49. Occurrences: 136. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.]
50. Occurrences: 130. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]

FACTORED SPACE 1, factorisation 4
2822 different trajectories
1. Occurrences: 71869. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 1077. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 896. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 876. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 792. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 723. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 661. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 511. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 439. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 400. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 383. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 360. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 347. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 334. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 325. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 290. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 276. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
18. Occurrences: 275. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
19. Occurrences: 262. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 248. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
21. Occurrences: 239. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 223. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
23. Occurrences: 222. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
24. Occurrences: 216. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 203. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
26. Occurrences: 193. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 186. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 181. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 164. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.]
30. Occurrences: 144. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 140. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 138. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.]
33. Occurrences: 134. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.]
34. Occurrences: 134. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
35. Occurrences: 130. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.]
36. Occurrences: 121. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.]
37. Occurrences: 121. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 119. Action Sequence: [1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 117. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 113. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.]
41. Occurrences: 109. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 102. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.]
43. Occurrences: 100. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 92. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 89. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 89. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 87. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 87. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.]
49. Occurrences: 85. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 84. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.]

FACTORED SPACE 2, factorisation 4
779 different trajectories
1. Occurrences: 26198. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 8605. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
3. Occurrences: 7773. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 7603. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 7248. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 5603. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 4742. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 3654. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 3206. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 2636. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 2105. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 1818. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 1560. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 1332. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 1217. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 1101. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
17. Occurrences: 1019. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.]
18. Occurrences: 922. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.]
19. Occurrences: 803. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
20. Occurrences: 713. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.]
21. Occurrences: 546. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 417. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 337. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 336. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 301. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 272. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 250. Action Sequence: [1.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
28. Occurrences: 247. Action Sequence: [1.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 198. Action Sequence: [1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 196. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 185. Action Sequence: [1.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 159. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
33. Occurrences: 143. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 137. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 127. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 121. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 102. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 101. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
39. Occurrences: 101. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 99. Action Sequence: [1.,0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 92. Action Sequence: [1.,0.,0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 91. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 90. Action Sequence: [1.,0.,0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 90. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 89. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 89. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 88. Action Sequence: [1.,0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 86. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 80. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 77. Action Sequence: [1.,0.,0.,0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

--------------------------------------------------------
'''

### State 56, Non Diabetes

In [ ]:
'''
NON FACTORED SPACE
5315 different trajectories
1. Occurrences: 13451. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
2. Occurrences: 3714. Action Sequence: [3.,0.]
3. Occurrences: 1949. Action Sequence: [3.,3.,0.]
4. Occurrences: 1487. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.]
5. Occurrences: 1460. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.]
6. Occurrences: 1326. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.]
7. Occurrences: 1296. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.]
8. Occurrences: 1172. Action Sequence: [3.,7.,0.]
9. Occurrences: 1167. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.,7.]
10. Occurrences: 1165. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.,7.,7.]
11. Occurrences: 1024. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.,7.,7.,7.]
12. Occurrences: 1018. Action Sequence: [3.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
13. Occurrences: 968. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.,7.,7.,7.,7.]
14. Occurrences: 889. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.,7.,7.,7.,7.,7.]
15. Occurrences: 884. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.]
16. Occurrences: 847. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.]
17. Occurrences: 803. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.]
18. Occurrences: 775. Action Sequence: [3.,3.,3.,0.]
19. Occurrences: 766. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.]
20. Occurrences: 748. Action Sequence: [3.,3.,3.,3.,3.,3.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.]
21. Occurrences: 647. Action Sequence: [3.,3.,7.,0.]
22. Occurrences: 643. Action Sequence: [3.,3.,3.,3.,3.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.]
23. Occurrences: 627. Action Sequence: [3.,3.,3.,3.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.]
24. Occurrences: 605. Action Sequence: [3.,3.,3.,7.,0.]
25. Occurrences: 556. Action Sequence: [3.,3.,3.,3.,7.,0.]
26. Occurrences: 550. Action Sequence: [3.,3.,3.,3.,3.,7.,0.]
27. Occurrences: 543. Action Sequence: [3.,3.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
28. Occurrences: 494. Action Sequence: [3.,3.,3.,3.,7.,7.,0.]
29. Occurrences: 485. Action Sequence: [3.,3.,3.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.]
30. Occurrences: 458. Action Sequence: [3.,3.,3.,7.,7.,0.]
31. Occurrences: 452. Action Sequence: [3.,3.,3.,3.,3.,3.,7.,0.]
32. Occurrences: 448. Action Sequence: [3.,3.,3.,3.,3.,7.,7.,0.]
33. Occurrences: 425. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,7.,0.]
34. Occurrences: 410. Action Sequence: [3.,3.,3.,7.,7.,7.,0.]
35. Occurrences: 407. Action Sequence: [3.,3.,3.,3.,7.,7.,7.,0.]
36. Occurrences: 393. Action Sequence: [3.,3.,3.,3.,3.,3.,7.,7.,0.]
37. Occurrences: 383. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,7.,7.,0.]
38. Occurrences: 381. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,7.,0.]
39. Occurrences: 374. Action Sequence: [3.,3.,7.,7.,0.]
40. Occurrences: 368. Action Sequence: [3.,3.,3.,3.,3.,7.,7.,7.,0.]
41. Occurrences: 353. Action Sequence: [3.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
42. Occurrences: 352. Action Sequence: [3.,3.,3.,3.,7.,7.,7.,7.,0.]
43. Occurrences: 343. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,0.]
44. Occurrences: 339. Action Sequence: [3.,3.,3.,3.,3.,3.,7.,7.,7.,0.]
45. Occurrences: 335. Action Sequence: [3.,3.,3.,3.,3.,7.,7.,7.,7.,0.]
46. Occurrences: 317. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.,0.]
47. Occurrences: 316. Action Sequence: [3.,3.,3.,3.,0.]
48. Occurrences: 311. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,0.]
49. Occurrences: 310. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,0.]
50. Occurrences: 304. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,0.]

FACTORED SPACE 0, factorisation 1
1364 different trajectories
1. Occurrences: 30287. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 4239. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 3472. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2665. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
5. Occurrences: 2455. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 2435. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 2427. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
8. Occurrences: 2418. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
9. Occurrences: 2373. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
10. Occurrences: 2340. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 2332. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 2317. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 2300. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
14. Occurrences: 2284. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
15. Occurrences: 2207. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 1941. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 1887. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 1832. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 1533. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 1474. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 1018. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 896. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 580. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.]
24. Occurrences: 570. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
25. Occurrences: 569. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 549. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.]
27. Occurrences: 549. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
28. Occurrences: 541. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 511. Action Sequence: [1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 510. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
31. Occurrences: 506. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 505. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.]
33. Occurrences: 504. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.]
34. Occurrences: 503. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.]
35. Occurrences: 441. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.]
36. Occurrences: 434. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
37. Occurrences: 397. Action Sequence: [1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 390. Action Sequence: [1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
39. Occurrences: 310. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
40. Occurrences: 261. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 256. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 240. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.]
43. Occurrences: 206. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.]
44. Occurrences: 203. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 202. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 193. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 189. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.]
48. Occurrences: 182. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 181. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.]
50. Occurrences: 177. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 1
4344 different trajectories
1. Occurrences: 13451. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 3714. Action Sequence: [1.,0.]
3. Occurrences: 1949. Action Sequence: [1.,1.,0.]
4. Occurrences: 1487. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.]
5. Occurrences: 1460. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.]
6. Occurrences: 1363. Action Sequence: [1.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
7. Occurrences: 1326. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.]
8. Occurrences: 1296. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.]
9. Occurrences: 1172. Action Sequence: [1.,3.,0.]
10. Occurrences: 1167. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.]
11. Occurrences: 1165. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.]
12. Occurrences: 1024. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.]
13. Occurrences: 968. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.]
14. Occurrences: 889. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
15. Occurrences: 884. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
16. Occurrences: 847. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
17. Occurrences: 803. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
18. Occurrences: 775. Action Sequence: [1.,1.,1.,0.]
19. Occurrences: 766. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
20. Occurrences: 748. Action Sequence: [1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
21. Occurrences: 732. Action Sequence: [1.,1.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
22. Occurrences: 647. Action Sequence: [1.,1.,3.,0.]
23. Occurrences: 643. Action Sequence: [1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
24. Occurrences: 627. Action Sequence: [1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
25. Occurrences: 605. Action Sequence: [1.,1.,1.,3.,0.]
26. Occurrences: 556. Action Sequence: [1.,1.,1.,1.,3.,0.]
27. Occurrences: 550. Action Sequence: [1.,1.,1.,1.,1.,3.,0.]
28. Occurrences: 494. Action Sequence: [1.,1.,1.,1.,3.,3.,0.]
29. Occurrences: 485. Action Sequence: [1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
30. Occurrences: 462. Action Sequence: [1.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
31. Occurrences: 458. Action Sequence: [1.,1.,1.,3.,3.,0.]
32. Occurrences: 452. Action Sequence: [1.,1.,1.,1.,1.,1.,3.,0.]
33. Occurrences: 448. Action Sequence: [1.,1.,1.,1.,1.,3.,3.,0.]
34. Occurrences: 425. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,3.,0.]
35. Occurrences: 410. Action Sequence: [1.,1.,1.,3.,3.,3.,0.]
36. Occurrences: 407. Action Sequence: [1.,1.,1.,1.,3.,3.,3.,0.]
37. Occurrences: 393. Action Sequence: [1.,1.,1.,1.,1.,1.,3.,3.,0.]
38. Occurrences: 383. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,3.,3.,0.]
39. Occurrences: 381. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,3.,0.]
40. Occurrences: 374. Action Sequence: [1.,1.,3.,3.,0.]
41. Occurrences: 368. Action Sequence: [1.,1.,1.,1.,1.,3.,3.,3.,0.]
42. Occurrences: 352. Action Sequence: [1.,1.,1.,1.,3.,3.,3.,3.,0.]
43. Occurrences: 343. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,0.]
44. Occurrences: 339. Action Sequence: [1.,1.,1.,1.,1.,1.,3.,3.,3.,0.]
45. Occurrences: 335. Action Sequence: [1.,1.,1.,1.,1.,3.,3.,3.,3.,0.]
46. Occurrences: 317. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,0.]
47. Occurrences: 316. Action Sequence: [1.,1.,1.,1.,0.]
48. Occurrences: 311. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,0.]
49. Occurrences: 310. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,0.]
50. Occurrences: 304. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,0.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 2
1272 different trajectories
1. Occurrences: 30287. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 4009. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 3309. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2665. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
5. Occurrences: 2403. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 2380. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 2372. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
8. Occurrences: 2290. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
9. Occurrences: 2286. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 2279. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
11. Occurrences: 2277. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 2263. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 2229. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
14. Occurrences: 2205. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
15. Occurrences: 2149. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 1897. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 1818. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 1784. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 1491. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 1439. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 1363. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 1194. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 771. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.]
24. Occurrences: 739. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.]
25. Occurrences: 727. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 716. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
27. Occurrences: 706. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.]
28. Occurrences: 691. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 663. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 656. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.]
31. Occurrences: 654. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.]
32. Occurrences: 647. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.]
33. Occurrences: 641. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
34. Occurrences: 638. Action Sequence: [1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
35. Occurrences: 583. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
36. Occurrences: 569. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.]
37. Occurrences: 496. Action Sequence: [1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 490. Action Sequence: [1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
39. Occurrences: 253. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 253. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
41. Occurrences: 250. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 226. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.]
43. Occurrences: 202. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 199. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 198. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.]
46. Occurrences: 190. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 184. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.]
48. Occurrences: 174. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 174. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.]
50. Occurrences: 173. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 2
5035 different trajectories
1. Occurrences: 13451. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 3714. Action Sequence: [1.,0.]
3. Occurrences: 1949. Action Sequence: [1.,1.,0.]
4. Occurrences: 1487. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.]
5. Occurrences: 1460. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.]
6. Occurrences: 1326. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.]
7. Occurrences: 1296. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.]
8. Occurrences: 1172. Action Sequence: [1.,3.,0.]
9. Occurrences: 1167. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.]
10. Occurrences: 1165. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.]
11. Occurrences: 1024. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.]
12. Occurrences: 1018. Action Sequence: [1.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
13. Occurrences: 968. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.]
14. Occurrences: 889. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
15. Occurrences: 884. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
16. Occurrences: 847. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
17. Occurrences: 803. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
18. Occurrences: 775. Action Sequence: [1.,1.,1.,0.]
19. Occurrences: 766. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
20. Occurrences: 748. Action Sequence: [1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
21. Occurrences: 647. Action Sequence: [1.,1.,3.,0.]
22. Occurrences: 643. Action Sequence: [1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
23. Occurrences: 627. Action Sequence: [1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
24. Occurrences: 605. Action Sequence: [1.,1.,1.,3.,0.]
25. Occurrences: 556. Action Sequence: [1.,1.,1.,1.,3.,0.]
26. Occurrences: 550. Action Sequence: [1.,1.,1.,1.,1.,3.,0.]
27. Occurrences: 543. Action Sequence: [1.,1.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
28. Occurrences: 494. Action Sequence: [1.,1.,1.,1.,3.,3.,0.]
29. Occurrences: 485. Action Sequence: [1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
30. Occurrences: 458. Action Sequence: [1.,1.,1.,3.,3.,0.]
31. Occurrences: 452. Action Sequence: [1.,1.,1.,1.,1.,1.,3.,0.]
32. Occurrences: 448. Action Sequence: [1.,1.,1.,1.,1.,3.,3.,0.]
33. Occurrences: 425. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,3.,0.]
34. Occurrences: 410. Action Sequence: [1.,1.,1.,3.,3.,3.,0.]
35. Occurrences: 407. Action Sequence: [1.,1.,1.,1.,3.,3.,3.,0.]
36. Occurrences: 393. Action Sequence: [1.,1.,1.,1.,1.,1.,3.,3.,0.]
37. Occurrences: 383. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,3.,3.,0.]
38. Occurrences: 381. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,3.,0.]
39. Occurrences: 374. Action Sequence: [1.,1.,3.,3.,0.]
40. Occurrences: 368. Action Sequence: [1.,1.,1.,1.,1.,3.,3.,3.,0.]
41. Occurrences: 353. Action Sequence: [1.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
42. Occurrences: 352. Action Sequence: [1.,1.,1.,1.,3.,3.,3.,3.,0.]
43. Occurrences: 343. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,0.]
44. Occurrences: 339. Action Sequence: [1.,1.,1.,1.,1.,1.,3.,3.,3.,0.]
45. Occurrences: 335. Action Sequence: [1.,1.,1.,1.,1.,3.,3.,3.,3.,0.]
46. Occurrences: 317. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,0.]
47. Occurrences: 316. Action Sequence: [1.,1.,1.,1.,0.]
48. Occurrences: 311. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,0.]
49. Occurrences: 310. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,0.]
50. Occurrences: 304. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,0.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 3
2857 different trajectories
1. Occurrences: 13649. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
2. Occurrences: 3714. Action Sequence: [0.,0.]
3. Occurrences: 1957. Action Sequence: [0.,0.,0.]
4. Occurrences: 1688. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1507. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.]
6. Occurrences: 1475. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
7. Occurrences: 1342. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.]
8. Occurrences: 1317. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.]
9. Occurrences: 1251. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 1187. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
11. Occurrences: 1181. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 1172. Action Sequence: [0.,1.,0.]
13. Occurrences: 1038. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 979. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 958. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 900. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 897. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 880. Action Sequence: [0.,0.,1.,0.]
19. Occurrences: 859. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 811. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 793. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 793. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 792. Action Sequence: [0.,0.,0.,0.]
24. Occurrences: 768. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 764. Action Sequence: [0.,0.,0.,1.,0.]
26. Occurrences: 628. Action Sequence: [0.,0.,0.,0.,1.,0.]
27. Occurrences: 578. Action Sequence: [0.,0.,0.,0.,0.,1.,0.]
28. Occurrences: 517. Action Sequence: [0.,0.,0.,0.,1.,1.,0.]
29. Occurrences: 515. Action Sequence: [0.,0.,0.,1.,1.,0.]
30. Occurrences: 498. Action Sequence: [0.,0.,1.,1.,0.]
31. Occurrences: 495. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 471. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,0.]
33. Occurrences: 462. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,0.]
34. Occurrences: 451. Action Sequence: [0.,0.,0.,1.,1.,1.,0.]
35. Occurrences: 436. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,0.]
36. Occurrences: 430. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,0.]
37. Occurrences: 407. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,0.]
38. Occurrences: 388. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,0.]
39. Occurrences: 388. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,0.]
40. Occurrences: 378. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,0.]
41. Occurrences: 361. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,0.]
42. Occurrences: 345. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,0.]
43. Occurrences: 344. Action Sequence: [0.,0.,1.,1.,1.,0.]
44. Occurrences: 344. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,0.]
45. Occurrences: 344. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.]
46. Occurrences: 339. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 324. Action Sequence: [0.,0.,0.,0.,0.]
48. Occurrences: 321. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,0.]
49. Occurrences: 319. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,0.]
50. Occurrences: 315. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,0.]

FACTORED SPACE 1, factorisation 3
1690 different trajectories
1. Occurrences: 30287. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
2. Occurrences: 4009. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
3. Occurrences: 3309. Action Sequence: [3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
4. Occurrences: 2665. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
5. Occurrences: 2403. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.]
6. Occurrences: 2380. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.]
7. Occurrences: 2372. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.]
8. Occurrences: 2290. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.]
9. Occurrences: 2286. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.]
10. Occurrences: 2279. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.]
11. Occurrences: 2277. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
12. Occurrences: 2263. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
13. Occurrences: 2229. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.]
14. Occurrences: 2205. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.]
15. Occurrences: 2149. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
16. Occurrences: 1897. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
17. Occurrences: 1818. Action Sequence: [3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
18. Occurrences: 1784. Action Sequence: [3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
19. Occurrences: 1491. Action Sequence: [3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
20. Occurrences: 1439. Action Sequence: [3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
21. Occurrences: 1018. Action Sequence: [3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 896. Action Sequence: [3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 580. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.]
24. Occurrences: 570. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
25. Occurrences: 569. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 549. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.]
27. Occurrences: 549. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
28. Occurrences: 541. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 511. Action Sequence: [3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 510. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
31. Occurrences: 506. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 505. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.]
33. Occurrences: 504. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.]
34. Occurrences: 503. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.]
35. Occurrences: 441. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.]
36. Occurrences: 434. Action Sequence: [3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
37. Occurrences: 397. Action Sequence: [3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 390. Action Sequence: [3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
39. Occurrences: 253. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.]
40. Occurrences: 253. Action Sequence: [3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
41. Occurrences: 250. Action Sequence: [3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
42. Occurrences: 230. Action Sequence: [3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
43. Occurrences: 226. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.,3.]
44. Occurrences: 202. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.]
45. Occurrences: 199. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.]
46. Occurrences: 198. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.]
47. Occurrences: 190. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.]
48. Occurrences: 184. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.]
49. Occurrences: 174. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.]
50. Occurrences: 174. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 4
2857 different trajectories
1. Occurrences: 13649. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
2. Occurrences: 3714. Action Sequence: [0.,0.]
3. Occurrences: 1957. Action Sequence: [0.,0.,0.]
4. Occurrences: 1688. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1507. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.]
6. Occurrences: 1475. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
7. Occurrences: 1342. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.]
8. Occurrences: 1317. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.]
9. Occurrences: 1251. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 1187. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
11. Occurrences: 1181. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 1172. Action Sequence: [0.,1.,0.]
13. Occurrences: 1038. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 979. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 958. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 900. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 897. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 880. Action Sequence: [0.,0.,1.,0.]
19. Occurrences: 859. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 811. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 793. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 793. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 792. Action Sequence: [0.,0.,0.,0.]
24. Occurrences: 768. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 764. Action Sequence: [0.,0.,0.,1.,0.]
26. Occurrences: 628. Action Sequence: [0.,0.,0.,0.,1.,0.]
27. Occurrences: 578. Action Sequence: [0.,0.,0.,0.,0.,1.,0.]
28. Occurrences: 517. Action Sequence: [0.,0.,0.,0.,1.,1.,0.]
29. Occurrences: 515. Action Sequence: [0.,0.,0.,1.,1.,0.]
30. Occurrences: 498. Action Sequence: [0.,0.,1.,1.,0.]
31. Occurrences: 495. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 471. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,0.]
33. Occurrences: 462. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,0.]
34. Occurrences: 451. Action Sequence: [0.,0.,0.,1.,1.,1.,0.]
35. Occurrences: 436. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,0.]
36. Occurrences: 430. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,0.]
37. Occurrences: 407. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,0.]
38. Occurrences: 388. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,0.]
39. Occurrences: 388. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,0.]
40. Occurrences: 378. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,0.]
41. Occurrences: 361. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,0.]
42. Occurrences: 345. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,0.]
43. Occurrences: 344. Action Sequence: [0.,0.,1.,1.,1.,0.]
44. Occurrences: 344. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,0.]
45. Occurrences: 344. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.]
46. Occurrences: 339. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 324. Action Sequence: [0.,0.,0.,0.,0.]
48. Occurrences: 321. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,0.]
49. Occurrences: 319. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,0.]
50. Occurrences: 315. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,0.]

FACTORED SPACE 1, factorisation 4
1364 different trajectories
1. Occurrences: 30287. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 4239. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 3472. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2665. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
5. Occurrences: 2455. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 2435. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 2427. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
8. Occurrences: 2418. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
9. Occurrences: 2373. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
10. Occurrences: 2340. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 2332. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 2317. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 2300. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
14. Occurrences: 2284. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
15. Occurrences: 2207. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 1941. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 1887. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 1832. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 1533. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 1474. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 1018. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 896. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 580. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.]
24. Occurrences: 570. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
25. Occurrences: 569. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 549. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.]
27. Occurrences: 549. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
28. Occurrences: 541. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 511. Action Sequence: [1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 510. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
31. Occurrences: 506. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 505. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.]
33. Occurrences: 504. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.]
34. Occurrences: 503. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.]
35. Occurrences: 441. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.]
36. Occurrences: 434. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
37. Occurrences: 397. Action Sequence: [1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 390. Action Sequence: [1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
39. Occurrences: 310. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
40. Occurrences: 261. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 256. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 240. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.]
43. Occurrences: 206. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.]
44. Occurrences: 203. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 202. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 193. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 189. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.]
48. Occurrences: 182. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 181. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.]
50. Occurrences: 177. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.]

FACTORED SPACE 2, factorisation 4
1272 different trajectories
1. Occurrences: 30287. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 4009. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 3309. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2665. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
5. Occurrences: 2403. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 2380. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 2372. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
8. Occurrences: 2290. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
9. Occurrences: 2286. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 2279. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
11. Occurrences: 2277. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 2263. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 2229. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
14. Occurrences: 2205. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
15. Occurrences: 2149. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 1897. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 1818. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 1784. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 1491. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 1439. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 1363. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 1194. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 771. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.]
24. Occurrences: 739. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.]
25. Occurrences: 727. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 716. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
27. Occurrences: 706. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.]
28. Occurrences: 691. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 663. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 656. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.]
31. Occurrences: 654. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.]
32. Occurrences: 647. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.]
33. Occurrences: 641. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
34. Occurrences: 638. Action Sequence: [1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
35. Occurrences: 583. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
36. Occurrences: 569. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.]
37. Occurrences: 496. Action Sequence: [1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 490. Action Sequence: [1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
39. Occurrences: 253. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 253. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
41. Occurrences: 250. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 226. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.]
43. Occurrences: 202. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 199. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 198. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.]
46. Occurrences: 190. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 184. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.]
48. Occurrences: 174. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 174. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.]
50. Occurrences: 173. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.]

--------------------------------------------------------
'''

### State 56, Diabetes

In [ ]:
'''
43269
1. Occurrences: 18105. Trajectory: [227.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['H ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['On']
2. Occurrences: 4473. Trajectory: [67.] Num abnormal of last state: 3
HRs : ['L ']
SBPs: ['L ']
POs : ['N ']
GL  : ['H ']
AB  : ['Of']
VA  : ['On']
VE  : ['On']
3. Occurrences: 2579. Trajectory: [219.,213.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['On', 'Of']
VE  : ['On', 'On']
4. Occurrences: 2556. Trajectory: [219.,229.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['On', 'Of']
VE  : ['On', 'On']
5. Occurrences: 804. Trajectory: [ 59.,227.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'H ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
6. Occurrences: 499. Trajectory: [219.,221.,229.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'H ']
AB  : ['Of', 'On', 'On']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
7. Occurrences: 497. Trajectory: [219.,221.,213.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['H ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'L ']
AB  : ['Of', 'On', 'On']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
8. Occurrences: 396. Trajectory: [147., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'L ']
POs : ['N ', 'N ']
GL  : ['H ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
9. Occurrences: 393. Trajectory: [147.,153.,233.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
10. Occurrences: 325. Trajectory: [147.,153., 73.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
11. Occurrences: 317. Trajectory: [147., 73.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'L ']
POs : ['N ', 'N ']
GL  : ['H ', 'VH']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
12. Occurrences: 308. Trajectory: [139., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
13. Occurrences: 298. Trajectory: [139., 49.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['N ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'Of']
VA  : ['On', 'Of']
VE  : ['On', 'On']
14. Occurrences: 284. Trajectory: [379.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
15. Occurrences: 283. Trajectory: [147.,145.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
16. Occurrences: 273. Trajectory: [147.,145., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
17. Occurrences: 229. Trajectory: [139.,145., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
18. Occurrences: 226. Trajectory: [139.,129.,209.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
19. Occurrences: 223. Trajectory: [139.,145.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
20. Occurrences: 221. Trajectory: [139.,137.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
21. Occurrences: 220. Trajectory: [139.,137., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
22. Occurrences: 208. Trajectory: [387.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
23. Occurrences: 206. Trajectory: [59.,67.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['L ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'Of']
VA  : ['On', 'On']
VE  : ['On', 'On']
24. Occurrences: 203. Trajectory: [147.,145., 73.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
25. Occurrences: 197. Trajectory: [139.,129., 49.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
26. Occurrences: 196. Trajectory: [139.,137.,209.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
27. Occurrences: 194. Trajectory: [139.,137., 49.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
28. Occurrences: 186. Trajectory: [387.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'N ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
29. Occurrences: 183. Trajectory: [147.,153.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
30. Occurrences: 181. Trajectory: [147.,145.,233.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'H ', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
31. Occurrences: 170. Trajectory: [147.,153., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
32. Occurrences: 170. Trajectory: [379.,373.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
33. Occurrences: 165. Trajectory: [139.,145.,233.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
34. Occurrences: 162. Trajectory: [139.,129.,201.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'VL']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
35. Occurrences: 157. Trajectory: [147.,153.,153.,233.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'VH', 'VH']
AB  : ['Of', 'Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On']
36. Occurrences: 156. Trajectory: [147.,137., 65.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'N ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
37. Occurrences: 155. Trajectory: [147.,137., 49.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'N ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
38. Occurrences: 155. Trajectory: [147.,153.,153., 73.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ', 'N ']
GL  : ['H ', 'VH', 'VH', 'VH']
AB  : ['Of', 'Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of', 'Of']
VE  : ['On', 'On', 'On', 'On']
39. Occurrences: 149. Trajectory: [139.,145., 73.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'VH']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
40. Occurrences: 147. Trajectory: [147.,137.,209.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'N ', 'L ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
41. Occurrences: 147. Trajectory: [139.,129., 41.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'L ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'L ', 'VL']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
42. Occurrences: 139. Trajectory: [147.,137.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['N ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['H ', 'N ', 'H ']
AB  : ['Of', 'Of', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
43. Occurrences: 134. Trajectory: [379.,389.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['N ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
44. Occurrences: 130. Trajectory: [ 59.,219.,229.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['L ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'H ']
AB  : ['Of', 'Of', 'On']
VA  : ['On', 'On', 'Of']
VE  : ['On', 'On', 'On']
45. Occurrences: 127. Trajectory: [219.,141.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['H ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'H ']
AB  : ['Of', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
46. Occurrences: 123. Trajectory: [219., 53.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'L ']
AB  : ['Of', 'On']
VA  : ['On', 'Of']
VE  : ['On', 'On']
47. Occurrences: 123. Trajectory: [219., 69.] Num abnormal of last state: 3
HRs : ['L ', 'L ']
SBPs: ['H ', 'L ']
POs : ['N ', 'N ']
GL  : ['N ', 'H ']
AB  : ['Of', 'On']
VA  : ['On', 'Of']
VE  : ['On', 'On']
48. Occurrences: 119. Trajectory: [219.,149.,225.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['H ', 'N ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'H ', 'H ']
AB  : ['Of', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'On']
49. Occurrences: 115. Trajectory: [459.,381.,376.] Num abnormal of last state: 0
HRs : ['N ', 'N ', 'N ']
SBPs: ['H ', 'N ', 'N ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'N ']
AB  : ['Of', 'On', 'Of']
VA  : ['On', 'Of', 'Of']
VE  : ['On', 'On', 'Of']
50. Occurrences: 111. Trajectory: [ 59.,219.,213.] Num abnormal of last state: 3
HRs : ['L ', 'L ', 'L ']
SBPs: ['L ', 'H ', 'H ']
POs : ['N ', 'N ', 'N ']
GL  : ['N ', 'N ', 'L ']
AB  : ['Of', 'Of', 'On']
VA  : ['On', 'On', 'Of']
VE  : ['On', 'On', 'On']
'''

### State 377, Diabetes

In [ ]:
'''
NON FACTORED SPACE
6093 different trajectories
1. Occurrences: 26689. Action Sequence: [6.,0.]
2. Occurrences: 10154. Action Sequence: [6.,0.,6.,0.]
3. Occurrences: 3936. Action Sequence: [6.,0.,6.,0.,6.,0.]
4. Occurrences: 2777. Action Sequence: [6.,0.,6.,6.,0.]
5. Occurrences: 1746. Action Sequence: [6.,0.,6.,6.,6.,0.]
6. Occurrences: 1585. Action Sequence: [6.,0.,6.,0.,6.,0.,6.,0.]
7. Occurrences: 1225. Action Sequence: [6.,0.,6.,6.,6.,6.,0.]
8. Occurrences: 1078. Action Sequence: [6.,0.,6.,6.,0.,6.,0.]
9. Occurrences: 1009. Action Sequence: [6.,0.,6.,0.,6.,6.,0.]
10. Occurrences: 791. Action Sequence: [6.,0.,6.,6.,6.,6.,6.,0.]
11. Occurrences: 703. Action Sequence: [6.,0.,6.,6.,6.,0.,6.,0.]
12. Occurrences: 682. Action Sequence: [6.,0.,6.,0.,6.,6.,6.,0.]
13. Occurrences: 585. Action Sequence: [6.,0.,6.,6.,6.,6.,6.,6.,0.]
14. Occurrences: 583. Action Sequence: [6.,0.,6.,0.,6.,0.,6.,0.,6.,0.]
15. Occurrences: 480. Action Sequence: [6.,0.,6.,6.,6.,6.,0.,6.,0.]
16. Occurrences: 438. Action Sequence: [6.,0.,6.,6.,0.,6.,0.,6.,0.]
17. Occurrences: 420. Action Sequence: [6.,0.,6.,0.,6.,6.,6.,6.,0.]
18. Occurrences: 416. Action Sequence: [6.,0.,6.,0.,6.,6.,0.,6.,0.]
19. Occurrences: 407. Action Sequence: [6.,0.,6.,0.,6.,0.,6.,6.,0.]
20. Occurrences: 388. Action Sequence: [6.,0.,6.,6.,6.,6.,6.,6.,6.,0.]
21. Occurrences: 364. Action Sequence: [6.,0.,6.,6.,6.,6.,6.,0.,6.,0.]
22. Occurrences: 310. Action Sequence: [6.,0.,6.,0.,6.,6.,6.,6.,6.,0.]
23. Occurrences: 298. Action Sequence: [6.,0.,6.,6.,0.,6.,6.,0.]
24. Occurrences: 277. Action Sequence: [6.,0.,6.,6.,6.,6.,6.,6.,6.,6.,0.]
25. Occurrences: 272. Action Sequence: [6.,0.,6.,6.,6.,0.,6.,0.,6.,0.]
26. Occurrences: 265. Action Sequence: [6.,0.,6.,0.,6.,6.,6.,0.,6.,0.]
27. Occurrences: 251. Action Sequence: [6.,0.,6.,0.,6.,0.,6.,6.,6.,0.]
28. Occurrences: 244. Action Sequence: [6.,0.,6.,6.,0.,6.,6.,6.,0.]
29. Occurrences: 239. Action Sequence: [6.,0.,6.,6.,6.,0.,6.,6.,0.]
30. Occurrences: 231. Action Sequence: [6.,0.,6.,6.,6.,6.,6.,6.,0.,6.,0.]
31. Occurrences: 224. Action Sequence: [6.,0.,6.,0.,6.,0.,6.,0.,6.,0.,6.,0.]
32. Occurrences: 223. Action Sequence: [6.,0.,6.,0.,6.,6.,6.,6.,6.,6.,0.]
33. Occurrences: 187. Action Sequence: [6.,0.,6.,6.,6.,6.,6.,6.,6.,6.,6.,0.]
34. Occurrences: 184. Action Sequence: [6.,0.,6.,6.,6.,6.,0.,6.,0.,6.,0.]
35. Occurrences: 178. Action Sequence: [6.,0.,6.,0.,6.,0.,6.,6.,0.,6.,0.]
36. Occurrences: 176. Action Sequence: [6.,0.,6.,0.,6.,6.,6.,6.,0.,6.,0.]
37. Occurrences: 162. Action Sequence: [6.,0.,6.,0.,6.,6.,0.,6.,0.,6.,0.]
38. Occurrences: 158. Action Sequence: [6.,0.,6.,0.,6.,0.,6.,6.,6.,6.,0.]
39. Occurrences: 154. Action Sequence: [6.,0.,6.,0.,6.,0.,6.,0.,6.,6.,0.]
40. Occurrences: 152. Action Sequence: [6.,0.,6.,6.,6.,6.,0.,6.,6.,0.]
41. Occurrences: 147. Action Sequence: [6.,0.,6.,6.,6.,0.,6.,6.,6.,0.]
42. Occurrences: 147. Action Sequence: [6.,0.,6.,0.,6.,6.,6.,6.,6.,6.,6.,0.]
43. Occurrences: 146. Action Sequence: [6.,0.,6.,6.,0.,6.,6.,6.,6.,0.]
44. Occurrences: 146. Action Sequence: [6.,0.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,0.]
45. Occurrences: 141. Action Sequence: [6.,0.,6.,6.,0.,6.,0.,6.,0.,6.,0.]
46. Occurrences: 138. Action Sequence: [6.,0.,6.,6.,6.,6.,6.,6.,6.,0.,6.,0.]
47. Occurrences: 137. Action Sequence: [6.,0.,6.,6.,0.,6.,6.,0.,6.,0.]
48. Occurrences: 136. Action Sequence: [6.,0.,6.,0.,6.,6.,0.,6.,6.,0.]
49. Occurrences: 132. Action Sequence: [6.,0.,6.,0.,6.,0.,6.,6.,6.,0.,6.,0.]
50. Occurrences: 122. Action Sequence: [6.,0.,6.,6.,0.,6.,0.,6.,6.,0.]

FACTORED SPACE 0, factorisation 1
3935 different trajectories
1. Occurrences: 26930. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 10264. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 4025. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2826. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1798. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1650. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1267. Action Sequence: [1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1119. Action Sequence: [1.,0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 1039. Action Sequence: [1.,0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 837. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 731. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 713. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 634. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 626. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 503. Action Sequence: [1.,0.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 460. Action Sequence: [1.,0.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 455. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 437. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 436. Action Sequence: [1.,0.,1.,0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 433. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 390. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 342. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 312. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 307. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 294. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 288. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 288. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 270. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 253. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 251. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 251. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 249. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 231. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 208. Action Sequence: [1.,0.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 200. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 191. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 184. Action Sequence: [1.,0.,1.,0.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 182. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 176. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 173. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 169. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 169. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 165. Action Sequence: [1.,0.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 163. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 162. Action Sequence: [1.,0.,1.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 160. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 153. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 147. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 146. Action Sequence: [1.,0.,1.,0.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 144. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 1
6093 different trajectories
1. Occurrences: 26689. Action Sequence: [2.,0.]
2. Occurrences: 10154. Action Sequence: [2.,0.,2.,0.]
3. Occurrences: 3936. Action Sequence: [2.,0.,2.,0.,2.,0.]
4. Occurrences: 2777. Action Sequence: [2.,0.,2.,2.,0.]
5. Occurrences: 1746. Action Sequence: [2.,0.,2.,2.,2.,0.]
6. Occurrences: 1585. Action Sequence: [2.,0.,2.,0.,2.,0.,2.,0.]
7. Occurrences: 1225. Action Sequence: [2.,0.,2.,2.,2.,2.,0.]
8. Occurrences: 1078. Action Sequence: [2.,0.,2.,2.,0.,2.,0.]
9. Occurrences: 1009. Action Sequence: [2.,0.,2.,0.,2.,2.,0.]
10. Occurrences: 791. Action Sequence: [2.,0.,2.,2.,2.,2.,2.,0.]
11. Occurrences: 703. Action Sequence: [2.,0.,2.,2.,2.,0.,2.,0.]
12. Occurrences: 682. Action Sequence: [2.,0.,2.,0.,2.,2.,2.,0.]
13. Occurrences: 585. Action Sequence: [2.,0.,2.,2.,2.,2.,2.,2.,0.]
14. Occurrences: 583. Action Sequence: [2.,0.,2.,0.,2.,0.,2.,0.,2.,0.]
15. Occurrences: 480. Action Sequence: [2.,0.,2.,2.,2.,2.,0.,2.,0.]
16. Occurrences: 438. Action Sequence: [2.,0.,2.,2.,0.,2.,0.,2.,0.]
17. Occurrences: 420. Action Sequence: [2.,0.,2.,0.,2.,2.,2.,2.,0.]
18. Occurrences: 416. Action Sequence: [2.,0.,2.,0.,2.,2.,0.,2.,0.]
19. Occurrences: 407. Action Sequence: [2.,0.,2.,0.,2.,0.,2.,2.,0.]
20. Occurrences: 388. Action Sequence: [2.,0.,2.,2.,2.,2.,2.,2.,2.,0.]
21. Occurrences: 364. Action Sequence: [2.,0.,2.,2.,2.,2.,2.,0.,2.,0.]
22. Occurrences: 310. Action Sequence: [2.,0.,2.,0.,2.,2.,2.,2.,2.,0.]
23. Occurrences: 298. Action Sequence: [2.,0.,2.,2.,0.,2.,2.,0.]
24. Occurrences: 277. Action Sequence: [2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,0.]
25. Occurrences: 272. Action Sequence: [2.,0.,2.,2.,2.,0.,2.,0.,2.,0.]
26. Occurrences: 265. Action Sequence: [2.,0.,2.,0.,2.,2.,2.,0.,2.,0.]
27. Occurrences: 251. Action Sequence: [2.,0.,2.,0.,2.,0.,2.,2.,2.,0.]
28. Occurrences: 244. Action Sequence: [2.,0.,2.,2.,0.,2.,2.,2.,0.]
29. Occurrences: 239. Action Sequence: [2.,0.,2.,2.,2.,0.,2.,2.,0.]
30. Occurrences: 231. Action Sequence: [2.,0.,2.,2.,2.,2.,2.,2.,0.,2.,0.]
31. Occurrences: 224. Action Sequence: [2.,0.,2.,0.,2.,0.,2.,0.,2.,0.,2.,0.]
32. Occurrences: 223. Action Sequence: [2.,0.,2.,0.,2.,2.,2.,2.,2.,2.,0.]
33. Occurrences: 187. Action Sequence: [2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.]
34. Occurrences: 184. Action Sequence: [2.,0.,2.,2.,2.,2.,0.,2.,0.,2.,0.]
35. Occurrences: 178. Action Sequence: [2.,0.,2.,0.,2.,0.,2.,2.,0.,2.,0.]
36. Occurrences: 176. Action Sequence: [2.,0.,2.,0.,2.,2.,2.,2.,0.,2.,0.]
37. Occurrences: 162. Action Sequence: [2.,0.,2.,0.,2.,2.,0.,2.,0.,2.,0.]
38. Occurrences: 158. Action Sequence: [2.,0.,2.,0.,2.,0.,2.,2.,2.,2.,0.]
39. Occurrences: 154. Action Sequence: [2.,0.,2.,0.,2.,0.,2.,0.,2.,2.,0.]
40. Occurrences: 152. Action Sequence: [2.,0.,2.,2.,2.,2.,0.,2.,2.,0.]
41. Occurrences: 147. Action Sequence: [2.,0.,2.,2.,2.,0.,2.,2.,2.,0.]
42. Occurrences: 147. Action Sequence: [2.,0.,2.,0.,2.,2.,2.,2.,2.,2.,2.,0.]
43. Occurrences: 146. Action Sequence: [2.,0.,2.,2.,0.,2.,2.,2.,2.,0.]
44. Occurrences: 146. Action Sequence: [2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.]
45. Occurrences: 141. Action Sequence: [2.,0.,2.,2.,0.,2.,0.,2.,0.,2.,0.]
46. Occurrences: 138. Action Sequence: [2.,0.,2.,2.,2.,2.,2.,2.,2.,0.,2.,0.]
47. Occurrences: 137. Action Sequence: [2.,0.,2.,2.,0.,2.,2.,0.,2.,0.]
48. Occurrences: 136. Action Sequence: [2.,0.,2.,0.,2.,2.,0.,2.,2.,0.]
49. Occurrences: 132. Action Sequence: [2.,0.,2.,0.,2.,0.,2.,2.,2.,0.,2.,0.]
50. Occurrences: 122. Action Sequence: [2.,0.,2.,2.,0.,2.,0.,2.,2.,0.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 2
19 different trajectories
1. Occurrences: 26689. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 20718. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
3. Occurrences: 10161. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 5697. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 4071. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 3333. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 3288. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 3247. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 2827. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 2809. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 2691. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 2454. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 2427. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 2202. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
15. Occurrences: 1961. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.]
16. Occurrences: 1920. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.]
17. Occurrences: 1757. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
18. Occurrences: 1627. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.]
19. Occurrences: 121. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 2
6093 different trajectories
1. Occurrences: 26689. Action Sequence: [3.,0.]
2. Occurrences: 10154. Action Sequence: [3.,0.,3.,0.]
3. Occurrences: 3936. Action Sequence: [3.,0.,3.,0.,3.,0.]
4. Occurrences: 2777. Action Sequence: [3.,0.,3.,3.,0.]
5. Occurrences: 1746. Action Sequence: [3.,0.,3.,3.,3.,0.]
6. Occurrences: 1585. Action Sequence: [3.,0.,3.,0.,3.,0.,3.,0.]
7. Occurrences: 1225. Action Sequence: [3.,0.,3.,3.,3.,3.,0.]
8. Occurrences: 1078. Action Sequence: [3.,0.,3.,3.,0.,3.,0.]
9. Occurrences: 1009. Action Sequence: [3.,0.,3.,0.,3.,3.,0.]
10. Occurrences: 791. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,0.]
11. Occurrences: 703. Action Sequence: [3.,0.,3.,3.,3.,0.,3.,0.]
12. Occurrences: 682. Action Sequence: [3.,0.,3.,0.,3.,3.,3.,0.]
13. Occurrences: 585. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,3.,0.]
14. Occurrences: 583. Action Sequence: [3.,0.,3.,0.,3.,0.,3.,0.,3.,0.]
15. Occurrences: 480. Action Sequence: [3.,0.,3.,3.,3.,3.,0.,3.,0.]
16. Occurrences: 438. Action Sequence: [3.,0.,3.,3.,0.,3.,0.,3.,0.]
17. Occurrences: 420. Action Sequence: [3.,0.,3.,0.,3.,3.,3.,3.,0.]
18. Occurrences: 416. Action Sequence: [3.,0.,3.,0.,3.,3.,0.,3.,0.]
19. Occurrences: 407. Action Sequence: [3.,0.,3.,0.,3.,0.,3.,3.,0.]
20. Occurrences: 388. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,3.,3.,0.]
21. Occurrences: 364. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,0.,3.,0.]
22. Occurrences: 310. Action Sequence: [3.,0.,3.,0.,3.,3.,3.,3.,3.,0.]
23. Occurrences: 298. Action Sequence: [3.,0.,3.,3.,0.,3.,3.,0.]
24. Occurrences: 277. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
25. Occurrences: 272. Action Sequence: [3.,0.,3.,3.,3.,0.,3.,0.,3.,0.]
26. Occurrences: 265. Action Sequence: [3.,0.,3.,0.,3.,3.,3.,0.,3.,0.]
27. Occurrences: 251. Action Sequence: [3.,0.,3.,0.,3.,0.,3.,3.,3.,0.]
28. Occurrences: 244. Action Sequence: [3.,0.,3.,3.,0.,3.,3.,3.,0.]
29. Occurrences: 239. Action Sequence: [3.,0.,3.,3.,3.,0.,3.,3.,0.]
30. Occurrences: 231. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,3.,0.,3.,0.]
31. Occurrences: 224. Action Sequence: [3.,0.,3.,0.,3.,0.,3.,0.,3.,0.,3.,0.]
32. Occurrences: 223. Action Sequence: [3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,0.]
33. Occurrences: 187. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
34. Occurrences: 184. Action Sequence: [3.,0.,3.,3.,3.,3.,0.,3.,0.,3.,0.]
35. Occurrences: 178. Action Sequence: [3.,0.,3.,0.,3.,0.,3.,3.,0.,3.,0.]
36. Occurrences: 176. Action Sequence: [3.,0.,3.,0.,3.,3.,3.,3.,0.,3.,0.]
37. Occurrences: 162. Action Sequence: [3.,0.,3.,0.,3.,3.,0.,3.,0.,3.,0.]
38. Occurrences: 158. Action Sequence: [3.,0.,3.,0.,3.,0.,3.,3.,3.,3.,0.]
39. Occurrences: 154. Action Sequence: [3.,0.,3.,0.,3.,0.,3.,0.,3.,3.,0.]
40. Occurrences: 152. Action Sequence: [3.,0.,3.,3.,3.,3.,0.,3.,3.,0.]
41. Occurrences: 147. Action Sequence: [3.,0.,3.,3.,3.,0.,3.,3.,3.,0.]
42. Occurrences: 147. Action Sequence: [3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,0.]
43. Occurrences: 146. Action Sequence: [3.,0.,3.,3.,0.,3.,3.,3.,3.,0.]
44. Occurrences: 146. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
45. Occurrences: 141. Action Sequence: [3.,0.,3.,3.,0.,3.,0.,3.,0.,3.,0.]
46. Occurrences: 138. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.]
47. Occurrences: 137. Action Sequence: [3.,0.,3.,3.,0.,3.,3.,0.,3.,0.]
48. Occurrences: 136. Action Sequence: [3.,0.,3.,0.,3.,3.,0.,3.,3.,0.]
49. Occurrences: 132. Action Sequence: [3.,0.,3.,0.,3.,0.,3.,3.,3.,0.,3.,0.]
50. Occurrences: 122. Action Sequence: [3.,0.,3.,3.,0.,3.,0.,3.,3.,0.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 3
6093 different trajectories
1. Occurrences: 26689. Action Sequence: [1.,0.]
2. Occurrences: 10154. Action Sequence: [1.,0.,1.,0.]
3. Occurrences: 3936. Action Sequence: [1.,0.,1.,0.,1.,0.]
4. Occurrences: 2777. Action Sequence: [1.,0.,1.,1.,0.]
5. Occurrences: 1746. Action Sequence: [1.,0.,1.,1.,1.,0.]
6. Occurrences: 1585. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.]
7. Occurrences: 1225. Action Sequence: [1.,0.,1.,1.,1.,1.,0.]
8. Occurrences: 1078. Action Sequence: [1.,0.,1.,1.,0.,1.,0.]
9. Occurrences: 1009. Action Sequence: [1.,0.,1.,0.,1.,1.,0.]
10. Occurrences: 791. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,0.]
11. Occurrences: 703. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,0.]
12. Occurrences: 682. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,0.]
13. Occurrences: 585. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,0.]
14. Occurrences: 583. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.,1.,0.]
15. Occurrences: 480. Action Sequence: [1.,0.,1.,1.,1.,1.,0.,1.,0.]
16. Occurrences: 438. Action Sequence: [1.,0.,1.,1.,0.,1.,0.,1.,0.]
17. Occurrences: 420. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,0.]
18. Occurrences: 416. Action Sequence: [1.,0.,1.,0.,1.,1.,0.,1.,0.]
19. Occurrences: 407. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,0.]
20. Occurrences: 388. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,0.]
21. Occurrences: 364. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,0.,1.,0.]
22. Occurrences: 310. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,0.]
23. Occurrences: 298. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,0.]
24. Occurrences: 277. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
25. Occurrences: 272. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,0.,1.,0.]
26. Occurrences: 265. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,0.,1.,0.]
27. Occurrences: 251. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,0.]
28. Occurrences: 244. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,0.]
29. Occurrences: 239. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,0.]
30. Occurrences: 231. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
31. Occurrences: 224. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,0.]
32. Occurrences: 223. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,0.]
33. Occurrences: 187. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
34. Occurrences: 184. Action Sequence: [1.,0.,1.,1.,1.,1.,0.,1.,0.,1.,0.]
35. Occurrences: 178. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,0.,1.,0.]
36. Occurrences: 176. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,0.,1.,0.]
37. Occurrences: 162. Action Sequence: [1.,0.,1.,0.,1.,1.,0.,1.,0.,1.,0.]
38. Occurrences: 158. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,0.]
39. Occurrences: 154. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,0.]
40. Occurrences: 152. Action Sequence: [1.,0.,1.,1.,1.,1.,0.,1.,1.,0.]
41. Occurrences: 147. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,0.]
42. Occurrences: 147. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,0.]
43. Occurrences: 146. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,0.]
44. Occurrences: 146. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
45. Occurrences: 141. Action Sequence: [1.,0.,1.,1.,0.,1.,0.,1.,0.,1.,0.]
46. Occurrences: 138. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
47. Occurrences: 137. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,0.,1.,0.]
48. Occurrences: 136. Action Sequence: [1.,0.,1.,0.,1.,1.,0.,1.,1.,0.]
49. Occurrences: 132. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,0.,1.,0.]
50. Occurrences: 122. Action Sequence: [1.,0.,1.,1.,0.,1.,0.,1.,1.,0.]

FACTORED SPACE 1, factorisation 3
6093 different trajectories
1. Occurrences: 26689. Action Sequence: [2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
2. Occurrences: 10154. Action Sequence: [2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
3. Occurrences: 3936. Action Sequence: [2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
4. Occurrences: 2777. Action Sequence: [2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
5. Occurrences: 1746. Action Sequence: [2.,0.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
6. Occurrences: 1585. Action Sequence: [2.,0.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
7. Occurrences: 1225. Action Sequence: [2.,0.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
8. Occurrences: 1078. Action Sequence: [2.,0.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
9. Occurrences: 1009. Action Sequence: [2.,0.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
10. Occurrences: 791. Action Sequence: [2.,0.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
11. Occurrences: 703. Action Sequence: [2.,0.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
12. Occurrences: 682. Action Sequence: [2.,0.,2.,0.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
13. Occurrences: 585. Action Sequence: [2.,0.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
14. Occurrences: 583. Action Sequence: [2.,0.,2.,0.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
15. Occurrences: 480. Action Sequence: [2.,0.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
16. Occurrences: 438. Action Sequence: [2.,0.,2.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
17. Occurrences: 420. Action Sequence: [2.,0.,2.,0.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
18. Occurrences: 416. Action Sequence: [2.,0.,2.,0.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
19. Occurrences: 407. Action Sequence: [2.,0.,2.,0.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
20. Occurrences: 388. Action Sequence: [2.,0.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
21. Occurrences: 364. Action Sequence: [2.,0.,2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
22. Occurrences: 310. Action Sequence: [2.,0.,2.,0.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
23. Occurrences: 298. Action Sequence: [2.,0.,2.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
24. Occurrences: 277. Action Sequence: [2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
25. Occurrences: 272. Action Sequence: [2.,0.,2.,2.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
26. Occurrences: 265. Action Sequence: [2.,0.,2.,0.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
27. Occurrences: 251. Action Sequence: [2.,0.,2.,0.,2.,0.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
28. Occurrences: 244. Action Sequence: [2.,0.,2.,2.,0.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
29. Occurrences: 239. Action Sequence: [2.,0.,2.,2.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
30. Occurrences: 231. Action Sequence: [2.,0.,2.,2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
31. Occurrences: 224. Action Sequence: [2.,0.,2.,0.,2.,0.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.]
32. Occurrences: 223. Action Sequence: [2.,0.,2.,0.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
33. Occurrences: 187. Action Sequence: [2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.]
34. Occurrences: 184. Action Sequence: [2.,0.,2.,2.,2.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
35. Occurrences: 178. Action Sequence: [2.,0.,2.,0.,2.,0.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
36. Occurrences: 176. Action Sequence: [2.,0.,2.,0.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
37. Occurrences: 162. Action Sequence: [2.,0.,2.,0.,2.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
38. Occurrences: 158. Action Sequence: [2.,0.,2.,0.,2.,0.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
39. Occurrences: 154. Action Sequence: [2.,0.,2.,0.,2.,0.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
40. Occurrences: 152. Action Sequence: [2.,0.,2.,2.,2.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
41. Occurrences: 147. Action Sequence: [2.,0.,2.,2.,2.,0.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
42. Occurrences: 147. Action Sequence: [2.,0.,2.,0.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.]
43. Occurrences: 146. Action Sequence: [2.,0.,2.,2.,0.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
44. Occurrences: 146. Action Sequence: [2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.]
45. Occurrences: 141. Action Sequence: [2.,0.,2.,2.,0.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
46. Occurrences: 138. Action Sequence: [2.,0.,2.,2.,2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.]
47. Occurrences: 137. Action Sequence: [2.,0.,2.,2.,0.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
48. Occurrences: 136. Action Sequence: [2.,0.,2.,0.,2.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
49. Occurrences: 132. Action Sequence: [2.,0.,2.,0.,2.,0.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.]
50. Occurrences: 122. Action Sequence: [2.,0.,2.,2.,0.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 4
6093 different trajectories
1. Occurrences: 26689. Action Sequence: [1.,0.]
2. Occurrences: 10154. Action Sequence: [1.,0.,1.,0.]
3. Occurrences: 3936. Action Sequence: [1.,0.,1.,0.,1.,0.]
4. Occurrences: 2777. Action Sequence: [1.,0.,1.,1.,0.]
5. Occurrences: 1746. Action Sequence: [1.,0.,1.,1.,1.,0.]
6. Occurrences: 1585. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.]
7. Occurrences: 1225. Action Sequence: [1.,0.,1.,1.,1.,1.,0.]
8. Occurrences: 1078. Action Sequence: [1.,0.,1.,1.,0.,1.,0.]
9. Occurrences: 1009. Action Sequence: [1.,0.,1.,0.,1.,1.,0.]
10. Occurrences: 791. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,0.]
11. Occurrences: 703. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,0.]
12. Occurrences: 682. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,0.]
13. Occurrences: 585. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,0.]
14. Occurrences: 583. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.,1.,0.]
15. Occurrences: 480. Action Sequence: [1.,0.,1.,1.,1.,1.,0.,1.,0.]
16. Occurrences: 438. Action Sequence: [1.,0.,1.,1.,0.,1.,0.,1.,0.]
17. Occurrences: 420. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,0.]
18. Occurrences: 416. Action Sequence: [1.,0.,1.,0.,1.,1.,0.,1.,0.]
19. Occurrences: 407. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,0.]
20. Occurrences: 388. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,0.]
21. Occurrences: 364. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,0.,1.,0.]
22. Occurrences: 310. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,0.]
23. Occurrences: 298. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,0.]
24. Occurrences: 277. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
25. Occurrences: 272. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,0.,1.,0.]
26. Occurrences: 265. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,0.,1.,0.]
27. Occurrences: 251. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,0.]
28. Occurrences: 244. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,0.]
29. Occurrences: 239. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,0.]
30. Occurrences: 231. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
31. Occurrences: 224. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,0.]
32. Occurrences: 223. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,0.]
33. Occurrences: 187. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
34. Occurrences: 184. Action Sequence: [1.,0.,1.,1.,1.,1.,0.,1.,0.,1.,0.]
35. Occurrences: 178. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,0.,1.,0.]
36. Occurrences: 176. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,0.,1.,0.]
37. Occurrences: 162. Action Sequence: [1.,0.,1.,0.,1.,1.,0.,1.,0.,1.,0.]
38. Occurrences: 158. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,0.]
39. Occurrences: 154. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,0.]
40. Occurrences: 152. Action Sequence: [1.,0.,1.,1.,1.,1.,0.,1.,1.,0.]
41. Occurrences: 147. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,0.]
42. Occurrences: 147. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,0.]
43. Occurrences: 146. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,0.]
44. Occurrences: 146. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
45. Occurrences: 141. Action Sequence: [1.,0.,1.,1.,0.,1.,0.,1.,0.,1.,0.]
46. Occurrences: 138. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
47. Occurrences: 137. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,0.,1.,0.]
48. Occurrences: 136. Action Sequence: [1.,0.,1.,0.,1.,1.,0.,1.,1.,0.]
49. Occurrences: 132. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,0.,1.,0.]
50. Occurrences: 122. Action Sequence: [1.,0.,1.,1.,0.,1.,0.,1.,1.,0.]

FACTORED SPACE 1, factorisation 4
3935 different trajectories
1. Occurrences: 26930. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 10264. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 4025. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2826. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1798. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1650. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1267. Action Sequence: [1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1119. Action Sequence: [1.,0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 1039. Action Sequence: [1.,0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 837. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 731. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 713. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 634. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 626. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 503. Action Sequence: [1.,0.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 460. Action Sequence: [1.,0.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 455. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 437. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 436. Action Sequence: [1.,0.,1.,0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 433. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 390. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 342. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 312. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 307. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 294. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 288. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 288. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 270. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 253. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 251. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 251. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 249. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 231. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 208. Action Sequence: [1.,0.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 200. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 191. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 184. Action Sequence: [1.,0.,1.,0.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 182. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 176. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 173. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 169. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 169. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 165. Action Sequence: [1.,0.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 163. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 162. Action Sequence: [1.,0.,1.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 160. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 153. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 147. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 146. Action Sequence: [1.,0.,1.,0.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 144. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 2, factorisation 4
19 different trajectories
1. Occurrences: 26689. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 20718. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
3. Occurrences: 10161. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 5697. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 4071. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 3333. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 3288. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 3247. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 2827. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 2809. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 2691. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 2454. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 2427. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 2202. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
15. Occurrences: 1961. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.]
16. Occurrences: 1920. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.]
17. Occurrences: 1757. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
18. Occurrences: 1627. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.]
19. Occurrences: 121. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

--------------------------------------------------------
'''

### State 377, No Diabetes

In [ ]:
'''
NON FACTORED SPACE
2629 different trajectories
1. Occurrences: 47680. Action Sequence: [0.]
2. Occurrences: 12976. Action Sequence: [0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
3. Occurrences: 4097. Action Sequence: [0.,7.,0.]
4. Occurrences: 3027. Action Sequence: [0.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.]
5. Occurrences: 1996. Action Sequence: [0.,7.,7.,0.]
6. Occurrences: 1430. Action Sequence: [0.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
7. Occurrences: 1302. Action Sequence: [0.,7.,7.,7.,0.]
8. Occurrences: 1187. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
9. Occurrences: 1160. Action Sequence: [0.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
10. Occurrences: 1022. Action Sequence: [0.,7.,7.,7.,7.,0.]
11. Occurrences: 996. Action Sequence: [0.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.]
12. Occurrences: 788. Action Sequence: [0.,7.,7.,7.,7.,7.,0.]
13. Occurrences: 690. Action Sequence: [0.,7.,7.,7.,7.,7.,7.,0.]
14. Occurrences: 666. Action Sequence: [0.,6.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
15. Occurrences: 619. Action Sequence: [0.,7.,7.,7.,7.,7.,7.,7.,0.]
16. Occurrences: 551. Action Sequence: [0.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
17. Occurrences: 466. Action Sequence: [0.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
18. Occurrences: 441. Action Sequence: [0.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
19. Occurrences: 375. Action Sequence: [0.,6.,6.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
20. Occurrences: 368. Action Sequence: [0.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
21. Occurrences: 348. Action Sequence: [0.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
22. Occurrences: 331. Action Sequence: [0.,7.,0.,7.,0.]
23. Occurrences: 313. Action Sequence: [0.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
24. Occurrences: 270. Action Sequence: [0.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
25. Occurrences: 231. Action Sequence: [0.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
26. Occurrences: 231. Action Sequence: [0.,7.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
27. Occurrences: 215. Action Sequence: [0.,7.,7.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
28. Occurrences: 206. Action Sequence: [0.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
29. Occurrences: 188. Action Sequence: [0.,7.,7.,7.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
30. Occurrences: 186. Action Sequence: [0.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
31. Occurrences: 179. Action Sequence: [0.,6.,6.,6.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
32. Occurrences: 176. Action Sequence: [0.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
33. Occurrences: 161. Action Sequence: [0.,7.,7.,7.,7.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
34. Occurrences: 153. Action Sequence: [0.,7.,0.,7.,7.,0.]
35. Occurrences: 149. Action Sequence: [0.,7.,7.,0.,7.,0.]
36. Occurrences: 144. Action Sequence: [0.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
37. Occurrences: 137. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.]
38. Occurrences: 133. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.]
39. Occurrences: 130. Action Sequence: [0.,3.]
40. Occurrences: 129. Action Sequence: [0.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
41. Occurrences: 127. Action Sequence: [0.,3.,0.]
42. Occurrences: 127. Action Sequence: [0.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.]
43. Occurrences: 125. Action Sequence: [0.,7.,7.,7.,7.,7.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
44. Occurrences: 122. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.,7.]
45. Occurrences: 121. Action Sequence: [0.,7.,0.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.]
46. Occurrences: 116. Action Sequence: [0.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
47. Occurrences: 113. Action Sequence: [0.,7.,7.,7.,0.,7.,0.]
48. Occurrences: 112. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.,7.]
49. Occurrences: 105. Action Sequence: [0.,7.,0.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
50. Occurrences: 98. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,7.,7.,7.]

FACTORED SPACE 0, factorisation 1
981 different trajectories
1. Occurrences: 54572. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 12976. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
3. Occurrences: 4462. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2626. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
5. Occurrences: 2219. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1483. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1238. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
8. Occurrences: 1225. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 1002. Action Sequence: [0.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 927. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 875. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 747. Action Sequence: [0.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
13. Occurrences: 727. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 694. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 620. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 583. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 553. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 520. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
19. Occurrences: 478. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
20. Occurrences: 472. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
21. Occurrences: 446. Action Sequence: [0.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 428. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
23. Occurrences: 394. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
24. Occurrences: 385. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
25. Occurrences: 358. Action Sequence: [0.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 355. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 270. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
28. Occurrences: 234. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 192. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 184. Action Sequence: [0.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
31. Occurrences: 178. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 175. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
33. Occurrences: 171. Action Sequence: [0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 170. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 133. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.]
36. Occurrences: 128. Action Sequence: [0.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 126. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 112. Action Sequence: [0.,1.,0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
39. Occurrences: 100. Action Sequence: [0.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 97. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.]
41. Occurrences: 97. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.]
42. Occurrences: 90. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 86. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 85. Action Sequence: [0.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 83. Action Sequence: [0.,1.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
46. Occurrences: 82. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.]
47. Occurrences: 78. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 71. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.]
49. Occurrences: 71. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 70. Action Sequence: [0.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 1
2181 different trajectories
1. Occurrences: 47680. Action Sequence: [0.]
2. Occurrences: 18821. Action Sequence: [0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
3. Occurrences: 4097. Action Sequence: [0.,3.,0.]
4. Occurrences: 1996. Action Sequence: [0.,3.,3.,0.]
5. Occurrences: 1503. Action Sequence: [0.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
6. Occurrences: 1302. Action Sequence: [0.,3.,3.,3.,0.]
7. Occurrences: 1187. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1022. Action Sequence: [0.,3.,3.,3.,3.,0.]
9. Occurrences: 996. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
10. Occurrences: 788. Action Sequence: [0.,3.,3.,3.,3.,3.,0.]
11. Occurrences: 722. Action Sequence: [0.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
12. Occurrences: 690. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,0.]
13. Occurrences: 619. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,0.]
14. Occurrences: 466. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
15. Occurrences: 441. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
16. Occurrences: 436. Action Sequence: [0.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
17. Occurrences: 368. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
18. Occurrences: 331. Action Sequence: [0.,3.,0.,3.,0.]
19. Occurrences: 319. Action Sequence: [0.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
20. Occurrences: 313. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
21. Occurrences: 285. Action Sequence: [0.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
22. Occurrences: 270. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
23. Occurrences: 257. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
24. Occurrences: 231. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
25. Occurrences: 218. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
26. Occurrences: 206. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
27. Occurrences: 186. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
28. Occurrences: 176. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
29. Occurrences: 166. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
30. Occurrences: 162. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
31. Occurrences: 153. Action Sequence: [0.,3.,0.,3.,3.,0.]
32. Occurrences: 153. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.]
33. Occurrences: 149. Action Sequence: [0.,3.,3.,0.,3.,0.]
34. Occurrences: 144. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
35. Occurrences: 137. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.]
36. Occurrences: 133. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.]
37. Occurrences: 130. Action Sequence: [0.,1.]
38. Occurrences: 127. Action Sequence: [0.,1.,0.]
39. Occurrences: 122. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.]
40. Occurrences: 116. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
41. Occurrences: 113. Action Sequence: [0.,3.,3.,3.,0.,3.,0.]
42. Occurrences: 112. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.]
43. Occurrences: 106. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.]
44. Occurrences: 99. Action Sequence: [0.,3.,0.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
45. Occurrences: 98. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.]
46. Occurrences: 91. Action Sequence: [0.,3.,0.,3.,3.,3.,0.]
47. Occurrences: 90. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.]
48. Occurrences: 89. Action Sequence: [0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
49. Occurrences: 87. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
50. Occurrences: 87. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 2
930 different trajectories
1. Occurrences: 51545. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 18821. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
3. Occurrences: 4334. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2158. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1548. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
6. Occurrences: 1452. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1189. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 970. Action Sequence: [0.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 891. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 849. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 757. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
12. Occurrences: 710. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 677. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 607. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 568. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 545. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 508. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
18. Occurrences: 471. Action Sequence: [0.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
19. Occurrences: 471. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
20. Occurrences: 457. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
21. Occurrences: 428. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
22. Occurrences: 379. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
23. Occurrences: 372. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
24. Occurrences: 370. Action Sequence: [0.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
25. Occurrences: 360. Action Sequence: [0.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 350. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 311. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
28. Occurrences: 285. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 232. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 224. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
31. Occurrences: 222. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 168. Action Sequence: [0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 168. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.]
34. Occurrences: 163. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 158. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.]
36. Occurrences: 125. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.]
37. Occurrences: 123. Action Sequence: [0.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 120. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.]
39. Occurrences: 112. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.]
40. Occurrences: 100. Action Sequence: [0.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
41. Occurrences: 99. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.]
42. Occurrences: 98. Action Sequence: [0.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 88. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 85. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.]
45. Occurrences: 84. Action Sequence: [0.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 81. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 76. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 70. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 69. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 68. Action Sequence: [0.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 2
2494 different trajectories
1. Occurrences: 47680. Action Sequence: [0.]
2. Occurrences: 12976. Action Sequence: [0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
3. Occurrences: 4097. Action Sequence: [0.,3.,0.]
4. Occurrences: 4023. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
5. Occurrences: 1996. Action Sequence: [0.,3.,3.,0.]
6. Occurrences: 1430. Action Sequence: [0.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
7. Occurrences: 1302. Action Sequence: [0.,3.,3.,3.,0.]
8. Occurrences: 1187. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 1160. Action Sequence: [0.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
10. Occurrences: 1022. Action Sequence: [0.,3.,3.,3.,3.,0.]
11. Occurrences: 788. Action Sequence: [0.,3.,3.,3.,3.,3.,0.]
12. Occurrences: 690. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,0.]
13. Occurrences: 666. Action Sequence: [0.,3.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
14. Occurrences: 619. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,0.]
15. Occurrences: 551. Action Sequence: [0.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
16. Occurrences: 466. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
17. Occurrences: 441. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
18. Occurrences: 375. Action Sequence: [0.,3.,3.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
19. Occurrences: 368. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
20. Occurrences: 348. Action Sequence: [0.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
21. Occurrences: 331. Action Sequence: [0.,3.,0.,3.,0.]
22. Occurrences: 313. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
23. Occurrences: 270. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
24. Occurrences: 231. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
25. Occurrences: 231. Action Sequence: [0.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
26. Occurrences: 215. Action Sequence: [0.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
27. Occurrences: 210. Action Sequence: [0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
28. Occurrences: 206. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
29. Occurrences: 188. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
30. Occurrences: 186. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
31. Occurrences: 179. Action Sequence: [0.,3.,3.,3.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
32. Occurrences: 176. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
33. Occurrences: 161. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
34. Occurrences: 153. Action Sequence: [0.,3.,0.,3.,3.,0.]
35. Occurrences: 149. Action Sequence: [0.,3.,3.,0.,3.,0.]
36. Occurrences: 144. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
37. Occurrences: 137. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.]
38. Occurrences: 133. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.]
39. Occurrences: 130. Action Sequence: [0.,1.]
40. Occurrences: 129. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
41. Occurrences: 127. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.]
42. Occurrences: 127. Action Sequence: [0.,1.,0.]
43. Occurrences: 126. Action Sequence: [0.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
44. Occurrences: 125. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
45. Occurrences: 122. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.,3.]
46. Occurrences: 116. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
47. Occurrences: 113. Action Sequence: [0.,3.,3.,3.,0.,3.,0.]
48. Occurrences: 112. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.,3.]
49. Occurrences: 105. Action Sequence: [0.,3.,0.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
50. Occurrences: 98. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,3.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 3
1830 different trajectories
1. Occurrences: 47680. Action Sequence: [0.]
2. Occurrences: 19817. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 4097. Action Sequence: [0.,1.,0.]
4. Occurrences: 1996. Action Sequence: [0.,1.,1.,0.]
5. Occurrences: 1592. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1302. Action Sequence: [0.,1.,1.,1.,0.]
7. Occurrences: 1195. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
8. Occurrences: 1022. Action Sequence: [0.,1.,1.,1.,1.,0.]
9. Occurrences: 790. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 788. Action Sequence: [0.,1.,1.,1.,1.,1.,0.]
11. Occurrences: 690. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,0.]
12. Occurrences: 619. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,0.]
13. Occurrences: 477. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 466. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
15. Occurrences: 441. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
16. Occurrences: 368. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
17. Occurrences: 358. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 331. Action Sequence: [0.,1.,0.,1.,0.]
19. Occurrences: 315. Action Sequence: [0.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 313. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
21. Occurrences: 293. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 270. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
23. Occurrences: 259. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 231. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
25. Occurrences: 212. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 211. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 206. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
28. Occurrences: 188. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 186. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
30. Occurrences: 176. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
31. Occurrences: 153. Action Sequence: [0.,1.,0.,1.,1.,0.]
32. Occurrences: 149. Action Sequence: [0.,1.,1.,0.,1.,0.]
33. Occurrences: 148. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 144. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
35. Occurrences: 140. Action Sequence: [0.,0.,0.]
36. Occurrences: 139. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 137. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.]
38. Occurrences: 133. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
39. Occurrences: 130. Action Sequence: [0.,0.]
40. Occurrences: 122. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
41. Occurrences: 116. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
42. Occurrences: 114. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.]
43. Occurrences: 113. Action Sequence: [0.,1.,1.,1.,0.,1.,0.]
44. Occurrences: 113. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
45. Occurrences: 112. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
46. Occurrences: 111. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
47. Occurrences: 109. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 100. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
49. Occurrences: 99. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 98. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.]

FACTORED SPACE 1, factorisation 3
1243 different trajectories
1. Occurrences: 51545. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
2. Occurrences: 12976. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
3. Occurrences: 4334. Action Sequence: [0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
4. Occurrences: 3027. Action Sequence: [0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
5. Occurrences: 2158. Action Sequence: [0.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
6. Occurrences: 1452. Action Sequence: [0.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
7. Occurrences: 1430. Action Sequence: [0.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
8. Occurrences: 1196. Action Sequence: [0.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
9. Occurrences: 1189. Action Sequence: [0.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
10. Occurrences: 970. Action Sequence: [0.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
11. Occurrences: 891. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
12. Occurrences: 849. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
13. Occurrences: 710. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
14. Occurrences: 677. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
15. Occurrences: 666. Action Sequence: [0.,2.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
16. Occurrences: 607. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.]
17. Occurrences: 572. Action Sequence: [0.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
18. Occurrences: 568. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.]
19. Occurrences: 545. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.]
20. Occurrences: 508. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.]
21. Occurrences: 471. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.]
22. Occurrences: 457. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.]
23. Occurrences: 428. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
24. Occurrences: 379. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.]
25. Occurrences: 375. Action Sequence: [0.,2.,2.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 372. Action Sequence: [0.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
27. Occurrences: 372. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.]
28. Occurrences: 350. Action Sequence: [0.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
29. Occurrences: 275. Action Sequence: [0.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 267. Action Sequence: [0.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
31. Occurrences: 229. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 214. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
33. Occurrences: 188. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
34. Occurrences: 179. Action Sequence: [0.,2.,2.,2.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
35. Occurrences: 171. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
36. Occurrences: 168. Action Sequence: [0.,3.,0.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
37. Occurrences: 166. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 163. Action Sequence: [0.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
39. Occurrences: 132. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.]
40. Occurrences: 128. Action Sequence: [0.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
41. Occurrences: 125. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.]
42. Occurrences: 123. Action Sequence: [0.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
43. Occurrences: 105. Action Sequence: [0.,3.,0.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
44. Occurrences: 98. Action Sequence: [0.,3.,0.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
45. Occurrences: 97. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.]
46. Occurrences: 96. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.]
47. Occurrences: 88. Action Sequence: [0.,3.,0.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
48. Occurrences: 84. Action Sequence: [0.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
49. Occurrences: 83. Action Sequence: [0.,2.,2.,2.,2.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
50. Occurrences: 81. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 4
1830 different trajectories
1. Occurrences: 47680. Action Sequence: [0.]
2. Occurrences: 19817. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 4097. Action Sequence: [0.,1.,0.]
4. Occurrences: 1996. Action Sequence: [0.,1.,1.,0.]
5. Occurrences: 1592. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1302. Action Sequence: [0.,1.,1.,1.,0.]
7. Occurrences: 1195. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
8. Occurrences: 1022. Action Sequence: [0.,1.,1.,1.,1.,0.]
9. Occurrences: 790. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 788. Action Sequence: [0.,1.,1.,1.,1.,1.,0.]
11. Occurrences: 690. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,0.]
12. Occurrences: 619. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,0.]
13. Occurrences: 477. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 466. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
15. Occurrences: 441. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
16. Occurrences: 368. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
17. Occurrences: 358. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 331. Action Sequence: [0.,1.,0.,1.,0.]
19. Occurrences: 315. Action Sequence: [0.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 313. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
21. Occurrences: 293. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 270. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
23. Occurrences: 259. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 231. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
25. Occurrences: 212. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 211. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 206. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
28. Occurrences: 188. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 186. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
30. Occurrences: 176. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
31. Occurrences: 153. Action Sequence: [0.,1.,0.,1.,1.,0.]
32. Occurrences: 149. Action Sequence: [0.,1.,1.,0.,1.,0.]
33. Occurrences: 148. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 144. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
35. Occurrences: 140. Action Sequence: [0.,0.,0.]
36. Occurrences: 139. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 137. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.]
38. Occurrences: 133. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
39. Occurrences: 130. Action Sequence: [0.,0.]
40. Occurrences: 122. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
41. Occurrences: 116. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
42. Occurrences: 114. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.]
43. Occurrences: 113. Action Sequence: [0.,1.,1.,1.,0.,1.,0.]
44. Occurrences: 113. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
45. Occurrences: 112. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
46. Occurrences: 111. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
47. Occurrences: 109. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 100. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
49. Occurrences: 99. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 98. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.]

FACTORED SPACE 1, factorisation 4
981 different trajectories
1. Occurrences: 54572. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 12976. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
3. Occurrences: 4462. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2626. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
5. Occurrences: 2219. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1483. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1238. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
8. Occurrences: 1225. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 1002. Action Sequence: [0.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 927. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 875. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 747. Action Sequence: [0.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
13. Occurrences: 727. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 694. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 620. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 583. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 553. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 520. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
19. Occurrences: 478. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
20. Occurrences: 472. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
21. Occurrences: 446. Action Sequence: [0.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 428. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
23. Occurrences: 394. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
24. Occurrences: 385. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
25. Occurrences: 358. Action Sequence: [0.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 355. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 270. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
28. Occurrences: 234. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 192. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 184. Action Sequence: [0.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
31. Occurrences: 178. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 175. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
33. Occurrences: 171. Action Sequence: [0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 170. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 133. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.]
36. Occurrences: 128. Action Sequence: [0.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 126. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 112. Action Sequence: [0.,1.,0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
39. Occurrences: 100. Action Sequence: [0.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 97. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.]
41. Occurrences: 97. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.]
42. Occurrences: 90. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 86. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 85. Action Sequence: [0.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 83. Action Sequence: [0.,1.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
46. Occurrences: 82. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.]
47. Occurrences: 78. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 71. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.]
49. Occurrences: 71. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 70. Action Sequence: [0.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 2, factorisation 4
930 different trajectories
1. Occurrences: 51545. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 18821. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
3. Occurrences: 4334. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2158. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1548. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
6. Occurrences: 1452. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1189. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 970. Action Sequence: [0.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 891. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 849. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 757. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
12. Occurrences: 710. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 677. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 607. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 568. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 545. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 508. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
18. Occurrences: 471. Action Sequence: [0.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
19. Occurrences: 471. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
20. Occurrences: 457. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
21. Occurrences: 428. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
22. Occurrences: 379. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
23. Occurrences: 372. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
24. Occurrences: 370. Action Sequence: [0.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
25. Occurrences: 360. Action Sequence: [0.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 350. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 311. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
28. Occurrences: 285. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 232. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 224. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
31. Occurrences: 222. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 168. Action Sequence: [0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 168. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.]
34. Occurrences: 163. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 158. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.]
36. Occurrences: 125. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.]
37. Occurrences: 123. Action Sequence: [0.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 120. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.]
39. Occurrences: 112. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.]
40. Occurrences: 100. Action Sequence: [0.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
41. Occurrences: 99. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.]
42. Occurrences: 98. Action Sequence: [0.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 88. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 85. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.]
45. Occurrences: 84. Action Sequence: [0.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 81. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 76. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 70. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 69. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 68. Action Sequence: [0.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

--------------------------------------------------------
'''

### State 696, Diabetes

In [ ]:
'''
NON FACTORED SPACE
11850 different trajectories
1. Occurrences: 14914. Action Sequence: [6.]
2. Occurrences: 6739. Action Sequence: [6.,0.]
3. Occurrences: 6174. Action Sequence: [6.,6.,0.]
4. Occurrences: 3354. Action Sequence: [6.,6.,6.,0.]
5. Occurrences: 2509. Action Sequence: [6.,0.,6.,0.]
6. Occurrences: 2313. Action Sequence: [6.,6.,0.,6.,0.]
7. Occurrences: 1935. Action Sequence: [6.,6.,6.,6.,0.]
8. Occurrences: 1494. Action Sequence: [6.,6.]
9. Occurrences: 1278. Action Sequence: [6.,6.,6.,0.,6.,0.]
10. Occurrences: 1208. Action Sequence: [6.,6.,6.,6.,6.,0.]
11. Occurrences: 1031. Action Sequence: [6.,0.,6.,0.,6.,0.]
12. Occurrences: 925. Action Sequence: [6.,6.,0.,6.,0.,6.,0.]
13. Occurrences: 746. Action Sequence: [6.,6.,6.,6.,6.,6.,0.]
14. Occurrences: 743. Action Sequence: [6.,6.,6.,6.,0.,6.,0.]
15. Occurrences: 693. Action Sequence: [6.,0.,6.,6.,0.]
16. Occurrences: 621. Action Sequence: [6.,6.,0.,6.,6.,0.]
17. Occurrences: 562. Action Sequence: [6.,6.,6.,6.,6.,6.,6.,0.]
18. Occurrences: 550. Action Sequence: [6.,6.,6.,0.,6.,0.,6.,0.]
19. Occurrences: 478. Action Sequence: [6.,6.,6.,6.,6.,0.,6.,0.]
20. Occurrences: 468. Action Sequence: [6.,0.,6.,6.,6.,0.]
21. Occurrences: 374. Action Sequence: [6.,6.,0.,6.,6.,6.,0.]
22. Occurrences: 359. Action Sequence: [6.,6.,6.,6.,6.,6.,6.,6.,0.]
23. Occurrences: 351. Action Sequence: [6.,6.,0.,6.,0.,6.,0.,6.,0.]
24. Occurrences: 340. Action Sequence: [6.,6.,6.,0.,6.,6.,0.]
25. Occurrences: 334. Action Sequence: [6.,0.,6.,0.,6.,0.,6.,0.]
26. Occurrences: 326. Action Sequence: [6.,0.,6.,6.,6.,6.,0.]
27. Occurrences: 325. Action Sequence: [6.,6.,6.,6.,6.,6.,0.,6.,0.]
28. Occurrences: 290. Action Sequence: [6.,6.,6.,6.,0.,6.,0.,6.,0.]
29. Occurrences: 270. Action Sequence: [6.,0.,6.,0.,6.,6.,0.]
30. Occurrences: 265. Action Sequence: [6.,0.,6.,6.,0.,6.,0.]
31. Occurrences: 256. Action Sequence: [6.,6.,0.,6.,6.,6.,6.,0.]
32. Occurrences: 248. Action Sequence: [6.,6.,6.,6.,6.,6.,6.,6.,6.,0.]
33. Occurrences: 230. Action Sequence: [6.,6.,0.,6.,6.,0.,6.,0.]
34. Occurrences: 225. Action Sequence: [6.,6.,6.,0.,6.,6.,6.,0.]
35. Occurrences: 221. Action Sequence: [6.,6.,0.,6.,0.,6.,6.,0.]
36. Occurrences: 208. Action Sequence: [6.,0.,6.,6.,6.,6.,6.,0.]
37. Occurrences: 208. Action Sequence: [6.,6.,6.,6.,6.,6.,6.,0.,6.,0.]
38. Occurrences: 194. Action Sequence: [6.,6.,6.,6.,6.,0.,6.,0.,6.,0.]
39. Occurrences: 187. Action Sequence: [6.,6.,6.,6.,0.,6.,6.,0.]
40. Occurrences: 184. Action Sequence: [6.,0.,6.,6.,6.,0.,6.,0.]
41. Occurrences: 181. Action Sequence: [6.,6.,0.,6.,6.,6.,6.,6.,0.]
42. Occurrences: 178. Action Sequence: [6.,6.,6.,0.,6.,0.,6.,0.,6.,0.]
43. Occurrences: 167. Action Sequence: [6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,0.]
44. Occurrences: 163. Action Sequence: [6.,0.,6.,6.,6.,6.,6.,6.,0.]
45. Occurrences: 156. Action Sequence: [6.,6.,6.,0.,6.,0.,6.,6.,0.]
46. Occurrences: 155. Action Sequence: [6.,0.,6.,0.,6.,6.,6.,0.]
47. Occurrences: 154. Action Sequence: [6.,6.,0.,6.,0.,6.,6.,6.,0.]
48. Occurrences: 152. Action Sequence: [6.,6.,0.,6.,6.,6.,0.,6.,0.]
49. Occurrences: 147. Action Sequence: [6.,0.,6.,0.,6.,0.,6.,0.,6.,0.]
50. Occurrences: 146. Action Sequence: [6.,6.,6.,6.,6.,6.,6.,6.,0.,6.,0.]

FACTORED SPACE 0, factorisation 1
8316 different trajectories
1. Occurrences: 16651. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 6798. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 6222. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 3397. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 2540. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 2353. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1961. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1313. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 1235. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 1049. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 950. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 774. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 767. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 708. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 641. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 587. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 573. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 495. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 478. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 388. Action Sequence: [1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 379. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 364. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 357. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 351. Action Sequence: [1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 345. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 338. Action Sequence: [1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 311. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 279. Action Sequence: [1.,0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 277. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 275. Action Sequence: [1.,0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 264. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 241. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 233. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 231. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 220. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 219. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 214. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 196. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 195. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 194. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 193. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 181. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 172. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 165. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 164. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 161. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 160. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 160. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 158. Action Sequence: [1.,1.,0.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 153. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 1
11850 different trajectories
1. Occurrences: 14914. Action Sequence: [2.]
2. Occurrences: 6739. Action Sequence: [2.,0.]
3. Occurrences: 6174. Action Sequence: [2.,2.,0.]
4. Occurrences: 3354. Action Sequence: [2.,2.,2.,0.]
5. Occurrences: 2509. Action Sequence: [2.,0.,2.,0.]
6. Occurrences: 2313. Action Sequence: [2.,2.,0.,2.,0.]
7. Occurrences: 1935. Action Sequence: [2.,2.,2.,2.,0.]
8. Occurrences: 1494. Action Sequence: [2.,2.]
9. Occurrences: 1278. Action Sequence: [2.,2.,2.,0.,2.,0.]
10. Occurrences: 1208. Action Sequence: [2.,2.,2.,2.,2.,0.]
11. Occurrences: 1031. Action Sequence: [2.,0.,2.,0.,2.,0.]
12. Occurrences: 925. Action Sequence: [2.,2.,0.,2.,0.,2.,0.]
13. Occurrences: 746. Action Sequence: [2.,2.,2.,2.,2.,2.,0.]
14. Occurrences: 743. Action Sequence: [2.,2.,2.,2.,0.,2.,0.]
15. Occurrences: 693. Action Sequence: [2.,0.,2.,2.,0.]
16. Occurrences: 621. Action Sequence: [2.,2.,0.,2.,2.,0.]
17. Occurrences: 562. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,0.]
18. Occurrences: 550. Action Sequence: [2.,2.,2.,0.,2.,0.,2.,0.]
19. Occurrences: 478. Action Sequence: [2.,2.,2.,2.,2.,0.,2.,0.]
20. Occurrences: 468. Action Sequence: [2.,0.,2.,2.,2.,0.]
21. Occurrences: 374. Action Sequence: [2.,2.,0.,2.,2.,2.,0.]
22. Occurrences: 359. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,0.]
23. Occurrences: 351. Action Sequence: [2.,2.,0.,2.,0.,2.,0.,2.,0.]
24. Occurrences: 340. Action Sequence: [2.,2.,2.,0.,2.,2.,0.]
25. Occurrences: 334. Action Sequence: [2.,0.,2.,0.,2.,0.,2.,0.]
26. Occurrences: 326. Action Sequence: [2.,0.,2.,2.,2.,2.,0.]
27. Occurrences: 325. Action Sequence: [2.,2.,2.,2.,2.,2.,0.,2.,0.]
28. Occurrences: 290. Action Sequence: [2.,2.,2.,2.,0.,2.,0.,2.,0.]
29. Occurrences: 270. Action Sequence: [2.,0.,2.,0.,2.,2.,0.]
30. Occurrences: 265. Action Sequence: [2.,0.,2.,2.,0.,2.,0.]
31. Occurrences: 256. Action Sequence: [2.,2.,0.,2.,2.,2.,2.,0.]
32. Occurrences: 248. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,0.]
33. Occurrences: 230. Action Sequence: [2.,2.,0.,2.,2.,0.,2.,0.]
34. Occurrences: 225. Action Sequence: [2.,2.,2.,0.,2.,2.,2.,0.]
35. Occurrences: 221. Action Sequence: [2.,2.,0.,2.,0.,2.,2.,0.]
36. Occurrences: 208. Action Sequence: [2.,0.,2.,2.,2.,2.,2.,0.]
37. Occurrences: 208. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,0.,2.,0.]
38. Occurrences: 194. Action Sequence: [2.,2.,2.,2.,2.,0.,2.,0.,2.,0.]
39. Occurrences: 187. Action Sequence: [2.,2.,2.,2.,0.,2.,2.,0.]
40. Occurrences: 184. Action Sequence: [2.,0.,2.,2.,2.,0.,2.,0.]
41. Occurrences: 181. Action Sequence: [2.,2.,0.,2.,2.,2.,2.,2.,0.]
42. Occurrences: 178. Action Sequence: [2.,2.,2.,0.,2.,0.,2.,0.,2.,0.]
43. Occurrences: 167. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.]
44. Occurrences: 163. Action Sequence: [2.,0.,2.,2.,2.,2.,2.,2.,0.]
45. Occurrences: 156. Action Sequence: [2.,2.,2.,0.,2.,0.,2.,2.,0.]
46. Occurrences: 155. Action Sequence: [2.,0.,2.,0.,2.,2.,2.,0.]
47. Occurrences: 154. Action Sequence: [2.,2.,0.,2.,0.,2.,2.,2.,0.]
48. Occurrences: 152. Action Sequence: [2.,2.,0.,2.,2.,2.,0.,2.,0.]
49. Occurrences: 147. Action Sequence: [2.,0.,2.,0.,2.,0.,2.,0.,2.,0.]
50. Occurrences: 146. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,0.,2.,0.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 2
20 different trajectories
1. Occurrences: 20647. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
2. Occurrences: 14914. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 8233. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 6341. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 5900. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 4968. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 4633. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 4014. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 3689. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 3375. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 3199. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 2969. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 2741. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 2524. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 2310. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 2178. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
17. Occurrences: 2035. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.]
18. Occurrences: 1927. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.]
19. Occurrences: 1781. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
20. Occurrences: 1622. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.]

FACTORED SPACE 1, factorisation 2
11850 different trajectories
1. Occurrences: 14914. Action Sequence: [3.]
2. Occurrences: 6739. Action Sequence: [3.,0.]
3. Occurrences: 6174. Action Sequence: [3.,3.,0.]
4. Occurrences: 3354. Action Sequence: [3.,3.,3.,0.]
5. Occurrences: 2509. Action Sequence: [3.,0.,3.,0.]
6. Occurrences: 2313. Action Sequence: [3.,3.,0.,3.,0.]
7. Occurrences: 1935. Action Sequence: [3.,3.,3.,3.,0.]
8. Occurrences: 1494. Action Sequence: [3.,3.]
9. Occurrences: 1278. Action Sequence: [3.,3.,3.,0.,3.,0.]
10. Occurrences: 1208. Action Sequence: [3.,3.,3.,3.,3.,0.]
11. Occurrences: 1031. Action Sequence: [3.,0.,3.,0.,3.,0.]
12. Occurrences: 925. Action Sequence: [3.,3.,0.,3.,0.,3.,0.]
13. Occurrences: 746. Action Sequence: [3.,3.,3.,3.,3.,3.,0.]
14. Occurrences: 743. Action Sequence: [3.,3.,3.,3.,0.,3.,0.]
15. Occurrences: 693. Action Sequence: [3.,0.,3.,3.,0.]
16. Occurrences: 621. Action Sequence: [3.,3.,0.,3.,3.,0.]
17. Occurrences: 562. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.]
18. Occurrences: 550. Action Sequence: [3.,3.,3.,0.,3.,0.,3.,0.]
19. Occurrences: 478. Action Sequence: [3.,3.,3.,3.,3.,0.,3.,0.]
20. Occurrences: 468. Action Sequence: [3.,0.,3.,3.,3.,0.]
21. Occurrences: 374. Action Sequence: [3.,3.,0.,3.,3.,3.,0.]
22. Occurrences: 359. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,0.]
23. Occurrences: 351. Action Sequence: [3.,3.,0.,3.,0.,3.,0.,3.,0.]
24. Occurrences: 340. Action Sequence: [3.,3.,3.,0.,3.,3.,0.]
25. Occurrences: 334. Action Sequence: [3.,0.,3.,0.,3.,0.,3.,0.]
26. Occurrences: 326. Action Sequence: [3.,0.,3.,3.,3.,3.,0.]
27. Occurrences: 325. Action Sequence: [3.,3.,3.,3.,3.,3.,0.,3.,0.]
28. Occurrences: 290. Action Sequence: [3.,3.,3.,3.,0.,3.,0.,3.,0.]
29. Occurrences: 270. Action Sequence: [3.,0.,3.,0.,3.,3.,0.]
30. Occurrences: 265. Action Sequence: [3.,0.,3.,3.,0.,3.,0.]
31. Occurrences: 256. Action Sequence: [3.,3.,0.,3.,3.,3.,3.,0.]
32. Occurrences: 248. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
33. Occurrences: 230. Action Sequence: [3.,3.,0.,3.,3.,0.,3.,0.]
34. Occurrences: 225. Action Sequence: [3.,3.,3.,0.,3.,3.,3.,0.]
35. Occurrences: 221. Action Sequence: [3.,3.,0.,3.,0.,3.,3.,0.]
36. Occurrences: 208. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,0.]
37. Occurrences: 208. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.,3.,0.]
38. Occurrences: 194. Action Sequence: [3.,3.,3.,3.,3.,0.,3.,0.,3.,0.]
39. Occurrences: 187. Action Sequence: [3.,3.,3.,3.,0.,3.,3.,0.]
40. Occurrences: 184. Action Sequence: [3.,0.,3.,3.,3.,0.,3.,0.]
41. Occurrences: 181. Action Sequence: [3.,3.,0.,3.,3.,3.,3.,3.,0.]
42. Occurrences: 178. Action Sequence: [3.,3.,3.,0.,3.,0.,3.,0.,3.,0.]
43. Occurrences: 167. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
44. Occurrences: 163. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,3.,0.]
45. Occurrences: 156. Action Sequence: [3.,3.,3.,0.,3.,0.,3.,3.,0.]
46. Occurrences: 155. Action Sequence: [3.,0.,3.,0.,3.,3.,3.,0.]
47. Occurrences: 154. Action Sequence: [3.,3.,0.,3.,0.,3.,3.,3.,0.]
48. Occurrences: 152. Action Sequence: [3.,3.,0.,3.,3.,3.,0.,3.,0.]
49. Occurrences: 147. Action Sequence: [3.,0.,3.,0.,3.,0.,3.,0.,3.,0.]
50. Occurrences: 146. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 3
11850 different trajectories
1. Occurrences: 14914. Action Sequence: [1.]
2. Occurrences: 6739. Action Sequence: [1.,0.]
3. Occurrences: 6174. Action Sequence: [1.,1.,0.]
4. Occurrences: 3354. Action Sequence: [1.,1.,1.,0.]
5. Occurrences: 2509. Action Sequence: [1.,0.,1.,0.]
6. Occurrences: 2313. Action Sequence: [1.,1.,0.,1.,0.]
7. Occurrences: 1935. Action Sequence: [1.,1.,1.,1.,0.]
8. Occurrences: 1494. Action Sequence: [1.,1.]
9. Occurrences: 1278. Action Sequence: [1.,1.,1.,0.,1.,0.]
10. Occurrences: 1208. Action Sequence: [1.,1.,1.,1.,1.,0.]
11. Occurrences: 1031. Action Sequence: [1.,0.,1.,0.,1.,0.]
12. Occurrences: 925. Action Sequence: [1.,1.,0.,1.,0.,1.,0.]
13. Occurrences: 746. Action Sequence: [1.,1.,1.,1.,1.,1.,0.]
14. Occurrences: 743. Action Sequence: [1.,1.,1.,1.,0.,1.,0.]
15. Occurrences: 693. Action Sequence: [1.,0.,1.,1.,0.]
16. Occurrences: 621. Action Sequence: [1.,1.,0.,1.,1.,0.]
17. Occurrences: 562. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.]
18. Occurrences: 550. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.]
19. Occurrences: 478. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.]
20. Occurrences: 468. Action Sequence: [1.,0.,1.,1.,1.,0.]
21. Occurrences: 374. Action Sequence: [1.,1.,0.,1.,1.,1.,0.]
22. Occurrences: 359. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.]
23. Occurrences: 351. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,0.]
24. Occurrences: 340. Action Sequence: [1.,1.,1.,0.,1.,1.,0.]
25. Occurrences: 334. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.]
26. Occurrences: 326. Action Sequence: [1.,0.,1.,1.,1.,1.,0.]
27. Occurrences: 325. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.]
28. Occurrences: 290. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,0.]
29. Occurrences: 270. Action Sequence: [1.,0.,1.,0.,1.,1.,0.]
30. Occurrences: 265. Action Sequence: [1.,0.,1.,1.,0.,1.,0.]
31. Occurrences: 256. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,0.]
32. Occurrences: 248. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
33. Occurrences: 230. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,0.]
34. Occurrences: 225. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,0.]
35. Occurrences: 221. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,0.]
36. Occurrences: 208. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,0.]
37. Occurrences: 208. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
38. Occurrences: 194. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,0.]
39. Occurrences: 187. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.]
40. Occurrences: 184. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,0.]
41. Occurrences: 181. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,0.]
42. Occurrences: 178. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.,1.,0.]
43. Occurrences: 167. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
44. Occurrences: 163. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,0.]
45. Occurrences: 156. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,0.]
46. Occurrences: 155. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,0.]
47. Occurrences: 154. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,0.]
48. Occurrences: 152. Action Sequence: [1.,1.,0.,1.,1.,1.,0.,1.,0.]
49. Occurrences: 147. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.,1.,0.]
50. Occurrences: 146. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]

FACTORED SPACE 1, factorisation 3
11850 different trajectories
1. Occurrences: 14914. Action Sequence: [2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
2. Occurrences: 6739. Action Sequence: [2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
3. Occurrences: 6174. Action Sequence: [2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
4. Occurrences: 3354. Action Sequence: [2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
5. Occurrences: 2509. Action Sequence: [2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
6. Occurrences: 2313. Action Sequence: [2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
7. Occurrences: 1935. Action Sequence: [2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
8. Occurrences: 1494. Action Sequence: [2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
9. Occurrences: 1278. Action Sequence: [2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
10. Occurrences: 1208. Action Sequence: [2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
11. Occurrences: 1031. Action Sequence: [2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
12. Occurrences: 925. Action Sequence: [2.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
13. Occurrences: 746. Action Sequence: [2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
14. Occurrences: 743. Action Sequence: [2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
15. Occurrences: 693. Action Sequence: [2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
16. Occurrences: 621. Action Sequence: [2.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
17. Occurrences: 562. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
18. Occurrences: 550. Action Sequence: [2.,2.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
19. Occurrences: 478. Action Sequence: [2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
20. Occurrences: 468. Action Sequence: [2.,0.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
21. Occurrences: 374. Action Sequence: [2.,2.,0.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
22. Occurrences: 359. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
23. Occurrences: 351. Action Sequence: [2.,2.,0.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
24. Occurrences: 340. Action Sequence: [2.,2.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
25. Occurrences: 334. Action Sequence: [2.,0.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
26. Occurrences: 326. Action Sequence: [2.,0.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
27. Occurrences: 325. Action Sequence: [2.,2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
28. Occurrences: 290. Action Sequence: [2.,2.,2.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
29. Occurrences: 270. Action Sequence: [2.,0.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
30. Occurrences: 265. Action Sequence: [2.,0.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
31. Occurrences: 256. Action Sequence: [2.,2.,0.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
32. Occurrences: 248. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
33. Occurrences: 230. Action Sequence: [2.,2.,0.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
34. Occurrences: 225. Action Sequence: [2.,2.,2.,0.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
35. Occurrences: 221. Action Sequence: [2.,2.,0.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
36. Occurrences: 208. Action Sequence: [2.,0.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
37. Occurrences: 208. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
38. Occurrences: 194. Action Sequence: [2.,2.,2.,2.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
39. Occurrences: 187. Action Sequence: [2.,2.,2.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
40. Occurrences: 184. Action Sequence: [2.,0.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
41. Occurrences: 181. Action Sequence: [2.,2.,0.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
42. Occurrences: 178. Action Sequence: [2.,2.,2.,0.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
43. Occurrences: 167. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
44. Occurrences: 163. Action Sequence: [2.,0.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
45. Occurrences: 156. Action Sequence: [2.,2.,2.,0.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
46. Occurrences: 155. Action Sequence: [2.,0.,2.,0.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
47. Occurrences: 154. Action Sequence: [2.,2.,0.,2.,0.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
48. Occurrences: 152. Action Sequence: [2.,2.,0.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
49. Occurrences: 147. Action Sequence: [2.,0.,2.,0.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
50. Occurrences: 146. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 4
11850 different trajectories
1. Occurrences: 14914. Action Sequence: [1.]
2. Occurrences: 6739. Action Sequence: [1.,0.]
3. Occurrences: 6174. Action Sequence: [1.,1.,0.]
4. Occurrences: 3354. Action Sequence: [1.,1.,1.,0.]
5. Occurrences: 2509. Action Sequence: [1.,0.,1.,0.]
6. Occurrences: 2313. Action Sequence: [1.,1.,0.,1.,0.]
7. Occurrences: 1935. Action Sequence: [1.,1.,1.,1.,0.]
8. Occurrences: 1494. Action Sequence: [1.,1.]
9. Occurrences: 1278. Action Sequence: [1.,1.,1.,0.,1.,0.]
10. Occurrences: 1208. Action Sequence: [1.,1.,1.,1.,1.,0.]
11. Occurrences: 1031. Action Sequence: [1.,0.,1.,0.,1.,0.]
12. Occurrences: 925. Action Sequence: [1.,1.,0.,1.,0.,1.,0.]
13. Occurrences: 746. Action Sequence: [1.,1.,1.,1.,1.,1.,0.]
14. Occurrences: 743. Action Sequence: [1.,1.,1.,1.,0.,1.,0.]
15. Occurrences: 693. Action Sequence: [1.,0.,1.,1.,0.]
16. Occurrences: 621. Action Sequence: [1.,1.,0.,1.,1.,0.]
17. Occurrences: 562. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.]
18. Occurrences: 550. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.]
19. Occurrences: 478. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.]
20. Occurrences: 468. Action Sequence: [1.,0.,1.,1.,1.,0.]
21. Occurrences: 374. Action Sequence: [1.,1.,0.,1.,1.,1.,0.]
22. Occurrences: 359. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.]
23. Occurrences: 351. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,0.]
24. Occurrences: 340. Action Sequence: [1.,1.,1.,0.,1.,1.,0.]
25. Occurrences: 334. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.]
26. Occurrences: 326. Action Sequence: [1.,0.,1.,1.,1.,1.,0.]
27. Occurrences: 325. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.]
28. Occurrences: 290. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,0.]
29. Occurrences: 270. Action Sequence: [1.,0.,1.,0.,1.,1.,0.]
30. Occurrences: 265. Action Sequence: [1.,0.,1.,1.,0.,1.,0.]
31. Occurrences: 256. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,0.]
32. Occurrences: 248. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
33. Occurrences: 230. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,0.]
34. Occurrences: 225. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,0.]
35. Occurrences: 221. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,0.]
36. Occurrences: 208. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,0.]
37. Occurrences: 208. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
38. Occurrences: 194. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,0.]
39. Occurrences: 187. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.]
40. Occurrences: 184. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,0.]
41. Occurrences: 181. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,0.]
42. Occurrences: 178. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.,1.,0.]
43. Occurrences: 167. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
44. Occurrences: 163. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,0.]
45. Occurrences: 156. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,0.]
46. Occurrences: 155. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,0.]
47. Occurrences: 154. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,0.]
48. Occurrences: 152. Action Sequence: [1.,1.,0.,1.,1.,1.,0.,1.,0.]
49. Occurrences: 147. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.,1.,0.]
50. Occurrences: 146. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]

FACTORED SPACE 1, factorisation 4
8316 different trajectories
1. Occurrences: 16651. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 6798. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 6222. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 3397. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 2540. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 2353. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1961. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1313. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 1235. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 1049. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 950. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 774. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 767. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 708. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 641. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 587. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 573. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 495. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 478. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 388. Action Sequence: [1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 379. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 364. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 357. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 351. Action Sequence: [1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 345. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 338. Action Sequence: [1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 311. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 279. Action Sequence: [1.,0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 277. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 275. Action Sequence: [1.,0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 264. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 241. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 233. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 231. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 220. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 219. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 214. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 196. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 195. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 194. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 193. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 181. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 172. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 165. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 164. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 161. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 160. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 160. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 158. Action Sequence: [1.,1.,0.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 153. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 2, factorisation 4
20 different trajectories
1. Occurrences: 20647. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
2. Occurrences: 14914. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 8233. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 6341. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 5900. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 4968. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 4633. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 4014. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 3689. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 3375. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 3199. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 2969. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 2741. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 2524. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 2310. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 2178. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
17. Occurrences: 2035. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.]
18. Occurrences: 1927. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.]
19. Occurrences: 1781. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
20. Occurrences: 1622. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.]

--------------------------------------------------------
'''

### State 696, No Diabetes

In [ ]:
'''
NON FACTORED SPACE
2745 different trajectories
1. Occurrences: 5873. Action Sequence: [7.,7.,7.,0.]
2. Occurrences: 5868. Action Sequence: [7.,7.,0.]
3. Occurrences: 5384. Action Sequence: [7.,7.,7.,7.,0.]
4. Occurrences: 5369. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.]
5. Occurrences: 4771. Action Sequence: [7.,7.,7.,7.,7.,0.]
6. Occurrences: 4158. Action Sequence: [7.,7.,7.,7.,7.,7.,0.]
7. Occurrences: 4090. Action Sequence: [7.,0.]
8. Occurrences: 3684. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,0.]
9. Occurrences: 3062. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,0.]
10. Occurrences: 2614. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
11. Occurrences: 2217. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
12. Occurrences: 1845. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
13. Occurrences: 1647. Action Sequence: [7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
14. Occurrences: 1622. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
15. Occurrences: 1524. Action Sequence: [7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
16. Occurrences: 1445. Action Sequence: [7.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
17. Occurrences: 1410. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
18. Occurrences: 1333. Action Sequence: [7.,7.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
19. Occurrences: 1138. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
20. Occurrences: 1111. Action Sequence: [7.,7.,7.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
21. Occurrences: 1081. Action Sequence: [7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
22. Occurrences: 1002. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
23. Occurrences: 990. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
24. Occurrences: 972. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
25. Occurrences: 818. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
26. Occurrences: 792. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
27. Occurrences: 700. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
28. Occurrences: 693. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
29. Occurrences: 628. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
30. Occurrences: 612. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
31. Occurrences: 526. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.]
32. Occurrences: 488. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.]
33. Occurrences: 463. Action Sequence: [7.,7.,7.,0.,7.,0.]
34. Occurrences: 455. Action Sequence: [7.,7.,0.,7.,0.]
35. Occurrences: 402. Action Sequence: [7.,7.,7.,7.,0.,7.,0.]
36. Occurrences: 394. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.]
37. Occurrences: 382. Action Sequence: [7.,7.,7.,7.,7.,0.,7.,0.]
38. Occurrences: 306. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,4.,4.,4.,4.,4.]
39. Occurrences: 305. Action Sequence: [7.,7.,7.,7.,7.,7.,0.,7.,0.]
40. Occurrences: 284. Action Sequence: [7.,0.,7.,0.]
41. Occurrences: 278. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,4.,4.,4.,4.]
42. Occurrences: 271. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,7.]
43. Occurrences: 259. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,0.,7.,0.]
44. Occurrences: 259. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,7.,7.,7.,7.,7.,7.,7.]
45. Occurrences: 248. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,7.,7.,7.,7.]
46. Occurrences: 246. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,7.,7.]
47. Occurrences: 238. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,7.,7.,7.,7.,7.,7.,7.,7.,7.]
48. Occurrences: 231. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,4.,4.,4.]
49. Occurrences: 231. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,7.,7.,7.]
50. Occurrences: 228. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,0.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.]

FACTORED SPACE 0, factorisation 1
1859 different trajectories
1. Occurrences: 6249. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 6225. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 5730. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 5369. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 5122. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 4480. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 4303. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 4009. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 3382. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 2903. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 2523. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 2090. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 1935. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 1647. Action Sequence: [1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
15. Occurrences: 1646. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 1524. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
17. Occurrences: 1445. Action Sequence: [1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
18. Occurrences: 1384. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
19. Occurrences: 1333. Action Sequence: [1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
20. Occurrences: 1283. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
21. Occurrences: 1111. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 1081. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 1075. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
24. Occurrences: 991. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
25. Occurrences: 990. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 972. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
27. Occurrences: 942. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
28. Occurrences: 792. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 693. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 628. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
31. Occurrences: 526. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 499. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 488. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.]
34. Occurrences: 482. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 441. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 420. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 394. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 342. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 306. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.]
40. Occurrences: 304. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 300. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 278. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.]
43. Occurrences: 275. Action Sequence: [1.,1.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
44. Occurrences: 274. Action Sequence: [1.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
45. Occurrences: 258. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 255. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
47. Occurrences: 231. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.]
48. Occurrences: 228. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
49. Occurrences: 226. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 226. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 1
2174 different trajectories
1. Occurrences: 5873. Action Sequence: [3.,3.,3.,0.]
2. Occurrences: 5868. Action Sequence: [3.,3.,0.]
3. Occurrences: 5384. Action Sequence: [3.,3.,3.,3.,0.]
4. Occurrences: 5369. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
5. Occurrences: 4771. Action Sequence: [3.,3.,3.,3.,3.,0.]
6. Occurrences: 4158. Action Sequence: [3.,3.,3.,3.,3.,3.,0.]
7. Occurrences: 4090. Action Sequence: [3.,0.]
8. Occurrences: 3684. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.]
9. Occurrences: 3062. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,0.]
10. Occurrences: 2614. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
11. Occurrences: 2217. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
12. Occurrences: 2131. Action Sequence: [3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
13. Occurrences: 2023. Action Sequence: [3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
14. Occurrences: 1875. Action Sequence: [3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
15. Occurrences: 1845. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
16. Occurrences: 1738. Action Sequence: [3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
17. Occurrences: 1622. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
18. Occurrences: 1423. Action Sequence: [3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
19. Occurrences: 1410. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
20. Occurrences: 1387. Action Sequence: [3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
21. Occurrences: 1293. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
22. Occurrences: 1138. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
23. Occurrences: 1033. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
24. Occurrences: 1002. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
25. Occurrences: 972. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
26. Occurrences: 899. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
27. Occurrences: 818. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
28. Occurrences: 813. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
29. Occurrences: 700. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
30. Occurrences: 654. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.]
31. Occurrences: 629. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.]
32. Occurrences: 612. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
33. Occurrences: 508. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.]
34. Occurrences: 463. Action Sequence: [3.,3.,3.,0.,3.,0.]
35. Occurrences: 455. Action Sequence: [3.,3.,0.,3.,0.]
36. Occurrences: 424. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.]
37. Occurrences: 402. Action Sequence: [3.,3.,3.,3.,0.,3.,0.]
38. Occurrences: 382. Action Sequence: [3.,3.,3.,3.,3.,0.,3.,0.]
39. Occurrences: 366. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.]
40. Occurrences: 305. Action Sequence: [3.,3.,3.,3.,3.,3.,0.,3.,0.]
41. Occurrences: 287. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.]
42. Occurrences: 284. Action Sequence: [3.,0.,3.,0.]
43. Occurrences: 271. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.]
44. Occurrences: 259. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.]
45. Occurrences: 259. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.,3.,0.]
46. Occurrences: 248. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.]
47. Occurrences: 246. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.]
48. Occurrences: 240. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.]
49. Occurrences: 238. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
50. Occurrences: 231. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 2
1754 different trajectories
1. Occurrences: 6080. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 6063. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 5600. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 5369. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 4975. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 4379. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 4180. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 3915. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 3291. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 2829. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 2457. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 2131. Action Sequence: [1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
13. Occurrences: 2052. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 2023. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
15. Occurrences: 1882. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 1875. Action Sequence: [1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
17. Occurrences: 1738. Action Sequence: [1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
18. Occurrences: 1609. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 1423. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
20. Occurrences: 1387. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
21. Occurrences: 1347. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
22. Occurrences: 1293. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 1251. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
24. Occurrences: 1051. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
25. Occurrences: 1033. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 972. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
27. Occurrences: 946. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
28. Occurrences: 899. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 883. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
30. Occurrences: 813. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
31. Occurrences: 654. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 629. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.]
33. Occurrences: 508. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.]
34. Occurrences: 487. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 474. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 431. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 424. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 408. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 366. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.]
40. Occurrences: 332. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 296. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 293. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 287. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.]
44. Occurrences: 249. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 240. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.]
46. Occurrences: 222. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 221. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 218. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 218. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 212. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.]

FACTORED SPACE 1, factorisation 2
2579 different trajectories
1. Occurrences: 5873. Action Sequence: [3.,3.,3.,0.]
2. Occurrences: 5868. Action Sequence: [3.,3.,0.]
3. Occurrences: 5384. Action Sequence: [3.,3.,3.,3.,0.]
4. Occurrences: 5369. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
5. Occurrences: 4771. Action Sequence: [3.,3.,3.,3.,3.,0.]
6. Occurrences: 4158. Action Sequence: [3.,3.,3.,3.,3.,3.,0.]
7. Occurrences: 4090. Action Sequence: [3.,0.]
8. Occurrences: 3684. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.]
9. Occurrences: 3062. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,0.]
10. Occurrences: 2614. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
11. Occurrences: 2217. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
12. Occurrences: 1845. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
13. Occurrences: 1647. Action Sequence: [3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
14. Occurrences: 1622. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
15. Occurrences: 1524. Action Sequence: [3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
16. Occurrences: 1445. Action Sequence: [3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
17. Occurrences: 1410. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
18. Occurrences: 1333. Action Sequence: [3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
19. Occurrences: 1138. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
20. Occurrences: 1111. Action Sequence: [3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
21. Occurrences: 1081. Action Sequence: [3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
22. Occurrences: 1002. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
23. Occurrences: 990. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
24. Occurrences: 972. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
25. Occurrences: 818. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
26. Occurrences: 792. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
27. Occurrences: 700. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
28. Occurrences: 693. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
29. Occurrences: 628. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
30. Occurrences: 612. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
31. Occurrences: 526. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.]
32. Occurrences: 488. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.]
33. Occurrences: 463. Action Sequence: [3.,3.,3.,0.,3.,0.]
34. Occurrences: 455. Action Sequence: [3.,3.,0.,3.,0.]
35. Occurrences: 402. Action Sequence: [3.,3.,3.,3.,0.,3.,0.]
36. Occurrences: 394. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.]
37. Occurrences: 382. Action Sequence: [3.,3.,3.,3.,3.,0.,3.,0.]
38. Occurrences: 368. Action Sequence: [3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
39. Occurrences: 349. Action Sequence: [3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
40. Occurrences: 347. Action Sequence: [3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
41. Occurrences: 341. Action Sequence: [3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
42. Occurrences: 330. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.]
43. Occurrences: 322. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
44. Occurrences: 317. Action Sequence: [3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
45. Occurrences: 315. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
46. Occurrences: 312. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.]
47. Occurrences: 306. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.]
48. Occurrences: 305. Action Sequence: [3.,3.,3.,3.,3.,3.,0.,3.,0.]
49. Occurrences: 304. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
50. Occurrences: 291. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 3
1851 different trajectories
1. Occurrences: 5873. Action Sequence: [1.,1.,1.,0.]
2. Occurrences: 5868. Action Sequence: [1.,1.,0.]
3. Occurrences: 5384. Action Sequence: [1.,1.,1.,1.,0.]
4. Occurrences: 5369. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 4771. Action Sequence: [1.,1.,1.,1.,1.,0.]
6. Occurrences: 4158. Action Sequence: [1.,1.,1.,1.,1.,1.,0.]
7. Occurrences: 4090. Action Sequence: [1.,0.]
8. Occurrences: 3684. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.]
9. Occurrences: 3062. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.]
10. Occurrences: 2614. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
11. Occurrences: 2330. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 2217. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
13. Occurrences: 2210. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 2086. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 1938. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 1845. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
17. Occurrences: 1639. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 1622. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
19. Occurrences: 1521. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 1473. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 1410. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
22. Occurrences: 1257. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 1138. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
24. Occurrences: 1112. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 1051. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 1002. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
27. Occurrences: 972. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
28. Occurrences: 888. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 859. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 818. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
31. Occurrences: 705. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 700. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
33. Occurrences: 631. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
34. Occurrences: 614. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
35. Occurrences: 612. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
36. Occurrences: 518. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
37. Occurrences: 486. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
38. Occurrences: 483. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
39. Occurrences: 463. Action Sequence: [1.,1.,1.,0.,1.,0.]
40. Occurrences: 455. Action Sequence: [1.,1.,0.,1.,0.]
41. Occurrences: 402. Action Sequence: [1.,1.,1.,1.,0.,1.,0.]
42. Occurrences: 382. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.]
43. Occurrences: 305. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.]
44. Occurrences: 284. Action Sequence: [1.,0.,1.,0.]
45. Occurrences: 259. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
46. Occurrences: 218. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
47. Occurrences: 216. Action Sequence: [1.,1.,0.,1.,1.,0.]
48. Occurrences: 206. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.]
49. Occurrences: 206. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
50. Occurrences: 200. Action Sequence: [1.,1.,1.,0.,1.,1.,0.]

FACTORED SPACE 1, factorisation 3
2325 different trajectories
1. Occurrences: 6080. Action Sequence: [3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
2. Occurrences: 6063. Action Sequence: [3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
3. Occurrences: 5600. Action Sequence: [3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
4. Occurrences: 5369. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
5. Occurrences: 4975. Action Sequence: [3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
6. Occurrences: 4379. Action Sequence: [3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
7. Occurrences: 4180. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
8. Occurrences: 3915. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
9. Occurrences: 3291. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
10. Occurrences: 2829. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
11. Occurrences: 2457. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
12. Occurrences: 2052. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.]
13. Occurrences: 1882. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.]
14. Occurrences: 1647. Action Sequence: [3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
15. Occurrences: 1609. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.]
16. Occurrences: 1524. Action Sequence: [3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
17. Occurrences: 1445. Action Sequence: [3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
18. Occurrences: 1347. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.]
19. Occurrences: 1333. Action Sequence: [3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
20. Occurrences: 1251. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.]
21. Occurrences: 1111. Action Sequence: [3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 1081. Action Sequence: [3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 1051. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.]
24. Occurrences: 990. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
25. Occurrences: 972. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
26. Occurrences: 946. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.]
27. Occurrences: 883. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.]
28. Occurrences: 792. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 693. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 628. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
31. Occurrences: 526. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 488. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.]
33. Occurrences: 487. Action Sequence: [3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
34. Occurrences: 474. Action Sequence: [3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
35. Occurrences: 431. Action Sequence: [3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
36. Occurrences: 408. Action Sequence: [3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
37. Occurrences: 394. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 332. Action Sequence: [3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
39. Occurrences: 306. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.]
40. Occurrences: 296. Action Sequence: [3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
41. Occurrences: 293. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
42. Occurrences: 278. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.]
43. Occurrences: 249. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
44. Occurrences: 231. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.]
45. Occurrences: 222. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.]
46. Occurrences: 221. Action Sequence: [3.,3.,0.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
47. Occurrences: 218. Action Sequence: [3.,3.,3.,3.,0.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
48. Occurrences: 218. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.]
49. Occurrences: 209. Action Sequence: [3.,3.,3.,0.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
50. Occurrences: 205. Action Sequence: [3.,3.,3.,3.,3.,0.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 4
1851 different trajectories
1. Occurrences: 5873. Action Sequence: [1.,1.,1.,0.]
2. Occurrences: 5868. Action Sequence: [1.,1.,0.]
3. Occurrences: 5384. Action Sequence: [1.,1.,1.,1.,0.]
4. Occurrences: 5369. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 4771. Action Sequence: [1.,1.,1.,1.,1.,0.]
6. Occurrences: 4158. Action Sequence: [1.,1.,1.,1.,1.,1.,0.]
7. Occurrences: 4090. Action Sequence: [1.,0.]
8. Occurrences: 3684. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.]
9. Occurrences: 3062. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.]
10. Occurrences: 2614. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
11. Occurrences: 2330. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 2217. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
13. Occurrences: 2210. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 2086. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 1938. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 1845. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
17. Occurrences: 1639. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 1622. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
19. Occurrences: 1521. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 1473. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 1410. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
22. Occurrences: 1257. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 1138. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
24. Occurrences: 1112. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 1051. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 1002. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
27. Occurrences: 972. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
28. Occurrences: 888. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 859. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 818. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
31. Occurrences: 705. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 700. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
33. Occurrences: 631. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
34. Occurrences: 614. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
35. Occurrences: 612. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
36. Occurrences: 518. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
37. Occurrences: 486. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
38. Occurrences: 483. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
39. Occurrences: 463. Action Sequence: [1.,1.,1.,0.,1.,0.]
40. Occurrences: 455. Action Sequence: [1.,1.,0.,1.,0.]
41. Occurrences: 402. Action Sequence: [1.,1.,1.,1.,0.,1.,0.]
42. Occurrences: 382. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.]
43. Occurrences: 305. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.]
44. Occurrences: 284. Action Sequence: [1.,0.,1.,0.]
45. Occurrences: 259. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
46. Occurrences: 218. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
47. Occurrences: 216. Action Sequence: [1.,1.,0.,1.,1.,0.]
48. Occurrences: 206. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.]
49. Occurrences: 206. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
50. Occurrences: 200. Action Sequence: [1.,1.,1.,0.,1.,1.,0.]

FACTORED SPACE 1, factorisation 4
1859 different trajectories
1. Occurrences: 6249. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 6225. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 5730. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 5369. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 5122. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 4480. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 4303. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 4009. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 3382. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 2903. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 2523. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 2090. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 1935. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 1647. Action Sequence: [1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
15. Occurrences: 1646. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 1524. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
17. Occurrences: 1445. Action Sequence: [1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
18. Occurrences: 1384. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
19. Occurrences: 1333. Action Sequence: [1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
20. Occurrences: 1283. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
21. Occurrences: 1111. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 1081. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 1075. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
24. Occurrences: 991. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
25. Occurrences: 990. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 972. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
27. Occurrences: 942. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
28. Occurrences: 792. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 693. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 628. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
31. Occurrences: 526. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 499. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 488. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.]
34. Occurrences: 482. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 441. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 420. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 394. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 342. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 306. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.]
40. Occurrences: 304. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 300. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 278. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.]
43. Occurrences: 275. Action Sequence: [1.,1.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
44. Occurrences: 274. Action Sequence: [1.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
45. Occurrences: 258. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 255. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
47. Occurrences: 231. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.]
48. Occurrences: 228. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
49. Occurrences: 226. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 226. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 2, factorisation 4
1754 different trajectories
1. Occurrences: 6080. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 6063. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 5600. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 5369. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 4975. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 4379. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 4180. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 3915. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 3291. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 2829. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 2457. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 2131. Action Sequence: [1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
13. Occurrences: 2052. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 2023. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
15. Occurrences: 1882. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 1875. Action Sequence: [1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
17. Occurrences: 1738. Action Sequence: [1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
18. Occurrences: 1609. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 1423. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
20. Occurrences: 1387. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
21. Occurrences: 1347. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
22. Occurrences: 1293. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 1251. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
24. Occurrences: 1051. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
25. Occurrences: 1033. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 972. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
27. Occurrences: 946. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
28. Occurrences: 899. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 883. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
30. Occurrences: 813. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
31. Occurrences: 654. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 629. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.]
33. Occurrences: 508. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.]
34. Occurrences: 487. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 474. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 431. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 424. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 408. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 366. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.]
40. Occurrences: 332. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 296. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 293. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 287. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.]
44. Occurrences: 249. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 240. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.]
46. Occurrences: 222. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 221. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 218. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 218. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 212. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.]

--------------------------------------------------------
'''

### State 703, Diabetes

In [ ]:
'''
NON FACTORED SPACE
12683 different trajectories
1. Occurrences: 15065. Action Sequence: [6.]
2. Occurrences: 6698. Action Sequence: [6.,0.]
3. Occurrences: 6001. Action Sequence: [6.,6.,0.]
4. Occurrences: 3275. Action Sequence: [6.,6.,6.,0.]
5. Occurrences: 2563. Action Sequence: [6.,0.,6.,0.]
6. Occurrences: 2280. Action Sequence: [6.,6.,0.,6.,0.]
7. Occurrences: 1833. Action Sequence: [6.,6.,6.,6.,0.]
8. Occurrences: 1447. Action Sequence: [6.,6.]
9. Occurrences: 1305. Action Sequence: [6.,6.,6.,0.,6.,0.]
10. Occurrences: 1118. Action Sequence: [6.,6.,6.,6.,6.,0.]
11. Occurrences: 960. Action Sequence: [6.,0.,6.,0.,6.,0.]
12. Occurrences: 858. Action Sequence: [6.,6.,0.,6.,0.,6.,0.]
13. Occurrences: 803. Action Sequence: [6.,6.,6.,6.,6.,6.,0.]
14. Occurrences: 716. Action Sequence: [6.,0.,6.,6.,0.]
15. Occurrences: 711. Action Sequence: [6.,6.,6.,6.,0.,6.,0.]
16. Occurrences: 631. Action Sequence: [6.,6.,0.,6.,6.,0.]
17. Occurrences: 499. Action Sequence: [6.,6.,6.,6.,6.,6.,6.,0.]
18. Occurrences: 478. Action Sequence: [6.,6.,6.,0.,6.,0.,6.,0.]
19. Occurrences: 455. Action Sequence: [6.,0.,6.,6.,6.,0.]
20. Occurrences: 445. Action Sequence: [6.,6.,6.,6.,6.,0.,6.,0.]
21. Occurrences: 407. Action Sequence: [6.,6.,0.,6.,6.,6.,0.]
22. Occurrences: 385. Action Sequence: [6.,6.,6.,6.,6.,6.,6.,6.,0.]
23. Occurrences: 374. Action Sequence: [6.,0.,6.,0.,6.,0.,6.,0.]
24. Occurrences: 364. Action Sequence: [6.,6.,0.,6.,0.,6.,0.,6.,0.]
25. Occurrences: 325. Action Sequence: [6.,6.,6.,0.,6.,6.,0.]
26. Occurrences: 298. Action Sequence: [6.,0.,6.,6.,6.,6.,0.]
27. Occurrences: 291. Action Sequence: [6.,6.,6.,6.,6.,6.,0.,6.,0.]
28. Occurrences: 273. Action Sequence: [6.,0.,6.,6.,0.,6.,0.]
29. Occurrences: 266. Action Sequence: [6.,6.,0.,6.,6.,6.,6.,0.]
30. Occurrences: 254. Action Sequence: [6.,6.,0.,6.,6.,0.,6.,0.]
31. Occurrences: 252. Action Sequence: [6.,0.,6.,0.,6.,6.,0.]
32. Occurrences: 252. Action Sequence: [6.,6.,6.,6.,0.,6.,0.,6.,0.]
33. Occurrences: 247. Action Sequence: [6.,6.,6.,0.,6.,6.,6.,0.]
34. Occurrences: 236. Action Sequence: [6.,6.,0.,6.,0.,6.,6.,0.]
35. Occurrences: 222. Action Sequence: [6.,6.,6.,6.,0.,6.,6.,0.]
36. Occurrences: 211. Action Sequence: [6.,6.,6.,6.,6.,6.,6.,6.,6.,0.]
37. Occurrences: 207. Action Sequence: [6.,0.,6.,6.,6.,6.,6.,0.]
38. Occurrences: 206. Action Sequence: [6.,6.,6.,6.,6.,6.,6.,0.,6.,0.]
39. Occurrences: 194. Action Sequence: [6.,0.,6.,0.,6.,6.,6.,0.]
40. Occurrences: 179. Action Sequence: [6.,6.,0.,6.,6.,6.,6.,6.,0.]
41. Occurrences: 178. Action Sequence: [6.,6.,6.,0.,6.,0.,6.,0.,6.,0.]
42. Occurrences: 172. Action Sequence: [6.,6.,6.,6.,6.,0.,6.,0.,6.,0.]
43. Occurrences: 170. Action Sequence: [6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,0.]
44. Occurrences: 166. Action Sequence: [6.,7.,0.]
45. Occurrences: 163. Action Sequence: [6.,0.,6.,6.,6.,0.,6.,0.]
46. Occurrences: 155. Action Sequence: [6.,6.,0.,6.,6.,6.,0.,6.,0.]
47. Occurrences: 155. Action Sequence: [6.,6.,6.,6.,6.,0.,6.,6.,0.]
48. Occurrences: 152. Action Sequence: [6.,6.,6.]
49. Occurrences: 150. Action Sequence: [6.,0.,6.,0.,6.,0.,6.,0.,6.,0.]
50. Occurrences: 143. Action Sequence: [6.,6.,0.,6.,0.,6.,6.,6.,0.]

FACTORED SPACE 0, factorisation 1
8375 different trajectories
1. Occurrences: 16827. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 6757. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 6222. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 3417. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 2597. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 2372. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1928. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1362. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 1185. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 984. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 917. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 850. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 755. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 728. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 666. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 530. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 518. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 474. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 466. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 437. Action Sequence: [1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 428. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 392. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 383. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 344. Action Sequence: [1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 323. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 306. Action Sequence: [1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 294. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 279. Action Sequence: [1.,0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 274. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 269. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 261. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 260. Action Sequence: [1.,0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 257. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 242. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 242. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 223. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 212. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 203. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 201. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 195. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 195. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 192. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 171. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 170. Action Sequence: [1.,1.,0.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 167. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 160. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 160. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 160. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 157. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 152. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 1
12683 different trajectories
1. Occurrences: 15065. Action Sequence: [2.]
2. Occurrences: 6698. Action Sequence: [2.,0.]
3. Occurrences: 6001. Action Sequence: [2.,2.,0.]
4. Occurrences: 3275. Action Sequence: [2.,2.,2.,0.]
5. Occurrences: 2563. Action Sequence: [2.,0.,2.,0.]
6. Occurrences: 2280. Action Sequence: [2.,2.,0.,2.,0.]
7. Occurrences: 1833. Action Sequence: [2.,2.,2.,2.,0.]
8. Occurrences: 1447. Action Sequence: [2.,2.]
9. Occurrences: 1305. Action Sequence: [2.,2.,2.,0.,2.,0.]
10. Occurrences: 1118. Action Sequence: [2.,2.,2.,2.,2.,0.]
11. Occurrences: 960. Action Sequence: [2.,0.,2.,0.,2.,0.]
12. Occurrences: 858. Action Sequence: [2.,2.,0.,2.,0.,2.,0.]
13. Occurrences: 803. Action Sequence: [2.,2.,2.,2.,2.,2.,0.]
14. Occurrences: 716. Action Sequence: [2.,0.,2.,2.,0.]
15. Occurrences: 711. Action Sequence: [2.,2.,2.,2.,0.,2.,0.]
16. Occurrences: 631. Action Sequence: [2.,2.,0.,2.,2.,0.]
17. Occurrences: 499. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,0.]
18. Occurrences: 478. Action Sequence: [2.,2.,2.,0.,2.,0.,2.,0.]
19. Occurrences: 455. Action Sequence: [2.,0.,2.,2.,2.,0.]
20. Occurrences: 445. Action Sequence: [2.,2.,2.,2.,2.,0.,2.,0.]
21. Occurrences: 407. Action Sequence: [2.,2.,0.,2.,2.,2.,0.]
22. Occurrences: 385. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,0.]
23. Occurrences: 374. Action Sequence: [2.,0.,2.,0.,2.,0.,2.,0.]
24. Occurrences: 364. Action Sequence: [2.,2.,0.,2.,0.,2.,0.,2.,0.]
25. Occurrences: 325. Action Sequence: [2.,2.,2.,0.,2.,2.,0.]
26. Occurrences: 298. Action Sequence: [2.,0.,2.,2.,2.,2.,0.]
27. Occurrences: 291. Action Sequence: [2.,2.,2.,2.,2.,2.,0.,2.,0.]
28. Occurrences: 273. Action Sequence: [2.,0.,2.,2.,0.,2.,0.]
29. Occurrences: 266. Action Sequence: [2.,2.,0.,2.,2.,2.,2.,0.]
30. Occurrences: 254. Action Sequence: [2.,2.,0.,2.,2.,0.,2.,0.]
31. Occurrences: 252. Action Sequence: [2.,0.,2.,0.,2.,2.,0.]
32. Occurrences: 252. Action Sequence: [2.,2.,2.,2.,0.,2.,0.,2.,0.]
33. Occurrences: 247. Action Sequence: [2.,2.,2.,0.,2.,2.,2.,0.]
34. Occurrences: 236. Action Sequence: [2.,2.,0.,2.,0.,2.,2.,0.]
35. Occurrences: 222. Action Sequence: [2.,2.,2.,2.,0.,2.,2.,0.]
36. Occurrences: 211. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,0.]
37. Occurrences: 207. Action Sequence: [2.,0.,2.,2.,2.,2.,2.,0.]
38. Occurrences: 206. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,0.,2.,0.]
39. Occurrences: 194. Action Sequence: [2.,0.,2.,0.,2.,2.,2.,0.]
40. Occurrences: 179. Action Sequence: [2.,2.,0.,2.,2.,2.,2.,2.,0.]
41. Occurrences: 178. Action Sequence: [2.,2.,2.,0.,2.,0.,2.,0.,2.,0.]
42. Occurrences: 172. Action Sequence: [2.,2.,2.,2.,2.,0.,2.,0.,2.,0.]
43. Occurrences: 170. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.]
44. Occurrences: 166. Action Sequence: [2.,3.,0.]
45. Occurrences: 163. Action Sequence: [2.,0.,2.,2.,2.,0.,2.,0.]
46. Occurrences: 155. Action Sequence: [2.,2.,0.,2.,2.,2.,0.,2.,0.]
47. Occurrences: 155. Action Sequence: [2.,2.,2.,2.,2.,0.,2.,2.,0.]
48. Occurrences: 152. Action Sequence: [2.,2.,2.]
49. Occurrences: 150. Action Sequence: [2.,0.,2.,0.,2.,0.,2.,0.,2.,0.]
50. Occurrences: 143. Action Sequence: [2.,2.,0.,2.,0.,2.,2.,2.,0.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 2
88 different trajectories
1. Occurrences: 20263. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
2. Occurrences: 15130. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 8145. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 6187. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 5873. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 4862. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 4489. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 3949. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 3699. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 3316. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 3042. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 2811. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 2673. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 2453. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 2262. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 2128. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
17. Occurrences: 1994. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.]
18. Occurrences: 1805. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.]
19. Occurrences: 1697. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
20. Occurrences: 1528. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.]
21. Occurrences: 375. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 174. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 113. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 87. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 81. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 79. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 66. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 62. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 59. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 54. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 53. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.]
32. Occurrences: 48. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 45. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 44. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 42. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
36. Occurrences: 41. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.]
37. Occurrences: 39. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
38. Occurrences: 37. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
39. Occurrences: 35. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.]
40. Occurrences: 15. Action Sequence: [0.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
41. Occurrences: 11. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 10. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 9. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
44. Occurrences: 7. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 7. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 7. Action Sequence: [0.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
47. Occurrences: 7. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 5. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 5. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
50. Occurrences: 5. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 2
11866 different trajectories
1. Occurrences: 15065. Action Sequence: [3.]
2. Occurrences: 6698. Action Sequence: [3.,0.]
3. Occurrences: 6167. Action Sequence: [3.,3.,0.]
4. Occurrences: 3371. Action Sequence: [3.,3.,3.,0.]
5. Occurrences: 2563. Action Sequence: [3.,0.,3.,0.]
6. Occurrences: 2343. Action Sequence: [3.,3.,0.,3.,0.]
7. Occurrences: 1896. Action Sequence: [3.,3.,3.,3.,0.]
8. Occurrences: 1512. Action Sequence: [3.,3.]
9. Occurrences: 1339. Action Sequence: [3.,3.,3.,0.,3.,0.]
10. Occurrences: 1154. Action Sequence: [3.,3.,3.,3.,3.,0.]
11. Occurrences: 960. Action Sequence: [3.,0.,3.,0.,3.,0.]
12. Occurrences: 887. Action Sequence: [3.,3.,0.,3.,0.,3.,0.]
13. Occurrences: 825. Action Sequence: [3.,3.,3.,3.,3.,3.,0.]
14. Occurrences: 728. Action Sequence: [3.,3.,3.,3.,0.,3.,0.]
15. Occurrences: 716. Action Sequence: [3.,0.,3.,3.,0.]
16. Occurrences: 651. Action Sequence: [3.,3.,0.,3.,3.,0.]
17. Occurrences: 509. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.]
18. Occurrences: 496. Action Sequence: [3.,3.,3.,0.,3.,0.,3.,0.]
19. Occurrences: 455. Action Sequence: [3.,0.,3.,3.,3.,0.]
20. Occurrences: 455. Action Sequence: [3.,3.,3.,3.,3.,0.,3.,0.]
21. Occurrences: 421. Action Sequence: [3.,3.,0.,3.,3.,3.,0.]
22. Occurrences: 395. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,0.]
23. Occurrences: 374. Action Sequence: [3.,0.,3.,0.,3.,0.,3.,0.]
24. Occurrences: 371. Action Sequence: [3.,3.,0.,3.,0.,3.,0.,3.,0.]
25. Occurrences: 333. Action Sequence: [3.,3.,3.,0.,3.,3.,0.]
26. Occurrences: 301. Action Sequence: [3.,3.,3.,3.,3.,3.,0.,3.,0.]
27. Occurrences: 298. Action Sequence: [3.,0.,3.,3.,3.,3.,0.]
28. Occurrences: 275. Action Sequence: [3.,3.,0.,3.,3.,3.,3.,0.]
29. Occurrences: 273. Action Sequence: [3.,0.,3.,3.,0.,3.,0.]
30. Occurrences: 256. Action Sequence: [3.,3.,0.,3.,3.,0.,3.,0.]
31. Occurrences: 255. Action Sequence: [3.,3.,3.,3.,0.,3.,0.,3.,0.]
32. Occurrences: 253. Action Sequence: [3.,3.,3.,0.,3.,3.,3.,0.]
33. Occurrences: 252. Action Sequence: [3.,0.,3.,0.,3.,3.,0.]
34. Occurrences: 242. Action Sequence: [3.,3.,0.,3.,0.,3.,3.,0.]
35. Occurrences: 229. Action Sequence: [3.,3.,3.,3.,0.,3.,3.,0.]
36. Occurrences: 220. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
37. Occurrences: 211. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.,3.,0.]
38. Occurrences: 207. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,0.]
39. Occurrences: 194. Action Sequence: [3.,0.,3.,0.,3.,3.,3.,0.]
40. Occurrences: 185. Action Sequence: [3.,3.,3.,0.,3.,0.,3.,0.,3.,0.]
41. Occurrences: 183. Action Sequence: [3.,3.,0.,3.,3.,3.,3.,3.,0.]
42. Occurrences: 179. Action Sequence: [3.,3.,3.,3.,3.,0.,3.,0.,3.,0.]
43. Occurrences: 179. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
44. Occurrences: 163. Action Sequence: [3.,0.,3.,3.,3.,0.,3.,0.]
45. Occurrences: 162. Action Sequence: [3.,3.,0.,3.,3.,3.,0.,3.,0.]
46. Occurrences: 160. Action Sequence: [3.,3.,3.,3.,3.,0.,3.,3.,0.]
47. Occurrences: 160. Action Sequence: [3.,3.,3.]
48. Occurrences: 150. Action Sequence: [3.,0.,3.,0.,3.,0.,3.,0.,3.,0.]
49. Occurrences: 147. Action Sequence: [3.,3.,0.,3.,0.,3.,3.,3.,0.]
50. Occurrences: 143. Action Sequence: [3.,3.,0.,3.,0.,3.,0.,3.,0.,3.,0.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 3
11866 different trajectories
1. Occurrences: 15065. Action Sequence: [1.]
2. Occurrences: 6698. Action Sequence: [1.,0.]
3. Occurrences: 6167. Action Sequence: [1.,1.,0.]
4. Occurrences: 3371. Action Sequence: [1.,1.,1.,0.]
5. Occurrences: 2563. Action Sequence: [1.,0.,1.,0.]
6. Occurrences: 2343. Action Sequence: [1.,1.,0.,1.,0.]
7. Occurrences: 1896. Action Sequence: [1.,1.,1.,1.,0.]
8. Occurrences: 1512. Action Sequence: [1.,1.]
9. Occurrences: 1339. Action Sequence: [1.,1.,1.,0.,1.,0.]
10. Occurrences: 1154. Action Sequence: [1.,1.,1.,1.,1.,0.]
11. Occurrences: 960. Action Sequence: [1.,0.,1.,0.,1.,0.]
12. Occurrences: 887. Action Sequence: [1.,1.,0.,1.,0.,1.,0.]
13. Occurrences: 825. Action Sequence: [1.,1.,1.,1.,1.,1.,0.]
14. Occurrences: 728. Action Sequence: [1.,1.,1.,1.,0.,1.,0.]
15. Occurrences: 716. Action Sequence: [1.,0.,1.,1.,0.]
16. Occurrences: 651. Action Sequence: [1.,1.,0.,1.,1.,0.]
17. Occurrences: 509. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.]
18. Occurrences: 496. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.]
19. Occurrences: 455. Action Sequence: [1.,0.,1.,1.,1.,0.]
20. Occurrences: 455. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.]
21. Occurrences: 421. Action Sequence: [1.,1.,0.,1.,1.,1.,0.]
22. Occurrences: 395. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.]
23. Occurrences: 374. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.]
24. Occurrences: 371. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,0.]
25. Occurrences: 333. Action Sequence: [1.,1.,1.,0.,1.,1.,0.]
26. Occurrences: 301. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.]
27. Occurrences: 298. Action Sequence: [1.,0.,1.,1.,1.,1.,0.]
28. Occurrences: 275. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,0.]
29. Occurrences: 273. Action Sequence: [1.,0.,1.,1.,0.,1.,0.]
30. Occurrences: 256. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,0.]
31. Occurrences: 255. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,0.]
32. Occurrences: 253. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,0.]
33. Occurrences: 252. Action Sequence: [1.,0.,1.,0.,1.,1.,0.]
34. Occurrences: 242. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,0.]
35. Occurrences: 229. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.]
36. Occurrences: 220. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
37. Occurrences: 211. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
38. Occurrences: 207. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,0.]
39. Occurrences: 194. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,0.]
40. Occurrences: 185. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.,1.,0.]
41. Occurrences: 183. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,0.]
42. Occurrences: 179. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,0.]
43. Occurrences: 179. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
44. Occurrences: 163. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,0.]
45. Occurrences: 162. Action Sequence: [1.,1.,0.,1.,1.,1.,0.,1.,0.]
46. Occurrences: 160. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,0.]
47. Occurrences: 160. Action Sequence: [1.,1.,1.]
48. Occurrences: 150. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.,1.,0.]
49. Occurrences: 147. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,0.]
50. Occurrences: 143. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,0.,1.,0.]

FACTORED SPACE 1, factorisation 3
12682 different trajectories
1. Occurrences: 15130. Action Sequence: [2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
2. Occurrences: 6698. Action Sequence: [2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
3. Occurrences: 6001. Action Sequence: [2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
4. Occurrences: 3275. Action Sequence: [2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
5. Occurrences: 2563. Action Sequence: [2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
6. Occurrences: 2280. Action Sequence: [2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
7. Occurrences: 1833. Action Sequence: [2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
8. Occurrences: 1447. Action Sequence: [2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
9. Occurrences: 1305. Action Sequence: [2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
10. Occurrences: 1118. Action Sequence: [2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
11. Occurrences: 960. Action Sequence: [2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
12. Occurrences: 858. Action Sequence: [2.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
13. Occurrences: 803. Action Sequence: [2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
14. Occurrences: 716. Action Sequence: [2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
15. Occurrences: 711. Action Sequence: [2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
16. Occurrences: 631. Action Sequence: [2.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
17. Occurrences: 499. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
18. Occurrences: 478. Action Sequence: [2.,2.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
19. Occurrences: 455. Action Sequence: [2.,0.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
20. Occurrences: 445. Action Sequence: [2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
21. Occurrences: 407. Action Sequence: [2.,2.,0.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
22. Occurrences: 385. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
23. Occurrences: 374. Action Sequence: [2.,0.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
24. Occurrences: 364. Action Sequence: [2.,2.,0.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
25. Occurrences: 325. Action Sequence: [2.,2.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
26. Occurrences: 298. Action Sequence: [2.,0.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
27. Occurrences: 291. Action Sequence: [2.,2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
28. Occurrences: 273. Action Sequence: [2.,0.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
29. Occurrences: 266. Action Sequence: [2.,2.,0.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
30. Occurrences: 254. Action Sequence: [2.,2.,0.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
31. Occurrences: 252. Action Sequence: [2.,0.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
32. Occurrences: 252. Action Sequence: [2.,2.,2.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
33. Occurrences: 247. Action Sequence: [2.,2.,2.,0.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
34. Occurrences: 236. Action Sequence: [2.,2.,0.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
35. Occurrences: 222. Action Sequence: [2.,2.,2.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
36. Occurrences: 211. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
37. Occurrences: 207. Action Sequence: [2.,0.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
38. Occurrences: 206. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
39. Occurrences: 194. Action Sequence: [2.,0.,2.,0.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
40. Occurrences: 179. Action Sequence: [2.,2.,0.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
41. Occurrences: 178. Action Sequence: [2.,2.,2.,0.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
42. Occurrences: 172. Action Sequence: [2.,2.,2.,2.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
43. Occurrences: 170. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
44. Occurrences: 166. Action Sequence: [2.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
45. Occurrences: 163. Action Sequence: [2.,0.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
46. Occurrences: 155. Action Sequence: [2.,2.,0.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
47. Occurrences: 155. Action Sequence: [2.,2.,2.,2.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
48. Occurrences: 152. Action Sequence: [2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
49. Occurrences: 150. Action Sequence: [2.,0.,2.,0.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
50. Occurrences: 143. Action Sequence: [2.,2.,0.,2.,0.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 4
11866 different trajectories
1. Occurrences: 15065. Action Sequence: [1.]
2. Occurrences: 6698. Action Sequence: [1.,0.]
3. Occurrences: 6167. Action Sequence: [1.,1.,0.]
4. Occurrences: 3371. Action Sequence: [1.,1.,1.,0.]
5. Occurrences: 2563. Action Sequence: [1.,0.,1.,0.]
6. Occurrences: 2343. Action Sequence: [1.,1.,0.,1.,0.]
7. Occurrences: 1896. Action Sequence: [1.,1.,1.,1.,0.]
8. Occurrences: 1512. Action Sequence: [1.,1.]
9. Occurrences: 1339. Action Sequence: [1.,1.,1.,0.,1.,0.]
10. Occurrences: 1154. Action Sequence: [1.,1.,1.,1.,1.,0.]
11. Occurrences: 960. Action Sequence: [1.,0.,1.,0.,1.,0.]
12. Occurrences: 887. Action Sequence: [1.,1.,0.,1.,0.,1.,0.]
13. Occurrences: 825. Action Sequence: [1.,1.,1.,1.,1.,1.,0.]
14. Occurrences: 728. Action Sequence: [1.,1.,1.,1.,0.,1.,0.]
15. Occurrences: 716. Action Sequence: [1.,0.,1.,1.,0.]
16. Occurrences: 651. Action Sequence: [1.,1.,0.,1.,1.,0.]
17. Occurrences: 509. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.]
18. Occurrences: 496. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.]
19. Occurrences: 455. Action Sequence: [1.,0.,1.,1.,1.,0.]
20. Occurrences: 455. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.]
21. Occurrences: 421. Action Sequence: [1.,1.,0.,1.,1.,1.,0.]
22. Occurrences: 395. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.]
23. Occurrences: 374. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.]
24. Occurrences: 371. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,0.]
25. Occurrences: 333. Action Sequence: [1.,1.,1.,0.,1.,1.,0.]
26. Occurrences: 301. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.]
27. Occurrences: 298. Action Sequence: [1.,0.,1.,1.,1.,1.,0.]
28. Occurrences: 275. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,0.]
29. Occurrences: 273. Action Sequence: [1.,0.,1.,1.,0.,1.,0.]
30. Occurrences: 256. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,0.]
31. Occurrences: 255. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,0.]
32. Occurrences: 253. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,0.]
33. Occurrences: 252. Action Sequence: [1.,0.,1.,0.,1.,1.,0.]
34. Occurrences: 242. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,0.]
35. Occurrences: 229. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.]
36. Occurrences: 220. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
37. Occurrences: 211. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
38. Occurrences: 207. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,0.]
39. Occurrences: 194. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,0.]
40. Occurrences: 185. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.,1.,0.]
41. Occurrences: 183. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,0.]
42. Occurrences: 179. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,0.]
43. Occurrences: 179. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
44. Occurrences: 163. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,0.]
45. Occurrences: 162. Action Sequence: [1.,1.,0.,1.,1.,1.,0.,1.,0.]
46. Occurrences: 160. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,0.]
47. Occurrences: 160. Action Sequence: [1.,1.,1.]
48. Occurrences: 150. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.,1.,0.]
49. Occurrences: 147. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,0.]
50. Occurrences: 143. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,0.,1.,0.]

FACTORED SPACE 1, factorisation 4
8375 different trajectories
1. Occurrences: 16827. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 6757. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 6222. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 3417. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 2597. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 2372. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1928. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1362. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 1185. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 984. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 917. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 850. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 755. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 728. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 666. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 530. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 518. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 474. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 466. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 437. Action Sequence: [1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 428. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 392. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 383. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 344. Action Sequence: [1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 323. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 306. Action Sequence: [1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 294. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 279. Action Sequence: [1.,0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 274. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 269. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 261. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 260. Action Sequence: [1.,0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 257. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 242. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 242. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 223. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 212. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 203. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 201. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 195. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 195. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 192. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 171. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 170. Action Sequence: [1.,1.,0.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 167. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 160. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 160. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 160. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 157. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 152. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 2, factorisation 4
88 different trajectories
1. Occurrences: 20263. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
2. Occurrences: 15130. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 8145. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 6187. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 5873. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 4862. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 4489. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 3949. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 3699. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 3316. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 3042. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 2811. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 2673. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 2453. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 2262. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 2128. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
17. Occurrences: 1994. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.]
18. Occurrences: 1805. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.]
19. Occurrences: 1697. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
20. Occurrences: 1528. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.]
21. Occurrences: 375. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 174. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 113. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 87. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 81. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 79. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 66. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 62. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 59. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 54. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 53. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.]
32. Occurrences: 48. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 45. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 44. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 42. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
36. Occurrences: 41. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.]
37. Occurrences: 39. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
38. Occurrences: 37. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
39. Occurrences: 35. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.]
40. Occurrences: 15. Action Sequence: [0.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
41. Occurrences: 11. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 10. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 9. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
44. Occurrences: 7. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 7. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 7. Action Sequence: [0.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
47. Occurrences: 7. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 5. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 5. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
50. Occurrences: 5. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]

--------------------------------------------------------
'''

### State 703, No Diabetes

In [ ]:
'''
NON FACTORED SPACE
2745 different trajectories
1. Occurrences: 5873. Action Sequence: [7.,7.,7.,0.]
2. Occurrences: 5868. Action Sequence: [7.,7.,0.]
3. Occurrences: 5384. Action Sequence: [7.,7.,7.,7.,0.]
4. Occurrences: 5369. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.]
5. Occurrences: 4771. Action Sequence: [7.,7.,7.,7.,7.,0.]
6. Occurrences: 4158. Action Sequence: [7.,7.,7.,7.,7.,7.,0.]
7. Occurrences: 4090. Action Sequence: [7.,0.]
8. Occurrences: 3684. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,0.]
9. Occurrences: 3062. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,0.]
10. Occurrences: 2614. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
11. Occurrences: 2217. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
12. Occurrences: 1845. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
13. Occurrences: 1647. Action Sequence: [7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
14. Occurrences: 1622. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
15. Occurrences: 1524. Action Sequence: [7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
16. Occurrences: 1445. Action Sequence: [7.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
17. Occurrences: 1410. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
18. Occurrences: 1333. Action Sequence: [7.,7.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
19. Occurrences: 1138. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
20. Occurrences: 1111. Action Sequence: [7.,7.,7.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
21. Occurrences: 1081. Action Sequence: [7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
22. Occurrences: 1002. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
23. Occurrences: 990. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
24. Occurrences: 972. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
25. Occurrences: 818. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
26. Occurrences: 792. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
27. Occurrences: 700. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
28. Occurrences: 693. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
29. Occurrences: 628. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
30. Occurrences: 612. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.]
31. Occurrences: 526. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.,4.]
32. Occurrences: 488. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.,4.]
33. Occurrences: 463. Action Sequence: [7.,7.,7.,0.,7.,0.]
34. Occurrences: 455. Action Sequence: [7.,7.,0.,7.,0.]
35. Occurrences: 402. Action Sequence: [7.,7.,7.,7.,0.,7.,0.]
36. Occurrences: 394. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,4.,4.,4.,4.,4.,4.]
37. Occurrences: 382. Action Sequence: [7.,7.,7.,7.,7.,0.,7.,0.]
38. Occurrences: 306. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,4.,4.,4.,4.,4.]
39. Occurrences: 305. Action Sequence: [7.,7.,7.,7.,7.,7.,0.,7.,0.]
40. Occurrences: 284. Action Sequence: [7.,0.,7.,0.]
41. Occurrences: 278. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,4.,4.,4.,4.]
42. Occurrences: 271. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,7.]
43. Occurrences: 259. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,0.,7.,0.]
44. Occurrences: 259. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,7.,7.,7.,7.,7.,7.,7.]
45. Occurrences: 248. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,7.,7.,7.,7.]
46. Occurrences: 246. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,7.,7.]
47. Occurrences: 238. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,7.,7.,7.,7.,7.,7.,7.,7.,7.]
48. Occurrences: 231. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,4.,4.,4.]
49. Occurrences: 231. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,0.,7.,7.,7.]
50. Occurrences: 228. Action Sequence: [7.,7.,7.,7.,7.,7.,7.,0.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.,7.]

FACTORED SPACE 0, factorisation 1
1859 different trajectories
1. Occurrences: 6249. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 6225. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 5730. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 5369. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 5122. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 4480. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 4303. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 4009. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 3382. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 2903. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 2523. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 2090. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 1935. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 1647. Action Sequence: [1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
15. Occurrences: 1646. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 1524. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
17. Occurrences: 1445. Action Sequence: [1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
18. Occurrences: 1384. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
19. Occurrences: 1333. Action Sequence: [1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
20. Occurrences: 1283. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
21. Occurrences: 1111. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 1081. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 1075. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
24. Occurrences: 991. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
25. Occurrences: 990. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 972. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
27. Occurrences: 942. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
28. Occurrences: 792. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 693. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 628. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
31. Occurrences: 526. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 499. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 488. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.]
34. Occurrences: 482. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 441. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 420. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 394. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 342. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 306. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.]
40. Occurrences: 304. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 300. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 278. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.]
43. Occurrences: 275. Action Sequence: [1.,1.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
44. Occurrences: 274. Action Sequence: [1.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
45. Occurrences: 258. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 255. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
47. Occurrences: 231. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.]
48. Occurrences: 228. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
49. Occurrences: 226. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 226. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 1
2174 different trajectories
1. Occurrences: 5873. Action Sequence: [3.,3.,3.,0.]
2. Occurrences: 5868. Action Sequence: [3.,3.,0.]
3. Occurrences: 5384. Action Sequence: [3.,3.,3.,3.,0.]
4. Occurrences: 5369. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
5. Occurrences: 4771. Action Sequence: [3.,3.,3.,3.,3.,0.]
6. Occurrences: 4158. Action Sequence: [3.,3.,3.,3.,3.,3.,0.]
7. Occurrences: 4090. Action Sequence: [3.,0.]
8. Occurrences: 3684. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.]
9. Occurrences: 3062. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,0.]
10. Occurrences: 2614. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
11. Occurrences: 2217. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
12. Occurrences: 2131. Action Sequence: [3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
13. Occurrences: 2023. Action Sequence: [3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
14. Occurrences: 1875. Action Sequence: [3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
15. Occurrences: 1845. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
16. Occurrences: 1738. Action Sequence: [3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
17. Occurrences: 1622. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
18. Occurrences: 1423. Action Sequence: [3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
19. Occurrences: 1410. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
20. Occurrences: 1387. Action Sequence: [3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
21. Occurrences: 1293. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
22. Occurrences: 1138. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
23. Occurrences: 1033. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
24. Occurrences: 1002. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
25. Occurrences: 972. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
26. Occurrences: 899. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
27. Occurrences: 818. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
28. Occurrences: 813. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
29. Occurrences: 700. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
30. Occurrences: 654. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.]
31. Occurrences: 629. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.]
32. Occurrences: 612. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
33. Occurrences: 508. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.]
34. Occurrences: 463. Action Sequence: [3.,3.,3.,0.,3.,0.]
35. Occurrences: 455. Action Sequence: [3.,3.,0.,3.,0.]
36. Occurrences: 424. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.]
37. Occurrences: 402. Action Sequence: [3.,3.,3.,3.,0.,3.,0.]
38. Occurrences: 382. Action Sequence: [3.,3.,3.,3.,3.,0.,3.,0.]
39. Occurrences: 366. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.]
40. Occurrences: 305. Action Sequence: [3.,3.,3.,3.,3.,3.,0.,3.,0.]
41. Occurrences: 287. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.]
42. Occurrences: 284. Action Sequence: [3.,0.,3.,0.]
43. Occurrences: 271. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.]
44. Occurrences: 259. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.]
45. Occurrences: 259. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.,3.,0.]
46. Occurrences: 248. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.]
47. Occurrences: 246. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.]
48. Occurrences: 240. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.]
49. Occurrences: 238. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
50. Occurrences: 231. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 2
1754 different trajectories
1. Occurrences: 6080. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 6063. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 5600. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 5369. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 4975. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 4379. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 4180. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 3915. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 3291. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 2829. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 2457. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 2131. Action Sequence: [1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
13. Occurrences: 2052. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 2023. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
15. Occurrences: 1882. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 1875. Action Sequence: [1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
17. Occurrences: 1738. Action Sequence: [1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
18. Occurrences: 1609. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 1423. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
20. Occurrences: 1387. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
21. Occurrences: 1347. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
22. Occurrences: 1293. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 1251. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
24. Occurrences: 1051. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
25. Occurrences: 1033. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 972. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
27. Occurrences: 946. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
28. Occurrences: 899. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 883. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
30. Occurrences: 813. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
31. Occurrences: 654. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 629. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.]
33. Occurrences: 508. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.]
34. Occurrences: 487. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 474. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 431. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 424. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 408. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 366. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.]
40. Occurrences: 332. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 296. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 293. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 287. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.]
44. Occurrences: 249. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 240. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.]
46. Occurrences: 222. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 221. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 218. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 218. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 212. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.]

FACTORED SPACE 1, factorisation 2
2579 different trajectories
1. Occurrences: 5873. Action Sequence: [3.,3.,3.,0.]
2. Occurrences: 5868. Action Sequence: [3.,3.,0.]
3. Occurrences: 5384. Action Sequence: [3.,3.,3.,3.,0.]
4. Occurrences: 5369. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
5. Occurrences: 4771. Action Sequence: [3.,3.,3.,3.,3.,0.]
6. Occurrences: 4158. Action Sequence: [3.,3.,3.,3.,3.,3.,0.]
7. Occurrences: 4090. Action Sequence: [3.,0.]
8. Occurrences: 3684. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.]
9. Occurrences: 3062. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,0.]
10. Occurrences: 2614. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
11. Occurrences: 2217. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
12. Occurrences: 1845. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
13. Occurrences: 1647. Action Sequence: [3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
14. Occurrences: 1622. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
15. Occurrences: 1524. Action Sequence: [3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
16. Occurrences: 1445. Action Sequence: [3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
17. Occurrences: 1410. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
18. Occurrences: 1333. Action Sequence: [3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
19. Occurrences: 1138. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
20. Occurrences: 1111. Action Sequence: [3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
21. Occurrences: 1081. Action Sequence: [3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
22. Occurrences: 1002. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
23. Occurrences: 990. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
24. Occurrences: 972. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
25. Occurrences: 818. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
26. Occurrences: 792. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
27. Occurrences: 700. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
28. Occurrences: 693. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
29. Occurrences: 628. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
30. Occurrences: 612. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
31. Occurrences: 526. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.,2.]
32. Occurrences: 488. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.,2.]
33. Occurrences: 463. Action Sequence: [3.,3.,3.,0.,3.,0.]
34. Occurrences: 455. Action Sequence: [3.,3.,0.,3.,0.]
35. Occurrences: 402. Action Sequence: [3.,3.,3.,3.,0.,3.,0.]
36. Occurrences: 394. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.,2.]
37. Occurrences: 382. Action Sequence: [3.,3.,3.,3.,3.,0.,3.,0.]
38. Occurrences: 368. Action Sequence: [3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
39. Occurrences: 349. Action Sequence: [3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
40. Occurrences: 347. Action Sequence: [3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
41. Occurrences: 341. Action Sequence: [3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
42. Occurrences: 330. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.]
43. Occurrences: 322. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
44. Occurrences: 317. Action Sequence: [3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
45. Occurrences: 315. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
46. Occurrences: 312. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.]
47. Occurrences: 306. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,2.,2.,2.,2.,2.]
48. Occurrences: 305. Action Sequence: [3.,3.,3.,3.,3.,3.,0.,3.,0.]
49. Occurrences: 304. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
50. Occurrences: 291. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 3
1851 different trajectories
1. Occurrences: 5873. Action Sequence: [1.,1.,1.,0.]
2. Occurrences: 5868. Action Sequence: [1.,1.,0.]
3. Occurrences: 5384. Action Sequence: [1.,1.,1.,1.,0.]
4. Occurrences: 5369. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 4771. Action Sequence: [1.,1.,1.,1.,1.,0.]
6. Occurrences: 4158. Action Sequence: [1.,1.,1.,1.,1.,1.,0.]
7. Occurrences: 4090. Action Sequence: [1.,0.]
8. Occurrences: 3684. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.]
9. Occurrences: 3062. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.]
10. Occurrences: 2614. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
11. Occurrences: 2330. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 2217. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
13. Occurrences: 2210. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 2086. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 1938. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 1845. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
17. Occurrences: 1639. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 1622. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
19. Occurrences: 1521. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 1473. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 1410. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
22. Occurrences: 1257. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 1138. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
24. Occurrences: 1112. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 1051. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 1002. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
27. Occurrences: 972. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
28. Occurrences: 888. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 859. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 818. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
31. Occurrences: 705. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 700. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
33. Occurrences: 631. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
34. Occurrences: 614. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
35. Occurrences: 612. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
36. Occurrences: 518. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
37. Occurrences: 486. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
38. Occurrences: 483. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
39. Occurrences: 463. Action Sequence: [1.,1.,1.,0.,1.,0.]
40. Occurrences: 455. Action Sequence: [1.,1.,0.,1.,0.]
41. Occurrences: 402. Action Sequence: [1.,1.,1.,1.,0.,1.,0.]
42. Occurrences: 382. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.]
43. Occurrences: 305. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.]
44. Occurrences: 284. Action Sequence: [1.,0.,1.,0.]
45. Occurrences: 259. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
46. Occurrences: 218. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
47. Occurrences: 216. Action Sequence: [1.,1.,0.,1.,1.,0.]
48. Occurrences: 206. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.]
49. Occurrences: 206. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
50. Occurrences: 200. Action Sequence: [1.,1.,1.,0.,1.,1.,0.]

FACTORED SPACE 1, factorisation 3
2325 different trajectories
1. Occurrences: 6080. Action Sequence: [3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
2. Occurrences: 6063. Action Sequence: [3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
3. Occurrences: 5600. Action Sequence: [3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
4. Occurrences: 5369. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
5. Occurrences: 4975. Action Sequence: [3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
6. Occurrences: 4379. Action Sequence: [3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
7. Occurrences: 4180. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
8. Occurrences: 3915. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
9. Occurrences: 3291. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
10. Occurrences: 2829. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
11. Occurrences: 2457. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
12. Occurrences: 2052. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.]
13. Occurrences: 1882. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.]
14. Occurrences: 1647. Action Sequence: [3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
15. Occurrences: 1609. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.,3.]
16. Occurrences: 1524. Action Sequence: [3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
17. Occurrences: 1445. Action Sequence: [3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
18. Occurrences: 1347. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.,3.]
19. Occurrences: 1333. Action Sequence: [3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
20. Occurrences: 1251. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.,3.]
21. Occurrences: 1111. Action Sequence: [3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 1081. Action Sequence: [3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 1051. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.,3.]
24. Occurrences: 990. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
25. Occurrences: 972. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
26. Occurrences: 946. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,3.]
27. Occurrences: 883. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.]
28. Occurrences: 792. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 693. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 628. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
31. Occurrences: 526. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 488. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.,0.]
33. Occurrences: 487. Action Sequence: [3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
34. Occurrences: 474. Action Sequence: [3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
35. Occurrences: 431. Action Sequence: [3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
36. Occurrences: 408. Action Sequence: [3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
37. Occurrences: 394. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 332. Action Sequence: [3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
39. Occurrences: 306. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.,0.]
40. Occurrences: 296. Action Sequence: [3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
41. Occurrences: 293. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
42. Occurrences: 278. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.,0.]
43. Occurrences: 249. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
44. Occurrences: 231. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,0.,0.,0.]
45. Occurrences: 222. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.]
46. Occurrences: 221. Action Sequence: [3.,3.,0.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
47. Occurrences: 218. Action Sequence: [3.,3.,3.,3.,0.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
48. Occurrences: 218. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,3.,3.,3.,3.,3.,3.]
49. Occurrences: 209. Action Sequence: [3.,3.,3.,0.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
50. Occurrences: 205. Action Sequence: [3.,3.,3.,3.,3.,0.,3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 4
1851 different trajectories
1. Occurrences: 5873. Action Sequence: [1.,1.,1.,0.]
2. Occurrences: 5868. Action Sequence: [1.,1.,0.]
3. Occurrences: 5384. Action Sequence: [1.,1.,1.,1.,0.]
4. Occurrences: 5369. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 4771. Action Sequence: [1.,1.,1.,1.,1.,0.]
6. Occurrences: 4158. Action Sequence: [1.,1.,1.,1.,1.,1.,0.]
7. Occurrences: 4090. Action Sequence: [1.,0.]
8. Occurrences: 3684. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.]
9. Occurrences: 3062. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.]
10. Occurrences: 2614. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
11. Occurrences: 2330. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 2217. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
13. Occurrences: 2210. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 2086. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 1938. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 1845. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
17. Occurrences: 1639. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 1622. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
19. Occurrences: 1521. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 1473. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 1410. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
22. Occurrences: 1257. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 1138. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
24. Occurrences: 1112. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 1051. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 1002. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
27. Occurrences: 972. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
28. Occurrences: 888. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 859. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 818. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
31. Occurrences: 705. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 700. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
33. Occurrences: 631. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
34. Occurrences: 614. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
35. Occurrences: 612. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
36. Occurrences: 518. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
37. Occurrences: 486. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
38. Occurrences: 483. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
39. Occurrences: 463. Action Sequence: [1.,1.,1.,0.,1.,0.]
40. Occurrences: 455. Action Sequence: [1.,1.,0.,1.,0.]
41. Occurrences: 402. Action Sequence: [1.,1.,1.,1.,0.,1.,0.]
42. Occurrences: 382. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.]
43. Occurrences: 305. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.]
44. Occurrences: 284. Action Sequence: [1.,0.,1.,0.]
45. Occurrences: 259. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
46. Occurrences: 218. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
47. Occurrences: 216. Action Sequence: [1.,1.,0.,1.,1.,0.]
48. Occurrences: 206. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.]
49. Occurrences: 206. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
50. Occurrences: 200. Action Sequence: [1.,1.,1.,0.,1.,1.,0.]

FACTORED SPACE 1, factorisation 4
1859 different trajectories
1. Occurrences: 6249. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 6225. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 5730. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 5369. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 5122. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 4480. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 4303. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 4009. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 3382. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 2903. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 2523. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 2090. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 1935. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 1647. Action Sequence: [1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
15. Occurrences: 1646. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 1524. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
17. Occurrences: 1445. Action Sequence: [1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
18. Occurrences: 1384. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
19. Occurrences: 1333. Action Sequence: [1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
20. Occurrences: 1283. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
21. Occurrences: 1111. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 1081. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 1075. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
24. Occurrences: 991. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
25. Occurrences: 990. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 972. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
27. Occurrences: 942. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
28. Occurrences: 792. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 693. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 628. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
31. Occurrences: 526. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 499. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 488. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.]
34. Occurrences: 482. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 441. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 420. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 394. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 342. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 306. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.]
40. Occurrences: 304. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 300. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 278. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.]
43. Occurrences: 275. Action Sequence: [1.,1.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
44. Occurrences: 274. Action Sequence: [1.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
45. Occurrences: 258. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 255. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
47. Occurrences: 231. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.]
48. Occurrences: 228. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
49. Occurrences: 226. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 226. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 2, factorisation 4
1754 different trajectories
1. Occurrences: 6080. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 6063. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 5600. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 5369. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 4975. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 4379. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 4180. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 3915. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 3291. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 2829. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 2457. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 2131. Action Sequence: [1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
13. Occurrences: 2052. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 2023. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
15. Occurrences: 1882. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 1875. Action Sequence: [1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
17. Occurrences: 1738. Action Sequence: [1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
18. Occurrences: 1609. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 1423. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
20. Occurrences: 1387. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
21. Occurrences: 1347. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
22. Occurrences: 1293. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 1251. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.]
24. Occurrences: 1051. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.]
25. Occurrences: 1033. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 972. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
27. Occurrences: 946. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
28. Occurrences: 899. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 883. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
30. Occurrences: 813. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
31. Occurrences: 654. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 629. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.]
33. Occurrences: 508. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.]
34. Occurrences: 487. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 474. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 431. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 424. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 408. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 366. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.]
40. Occurrences: 332. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 296. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 293. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 287. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.]
44. Occurrences: 249. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 240. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.]
46. Occurrences: 222. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 221. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 218. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 218. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 212. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,0.]

--------------------------------------------------------
'''

### State 439, Diabetes

In [ ]:
'''
NON FACTORED SPACE
11808 different trajectories
1. Occurrences: 15043. Action Sequence: [6.]
2. Occurrences: 4535. Action Sequence: [6.,6.,0.]
3. Occurrences: 3862. Action Sequence: [6.,6.,6.,0.]
4. Occurrences: 2733. Action Sequence: [6.,6.,6.,6.,0.]
5. Occurrences: 2447. Action Sequence: [6.,0.]
6. Occurrences: 1973. Action Sequence: [6.,6.,6.,6.,6.,0.]
7. Occurrences: 1877. Action Sequence: [6.,6.,0.,6.,0.]
8. Occurrences: 1501. Action Sequence: [6.,6.,6.,0.,6.,0.]
9. Occurrences: 1384. Action Sequence: [6.,6.,6.,6.,6.,6.,0.]
10. Occurrences: 1042. Action Sequence: [6.,6.,6.,6.,0.,6.,0.]
11. Occurrences: 961. Action Sequence: [6.,6.,6.,6.,6.,6.,6.,0.]
12. Occurrences: 956. Action Sequence: [6.,0.,6.,0.]
13. Occurrences: 786. Action Sequence: [6.,6.,6.,6.,6.,0.,6.,0.]
14. Occurrences: 747. Action Sequence: [6.,6.,0.,6.,0.,6.,0.]
15. Occurrences: 696. Action Sequence: [6.,6.,6.,6.,6.,6.,6.,6.,0.]
16. Occurrences: 585. Action Sequence: [6.,6.,0.,6.,6.,0.]
17. Occurrences: 579. Action Sequence: [6.,6.,6.,0.,6.,0.,6.,0.]
18. Occurrences: 553. Action Sequence: [6.,6.,6.,6.,6.,6.,0.,6.,0.]
19. Occurrences: 473. Action Sequence: [6.,6.,6.,0.,6.,6.,0.]
20. Occurrences: 429. Action Sequence: [6.,6.,6.,6.,6.,6.,6.,6.,6.,0.]
21. Occurrences: 429. Action Sequence: [6.,6.,6.,6.,0.,6.,0.,6.,0.]
22. Occurrences: 412. Action Sequence: [6.,6.,6.,6.,6.,6.,6.,0.,6.,0.]
23. Occurrences: 377. Action Sequence: [6.,0.,6.,0.,6.,0.]
24. Occurrences: 358. Action Sequence: [6.,6.,6.,6.,0.,6.,6.,0.]
25. Occurrences: 352. Action Sequence: [6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,0.]
26. Occurrences: 351. Action Sequence: [6.,6.,0.,6.,6.,6.,0.]
27. Occurrences: 326. Action Sequence: [6.,0.,6.,6.,0.]
28. Occurrences: 304. Action Sequence: [6.,6.,6.,6.,6.,0.,6.,0.,6.,0.]
29. Occurrences: 271. Action Sequence: [6.,6.,6.,0.,6.,6.,6.,0.]
30. Occurrences: 258. Action Sequence: [6.,6.,0.,6.,0.,6.,0.,6.,0.]
31. Occurrences: 257. Action Sequence: [6.,6.,6.,6.,6.,6.,6.,6.,0.,6.,0.]
32. Occurrences: 246. Action Sequence: [6.,6.,6.,0.,6.,0.,6.,0.,6.,0.]
33. Occurrences: 245. Action Sequence: [6.,6.,6.,6.,6.,0.,6.,6.,0.]
34. Occurrences: 241. Action Sequence: [6.,6.,0.,6.,6.,6.,6.,0.]
35. Occurrences: 237. Action Sequence: [6.,6.,6.,6.,6.,6.,0.,6.,0.,6.,0.]
36. Occurrences: 236. Action Sequence: [6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,0.]
37. Occurrences: 229. Action Sequence: [6.,6.,0.,6.,6.,0.,6.,0.]
38. Occurrences: 227. Action Sequence: [6.,6.,6.,6.,0.,6.,6.,6.,0.]
39. Occurrences: 212. Action Sequence: [6.,0.,6.,6.,6.,0.]
40. Occurrences: 194. Action Sequence: [6.,6.,0.,6.,0.,6.,6.,0.]
41. Occurrences: 182. Action Sequence: [6.,6.,6.,0.,6.,0.,6.,6.,0.]
42. Occurrences: 177. Action Sequence: [6.,6.,6.,0.,6.,6.,6.,6.,0.]
43. Occurrences: 174. Action Sequence: [6.,6.,6.,0.,6.,6.,0.,6.,0.]
44. Occurrences: 172. Action Sequence: [6.,6.,6.,6.,6.,6.,6.,6.,6.,0.,6.,0.]
45. Occurrences: 168. Action Sequence: [6.,6.,6.,6.,0.,6.,0.,6.,0.,6.,0.]
46. Occurrences: 166. Action Sequence: [6.,6.,6.,6.,6.,6.,0.,6.,6.,0.]
47. Occurrences: 164. Action Sequence: [6.,6.,0.,6.,6.,6.,6.,6.,0.]
48. Occurrences: 164. Action Sequence: [6.,6.,6.,6.,6.,6.,6.,0.,6.,0.,6.,0.]
49. Occurrences: 162. Action Sequence: [6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,0.]
50. Occurrences: 155. Action Sequence: [6.,6.,6.,0.,6.,6.,6.,6.,6.,0.]

FACTORED SPACE 0, factorisation 1
7675 different trajectories
1. Occurrences: 15195. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 4603. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 4008. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2878. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 2471. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 2091. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1914. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1570. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 1495. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 1114. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 1053. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 972. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 846. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 770. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 768. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 609. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 607. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 594. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 505. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 492. Action Sequence: [1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 457. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 456. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 395. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 383. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 372. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 368. Action Sequence: [1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 352. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 330. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 305. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 294. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 288. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 281. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 274. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 270. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 265. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 254. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 253. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 238. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 218. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 210. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 203. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 202. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 198. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 194. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 193. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 191. Action Sequence: [1.,1.,1.,0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 186. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 178. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 177. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 175. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 1
11808 different trajectories
1. Occurrences: 15043. Action Sequence: [2.]
2. Occurrences: 4535. Action Sequence: [2.,2.,0.]
3. Occurrences: 3862. Action Sequence: [2.,2.,2.,0.]
4. Occurrences: 2733. Action Sequence: [2.,2.,2.,2.,0.]
5. Occurrences: 2447. Action Sequence: [2.,0.]
6. Occurrences: 1973. Action Sequence: [2.,2.,2.,2.,2.,0.]
7. Occurrences: 1877. Action Sequence: [2.,2.,0.,2.,0.]
8. Occurrences: 1501. Action Sequence: [2.,2.,2.,0.,2.,0.]
9. Occurrences: 1384. Action Sequence: [2.,2.,2.,2.,2.,2.,0.]
10. Occurrences: 1042. Action Sequence: [2.,2.,2.,2.,0.,2.,0.]
11. Occurrences: 961. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,0.]
12. Occurrences: 956. Action Sequence: [2.,0.,2.,0.]
13. Occurrences: 786. Action Sequence: [2.,2.,2.,2.,2.,0.,2.,0.]
14. Occurrences: 747. Action Sequence: [2.,2.,0.,2.,0.,2.,0.]
15. Occurrences: 696. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,0.]
16. Occurrences: 585. Action Sequence: [2.,2.,0.,2.,2.,0.]
17. Occurrences: 579. Action Sequence: [2.,2.,2.,0.,2.,0.,2.,0.]
18. Occurrences: 553. Action Sequence: [2.,2.,2.,2.,2.,2.,0.,2.,0.]
19. Occurrences: 473. Action Sequence: [2.,2.,2.,0.,2.,2.,0.]
20. Occurrences: 429. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,0.]
21. Occurrences: 429. Action Sequence: [2.,2.,2.,2.,0.,2.,0.,2.,0.]
22. Occurrences: 412. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,0.,2.,0.]
23. Occurrences: 377. Action Sequence: [2.,0.,2.,0.,2.,0.]
24. Occurrences: 358. Action Sequence: [2.,2.,2.,2.,0.,2.,2.,0.]
25. Occurrences: 352. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.]
26. Occurrences: 351. Action Sequence: [2.,2.,0.,2.,2.,2.,0.]
27. Occurrences: 326. Action Sequence: [2.,0.,2.,2.,0.]
28. Occurrences: 304. Action Sequence: [2.,2.,2.,2.,2.,0.,2.,0.,2.,0.]
29. Occurrences: 271. Action Sequence: [2.,2.,2.,0.,2.,2.,2.,0.]
30. Occurrences: 258. Action Sequence: [2.,2.,0.,2.,0.,2.,0.,2.,0.]
31. Occurrences: 257. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,0.,2.,0.]
32. Occurrences: 246. Action Sequence: [2.,2.,2.,0.,2.,0.,2.,0.,2.,0.]
33. Occurrences: 245. Action Sequence: [2.,2.,2.,2.,2.,0.,2.,2.,0.]
34. Occurrences: 241. Action Sequence: [2.,2.,0.,2.,2.,2.,2.,0.]
35. Occurrences: 237. Action Sequence: [2.,2.,2.,2.,2.,2.,0.,2.,0.,2.,0.]
36. Occurrences: 236. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.]
37. Occurrences: 229. Action Sequence: [2.,2.,0.,2.,2.,0.,2.,0.]
38. Occurrences: 227. Action Sequence: [2.,2.,2.,2.,0.,2.,2.,2.,0.]
39. Occurrences: 212. Action Sequence: [2.,0.,2.,2.,2.,0.]
40. Occurrences: 194. Action Sequence: [2.,2.,0.,2.,0.,2.,2.,0.]
41. Occurrences: 182. Action Sequence: [2.,2.,2.,0.,2.,0.,2.,2.,0.]
42. Occurrences: 177. Action Sequence: [2.,2.,2.,0.,2.,2.,2.,2.,0.]
43. Occurrences: 174. Action Sequence: [2.,2.,2.,0.,2.,2.,0.,2.,0.]
44. Occurrences: 172. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,2.,0.]
45. Occurrences: 168. Action Sequence: [2.,2.,2.,2.,0.,2.,0.,2.,0.,2.,0.]
46. Occurrences: 166. Action Sequence: [2.,2.,2.,2.,2.,2.,0.,2.,2.,0.]
47. Occurrences: 164. Action Sequence: [2.,2.,0.,2.,2.,2.,2.,2.,0.]
48. Occurrences: 164. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,0.,2.,0.,2.,0.]
49. Occurrences: 162. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.]
50. Occurrences: 155. Action Sequence: [2.,2.,2.,0.,2.,2.,2.,2.,2.,0.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 2
93 different trajectories
1. Occurrences: 23365. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
2. Occurrences: 15043. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 4958. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 4835. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 4664. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 4543. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 4404. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 4101. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 3837. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 3569. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 3389. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 3057. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 2785. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 2619. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 2447. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 2363. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
17. Occurrences: 2301. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.]
18. Occurrences: 2089. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.]
19. Occurrences: 2024. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
20. Occurrences: 1848. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.]
21. Occurrences: 471. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 105. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 87. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 84. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 84. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 83. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 82. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 82. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 75. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 53. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 52. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 49. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 48. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 47. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
35. Occurrences: 40. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.]
36. Occurrences: 34. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.]
37. Occurrences: 33. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.]
38. Occurrences: 33. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
39. Occurrences: 17. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 13. Action Sequence: [0.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
41. Occurrences: 13. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 13. Action Sequence: [0.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
43. Occurrences: 11. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 11. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 10. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 10. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 9. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 8. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 6. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
50. Occurrences: 6. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]

FACTORED SPACE 1, factorisation 2
10896 different trajectories
1. Occurrences: 15043. Action Sequence: [3.]
2. Occurrences: 4552. Action Sequence: [3.,3.,0.]
3. Occurrences: 3954. Action Sequence: [3.,3.,3.,0.]
4. Occurrences: 2838. Action Sequence: [3.,3.,3.,3.,0.]
5. Occurrences: 2447. Action Sequence: [3.,0.]
6. Occurrences: 2032. Action Sequence: [3.,3.,3.,3.,3.,0.]
7. Occurrences: 1889. Action Sequence: [3.,3.,0.,3.,0.]
8. Occurrences: 1535. Action Sequence: [3.,3.,3.,0.,3.,0.]
9. Occurrences: 1442. Action Sequence: [3.,3.,3.,3.,3.,3.,0.]
10. Occurrences: 1071. Action Sequence: [3.,3.,3.,3.,0.,3.,0.]
11. Occurrences: 997. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.]
12. Occurrences: 956. Action Sequence: [3.,0.,3.,0.]
13. Occurrences: 811. Action Sequence: [3.,3.,3.,3.,3.,0.,3.,0.]
14. Occurrences: 754. Action Sequence: [3.,3.,0.,3.,0.,3.,0.]
15. Occurrences: 721. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,0.]
16. Occurrences: 588. Action Sequence: [3.,3.,3.,0.,3.,0.,3.,0.]
17. Occurrences: 587. Action Sequence: [3.,3.,0.,3.,3.,0.]
18. Occurrences: 571. Action Sequence: [3.,3.,3.,3.,3.,3.,0.,3.,0.]
19. Occurrences: 477. Action Sequence: [3.,3.,3.,0.,3.,3.,0.]
20. Occurrences: 450. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
21. Occurrences: 441. Action Sequence: [3.,3.,3.,3.,0.,3.,0.,3.,0.]
22. Occurrences: 426. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.,3.,0.]
23. Occurrences: 377. Action Sequence: [3.,0.,3.,0.,3.,0.]
24. Occurrences: 365. Action Sequence: [3.,3.,3.,3.,0.,3.,3.,0.]
25. Occurrences: 361. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
26. Occurrences: 355. Action Sequence: [3.,3.,0.,3.,3.,3.,0.]
27. Occurrences: 326. Action Sequence: [3.,0.,3.,3.,0.]
28. Occurrences: 322. Action Sequence: [3.,3.,3.,3.,3.,0.,3.,0.,3.,0.]
29. Occurrences: 277. Action Sequence: [3.,3.,3.,0.,3.,3.,3.,0.]
30. Occurrences: 270. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.]
31. Occurrences: 260. Action Sequence: [3.,3.,0.,3.,0.,3.,0.,3.,0.]
32. Occurrences: 252. Action Sequence: [3.,3.,3.,3.,3.,0.,3.,3.,0.]
33. Occurrences: 252. Action Sequence: [3.,3.,3.,0.,3.,0.,3.,0.,3.,0.]
34. Occurrences: 245. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
35. Occurrences: 244. Action Sequence: [3.,3.,0.,3.,3.,3.,3.,0.]
36. Occurrences: 243. Action Sequence: [3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,0.]
37. Occurrences: 242. Action Sequence: [3.,3.,3.,3.,0.,3.,3.,3.,0.]
38. Occurrences: 229. Action Sequence: [3.,3.,0.,3.,3.,0.,3.,0.]
39. Occurrences: 212. Action Sequence: [3.,0.,3.,3.,3.,0.]
40. Occurrences: 194. Action Sequence: [3.,3.,0.,3.,0.,3.,3.,0.]
41. Occurrences: 187. Action Sequence: [3.,3.,3.,0.,3.,0.,3.,3.,0.]
42. Occurrences: 182. Action Sequence: [3.,3.,3.,0.,3.,3.,3.,3.,0.]
43. Occurrences: 180. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,0.,3.,0.]
44. Occurrences: 178. Action Sequence: [3.,3.,3.,0.,3.,3.,0.,3.,0.]
45. Occurrences: 173. Action Sequence: [3.,3.,3.,3.,0.,3.,0.,3.,0.,3.,0.]
46. Occurrences: 173. Action Sequence: [3.,3.,3.,3.,3.,3.,0.,3.,3.,0.]
47. Occurrences: 165. Action Sequence: [3.,3.,0.,3.,3.,3.,3.,3.,0.]
48. Occurrences: 165. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,0.]
49. Occurrences: 164. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,0.,3.,0.,3.,0.]
50. Occurrences: 161. Action Sequence: [3.,3.,3.,0.,3.,3.,3.,3.,3.,0.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 3
10896 different trajectories
1. Occurrences: 15043. Action Sequence: [1.]
2. Occurrences: 4552. Action Sequence: [1.,1.,0.]
3. Occurrences: 3954. Action Sequence: [1.,1.,1.,0.]
4. Occurrences: 2838. Action Sequence: [1.,1.,1.,1.,0.]
5. Occurrences: 2447. Action Sequence: [1.,0.]
6. Occurrences: 2032. Action Sequence: [1.,1.,1.,1.,1.,0.]
7. Occurrences: 1889. Action Sequence: [1.,1.,0.,1.,0.]
8. Occurrences: 1535. Action Sequence: [1.,1.,1.,0.,1.,0.]
9. Occurrences: 1442. Action Sequence: [1.,1.,1.,1.,1.,1.,0.]
10. Occurrences: 1071. Action Sequence: [1.,1.,1.,1.,0.,1.,0.]
11. Occurrences: 997. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.]
12. Occurrences: 956. Action Sequence: [1.,0.,1.,0.]
13. Occurrences: 811. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.]
14. Occurrences: 754. Action Sequence: [1.,1.,0.,1.,0.,1.,0.]
15. Occurrences: 721. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.]
16. Occurrences: 588. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.]
17. Occurrences: 587. Action Sequence: [1.,1.,0.,1.,1.,0.]
18. Occurrences: 571. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.]
19. Occurrences: 477. Action Sequence: [1.,1.,1.,0.,1.,1.,0.]
20. Occurrences: 450. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
21. Occurrences: 441. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,0.]
22. Occurrences: 426. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
23. Occurrences: 377. Action Sequence: [1.,0.,1.,0.,1.,0.]
24. Occurrences: 365. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.]
25. Occurrences: 361. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
26. Occurrences: 355. Action Sequence: [1.,1.,0.,1.,1.,1.,0.]
27. Occurrences: 326. Action Sequence: [1.,0.,1.,1.,0.]
28. Occurrences: 322. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,0.]
29. Occurrences: 277. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,0.]
30. Occurrences: 270. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
31. Occurrences: 260. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,0.]
32. Occurrences: 252. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,0.]
33. Occurrences: 252. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.,1.,0.]
34. Occurrences: 245. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
35. Occurrences: 244. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,0.]
36. Occurrences: 243. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.]
37. Occurrences: 242. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,0.]
38. Occurrences: 229. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,0.]
39. Occurrences: 212. Action Sequence: [1.,0.,1.,1.,1.,0.]
40. Occurrences: 194. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,0.]
41. Occurrences: 187. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,0.]
42. Occurrences: 182. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,0.]
43. Occurrences: 180. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
44. Occurrences: 178. Action Sequence: [1.,1.,1.,0.,1.,1.,0.,1.,0.]
45. Occurrences: 173. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,0.]
46. Occurrences: 173. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,0.]
47. Occurrences: 165. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,0.]
48. Occurrences: 165. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
49. Occurrences: 164. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.]
50. Occurrences: 161. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,0.]

FACTORED SPACE 1, factorisation 3
11808 different trajectories
1. Occurrences: 15043. Action Sequence: [2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
2. Occurrences: 4535. Action Sequence: [2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
3. Occurrences: 3862. Action Sequence: [2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
4. Occurrences: 2733. Action Sequence: [2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
5. Occurrences: 2447. Action Sequence: [2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
6. Occurrences: 1973. Action Sequence: [2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
7. Occurrences: 1877. Action Sequence: [2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
8. Occurrences: 1501. Action Sequence: [2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
9. Occurrences: 1384. Action Sequence: [2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
10. Occurrences: 1042. Action Sequence: [2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
11. Occurrences: 961. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
12. Occurrences: 956. Action Sequence: [2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
13. Occurrences: 786. Action Sequence: [2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
14. Occurrences: 747. Action Sequence: [2.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
15. Occurrences: 696. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
16. Occurrences: 585. Action Sequence: [2.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
17. Occurrences: 579. Action Sequence: [2.,2.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
18. Occurrences: 553. Action Sequence: [2.,2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
19. Occurrences: 473. Action Sequence: [2.,2.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
20. Occurrences: 429. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
21. Occurrences: 429. Action Sequence: [2.,2.,2.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
22. Occurrences: 412. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
23. Occurrences: 377. Action Sequence: [2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
24. Occurrences: 358. Action Sequence: [2.,2.,2.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
25. Occurrences: 352. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
26. Occurrences: 351. Action Sequence: [2.,2.,0.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
27. Occurrences: 326. Action Sequence: [2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
28. Occurrences: 304. Action Sequence: [2.,2.,2.,2.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
29. Occurrences: 271. Action Sequence: [2.,2.,2.,0.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
30. Occurrences: 258. Action Sequence: [2.,2.,0.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
31. Occurrences: 257. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
32. Occurrences: 246. Action Sequence: [2.,2.,2.,0.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
33. Occurrences: 245. Action Sequence: [2.,2.,2.,2.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
34. Occurrences: 241. Action Sequence: [2.,2.,0.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
35. Occurrences: 237. Action Sequence: [2.,2.,2.,2.,2.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
36. Occurrences: 236. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.]
37. Occurrences: 229. Action Sequence: [2.,2.,0.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
38. Occurrences: 227. Action Sequence: [2.,2.,2.,2.,0.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
39. Occurrences: 212. Action Sequence: [2.,0.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
40. Occurrences: 194. Action Sequence: [2.,2.,0.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
41. Occurrences: 182. Action Sequence: [2.,2.,2.,0.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
42. Occurrences: 177. Action Sequence: [2.,2.,2.,0.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
43. Occurrences: 174. Action Sequence: [2.,2.,2.,0.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
44. Occurrences: 172. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.]
45. Occurrences: 168. Action Sequence: [2.,2.,2.,2.,0.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
46. Occurrences: 166. Action Sequence: [2.,2.,2.,2.,2.,2.,0.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
47. Occurrences: 164. Action Sequence: [2.,2.,0.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
48. Occurrences: 164. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,0.,2.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.]
49. Occurrences: 162. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.]
50. Occurrences: 155. Action Sequence: [2.,2.,2.,0.,2.,2.,2.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 4
10896 different trajectories
1. Occurrences: 15043. Action Sequence: [1.]
2. Occurrences: 4552. Action Sequence: [1.,1.,0.]
3. Occurrences: 3954. Action Sequence: [1.,1.,1.,0.]
4. Occurrences: 2838. Action Sequence: [1.,1.,1.,1.,0.]
5. Occurrences: 2447. Action Sequence: [1.,0.]
6. Occurrences: 2032. Action Sequence: [1.,1.,1.,1.,1.,0.]
7. Occurrences: 1889. Action Sequence: [1.,1.,0.,1.,0.]
8. Occurrences: 1535. Action Sequence: [1.,1.,1.,0.,1.,0.]
9. Occurrences: 1442. Action Sequence: [1.,1.,1.,1.,1.,1.,0.]
10. Occurrences: 1071. Action Sequence: [1.,1.,1.,1.,0.,1.,0.]
11. Occurrences: 997. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.]
12. Occurrences: 956. Action Sequence: [1.,0.,1.,0.]
13. Occurrences: 811. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.]
14. Occurrences: 754. Action Sequence: [1.,1.,0.,1.,0.,1.,0.]
15. Occurrences: 721. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.]
16. Occurrences: 588. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.]
17. Occurrences: 587. Action Sequence: [1.,1.,0.,1.,1.,0.]
18. Occurrences: 571. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.]
19. Occurrences: 477. Action Sequence: [1.,1.,1.,0.,1.,1.,0.]
20. Occurrences: 450. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
21. Occurrences: 441. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,0.]
22. Occurrences: 426. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
23. Occurrences: 377. Action Sequence: [1.,0.,1.,0.,1.,0.]
24. Occurrences: 365. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.]
25. Occurrences: 361. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
26. Occurrences: 355. Action Sequence: [1.,1.,0.,1.,1.,1.,0.]
27. Occurrences: 326. Action Sequence: [1.,0.,1.,1.,0.]
28. Occurrences: 322. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,0.]
29. Occurrences: 277. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,0.]
30. Occurrences: 270. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
31. Occurrences: 260. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,0.]
32. Occurrences: 252. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,0.]
33. Occurrences: 252. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.,1.,0.]
34. Occurrences: 245. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
35. Occurrences: 244. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,0.]
36. Occurrences: 243. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.]
37. Occurrences: 242. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,0.]
38. Occurrences: 229. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,0.]
39. Occurrences: 212. Action Sequence: [1.,0.,1.,1.,1.,0.]
40. Occurrences: 194. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,0.]
41. Occurrences: 187. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,0.]
42. Occurrences: 182. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,0.]
43. Occurrences: 180. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.]
44. Occurrences: 178. Action Sequence: [1.,1.,1.,0.,1.,1.,0.,1.,0.]
45. Occurrences: 173. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,0.]
46. Occurrences: 173. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,0.]
47. Occurrences: 165. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,0.]
48. Occurrences: 165. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
49. Occurrences: 164. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.]
50. Occurrences: 161. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,0.]

FACTORED SPACE 1, factorisation 4
7675 different trajectories
1. Occurrences: 15195. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 4603. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 4008. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2878. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 2471. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 2091. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1914. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1570. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 1495. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 1114. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 1053. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 972. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 846. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 770. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 768. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 609. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 607. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 594. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 505. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 492. Action Sequence: [1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 457. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 456. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 395. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 383. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 372. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 368. Action Sequence: [1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 352. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 330. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 305. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 294. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 288. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 281. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 274. Action Sequence: [1.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 270. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 265. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 254. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 253. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 238. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 218. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 210. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 203. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 202. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 198. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 194. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 193. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 191. Action Sequence: [1.,1.,1.,0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 186. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 178. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 177. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 175. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 2, factorisation 4
93 different trajectories
1. Occurrences: 23365. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
2. Occurrences: 15043. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 4958. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 4835. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 4664. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 4543. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 4404. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 4101. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 3837. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 3569. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 3389. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 3057. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 2785. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 2619. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 2447. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 2363. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
17. Occurrences: 2301. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.]
18. Occurrences: 2089. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.]
19. Occurrences: 2024. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
20. Occurrences: 1848. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.]
21. Occurrences: 471. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 105. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 87. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 84. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 84. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 83. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 82. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 82. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 75. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 53. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 52. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 49. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 48. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 47. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
35. Occurrences: 40. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.]
36. Occurrences: 34. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.]
37. Occurrences: 33. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.]
38. Occurrences: 33. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
39. Occurrences: 17. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 13. Action Sequence: [0.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
41. Occurrences: 13. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 13. Action Sequence: [0.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
43. Occurrences: 11. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 11. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 10. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 10. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 9. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 8. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 6. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
50. Occurrences: 6. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]

--------------------------------------------------------
'''

### State 439 No Diabetes

In [ ]:
'''
NON FACTORED SPACE
16271 different trajectories
1. Occurrences: 15512. Action Sequence: [6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
2. Occurrences: 15013. Action Sequence: [6.]
3. Occurrences: 10909. Action Sequence: [6.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
4. Occurrences: 4662. Action Sequence: [6.,6.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
5. Occurrences: 2254. Action Sequence: [6.,6.,6.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
6. Occurrences: 1139. Action Sequence: [6.,6.,6.,6.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
7. Occurrences: 799. Action Sequence: [6.,4.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
8. Occurrences: 697. Action Sequence: [6.,7.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
9. Occurrences: 689. Action Sequence: [6.,4.,0.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
10. Occurrences: 688. Action Sequence: [6.,4.,0.]
11. Occurrences: 665. Action Sequence: [6.,6.,4.,0.]
12. Occurrences: 630. Action Sequence: [6.,6.,4.,0.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
13. Occurrences: 604. Action Sequence: [6.,6.,6.,6.,6.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
14. Occurrences: 533. Action Sequence: [6.,6.,4.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
15. Occurrences: 529. Action Sequence: [6.,4.,4.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
16. Occurrences: 434. Action Sequence: [6.,4.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
17. Occurrences: 431. Action Sequence: [6.,6.,4.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
18. Occurrences: 404. Action Sequence: [6.,4.,4.,4.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
19. Occurrences: 336. Action Sequence: [6.,4.,0.,6.,4.,0.]
20. Occurrences: 326. Action Sequence: [6.,6.,4.,4.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
21. Occurrences: 325. Action Sequence: [6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.,6.]
22. Occurrences: 305. Action Sequence: [6.,6.,6.,6.,6.,6.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
23. Occurrences: 295. Action Sequence: [6.,6.,6.,4.,0.]
24. Occurrences: 289. Action Sequence: [6.,6.,4.,0.,6.,4.,0.]
25. Occurrences: 259. Action Sequence: [6.,4.,4.,4.,4.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
26. Occurrences: 257. Action Sequence: [6.,6.,6.,4.,0.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
27. Occurrences: 254. Action Sequence: [6.,4.,0.,6.,4.,0.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
28. Occurrences: 248. Action Sequence: [6.,6.,4.,4.,4.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
29. Occurrences: 241. Action Sequence: [6.,4.,4.,0.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
30. Occurrences: 233. Action Sequence: [6.,6.,4.,0.,6.,4.,0.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
31. Occurrences: 208. Action Sequence: [6.,7.,7.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
32. Occurrences: 204. Action Sequence: [6.,4.,0.,6.,0.]
33. Occurrences: 201. Action Sequence: [6.,4.,4.,4.,4.,4.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
34. Occurrences: 199. Action Sequence: [6.,4.,6.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
35. Occurrences: 192. Action Sequence: [6.,6.,4.,0.,6.,4.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
36. Occurrences: 181. Action Sequence: [6.,6.,4.,4.,4.,4.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
37. Occurrences: 179. Action Sequence: [6.,6.,6.,4.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
38. Occurrences: 178. Action Sequence: [6.,6.,4.,0.,6.,0.]
39. Occurrences: 177. Action Sequence: [6.,4.,4.,0.]
40. Occurrences: 177. Action Sequence: [6.,6.,6.,4.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
41. Occurrences: 176. Action Sequence: [6.,4.,0.,6.,4.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
42. Occurrences: 169. Action Sequence: [6.,4.,4.,4.,0.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
43. Occurrences: 157. Action Sequence: [6.,4.,4.,0.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
44. Occurrences: 152. Action Sequence: [6.,4.,4.,4.,4.,4.,4.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
45. Occurrences: 142. Action Sequence: [6.,6.,0.]
46. Occurrences: 142. Action Sequence: [6.,6.,6.,6.,6.,6.,6.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
47. Occurrences: 134. Action Sequence: [6.,6.,6.,6.,4.,0.]
48. Occurrences: 133. Action Sequence: [6.,4.,4.,6.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]
49. Occurrences: 131. Action Sequence: [6.,4.,4.,0.,6.,4.,0.]
50. Occurrences: 122. Action Sequence: [6.,6.,4.,4.,4.,0.,6.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.,4.]

FACTORED SPACE 0, factorisation 1
9686 different trajectories
1. Occurrences: 16496. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
2. Occurrences: 15410. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 11740. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
4. Occurrences: 5733. Action Sequence: [1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
5. Occurrences: 2637. Action Sequence: [1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
6. Occurrences: 1491. Action Sequence: [1.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
7. Occurrences: 1307. Action Sequence: [1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
8. Occurrences: 1281. Action Sequence: [1.,1.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
9. Occurrences: 872. Action Sequence: [1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
10. Occurrences: 775. Action Sequence: [1.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
11. Occurrences: 721. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 716. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 674. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
14. Occurrences: 591. Action Sequence: [1.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
15. Occurrences: 557. Action Sequence: [1.,1.,1.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
16. Occurrences: 519. Action Sequence: [1.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
17. Occurrences: 454. Action Sequence: [1.,1.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
18. Occurrences: 420. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
19. Occurrences: 394. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 386. Action Sequence: [1.,1.,0.,0.,1.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
21. Occurrences: 355. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 355. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 351. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
24. Occurrences: 345. Action Sequence: [1.,1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 341. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 297. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
27. Occurrences: 254. Action Sequence: [1.,1.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
28. Occurrences: 249. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 248. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 242. Action Sequence: [1.,0.,0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
31. Occurrences: 222. Action Sequence: [1.,0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 220. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 219. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
34. Occurrences: 207. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
35. Occurrences: 193. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
36. Occurrences: 190. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 182. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 180. Action Sequence: [1.,0.,0.,0.,1.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
39. Occurrences: 178. Action Sequence: [1.,1.,1.,0.,0.,1.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
40. Occurrences: 169. Action Sequence: [1.,1.,1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 168. Action Sequence: [1.,0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 167. Action Sequence: [1.,1.,0.,0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
43. Occurrences: 159. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
44. Occurrences: 157. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
45. Occurrences: 156. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 153. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 145. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 143. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
49. Occurrences: 141. Action Sequence: [1.,1.,0.,0.,1.,0.,0.,1.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
50. Occurrences: 138. Action Sequence: [1.,0.,0.,0.,0.,1.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]

FACTORED SPACE 1, factorisation 1
5839 different trajectories
1. Occurrences: 45251. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
2. Occurrences: 15013. Action Sequence: [2.]
3. Occurrences: 1716. Action Sequence: [2.,2.,2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
4. Occurrences: 1359. Action Sequence: [2.,2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
5. Occurrences: 1217. Action Sequence: [2.,2.,2.,2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
6. Occurrences: 1002. Action Sequence: [2.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
7. Occurrences: 967. Action Sequence: [2.,2.,2.,0.]
8. Occurrences: 913. Action Sequence: [2.,2.,2.,2.,2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
9. Occurrences: 830. Action Sequence: [2.,2.,0.]
10. Occurrences: 736. Action Sequence: [2.,2.,2.,2.,2.,2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
11. Occurrences: 665. Action Sequence: [2.,2.,2.,0.,2.,2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
12. Occurrences: 660. Action Sequence: [2.,2.,2.,2.,0.]
13. Occurrences: 579. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
14. Occurrences: 501. Action Sequence: [2.,2.,2.,2.,2.,0.]
15. Occurrences: 500. Action Sequence: [2.,2.,2.,2.,0.,2.,2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
16. Occurrences: 497. Action Sequence: [2.,2.,0.,2.,2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
17. Occurrences: 464. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
18. Occurrences: 431. Action Sequence: [2.,2.,2.,0.,2.,2.,0.]
19. Occurrences: 388. Action Sequence: [2.,2.,2.,2.,2.,0.,2.,2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
20. Occurrences: 387. Action Sequence: [2.,2.,2.,2.,2.,2.,0.]
21. Occurrences: 342. Action Sequence: [2.,2.,0.,2.,2.,0.]
22. Occurrences: 322. Action Sequence: [2.,2.,2.,2.,0.,2.,2.,0.]
23. Occurrences: 319. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
24. Occurrences: 300. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,0.]
25. Occurrences: 293. Action Sequence: [2.,2.,2.,2.,2.,2.,0.,2.,2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
26. Occurrences: 284. Action Sequence: [2.,3.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
27. Occurrences: 270. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
28. Occurrences: 258. Action Sequence: [2.,2.,2.,2.,2.,0.,2.,2.,0.]
29. Occurrences: 251. Action Sequence: [2.,2.,2.,0.,2.,0.]
30. Occurrences: 247. Action Sequence: [2.,2.,2.,0.,2.,2.,0.,2.,2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
31. Occurrences: 244. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,0.,2.,2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
32. Occurrences: 226. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,0.]
33. Occurrences: 222. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,2.,2.,2.,2.,2.,2.,2.,2.]
34. Occurrences: 204. Action Sequence: [2.,2.,0.,2.,0.]
35. Occurrences: 201. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,0.,2.,2.,0.,2.,2.,2.,2.,2.,2.,2.,2.]
36. Occurrences: 194. Action Sequence: [2.,2.,2.,2.,0.,2.,0.]
37. Occurrences: 187. Action Sequence: [2.,2.,0.,2.,2.,0.,2.,2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
38. Occurrences: 177. Action Sequence: [2.,2.,2.,2.,2.,2.,0.,2.,2.,0.]
39. Occurrences: 173. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,2.,2.,2.,2.,2.,2.,2.]
40. Occurrences: 173. Action Sequence: [2.,2.,2.,2.,0.,2.,2.,0.,2.,2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
41. Occurrences: 164. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,2.,2.,2.,2.,2.,2.]
42. Occurrences: 162. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.]
43. Occurrences: 160. Action Sequence: [2.,2.,2.,0.,2.,2.,0.,2.,2.,0.]
44. Occurrences: 160. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,2.,2.]
45. Occurrences: 157. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,0.]
46. Occurrences: 154. Action Sequence: [2.,2.,2.,2.,2.,0.,2.,2.,0.,2.,2.,0.,2.,2.,2.,2.,2.,2.,2.,2.]
47. Occurrences: 153. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,0.,2.,2.,0.]
48. Occurrences: 153. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.]
49. Occurrences: 146. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,2.,2.,0.,2.,2.]
50. Occurrences: 146. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,0.,2.,2.,0.,2.,2.,2.,2.,2.,2.,2.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 2
1615 different trajectories
1. Occurrences: 62318. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
2. Occurrences: 15013. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 1477. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
4. Occurrences: 1147. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1110. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1012. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 938. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 916. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 911. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 875. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 807. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 756. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 643. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 558. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 541. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 456. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
17. Occurrences: 428. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
18. Occurrences: 397. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.]
19. Occurrences: 344. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.]
20. Occurrences: 294. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.]
21. Occurrences: 287. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
22. Occurrences: 210. Action Sequence: [0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 192. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
24. Occurrences: 184. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
25. Occurrences: 175. Action Sequence: [0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 172. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
27. Occurrences: 169. Action Sequence: [0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
28. Occurrences: 158. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 152. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 136. Action Sequence: [0.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
31. Occurrences: 113. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 104. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.]
33. Occurrences: 102. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.]
34. Occurrences: 97. Action Sequence: [0.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
35. Occurrences: 94. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.]
36. Occurrences: 89. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.]
37. Occurrences: 79. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.]
38. Occurrences: 66. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.]
39. Occurrences: 63. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.]
40. Occurrences: 55. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 49. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 44. Action Sequence: [0.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
43. Occurrences: 43. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 41. Action Sequence: [0.,0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 41. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 39. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 39. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 36. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 36. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 34. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 2
14002 different trajectories
1. Occurrences: 15512. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
2. Occurrences: 15013. Action Sequence: [3.]
3. Occurrences: 10932. Action Sequence: [3.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
4. Occurrences: 5364. Action Sequence: [3.,3.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
5. Occurrences: 2464. Action Sequence: [3.,3.,3.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
6. Occurrences: 1240. Action Sequence: [3.,3.,3.,3.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
7. Occurrences: 799. Action Sequence: [3.,2.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
8. Occurrences: 695. Action Sequence: [3.,3.,2.,0.]
9. Occurrences: 692. Action Sequence: [3.,2.,0.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
10. Occurrences: 688. Action Sequence: [3.,2.,0.]
11. Occurrences: 655. Action Sequence: [3.,3.,2.,0.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
12. Occurrences: 646. Action Sequence: [3.,3.,3.,3.,3.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
13. Occurrences: 535. Action Sequence: [3.,3.,2.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
14. Occurrences: 529. Action Sequence: [3.,2.,2.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
15. Occurrences: 445. Action Sequence: [3.,3.,2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
16. Occurrences: 434. Action Sequence: [3.,2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
17. Occurrences: 404. Action Sequence: [3.,2.,2.,2.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
18. Occurrences: 397. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
19. Occurrences: 341. Action Sequence: [3.,2.,0.,3.,2.,0.]
20. Occurrences: 337. Action Sequence: [3.,3.,3.,2.,0.]
21. Occurrences: 326. Action Sequence: [3.,3.,2.,2.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
22. Occurrences: 324. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
23. Occurrences: 302. Action Sequence: [3.,3.,2.,0.,3.,2.,0.]
24. Occurrences: 292. Action Sequence: [3.,3.,3.,2.,0.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
25. Occurrences: 263. Action Sequence: [3.,2.,0.,3.,2.,0.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
26. Occurrences: 259. Action Sequence: [3.,2.,2.,2.,2.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
27. Occurrences: 248. Action Sequence: [3.,3.,2.,2.,2.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
28. Occurrences: 246. Action Sequence: [3.,3.,2.,0.,3.,2.,0.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
29. Occurrences: 242. Action Sequence: [3.,2.,2.,0.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
30. Occurrences: 228. Action Sequence: [3.,2.,0.,3.,0.]
31. Occurrences: 218. Action Sequence: [3.,3.,3.,2.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
32. Occurrences: 202. Action Sequence: [3.,3.,2.,0.,3.,2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
33. Occurrences: 201. Action Sequence: [3.,2.,2.,2.,2.,2.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
34. Occurrences: 199. Action Sequence: [3.,2.,3.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
35. Occurrences: 199. Action Sequence: [3.,3.,2.,0.,3.,0.]
36. Occurrences: 195. Action Sequence: [3.,3.,3.,2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
37. Occurrences: 181. Action Sequence: [3.,3.,2.,2.,2.,2.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
38. Occurrences: 179. Action Sequence: [3.,2.,0.,3.,2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
39. Occurrences: 177. Action Sequence: [3.,2.,2.,0.]
40. Occurrences: 171. Action Sequence: [3.,2.,2.,2.,0.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
41. Occurrences: 157. Action Sequence: [3.,2.,2.,0.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
42. Occurrences: 152. Action Sequence: [3.,2.,2.,2.,2.,2.,2.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
43. Occurrences: 148. Action Sequence: [3.,3.,3.,3.,2.,0.]
44. Occurrences: 148. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
45. Occurrences: 147. Action Sequence: [3.,3.,3.,2.,0.,3.,2.,0.]
46. Occurrences: 142. Action Sequence: [3.,3.,0.]
47. Occurrences: 137. Action Sequence: [3.,3.,3.,2.,2.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
48. Occurrences: 136. Action Sequence: [3.,2.,2.,0.,3.,2.,0.]
49. Occurrences: 133. Action Sequence: [3.,2.,2.,3.,3.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
50. Occurrences: 131. Action Sequence: [3.,3.,3.,0.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 3
2029 different trajectories
1. Occurrences: 46877. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 15013. Action Sequence: [1.]
3. Occurrences: 1924. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 1462. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1428. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1056. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1024. Action Sequence: [1.,1.,1.,0.]
8. Occurrences: 868. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 830. Action Sequence: [1.,1.,0.]
10. Occurrences: 777. Action Sequence: [1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 740. Action Sequence: [1.,1.,1.,1.,0.]
12. Occurrences: 699. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 607. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 562. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 562. Action Sequence: [1.,1.,1.,1.,1.,0.]
16. Occurrences: 557. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 479. Action Sequence: [1.,1.,1.,0.,1.,1.,0.]
18. Occurrences: 467. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 444. Action Sequence: [1.,1.,1.,1.,1.,1.,0.]
20. Occurrences: 401. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 388. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.]
22. Occurrences: 378. Action Sequence: [1.,1.,0.,1.,1.,0.]
23. Occurrences: 360. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 358. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.]
25. Occurrences: 330. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 307. Action Sequence: [1.,1.,1.,0.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 304. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 291. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,0.]
29. Occurrences: 285. Action Sequence: [1.,1.,1.,0.,1.,0.]
30. Occurrences: 282. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.]
31. Occurrences: 280. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 252. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 235. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 232. Action Sequence: [1.,1.,0.,1.,0.]
35. Occurrences: 226. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 224. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 223. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 216. Action Sequence: [1.,1.,1.,1.,0.,1.,0.]
39. Occurrences: 212. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,0.]
40. Occurrences: 211. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
41. Occurrences: 204. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
42. Occurrences: 192. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
43. Occurrences: 192. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 183. Action Sequence: [1.,1.,1.,0.,1.,1.,0.,1.,1.,0.]
45. Occurrences: 180. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
46. Occurrences: 179. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,0.]
47. Occurrences: 178. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 177. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
49. Occurrences: 175. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
50. Occurrences: 169. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.]

FACTORED SPACE 1, factorisation 3
12733 different trajectories
1. Occurrences: 16496. Action Sequence: [2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
2. Occurrences: 15013. Action Sequence: [2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
3. Occurrences: 11703. Action Sequence: [2.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
4. Occurrences: 4975. Action Sequence: [2.,2.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
5. Occurrences: 2397. Action Sequence: [2.,2.,2.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
6. Occurrences: 1483. Action Sequence: [2.,0.,0.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
7. Occurrences: 1239. Action Sequence: [2.,2.,0.,0.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
8. Occurrences: 1192. Action Sequence: [2.,2.,2.,2.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
9. Occurrences: 870. Action Sequence: [2.,0.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
10. Occurrences: 772. Action Sequence: [2.,0.,0.,0.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
11. Occurrences: 742. Action Sequence: [2.,3.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
12. Occurrences: 704. Action Sequence: [2.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
13. Occurrences: 682. Action Sequence: [2.,2.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
14. Occurrences: 627. Action Sequence: [2.,2.,2.,2.,2.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
15. Occurrences: 588. Action Sequence: [2.,2.,0.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
16. Occurrences: 515. Action Sequence: [2.,0.,0.,0.,0.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
17. Occurrences: 467. Action Sequence: [2.,2.,2.,0.,0.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
18. Occurrences: 453. Action Sequence: [2.,2.,0.,0.,0.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
19. Occurrences: 405. Action Sequence: [2.,0.,0.,2.,0.,0.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
20. Occurrences: 383. Action Sequence: [2.,0.,0.,2.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
21. Occurrences: 364. Action Sequence: [2.,2.,0.,0.,2.,0.,0.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 350. Action Sequence: [2.,0.,0.,0.,0.,0.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 341. Action Sequence: [2.,2.,0.,0.,0.,0.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
24. Occurrences: 325. Action Sequence: [2.,2.,0.,0.,2.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
25. Occurrences: 325. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.,2.]
26. Occurrences: 322. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
27. Occurrences: 307. Action Sequence: [2.,2.,2.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
28. Occurrences: 297. Action Sequence: [2.,0.,0.,0.,0.,0.,0.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 228. Action Sequence: [2.,3.,3.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 222. Action Sequence: [2.,0.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
31. Occurrences: 222. Action Sequence: [2.,0.,2.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 218. Action Sequence: [2.,2.,2.,2.,0.,0.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
33. Occurrences: 215. Action Sequence: [2.,2.,0.,0.,0.,0.,0.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
34. Occurrences: 213. Action Sequence: [2.,0.,0.,2.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
35. Occurrences: 211. Action Sequence: [2.,2.,2.,0.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
36. Occurrences: 206. Action Sequence: [2.,0.,0.,0.,0.,0.,0.,0.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
37. Occurrences: 195. Action Sequence: [2.,2.,0.,0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
38. Occurrences: 187. Action Sequence: [2.,0.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
39. Occurrences: 180. Action Sequence: [2.,2.,0.,0.,0.,0.,0.,0.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
40. Occurrences: 175. Action Sequence: [2.,0.,0.,0.,2.,0.,0.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
41. Occurrences: 160. Action Sequence: [2.,0.,0.,0.,2.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
42. Occurrences: 155. Action Sequence: [2.,0.,0.,0.,0.,0.,0.,0.,0.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
43. Occurrences: 154. Action Sequence: [2.,2.,2.,0.,0.,0.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
44. Occurrences: 151. Action Sequence: [2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
45. Occurrences: 149. Action Sequence: [2.,2.,2.,0.,0.,2.,0.,0.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
46. Occurrences: 146. Action Sequence: [2.,2.,2.,2.,2.,2.,2.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
47. Occurrences: 137. Action Sequence: [2.,0.,0.,0.,0.,2.,0.,0.,2.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
48. Occurrences: 136. Action Sequence: [2.,2.,2.,2.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
49. Occurrences: 136. Action Sequence: [2.,2.,2.,0.,0.,2.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
50. Occurrences: 133. Action Sequence: [2.,0.,0.,2.,0.,0.,2.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 4
2029 different trajectories
1. Occurrences: 46877. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 15013. Action Sequence: [1.]
3. Occurrences: 1924. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 1462. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1428. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1056. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1024. Action Sequence: [1.,1.,1.,0.]
8. Occurrences: 868. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 830. Action Sequence: [1.,1.,0.]
10. Occurrences: 777. Action Sequence: [1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 740. Action Sequence: [1.,1.,1.,1.,0.]
12. Occurrences: 699. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 607. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 562. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 562. Action Sequence: [1.,1.,1.,1.,1.,0.]
16. Occurrences: 557. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 479. Action Sequence: [1.,1.,1.,0.,1.,1.,0.]
18. Occurrences: 467. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 444. Action Sequence: [1.,1.,1.,1.,1.,1.,0.]
20. Occurrences: 401. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 388. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.]
22. Occurrences: 378. Action Sequence: [1.,1.,0.,1.,1.,0.]
23. Occurrences: 360. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 358. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.]
25. Occurrences: 330. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 307. Action Sequence: [1.,1.,1.,0.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 304. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 291. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,0.]
29. Occurrences: 285. Action Sequence: [1.,1.,1.,0.,1.,0.]
30. Occurrences: 282. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.]
31. Occurrences: 280. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 252. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 235. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 232. Action Sequence: [1.,1.,0.,1.,0.]
35. Occurrences: 226. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 224. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 223. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 216. Action Sequence: [1.,1.,1.,1.,0.,1.,0.]
39. Occurrences: 212. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,1.,1.,0.]
40. Occurrences: 211. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.]
41. Occurrences: 204. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
42. Occurrences: 192. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
43. Occurrences: 192. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 183. Action Sequence: [1.,1.,1.,0.,1.,1.,0.,1.,1.,0.]
45. Occurrences: 180. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.]
46. Occurrences: 179. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,0.]
47. Occurrences: 178. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 177. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.]
49. Occurrences: 175. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.]
50. Occurrences: 169. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,0.,1.,1.,0.,1.,1.]

FACTORED SPACE 1, factorisation 4
9686 different trajectories
1. Occurrences: 16496. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
2. Occurrences: 15410. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 11740. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
4. Occurrences: 5733. Action Sequence: [1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
5. Occurrences: 2637. Action Sequence: [1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
6. Occurrences: 1491. Action Sequence: [1.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
7. Occurrences: 1307. Action Sequence: [1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
8. Occurrences: 1281. Action Sequence: [1.,1.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
9. Occurrences: 872. Action Sequence: [1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
10. Occurrences: 775. Action Sequence: [1.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
11. Occurrences: 721. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 716. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 674. Action Sequence: [1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
14. Occurrences: 591. Action Sequence: [1.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
15. Occurrences: 557. Action Sequence: [1.,1.,1.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
16. Occurrences: 519. Action Sequence: [1.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
17. Occurrences: 454. Action Sequence: [1.,1.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
18. Occurrences: 420. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
19. Occurrences: 394. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 386. Action Sequence: [1.,1.,0.,0.,1.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
21. Occurrences: 355. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
22. Occurrences: 355. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 351. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
24. Occurrences: 345. Action Sequence: [1.,1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 341. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 297. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
27. Occurrences: 254. Action Sequence: [1.,1.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
28. Occurrences: 249. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 248. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 242. Action Sequence: [1.,0.,0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
31. Occurrences: 222. Action Sequence: [1.,0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 220. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 219. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
34. Occurrences: 207. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
35. Occurrences: 193. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
36. Occurrences: 190. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 182. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
38. Occurrences: 180. Action Sequence: [1.,0.,0.,0.,1.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
39. Occurrences: 178. Action Sequence: [1.,1.,1.,0.,0.,1.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
40. Occurrences: 169. Action Sequence: [1.,1.,1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 168. Action Sequence: [1.,0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 167. Action Sequence: [1.,1.,0.,0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
43. Occurrences: 159. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
44. Occurrences: 157. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
45. Occurrences: 156. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 153. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 145. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 143. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
49. Occurrences: 141. Action Sequence: [1.,1.,0.,0.,1.,0.,0.,1.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
50. Occurrences: 138. Action Sequence: [1.,0.,0.,0.,0.,1.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]

FACTORED SPACE 2, factorisation 4
1615 different trajectories
1. Occurrences: 62318. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
2. Occurrences: 15013. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 1477. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
4. Occurrences: 1147. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1110. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1012. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 938. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 916. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 911. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 875. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 807. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 756. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 643. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 558. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 541. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 456. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.]
17. Occurrences: 428. Action Sequence: [0.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
18. Occurrences: 397. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.]
19. Occurrences: 344. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.]
20. Occurrences: 294. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.]
21. Occurrences: 287. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.]
22. Occurrences: 210. Action Sequence: [0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
23. Occurrences: 192. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
24. Occurrences: 184. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
25. Occurrences: 175. Action Sequence: [0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
26. Occurrences: 172. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
27. Occurrences: 169. Action Sequence: [0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
28. Occurrences: 158. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
29. Occurrences: 152. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
30. Occurrences: 136. Action Sequence: [0.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
31. Occurrences: 113. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.]
32. Occurrences: 104. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.]
33. Occurrences: 102. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.,0.,0.]
34. Occurrences: 97. Action Sequence: [0.,1.,0.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
35. Occurrences: 94. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.,0.]
36. Occurrences: 89. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.]
37. Occurrences: 79. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.]
38. Occurrences: 66. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.]
39. Occurrences: 63. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,0.,0.,0.]
40. Occurrences: 55. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 49. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 44. Action Sequence: [0.,1.,1.,1.,1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
43. Occurrences: 43. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 41. Action Sequence: [0.,0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 41. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 39. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 39. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 36. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 36. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 34. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.]

--------------------------------------------------------
'''

## Record Data for Random Policy (top 50 Trajectories over 100000 episodes)

### State 136 With Diabetes

In [ ]:
'''
NON FACTORED SPACE
21429 different trajectories
1. Occurrences: 6314. Action Sequence: [5.]
2. Occurrences: 5777. Action Sequence: [1.]
3. Occurrences: 5614. Action Sequence: [7.]
4. Occurrences: 4951. Action Sequence: [3.]
5. Occurrences: 2326. Action Sequence: [0.]
6. Occurrences: 1328. Action Sequence: [2.]
7. Occurrences: 1024. Action Sequence: [6.,1.]
8. Occurrences: 972. Action Sequence: [6.,3.]
9. Occurrences: 962. Action Sequence: [6.,5.]
10. Occurrences: 942. Action Sequence: [4.,1.]
11. Occurrences: 917. Action Sequence: [4.,3.]
12. Occurrences: 912. Action Sequence: [4.,5.]
13. Occurrences: 887. Action Sequence: [6.,7.]
14. Occurrences: 803. Action Sequence: [4.,7.]
15. Occurrences: 775. Action Sequence: [4.]
16. Occurrences: 761. Action Sequence: [2.,5.]
17. Occurrences: 725. Action Sequence: [0.,1.]
18. Occurrences: 720. Action Sequence: [0.,5.]
19. Occurrences: 663. Action Sequence: [0.,7.]
20. Occurrences: 661. Action Sequence: [2.,1.]
21. Occurrences: 659. Action Sequence: [2.,7.]
22. Occurrences: 638. Action Sequence: [0.,3.]
23. Occurrences: 620. Action Sequence: [2.,3.]
24. Occurrences: 495. Action Sequence: [5.,1.]
25. Occurrences: 492. Action Sequence: [5.,5.]
26. Occurrences: 476. Action Sequence: [7.,7.]
27. Occurrences: 473. Action Sequence: [7.,5.]
28. Occurrences: 472. Action Sequence: [7.,1.]
29. Occurrences: 458. Action Sequence: [7.,2.]
30. Occurrences: 457. Action Sequence: [3.,3.]
31. Occurrences: 440. Action Sequence: [7.,3.]
32. Occurrences: 439. Action Sequence: [3.,5.]
33. Occurrences: 439. Action Sequence: [1.,5.]
34. Occurrences: 435. Action Sequence: [5.,3.]
35. Occurrences: 424. Action Sequence: [3.,1.]
36. Occurrences: 419. Action Sequence: [7.,0.]
37. Occurrences: 414. Action Sequence: [5.,7.]
38. Occurrences: 411. Action Sequence: [3.,0.]
39. Occurrences: 410. Action Sequence: [1.,1.]
40. Occurrences: 396. Action Sequence: [5.,0.]
41. Occurrences: 389. Action Sequence: [3.,7.]
42. Occurrences: 372. Action Sequence: [5.,2.]
43. Occurrences: 369. Action Sequence: [1.,0.]
44. Occurrences: 364. Action Sequence: [1.,7.]
45. Occurrences: 361. Action Sequence: [1.,3.]
46. Occurrences: 351. Action Sequence: [3.,2.]
47. Occurrences: 322. Action Sequence: [2.,0.]
48. Occurrences: 311. Action Sequence: [0.,0.]
49. Occurrences: 296. Action Sequence: [1.,2.]
50. Occurrences: 245. Action Sequence: [5.,4.]

FACTORED SPACE 0, factorisation 1
4134 different trajectories
1. Occurrences: 24038. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
2. Occurrences: 21481. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
3. Occurrences: 10024. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
4. Occurrences: 9766. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
5. Occurrences: 3253. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
6. Occurrences: 3222. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
7. Occurrences: 3049. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
8. Occurrences: 2863. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
9. Occurrences: 996. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
10. Occurrences: 980. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
11. Occurrences: 979. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
12. Occurrences: 977. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
13. Occurrences: 953. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
14. Occurrences: 901. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
15. Occurrences: 901. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
16. Occurrences: 838. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
17. Occurrences: 339. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
18. Occurrences: 329. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
19. Occurrences: 314. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
20. Occurrences: 312. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
21. Occurrences: 312. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
22. Occurrences: 282. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
23. Occurrences: 277. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
24. Occurrences: 269. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
25. Occurrences: 268. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
26. Occurrences: 263. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
27. Occurrences: 257. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
28. Occurrences: 256. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
29. Occurrences: 248. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
30. Occurrences: 248. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
31. Occurrences: 241. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
32. Occurrences: 230. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
33. Occurrences: 103. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
34. Occurrences: 103. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
35. Occurrences: 102. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
36. Occurrences: 102. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
37. Occurrences: 101. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
38. Occurrences: 100. Action Sequence: [0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
39. Occurrences: 98. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
40. Occurrences: 98. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
41. Occurrences: 94. Action Sequence: [0.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
42. Occurrences: 94. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
43. Occurrences: 92. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
44. Occurrences: 91. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
45. Occurrences: 90. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
46. Occurrences: 88. Action Sequence: [0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
47. Occurrences: 87. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
48. Occurrences: 86. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
49. Occurrences: 86. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
50. Occurrences: 85. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]

FACTORED SPACE 1, factorisation 1
11833 different trajectories
1. Occurrences: 11928. Action Sequence: [3.]
(pi_b, pi_e): [(0.25, 0.0)]
2. Occurrences: 10728. Action Sequence: [1.]
(pi_b, pi_e): [(0.25, 1.0)]
3. Occurrences: 3855. Action Sequence: [2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0)]
4. Occurrences: 3654. Action Sequence: [0.]
(pi_b, pi_e): [(0.25, 0.0)]
5. Occurrences: 3564. Action Sequence: [2.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
6. Occurrences: 2803. Action Sequence: [0.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
7. Occurrences: 2644. Action Sequence: [0.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
8. Occurrences: 1855. Action Sequence: [3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
9. Occurrences: 1842. Action Sequence: [3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0)]
10. Occurrences: 1652. Action Sequence: [1.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
11. Occurrences: 1645. Action Sequence: [3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
12. Occurrences: 1631. Action Sequence: [1.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
13. Occurrences: 1427. Action Sequence: [1.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
14. Occurrences: 1038. Action Sequence: [0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
15. Occurrences: 978. Action Sequence: [2.,2.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
16. Occurrences: 943. Action Sequence: [2.,2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
17. Occurrences: 884. Action Sequence: [2.,0.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
18. Occurrences: 866. Action Sequence: [3.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
19. Occurrences: 816. Action Sequence: [1.,2.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
20. Occurrences: 775. Action Sequence: [2.]
(pi_b, pi_e): [(0.25, 0.0)]
21. Occurrences: 757. Action Sequence: [0.,2.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
22. Occurrences: 731. Action Sequence: [0.,2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
23. Occurrences: 723. Action Sequence: [2.,0.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
24. Occurrences: 622. Action Sequence: [2.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
25. Occurrences: 549. Action Sequence: [0.,0.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
26. Occurrences: 494. Action Sequence: [0.,0.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
27. Occurrences: 388. Action Sequence: [3.,2.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
28. Occurrences: 378. Action Sequence: [0.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
29. Occurrences: 344. Action Sequence: [3.,2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0)]
30. Occurrences: 342. Action Sequence: [2.,3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0)]
31. Occurrences: 340. Action Sequence: [1.,2.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
32. Occurrences: 338. Action Sequence: [2.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
33. Occurrences: 334. Action Sequence: [1.,2.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
34. Occurrences: 311. Action Sequence: [0.,3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0)]
35. Occurrences: 305. Action Sequence: [2.,1.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
36. Occurrences: 301. Action Sequence: [2.,1.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
37. Occurrences: 287. Action Sequence: [2.,3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
38. Occurrences: 275. Action Sequence: [0.,1.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
39. Occurrences: 265. Action Sequence: [0.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
40. Occurrences: 254. Action Sequence: [0.,3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
41. Occurrences: 244. Action Sequence: [2.,0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
42. Occurrences: 243. Action Sequence: [2.,0.,2.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
43. Occurrences: 240. Action Sequence: [2.,0.,2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
44. Occurrences: 232. Action Sequence: [0.,1.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
45. Occurrences: 230. Action Sequence: [2.,1.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
46. Occurrences: 227. Action Sequence: [0.,0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
47. Occurrences: 223. Action Sequence: [3.,0.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0)]
48. Occurrences: 222. Action Sequence: [3.,3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0)]
49. Occurrences: 220. Action Sequence: [2.,2.,2.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
50. Occurrences: 219. Action Sequence: [3.,0.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 2
3878 different trajectories
1. Occurrences: 31687. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
2. Occurrences: 20401. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
3. Occurrences: 9557. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
4. Occurrences: 7554. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
5. Occurrences: 4366. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
6. Occurrences: 2652. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
7. Occurrences: 2031. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
8. Occurrences: 1993. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
9. Occurrences: 1896. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
10. Occurrences: 1035. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
11. Occurrences: 870. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
12. Occurrences: 786. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
13. Occurrences: 752. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
14. Occurrences: 660. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
15. Occurrences: 479. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
16. Occurrences: 461. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
17. Occurrences: 426. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
18. Occurrences: 409. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
19. Occurrences: 399. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
20. Occurrences: 316. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
21. Occurrences: 297. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
22. Occurrences: 280. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
23. Occurrences: 238. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
24. Occurrences: 185. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
25. Occurrences: 180. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
26. Occurrences: 165. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
27. Occurrences: 164. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
28. Occurrences: 159. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
29. Occurrences: 158. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
30. Occurrences: 151. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
31. Occurrences: 140. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
32. Occurrences: 131. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
33. Occurrences: 126. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
34. Occurrences: 126. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
35. Occurrences: 120. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
36. Occurrences: 119. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
37. Occurrences: 114. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
38. Occurrences: 108. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
39. Occurrences: 106. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
40. Occurrences: 91. Action Sequence: [0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
41. Occurrences: 81. Action Sequence: [0.,0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
42. Occurrences: 78. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
43. Occurrences: 77. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
44. Occurrences: 74. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
45. Occurrences: 72. Action Sequence: [0.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
46. Occurrences: 71. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
47. Occurrences: 71. Action Sequence: [0.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
48. Occurrences: 70. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
49. Occurrences: 58. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
50. Occurrences: 57. Action Sequence: [0.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]

FACTORED SPACE 1, factorisation 2
12652 different trajectories
1. Occurrences: 8103. Action Sequence: [0.]
(pi_b, pi_e): [(0.25, 0.0)]
2. Occurrences: 7089. Action Sequence: [2.]
(pi_b, pi_e): [(0.25, 0.0)]
3. Occurrences: 6279. Action Sequence: [1.]
(pi_b, pi_e): [(0.25, 1.0)]
4. Occurrences: 5614. Action Sequence: [3.]
(pi_b, pi_e): [(0.25, 0.0)]
5. Occurrences: 2097. Action Sequence: [3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
6. Occurrences: 2036. Action Sequence: [2.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
7. Occurrences: 1977. Action Sequence: [3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0)]
8. Occurrences: 1854. Action Sequence: [2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
9. Occurrences: 1818. Action Sequence: [1.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
10. Occurrences: 1815. Action Sequence: [0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
11. Occurrences: 1774. Action Sequence: [3.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
12. Occurrences: 1758. Action Sequence: [2.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
13. Occurrences: 1651. Action Sequence: [1.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
14. Occurrences: 1576. Action Sequence: [3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
15. Occurrences: 1564. Action Sequence: [1.,2.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
16. Occurrences: 1528. Action Sequence: [0.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
17. Occurrences: 1477. Action Sequence: [0.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
18. Occurrences: 1391. Action Sequence: [2.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
19. Occurrences: 1294. Action Sequence: [1.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
20. Occurrences: 1242. Action Sequence: [0.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
21. Occurrences: 360. Action Sequence: [3.,3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
22. Occurrences: 356. Action Sequence: [1.,3.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
23. Occurrences: 346. Action Sequence: [3.,2.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
24. Occurrences: 345. Action Sequence: [3.,2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0)]
25. Occurrences: 335. Action Sequence: [3.,3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0)]
26. Occurrences: 330. Action Sequence: [2.,3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
27. Occurrences: 324. Action Sequence: [3.,0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
28. Occurrences: 320. Action Sequence: [2.,1.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
29. Occurrences: 318. Action Sequence: [1.,2.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
30. Occurrences: 313. Action Sequence: [3.,2.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
31. Occurrences: 311. Action Sequence: [2.,3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
32. Occurrences: 311. Action Sequence: [3.,3.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
33. Occurrences: 307. Action Sequence: [2.,2.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
34. Occurrences: 304. Action Sequence: [1.,1.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
35. Occurrences: 304. Action Sequence: [2.,3.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
36. Occurrences: 295. Action Sequence: [2.,2.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
37. Occurrences: 294. Action Sequence: [3.,1.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
38. Occurrences: 293. Action Sequence: [2.,1.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 1.0)]
39. Occurrences: 292. Action Sequence: [3.,0.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
40. Occurrences: 291. Action Sequence: [1.,3.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 1.0)]
41. Occurrences: 291. Action Sequence: [3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0)]
42. Occurrences: 289. Action Sequence: [2.,2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0)]
43. Occurrences: 287. Action Sequence: [3.,1.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 1.0)]
44. Occurrences: 285. Action Sequence: [0.,2.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
45. Occurrences: 283. Action Sequence: [0.,3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
46. Occurrences: 283. Action Sequence: [3.,1.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
47. Occurrences: 281. Action Sequence: [0.,0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
48. Occurrences: 281. Action Sequence: [0.,2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
49. Occurrences: 280. Action Sequence: [1.,0.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
50. Occurrences: 278. Action Sequence: [3.,1.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 3
4989 different trajectories
1. Occurrences: 14382. Action Sequence: [0.]
(pi_b, pi_e): [(0.5, 1.0)]
2. Occurrences: 12703. Action Sequence: [1.]
(pi_b, pi_e): [(0.5, 0.0)]
3. Occurrences: 7964. Action Sequence: [1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0)]
4. Occurrences: 6761. Action Sequence: [0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0)]
5. Occurrences: 6499. Action Sequence: [1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0)]
6. Occurrences: 5628. Action Sequence: [0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0)]
7. Occurrences: 2611. Action Sequence: [1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
8. Occurrences: 2333. Action Sequence: [0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
9. Occurrences: 2303. Action Sequence: [1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
10. Occurrences: 2255. Action Sequence: [1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
11. Occurrences: 2041. Action Sequence: [1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
12. Occurrences: 2017. Action Sequence: [0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
13. Occurrences: 1892. Action Sequence: [0.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
14. Occurrences: 1514. Action Sequence: [0.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
15. Occurrences: 854. Action Sequence: [1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
16. Occurrences: 800. Action Sequence: [1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
17. Occurrences: 772. Action Sequence: [0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
18. Occurrences: 749. Action Sequence: [1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
19. Occurrences: 722. Action Sequence: [1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
20. Occurrences: 690. Action Sequence: [0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
21. Occurrences: 654. Action Sequence: [1.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
22. Occurrences: 653. Action Sequence: [1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
23. Occurrences: 647. Action Sequence: [0.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
24. Occurrences: 625. Action Sequence: [1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
25. Occurrences: 615. Action Sequence: [0.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
26. Occurrences: 609. Action Sequence: [0.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
27. Occurrences: 549. Action Sequence: [0.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
28. Occurrences: 483. Action Sequence: [1.,0.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
29. Occurrences: 481. Action Sequence: [0.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
30. Occurrences: 433. Action Sequence: [0.,0.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
31. Occurrences: 263. Action Sequence: [0.,1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
32. Occurrences: 252. Action Sequence: [0.,1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
33. Occurrences: 247. Action Sequence: [1.,1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
34. Occurrences: 240. Action Sequence: [1.,1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
35. Occurrences: 236. Action Sequence: [1.,1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
36. Occurrences: 223. Action Sequence: [1.,1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
37. Occurrences: 219. Action Sequence: [0.,1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
38. Occurrences: 216. Action Sequence: [1.,0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
39. Occurrences: 213. Action Sequence: [0.,1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
40. Occurrences: 210. Action Sequence: [1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
41. Occurrences: 207. Action Sequence: [1.,0.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
42. Occurrences: 206. Action Sequence: [0.,0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
43. Occurrences: 201. Action Sequence: [1.,0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
44. Occurrences: 201. Action Sequence: [1.,1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
45. Occurrences: 200. Action Sequence: [0.,0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
46. Occurrences: 198. Action Sequence: [1.,1.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
47. Occurrences: 197. Action Sequence: [0.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
48. Occurrences: 186. Action Sequence: [0.,0.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
49. Occurrences: 184. Action Sequence: [0.,0.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
50. Occurrences: 182. Action Sequence: [0.,1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]

FACTORED SPACE 1, factorisation 3
10610 different trajectories
1. Occurrences: 13872. Action Sequence: [1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
2. Occurrences: 12572. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
3. Occurrences: 6451. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
4. Occurrences: 4841. Action Sequence: [2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
5. Occurrences: 3729. Action Sequence: [2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
6. Occurrences: 3579. Action Sequence: [0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
7. Occurrences: 2036. Action Sequence: [1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
8. Occurrences: 1996. Action Sequence: [3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
9. Occurrences: 1596. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
10. Occurrences: 1550. Action Sequence: [3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
11. Occurrences: 1533. Action Sequence: [2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
12. Occurrences: 1526. Action Sequence: [0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
13. Occurrences: 1502. Action Sequence: [1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
14. Occurrences: 1284. Action Sequence: [1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
15. Occurrences: 1224. Action Sequence: [2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
16. Occurrences: 1189. Action Sequence: [0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
17. Occurrences: 890. Action Sequence: [2.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
18. Occurrences: 859. Action Sequence: [2.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
19. Occurrences: 809. Action Sequence: [0.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
20. Occurrences: 806. Action Sequence: [0.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
21. Occurrences: 409. Action Sequence: [2.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
22. Occurrences: 359. Action Sequence: [3.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
23. Occurrences: 352. Action Sequence: [0.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
24. Occurrences: 350. Action Sequence: [2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
25. Occurrences: 349. Action Sequence: [0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
26. Occurrences: 346. Action Sequence: [0.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
27. Occurrences: 338. Action Sequence: [2.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
28. Occurrences: 337. Action Sequence: [3.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
29. Occurrences: 335. Action Sequence: [2.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
30. Occurrences: 319. Action Sequence: [0.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
31. Occurrences: 298. Action Sequence: [2.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
32. Occurrences: 292. Action Sequence: [1.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
33. Occurrences: 291. Action Sequence: [1.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
34. Occurrences: 287. Action Sequence: [0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
35. Occurrences: 284. Action Sequence: [0.,0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
36. Occurrences: 274. Action Sequence: [2.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
37. Occurrences: 271. Action Sequence: [2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
38. Occurrences: 271. Action Sequence: [2.,0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
39. Occurrences: 269. Action Sequence: [0.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 1.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
40. Occurrences: 260. Action Sequence: [3.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
41. Occurrences: 251. Action Sequence: [2.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
42. Occurrences: 235. Action Sequence: [0.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
43. Occurrences: 234. Action Sequence: [0.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
44. Occurrences: 234. Action Sequence: [1.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
45. Occurrences: 228. Action Sequence: [3.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
46. Occurrences: 227. Action Sequence: [2.,1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
47. Occurrences: 226. Action Sequence: [0.,1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
48. Occurrences: 222. Action Sequence: [1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
49. Occurrences: 217. Action Sequence: [0.,2.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
50. Occurrences: 209. Action Sequence: [3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 4
4989 different trajectories
1. Occurrences: 14382. Action Sequence: [0.]
(pi_b, pi_e): [(0.5, 1.0)]
2. Occurrences: 12703. Action Sequence: [1.]
(pi_b, pi_e): [(0.5, 0.0)]
3. Occurrences: 7964. Action Sequence: [1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0)]
4. Occurrences: 6761. Action Sequence: [0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0)]
5. Occurrences: 6499. Action Sequence: [1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0)]
6. Occurrences: 5628. Action Sequence: [0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0)]
7. Occurrences: 2611. Action Sequence: [1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
8. Occurrences: 2333. Action Sequence: [0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
9. Occurrences: 2303. Action Sequence: [1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
10. Occurrences: 2255. Action Sequence: [1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
11. Occurrences: 2041. Action Sequence: [1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
12. Occurrences: 2017. Action Sequence: [0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
13. Occurrences: 1892. Action Sequence: [0.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
14. Occurrences: 1514. Action Sequence: [0.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
15. Occurrences: 854. Action Sequence: [1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
16. Occurrences: 800. Action Sequence: [1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
17. Occurrences: 772. Action Sequence: [0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
18. Occurrences: 749. Action Sequence: [1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
19. Occurrences: 722. Action Sequence: [1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
20. Occurrences: 690. Action Sequence: [0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
21. Occurrences: 654. Action Sequence: [1.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
22. Occurrences: 653. Action Sequence: [1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
23. Occurrences: 647. Action Sequence: [0.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
24. Occurrences: 625. Action Sequence: [1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
25. Occurrences: 615. Action Sequence: [0.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
26. Occurrences: 609. Action Sequence: [0.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
27. Occurrences: 549. Action Sequence: [0.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
28. Occurrences: 483. Action Sequence: [1.,0.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
29. Occurrences: 481. Action Sequence: [0.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
30. Occurrences: 433. Action Sequence: [0.,0.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
31. Occurrences: 263. Action Sequence: [0.,1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
32. Occurrences: 252. Action Sequence: [0.,1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
33. Occurrences: 247. Action Sequence: [1.,1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
34. Occurrences: 240. Action Sequence: [1.,1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
35. Occurrences: 236. Action Sequence: [1.,1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
36. Occurrences: 223. Action Sequence: [1.,1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
37. Occurrences: 219. Action Sequence: [0.,1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
38. Occurrences: 216. Action Sequence: [1.,0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
39. Occurrences: 213. Action Sequence: [0.,1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
40. Occurrences: 210. Action Sequence: [1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
41. Occurrences: 207. Action Sequence: [1.,0.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
42. Occurrences: 206. Action Sequence: [0.,0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
43. Occurrences: 201. Action Sequence: [1.,0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
44. Occurrences: 201. Action Sequence: [1.,1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
45. Occurrences: 200. Action Sequence: [0.,0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
46. Occurrences: 198. Action Sequence: [1.,1.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
47. Occurrences: 197. Action Sequence: [0.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
48. Occurrences: 186. Action Sequence: [0.,0.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
49. Occurrences: 184. Action Sequence: [0.,0.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
50. Occurrences: 182. Action Sequence: [0.,1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]

FACTORED SPACE 1, factorisation 4
4134 different trajectories
1. Occurrences: 24038. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
2. Occurrences: 21481. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
3. Occurrences: 10024. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
4. Occurrences: 9766. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
5. Occurrences: 3253. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
6. Occurrences: 3222. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
7. Occurrences: 3049. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
8. Occurrences: 2863. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
9. Occurrences: 996. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
10. Occurrences: 980. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
11. Occurrences: 979. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
12. Occurrences: 977. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
13. Occurrences: 953. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
14. Occurrences: 901. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
15. Occurrences: 901. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
16. Occurrences: 838. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
17. Occurrences: 339. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
18. Occurrences: 329. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
19. Occurrences: 314. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
20. Occurrences: 312. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
21. Occurrences: 312. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
22. Occurrences: 282. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
23. Occurrences: 277. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
24. Occurrences: 269. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
25. Occurrences: 268. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
26. Occurrences: 263. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
27. Occurrences: 257. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
28. Occurrences: 256. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
29. Occurrences: 248. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
30. Occurrences: 248. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
31. Occurrences: 241. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
32. Occurrences: 230. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
33. Occurrences: 103. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
34. Occurrences: 103. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
35. Occurrences: 102. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
36. Occurrences: 102. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
37. Occurrences: 101. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
38. Occurrences: 100. Action Sequence: [0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
39. Occurrences: 98. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
40. Occurrences: 98. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
41. Occurrences: 94. Action Sequence: [0.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
42. Occurrences: 94. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
43. Occurrences: 92. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
44. Occurrences: 91. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
45. Occurrences: 90. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
46. Occurrences: 88. Action Sequence: [0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
47. Occurrences: 87. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
48. Occurrences: 86. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
49. Occurrences: 86. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
50. Occurrences: 85. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]

FACTORED SPACE 2, factorisation 4
3878 different trajectories
1. Occurrences: 31687. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
2. Occurrences: 20401. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
3. Occurrences: 9557. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
4. Occurrences: 7554. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
5. Occurrences: 4366. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
6. Occurrences: 2652. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
7. Occurrences: 2031. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
8. Occurrences: 1993. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
9. Occurrences: 1896. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
10. Occurrences: 1035. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
11. Occurrences: 870. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
12. Occurrences: 786. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
13. Occurrences: 752. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
14. Occurrences: 660. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
15. Occurrences: 479. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
16. Occurrences: 461. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
17. Occurrences: 426. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
18. Occurrences: 409. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
19. Occurrences: 399. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
20. Occurrences: 316. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
21. Occurrences: 297. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
22. Occurrences: 280. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
23. Occurrences: 238. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
24. Occurrences: 185. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
25. Occurrences: 180. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
26. Occurrences: 165. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
27. Occurrences: 164. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
28. Occurrences: 159. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
29. Occurrences: 158. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
30. Occurrences: 151. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
31. Occurrences: 140. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
32. Occurrences: 131. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
33. Occurrences: 126. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
34. Occurrences: 126. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
35. Occurrences: 120. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
36. Occurrences: 119. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
37. Occurrences: 114. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
38. Occurrences: 108. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
39. Occurrences: 106. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
40. Occurrences: 91. Action Sequence: [0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
41. Occurrences: 81. Action Sequence: [0.,0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
42. Occurrences: 78. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
43. Occurrences: 77. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
44. Occurrences: 74. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
45. Occurrences: 72. Action Sequence: [0.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
46. Occurrences: 71. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
47. Occurrences: 71. Action Sequence: [0.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
48. Occurrences: 70. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
49. Occurrences: 58. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
50. Occurrences: 57. Action Sequence: [0.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]

--------------------------------------------------------
'''

### State 136 Without Diabetes

In [ ]:
'''
NON FACTORED SPACE
69072 different trajectories
1. Occurrences: 1541. Action Sequence: [0.]
2. Occurrences: 841. Action Sequence: [5.]
3. Occurrences: 796. Action Sequence: [1.]
4. Occurrences: 416. Action Sequence: [2.]
5. Occurrences: 373. Action Sequence: [4.,1.]
6. Occurrences: 349. Action Sequence: [6.,1.]
7. Occurrences: 326. Action Sequence: [4.,5.]
8. Occurrences: 313. Action Sequence: [3.,0.]
9. Occurrences: 304. Action Sequence: [2.,0.]
10. Occurrences: 302. Action Sequence: [5.,0.]
11. Occurrences: 300. Action Sequence: [1.,0.]
12. Occurrences: 297. Action Sequence: [6.,5.]
13. Occurrences: 286. Action Sequence: [7.,0.]
14. Occurrences: 285. Action Sequence: [0.,0.]
15. Occurrences: 275. Action Sequence: [0.,5.]
16. Occurrences: 245. Action Sequence: [0.,1.]
17. Occurrences: 244. Action Sequence: [4.]
18. Occurrences: 241. Action Sequence: [6.,3.]
19. Occurrences: 234. Action Sequence: [4.,3.]
20. Occurrences: 230. Action Sequence: [2.,5.]
21. Occurrences: 230. Action Sequence: [7.,2.]
22. Occurrences: 227. Action Sequence: [2.,1.]
23. Occurrences: 226. Action Sequence: [6.,7.]
24. Occurrences: 197. Action Sequence: [7.,4.]
25. Occurrences: 195. Action Sequence: [5.,2.]
26. Occurrences: 195. Action Sequence: [4.,7.]
27. Occurrences: 181. Action Sequence: [5.,4.]
28. Occurrences: 172. Action Sequence: [5.,1.]
29. Occurrences: 168. Action Sequence: [1.,4.]
30. Occurrences: 166. Action Sequence: [2.,7.]
31. Occurrences: 164. Action Sequence: [3.,2.]
32. Occurrences: 157. Action Sequence: [3.,4.]
33. Occurrences: 152. Action Sequence: [0.,3.]
34. Occurrences: 149. Action Sequence: [1.,2.]
35. Occurrences: 144. Action Sequence: [0.,7.]
36. Occurrences: 144. Action Sequence: [5.,5.]
37. Occurrences: 140. Action Sequence: [7.,1.]
38. Occurrences: 138. Action Sequence: [1.,1.]
39. Occurrences: 138. Action Sequence: [4.,0.]
40. Occurrences: 133. Action Sequence: [2.,3.]
41. Occurrences: 131. Action Sequence: [5.,6.]
42. Occurrences: 127. Action Sequence: [1.,5.]
43. Occurrences: 121. Action Sequence: [0.,2.]
44. Occurrences: 120. Action Sequence: [7.,6.]
45. Occurrences: 117. Action Sequence: [3.,6.]
46. Occurrences: 116. Action Sequence: [3.,1.]
47. Occurrences: 115. Action Sequence: [6.,0.]
48. Occurrences: 107. Action Sequence: [3.,5.]
49. Occurrences: 105. Action Sequence: [0.,4.]
50. Occurrences: 103. Action Sequence: [7.,5.]

FACTORED SPACE 0, factorisation 1
24979 different trajectories
1. Occurrences: 6677. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
2. Occurrences: 4910. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
3. Occurrences: 4841. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
4. Occurrences: 3785. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
5. Occurrences: 2560. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
6. Occurrences: 2387. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
7. Occurrences: 2379. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
8. Occurrences: 2355. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
9. Occurrences: 1202. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
10. Occurrences: 1113. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
11. Occurrences: 1080. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
12. Occurrences: 1078. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
13. Occurrences: 1054. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
14. Occurrences: 1028. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
15. Occurrences: 1020. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
16. Occurrences: 983. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
17. Occurrences: 567. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
18. Occurrences: 532. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
19. Occurrences: 530. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
20. Occurrences: 501. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
21. Occurrences: 497. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
22. Occurrences: 493. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
23. Occurrences: 486. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
24. Occurrences: 485. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
25. Occurrences: 476. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
26. Occurrences: 447. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
27. Occurrences: 445. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
28. Occurrences: 442. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
29. Occurrences: 429. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
30. Occurrences: 418. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
31. Occurrences: 407. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
32. Occurrences: 403. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
33. Occurrences: 258. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
34. Occurrences: 237. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
35. Occurrences: 236. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
36. Occurrences: 236. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
37. Occurrences: 228. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
38. Occurrences: 227. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
39. Occurrences: 227. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
40. Occurrences: 222. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
41. Occurrences: 220. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
42. Occurrences: 220. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
43. Occurrences: 214. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
44. Occurrences: 213. Action Sequence: [0.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
45. Occurrences: 213. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
46. Occurrences: 213. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
47. Occurrences: 212. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
48. Occurrences: 208. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
49. Occurrences: 203. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
50. Occurrences: 202. Action Sequence: [0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]

FACTORED SPACE 1, factorisation 1
54151 different trajectories
1. Occurrences: 1957. Action Sequence: [0.]
(pi_b, pi_e): [(0.25, 0.0)]
2. Occurrences: 1197. Action Sequence: [2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
3. Occurrences: 1044. Action Sequence: [2.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
4. Occurrences: 1013. Action Sequence: [3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
5. Occurrences: 926. Action Sequence: [1.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
6. Occurrences: 841. Action Sequence: [3.]
(pi_b, pi_e): [(0.25, 0.0)]
7. Occurrences: 815. Action Sequence: [0.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
8. Occurrences: 803. Action Sequence: [0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
9. Occurrences: 796. Action Sequence: [1.]
(pi_b, pi_e): [(0.25, 1.0)]
10. Occurrences: 757. Action Sequence: [0.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
11. Occurrences: 629. Action Sequence: [3.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
12. Occurrences: 537. Action Sequence: [1.,2.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
13. Occurrences: 423. Action Sequence: [2.,2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
14. Occurrences: 419. Action Sequence: [2.,2.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
15. Occurrences: 415. Action Sequence: [2.,0.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
16. Occurrences: 380. Action Sequence: [0.,2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
17. Occurrences: 379. Action Sequence: [2.,0.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
18. Occurrences: 364. Action Sequence: [2.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
19. Occurrences: 323. Action Sequence: [3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0)]
20. Occurrences: 322. Action Sequence: [0.,2.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
21. Occurrences: 284. Action Sequence: [0.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
22. Occurrences: 264. Action Sequence: [1.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0)]
23. Occurrences: 263. Action Sequence: [3.,1.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
24. Occurrences: 262. Action Sequence: [1.,3.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 1.0)]
25. Occurrences: 260. Action Sequence: [0.,0.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
26. Occurrences: 252. Action Sequence: [3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
27. Occurrences: 251. Action Sequence: [3.,3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
28. Occurrences: 245. Action Sequence: [1.,1.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 0.0)]
29. Occurrences: 244. Action Sequence: [2.]
(pi_b, pi_e): [(0.25, 0.0)]
30. Occurrences: 242. Action Sequence: [1.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
31. Occurrences: 234. Action Sequence: [0.,0.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
32. Occurrences: 234. Action Sequence: [3.,0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
33. Occurrences: 230. Action Sequence: [2.,3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
34. Occurrences: 221. Action Sequence: [0.,3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
35. Occurrences: 209. Action Sequence: [3.,2.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
36. Occurrences: 203. Action Sequence: [2.,1.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
37. Occurrences: 202. Action Sequence: [3.,2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
38. Occurrences: 200. Action Sequence: [1.,0.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
39. Occurrences: 198. Action Sequence: [0.,0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
40. Occurrences: 189. Action Sequence: [0.,1.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
41. Occurrences: 189. Action Sequence: [2.,0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
42. Occurrences: 185. Action Sequence: [1.,2.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
43. Occurrences: 170. Action Sequence: [3.,1.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
44. Occurrences: 168. Action Sequence: [3.,3.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
45. Occurrences: 164. Action Sequence: [3.,2.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
46. Occurrences: 159. Action Sequence: [1.,3.,2.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
47. Occurrences: 151. Action Sequence: [1.,1.,2.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 0.0)]
48. Occurrences: 148. Action Sequence: [1.,2.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
49. Occurrences: 141. Action Sequence: [2.,2.,2.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
50. Occurrences: 140. Action Sequence: [2.,2.,2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 1.0), (0.25, 0.0)]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 2
24596 different trajectories
1. Occurrences: 7318. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
2. Occurrences: 5129. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
3. Occurrences: 4805. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
4. Occurrences: 3690. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
5. Occurrences: 2865. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
6. Occurrences: 2342. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
7. Occurrences: 2328. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
8. Occurrences: 2151. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
9. Occurrences: 1492. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
10. Occurrences: 1081. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
11. Occurrences: 1071. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
12. Occurrences: 992. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
13. Occurrences: 991. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
14. Occurrences: 983. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
15. Occurrences: 967. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
16. Occurrences: 952. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
17. Occurrences: 732. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
18. Occurrences: 571. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
19. Occurrences: 557. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
20. Occurrences: 498. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
21. Occurrences: 479. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
22. Occurrences: 462. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
23. Occurrences: 441. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
24. Occurrences: 439. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
25. Occurrences: 437. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
26. Occurrences: 435. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
27. Occurrences: 426. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
28. Occurrences: 418. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
29. Occurrences: 415. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
30. Occurrences: 410. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
31. Occurrences: 402. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
32. Occurrences: 385. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
33. Occurrences: 300. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
34. Occurrences: 275. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
35. Occurrences: 271. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
36. Occurrences: 251. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
37. Occurrences: 236. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
38. Occurrences: 234. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
39. Occurrences: 224. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
40. Occurrences: 219. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
41. Occurrences: 216. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
42. Occurrences: 207. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
43. Occurrences: 205. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
44. Occurrences: 205. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
45. Occurrences: 203. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
46. Occurrences: 198. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
47. Occurrences: 197. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
48. Occurrences: 197. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
49. Occurrences: 195. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
50. Occurrences: 194. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]

FACTORED SPACE 1, factorisation 2
54552 different trajectories
1. Occurrences: 2337. Action Sequence: [0.]
(pi_b, pi_e): [(0.25, 0.0)]
2. Occurrences: 1085. Action Sequence: [2.]
(pi_b, pi_e): [(0.25, 0.0)]
3. Occurrences: 985. Action Sequence: [2.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
4. Occurrences: 968. Action Sequence: [0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
5. Occurrences: 960. Action Sequence: [1.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
6. Occurrences: 890. Action Sequence: [3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
7. Occurrences: 713. Action Sequence: [2.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
8. Occurrences: 675. Action Sequence: [0.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
9. Occurrences: 643. Action Sequence: [3.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
10. Occurrences: 586. Action Sequence: [1.,2.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
11. Occurrences: 527. Action Sequence: [3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0)]
12. Occurrences: 495. Action Sequence: [2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
13. Occurrences: 432. Action Sequence: [0.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
14. Occurrences: 416. Action Sequence: [1.]
(pi_b, pi_e): [(0.25, 1.0)]
15. Occurrences: 390. Action Sequence: [1.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
16. Occurrences: 346. Action Sequence: [3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0)]
17. Occurrences: 336. Action Sequence: [2.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
18. Occurrences: 331. Action Sequence: [1.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0)]
19. Occurrences: 286. Action Sequence: [0.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0)]
20. Occurrences: 276. Action Sequence: [2.,1.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
21. Occurrences: 269. Action Sequence: [3.,3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
22. Occurrences: 267. Action Sequence: [3.,1.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
23. Occurrences: 257. Action Sequence: [1.,3.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
24. Occurrences: 251. Action Sequence: [1.,1.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 0.0)]
25. Occurrences: 247. Action Sequence: [2.,0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
26. Occurrences: 245. Action Sequence: [2.,2.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
27. Occurrences: 243. Action Sequence: [3.,2.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
28. Occurrences: 242. Action Sequence: [0.,0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
29. Occurrences: 236. Action Sequence: [2.,3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
30. Occurrences: 235. Action Sequence: [1.,2.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
31. Occurrences: 230. Action Sequence: [0.,2.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
32. Occurrences: 227. Action Sequence: [1.,0.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
33. Occurrences: 227. Action Sequence: [0.,3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0)]
34. Occurrences: 222. Action Sequence: [3.,0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
35. Occurrences: 219. Action Sequence: [0.,1.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
36. Occurrences: 200. Action Sequence: [3.,2.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
37. Occurrences: 200. Action Sequence: [2.,2.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
38. Occurrences: 198. Action Sequence: [2.,0.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
39. Occurrences: 198. Action Sequence: [2.,1.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
40. Occurrences: 194. Action Sequence: [2.,3.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
41. Occurrences: 189. Action Sequence: [3.,0.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
42. Occurrences: 187. Action Sequence: [3.,3.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
43. Occurrences: 186. Action Sequence: [0.,2.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
44. Occurrences: 185. Action Sequence: [3.,1.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
45. Occurrences: 172. Action Sequence: [1.,3.,2.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
46. Occurrences: 172. Action Sequence: [0.,0.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
47. Occurrences: 164. Action Sequence: [3.,1.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
48. Occurrences: 160. Action Sequence: [3.,3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0)]
49. Occurrences: 159. Action Sequence: [0.,3.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
50. Occurrences: 157. Action Sequence: [1.,2.,2.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 3
29448 different trajectories
1. Occurrences: 2897. Action Sequence: [1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0)]
2. Occurrences: 2753. Action Sequence: [0.]
(pi_b, pi_e): [(0.5, 1.0)]
3. Occurrences: 2750. Action Sequence: [0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0)]
4. Occurrences: 2038. Action Sequence: [1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0)]
5. Occurrences: 1878. Action Sequence: [0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0)]
6. Occurrences: 1614. Action Sequence: [1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
7. Occurrences: 1570. Action Sequence: [1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
8. Occurrences: 1483. Action Sequence: [0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
9. Occurrences: 1376. Action Sequence: [0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
10. Occurrences: 1223. Action Sequence: [1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
11. Occurrences: 1184. Action Sequence: [1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
12. Occurrences: 1085. Action Sequence: [1.]
(pi_b, pi_e): [(0.5, 0.0)]
13. Occurrences: 1038. Action Sequence: [0.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
14. Occurrences: 921. Action Sequence: [0.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
15. Occurrences: 780. Action Sequence: [1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
16. Occurrences: 743. Action Sequence: [1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
17. Occurrences: 710. Action Sequence: [0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
18. Occurrences: 682. Action Sequence: [1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
19. Occurrences: 664. Action Sequence: [0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
20. Occurrences: 653. Action Sequence: [1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
21. Occurrences: 614. Action Sequence: [1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
22. Occurrences: 594. Action Sequence: [0.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
23. Occurrences: 571. Action Sequence: [0.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
24. Occurrences: 567. Action Sequence: [1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
25. Occurrences: 539. Action Sequence: [1.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
26. Occurrences: 519. Action Sequence: [0.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
27. Occurrences: 453. Action Sequence: [0.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
28. Occurrences: 448. Action Sequence: [1.,0.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
29. Occurrences: 410. Action Sequence: [0.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
30. Occurrences: 355. Action Sequence: [1.,0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
31. Occurrences: 353. Action Sequence: [0.,0.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
32. Occurrences: 353. Action Sequence: [1.,1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
33. Occurrences: 346. Action Sequence: [0.,1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
34. Occurrences: 339. Action Sequence: [1.,1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
35. Occurrences: 328. Action Sequence: [1.,1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
36. Occurrences: 322. Action Sequence: [1.,0.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
37. Occurrences: 312. Action Sequence: [0.,1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
38. Occurrences: 312. Action Sequence: [1.,0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
39. Occurrences: 307. Action Sequence: [0.,0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
40. Occurrences: 307. Action Sequence: [0.,0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
41. Occurrences: 301. Action Sequence: [0.,1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
42. Occurrences: 300. Action Sequence: [0.,0.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
43. Occurrences: 289. Action Sequence: [0.,1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
44. Occurrences: 284. Action Sequence: [1.,0.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
45. Occurrences: 278. Action Sequence: [0.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
46. Occurrences: 267. Action Sequence: [1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
47. Occurrences: 263. Action Sequence: [1.,1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
48. Occurrences: 257. Action Sequence: [1.,1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
49. Occurrences: 256. Action Sequence: [0.,1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
50. Occurrences: 238. Action Sequence: [1.,0.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]

FACTORED SPACE 1, factorisation 3
52116 different trajectories
1. Occurrences: 2560. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
2. Occurrences: 1672. Action Sequence: [1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
3. Occurrences: 1284. Action Sequence: [0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
4. Occurrences: 1237. Action Sequence: [2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
5. Occurrences: 1166. Action Sequence: [2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
6. Occurrences: 1066. Action Sequence: [1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
7. Occurrences: 1047. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
8. Occurrences: 910. Action Sequence: [2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
9. Occurrences: 903. Action Sequence: [0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
10. Occurrences: 747. Action Sequence: [3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
11. Occurrences: 667. Action Sequence: [1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
12. Occurrences: 588. Action Sequence: [1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
13. Occurrences: 548. Action Sequence: [0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
14. Occurrences: 543. Action Sequence: [2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
15. Occurrences: 474. Action Sequence: [3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
16. Occurrences: 444. Action Sequence: [0.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
17. Occurrences: 425. Action Sequence: [2.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
18. Occurrences: 414. Action Sequence: [0.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
19. Occurrences: 404. Action Sequence: [2.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
20. Occurrences: 315. Action Sequence: [3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
21. Occurrences: 273. Action Sequence: [1.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
22. Occurrences: 269. Action Sequence: [2.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
23. Occurrences: 263. Action Sequence: [3.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
24. Occurrences: 262. Action Sequence: [3.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
25. Occurrences: 261. Action Sequence: [1.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
26. Occurrences: 254. Action Sequence: [1.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
27. Occurrences: 246. Action Sequence: [0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
28. Occurrences: 241. Action Sequence: [2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
29. Occurrences: 232. Action Sequence: [2.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
30. Occurrences: 232. Action Sequence: [0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
31. Occurrences: 230. Action Sequence: [1.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
32. Occurrences: 227. Action Sequence: [0.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
33. Occurrences: 223. Action Sequence: [3.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
34. Occurrences: 219. Action Sequence: [3.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
35. Occurrences: 217. Action Sequence: [0.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
36. Occurrences: 212. Action Sequence: [1.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
37. Occurrences: 212. Action Sequence: [2.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
38. Occurrences: 206. Action Sequence: [3.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
39. Occurrences: 202. Action Sequence: [3.,1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
40. Occurrences: 197. Action Sequence: [1.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
41. Occurrences: 191. Action Sequence: [1.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
42. Occurrences: 186. Action Sequence: [0.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
43. Occurrences: 185. Action Sequence: [2.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
44. Occurrences: 182. Action Sequence: [2.,0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
45. Occurrences: 171. Action Sequence: [0.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
46. Occurrences: 170. Action Sequence: [2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
47. Occurrences: 169. Action Sequence: [3.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
48. Occurrences: 168. Action Sequence: [0.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
49. Occurrences: 159. Action Sequence: [2.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
50. Occurrences: 156. Action Sequence: [2.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 4
29448 different trajectories
1. Occurrences: 2897. Action Sequence: [1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0)]
2. Occurrences: 2753. Action Sequence: [0.]
(pi_b, pi_e): [(0.5, 1.0)]
3. Occurrences: 2750. Action Sequence: [0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0)]
4. Occurrences: 2038. Action Sequence: [1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0)]
5. Occurrences: 1878. Action Sequence: [0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0)]
6. Occurrences: 1614. Action Sequence: [1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
7. Occurrences: 1570. Action Sequence: [1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
8. Occurrences: 1483. Action Sequence: [0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
9. Occurrences: 1376. Action Sequence: [0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
10. Occurrences: 1223. Action Sequence: [1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
11. Occurrences: 1184. Action Sequence: [1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
12. Occurrences: 1085. Action Sequence: [1.]
(pi_b, pi_e): [(0.5, 0.0)]
13. Occurrences: 1038. Action Sequence: [0.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
14. Occurrences: 921. Action Sequence: [0.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
15. Occurrences: 780. Action Sequence: [1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
16. Occurrences: 743. Action Sequence: [1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
17. Occurrences: 710. Action Sequence: [0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
18. Occurrences: 682. Action Sequence: [1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
19. Occurrences: 664. Action Sequence: [0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
20. Occurrences: 653. Action Sequence: [1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
21. Occurrences: 614. Action Sequence: [1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
22. Occurrences: 594. Action Sequence: [0.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
23. Occurrences: 571. Action Sequence: [0.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
24. Occurrences: 567. Action Sequence: [1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
25. Occurrences: 539. Action Sequence: [1.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
26. Occurrences: 519. Action Sequence: [0.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
27. Occurrences: 453. Action Sequence: [0.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
28. Occurrences: 448. Action Sequence: [1.,0.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
29. Occurrences: 410. Action Sequence: [0.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
30. Occurrences: 355. Action Sequence: [1.,0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
31. Occurrences: 353. Action Sequence: [0.,0.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
32. Occurrences: 353. Action Sequence: [1.,1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
33. Occurrences: 346. Action Sequence: [0.,1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
34. Occurrences: 339. Action Sequence: [1.,1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
35. Occurrences: 328. Action Sequence: [1.,1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
36. Occurrences: 322. Action Sequence: [1.,0.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
37. Occurrences: 312. Action Sequence: [0.,1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
38. Occurrences: 312. Action Sequence: [1.,0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
39. Occurrences: 307. Action Sequence: [0.,0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
40. Occurrences: 307. Action Sequence: [0.,0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
41. Occurrences: 301. Action Sequence: [0.,1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
42. Occurrences: 300. Action Sequence: [0.,0.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
43. Occurrences: 289. Action Sequence: [0.,1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
44. Occurrences: 284. Action Sequence: [1.,0.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
45. Occurrences: 278. Action Sequence: [0.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
46. Occurrences: 267. Action Sequence: [1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
47. Occurrences: 263. Action Sequence: [1.,1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
48. Occurrences: 257. Action Sequence: [1.,1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
49. Occurrences: 256. Action Sequence: [0.,1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
50. Occurrences: 238. Action Sequence: [1.,0.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]

FACTORED SPACE 1, factorisation 4
24979 different trajectories
1. Occurrences: 6677. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
2. Occurrences: 4910. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
3. Occurrences: 4841. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
4. Occurrences: 3785. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
5. Occurrences: 2560. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
6. Occurrences: 2387. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
7. Occurrences: 2379. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
8. Occurrences: 2355. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
9. Occurrences: 1202. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
10. Occurrences: 1113. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
11. Occurrences: 1080. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
12. Occurrences: 1078. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
13. Occurrences: 1054. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
14. Occurrences: 1028. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
15. Occurrences: 1020. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
16. Occurrences: 983. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
17. Occurrences: 567. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
18. Occurrences: 532. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
19. Occurrences: 530. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
20. Occurrences: 501. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
21. Occurrences: 497. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
22. Occurrences: 493. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
23. Occurrences: 486. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
24. Occurrences: 485. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
25. Occurrences: 476. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
26. Occurrences: 447. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
27. Occurrences: 445. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
28. Occurrences: 442. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
29. Occurrences: 429. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
30. Occurrences: 418. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
31. Occurrences: 407. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
32. Occurrences: 403. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
33. Occurrences: 258. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
34. Occurrences: 237. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
35. Occurrences: 236. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
36. Occurrences: 236. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
37. Occurrences: 228. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
38. Occurrences: 227. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
39. Occurrences: 227. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
40. Occurrences: 222. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
41. Occurrences: 220. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
42. Occurrences: 220. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
43. Occurrences: 214. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
44. Occurrences: 213. Action Sequence: [0.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
45. Occurrences: 213. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
46. Occurrences: 213. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
47. Occurrences: 212. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
48. Occurrences: 208. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
49. Occurrences: 203. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
50. Occurrences: 202. Action Sequence: [0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]

FACTORED SPACE 2, factorisation 4
24596 different trajectories
1. Occurrences: 7318. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
2. Occurrences: 5129. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
3. Occurrences: 4805. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
4. Occurrences: 3690. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
5. Occurrences: 2865. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
6. Occurrences: 2342. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
7. Occurrences: 2328. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
8. Occurrences: 2151. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
9. Occurrences: 1492. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
10. Occurrences: 1081. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
11. Occurrences: 1071. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
12. Occurrences: 992. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
13. Occurrences: 991. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
14. Occurrences: 983. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
15. Occurrences: 967. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
16. Occurrences: 952. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
17. Occurrences: 732. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
18. Occurrences: 571. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
19. Occurrences: 557. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
20. Occurrences: 498. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
21. Occurrences: 479. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
22. Occurrences: 462. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
23. Occurrences: 441. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
24. Occurrences: 439. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
25. Occurrences: 437. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
26. Occurrences: 435. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
27. Occurrences: 426. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
28. Occurrences: 418. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
29. Occurrences: 415. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
30. Occurrences: 410. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
31. Occurrences: 402. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
32. Occurrences: 385. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
33. Occurrences: 300. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
34. Occurrences: 275. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
35. Occurrences: 271. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
36. Occurrences: 251. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
37. Occurrences: 236. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
38. Occurrences: 234. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
39. Occurrences: 224. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
40. Occurrences: 219. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
41. Occurrences: 216. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
42. Occurrences: 207. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
43. Occurrences: 205. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
44. Occurrences: 205. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
45. Occurrences: 203. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
46. Occurrences: 198. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
47. Occurrences: 197. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
48. Occurrences: 197. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
49. Occurrences: 195. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
50. Occurrences: 194. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]

--------------------------------------------------------
'''

### State 143, Diabetes

In [ ]:
'''
NON FACTORED SPACE
18781 different trajectories
1. Occurrences: 6450. Action Sequence: [1.]
2. Occurrences: 6153. Action Sequence: [5.]
3. Occurrences: 5759. Action Sequence: [3.]
4. Occurrences: 5745. Action Sequence: [7.]
5. Occurrences: 1821. Action Sequence: [0.]
6. Occurrences: 1175. Action Sequence: [4.]
7. Occurrences: 1044. Action Sequence: [2.]
8. Occurrences: 954. Action Sequence: [6.,5.]
9. Occurrences: 918. Action Sequence: [6.,1.]
10. Occurrences: 906. Action Sequence: [6.,7.]
11. Occurrences: 904. Action Sequence: [4.,5.]
12. Occurrences: 895. Action Sequence: [4.,1.]
13. Occurrences: 877. Action Sequence: [6.,3.]
14. Occurrences: 875. Action Sequence: [2.,7.]
15. Occurrences: 855. Action Sequence: [2.,5.]
16. Occurrences: 830. Action Sequence: [4.,3.]
17. Occurrences: 830. Action Sequence: [0.,5.]
18. Occurrences: 829. Action Sequence: [4.,7.]
19. Occurrences: 788. Action Sequence: [2.,3.]
20. Occurrences: 781. Action Sequence: [0.,7.]
21. Occurrences: 767. Action Sequence: [2.,1.]
22. Occurrences: 738. Action Sequence: [0.,1.]
23. Occurrences: 699. Action Sequence: [0.,3.]
24. Occurrences: 497. Action Sequence: [3.,5.]
25. Occurrences: 484. Action Sequence: [7.,1.]
26. Occurrences: 479. Action Sequence: [7.,3.]
27. Occurrences: 464. Action Sequence: [7.,0.]
28. Occurrences: 461. Action Sequence: [7.,5.]
29. Occurrences: 460. Action Sequence: [1.,5.]
30. Occurrences: 451. Action Sequence: [3.,7.]
31. Occurrences: 450. Action Sequence: [3.,1.]
32. Occurrences: 448. Action Sequence: [5.,3.]
33. Occurrences: 446. Action Sequence: [7.,2.]
34. Occurrences: 438. Action Sequence: [5.,5.]
35. Occurrences: 430. Action Sequence: [5.,0.]
36. Occurrences: 425. Action Sequence: [5.,1.]
37. Occurrences: 423. Action Sequence: [7.,7.]
38. Occurrences: 422. Action Sequence: [1.,1.]
39. Occurrences: 419. Action Sequence: [3.,0.]
40. Occurrences: 408. Action Sequence: [5.,7.]
41. Occurrences: 401. Action Sequence: [3.,3.]
42. Occurrences: 396. Action Sequence: [1.,7.]
43. Occurrences: 373. Action Sequence: [1.,2.]
44. Occurrences: 370. Action Sequence: [1.,3.]
45. Occurrences: 365. Action Sequence: [5.,2.]
46. Occurrences: 365. Action Sequence: [6.]
47. Occurrences: 355. Action Sequence: [1.,0.]
48. Occurrences: 350. Action Sequence: [3.,2.]
49. Occurrences: 308. Action Sequence: [2.,0.]
50. Occurrences: 307. Action Sequence: [0.,0.]

FACTORED SPACE 0, factorisation 1
2706 different trajectories
1. Occurrences: 24864. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
2. Occurrences: 22870. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
3. Occurrences: 10364. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
4. Occurrences: 9710. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
5. Occurrences: 3425. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
6. Occurrences: 3221. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
7. Occurrences: 3111. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
8. Occurrences: 2858. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
9. Occurrences: 1028. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
10. Occurrences: 989. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
11. Occurrences: 984. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
12. Occurrences: 955. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
13. Occurrences: 897. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
14. Occurrences: 883. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
15. Occurrences: 812. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
16. Occurrences: 762. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
17. Occurrences: 324. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
18. Occurrences: 301. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
19. Occurrences: 298. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
20. Occurrences: 288. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
21. Occurrences: 280. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
22. Occurrences: 276. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
23. Occurrences: 273. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
24. Occurrences: 269. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
25. Occurrences: 268. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
26. Occurrences: 267. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
27. Occurrences: 253. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
28. Occurrences: 253. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
29. Occurrences: 238. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
30. Occurrences: 235. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
31. Occurrences: 220. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
32. Occurrences: 218. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
33. Occurrences: 97. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
34. Occurrences: 96. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
35. Occurrences: 95. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
36. Occurrences: 94. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
37. Occurrences: 93. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
38. Occurrences: 93. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
39. Occurrences: 92. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
40. Occurrences: 89. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
41. Occurrences: 85. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
42. Occurrences: 84. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
43. Occurrences: 80. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
44. Occurrences: 80. Action Sequence: [0.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
45. Occurrences: 80. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
46. Occurrences: 79. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
47. Occurrences: 78. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
48. Occurrences: 78. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
49. Occurrences: 77. Action Sequence: [0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
50. Occurrences: 77. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]

FACTORED SPACE 1, factorisation 1
9524 different trajectories
1. Occurrences: 12209. Action Sequence: [1.]
(pi_b, pi_e): [(0.25, 1.0)]
2. Occurrences: 11898. Action Sequence: [3.]
(pi_b, pi_e): [(0.25, 0.0)]
3. Occurrences: 3593. Action Sequence: [2.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
4. Occurrences: 3520. Action Sequence: [2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0)]
5. Occurrences: 3341. Action Sequence: [0.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
6. Occurrences: 2992. Action Sequence: [0.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
7. Occurrences: 2865. Action Sequence: [0.]
(pi_b, pi_e): [(0.25, 0.0)]
8. Occurrences: 1836. Action Sequence: [3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0)]
9. Occurrences: 1804. Action Sequence: [1.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
10. Occurrences: 1730. Action Sequence: [3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
11. Occurrences: 1705. Action Sequence: [3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
12. Occurrences: 1643. Action Sequence: [1.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0)]
13. Occurrences: 1540. Action Sequence: [2.]
(pi_b, pi_e): [(0.25, 0.0)]
14. Occurrences: 1497. Action Sequence: [1.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0)]
15. Occurrences: 1024. Action Sequence: [0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
16. Occurrences: 919. Action Sequence: [2.,2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
17. Occurrences: 902. Action Sequence: [2.,2.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
18. Occurrences: 897. Action Sequence: [0.,2.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
19. Occurrences: 892. Action Sequence: [0.,2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
20. Occurrences: 888. Action Sequence: [1.,2.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
21. Occurrences: 848. Action Sequence: [3.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
22. Occurrences: 807. Action Sequence: [2.,0.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
23. Occurrences: 775. Action Sequence: [2.,0.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
24. Occurrences: 696. Action Sequence: [2.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
25. Occurrences: 691. Action Sequence: [0.,0.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
26. Occurrences: 587. Action Sequence: [0.,0.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
27. Occurrences: 354. Action Sequence: [3.,2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0)]
28. Occurrences: 352. Action Sequence: [1.,2.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 1.0)]
29. Occurrences: 329. Action Sequence: [3.,2.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
30. Occurrences: 327. Action Sequence: [2.,3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0)]
31. Occurrences: 316. Action Sequence: [0.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
32. Occurrences: 313. Action Sequence: [0.,3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0)]
33. Occurrences: 312. Action Sequence: [2.,1.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
34. Occurrences: 310. Action Sequence: [0.,3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
35. Occurrences: 308. Action Sequence: [1.,2.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
36. Occurrences: 307. Action Sequence: [2.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
37. Occurrences: 304. Action Sequence: [2.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
38. Occurrences: 300. Action Sequence: [0.,1.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 1.0)]
39. Occurrences: 300. Action Sequence: [0.,1.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
40. Occurrences: 296. Action Sequence: [2.,1.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 1.0)]
41. Occurrences: 279. Action Sequence: [0.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
42. Occurrences: 279. Action Sequence: [2.,3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
43. Occurrences: 259. Action Sequence: [2.,0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
44. Occurrences: 251. Action Sequence: [2.,1.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
45. Occurrences: 251. Action Sequence: [2.,2.,2.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
46. Occurrences: 239. Action Sequence: [3.,0.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
47. Occurrences: 239. Action Sequence: [0.,2.,2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
48. Occurrences: 227. Action Sequence: [0.,0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
49. Occurrences: 223. Action Sequence: [3.,3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
50. Occurrences: 221. Action Sequence: [2.,2.,2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 1.0), (0.25, 0.0)]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 2
2482 different trajectories
1. Occurrences: 33146. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
2. Occurrences: 20970. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
3. Occurrences: 10039. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
4. Occurrences: 7698. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
5. Occurrences: 4525. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
6. Occurrences: 2707. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
7. Occurrences: 1958. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
8. Occurrences: 1937. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
9. Occurrences: 1835. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
10. Occurrences: 1047. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
11. Occurrences: 870. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
12. Occurrences: 702. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
13. Occurrences: 671. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
14. Occurrences: 645. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
15. Occurrences: 466. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
16. Occurrences: 462. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
17. Occurrences: 417. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
18. Occurrences: 363. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
19. Occurrences: 356. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
20. Occurrences: 291. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
21. Occurrences: 283. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
22. Occurrences: 245. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
23. Occurrences: 243. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
24. Occurrences: 205. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
25. Occurrences: 190. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
26. Occurrences: 167. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
27. Occurrences: 158. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
28. Occurrences: 148. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
29. Occurrences: 147. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
30. Occurrences: 124. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
31. Occurrences: 119. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
32. Occurrences: 116. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
33. Occurrences: 113. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
34. Occurrences: 113. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
35. Occurrences: 110. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
36. Occurrences: 109. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
37. Occurrences: 109. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
38. Occurrences: 101. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
39. Occurrences: 96. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
40. Occurrences: 82. Action Sequence: [0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
41. Occurrences: 79. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
42. Occurrences: 73. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
43. Occurrences: 73. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
44. Occurrences: 73. Action Sequence: [0.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
45. Occurrences: 71. Action Sequence: [0.,0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
46. Occurrences: 66. Action Sequence: [0.,0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
47. Occurrences: 65. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
48. Occurrences: 61. Action Sequence: [0.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
49. Occurrences: 58. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
50. Occurrences: 50. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]

FACTORED SPACE 1, factorisation 2
10332 different trajectories
1. Occurrences: 8271. Action Sequence: [0.]
(pi_b, pi_e): [(0.25, 0.0)]
2. Occurrences: 7328. Action Sequence: [2.]
(pi_b, pi_e): [(0.25, 0.0)]
3. Occurrences: 6803. Action Sequence: [1.]
(pi_b, pi_e): [(0.25, 1.0)]
4. Occurrences: 6110. Action Sequence: [3.]
(pi_b, pi_e): [(0.25, 0.0)]
5. Occurrences: 2074. Action Sequence: [3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
6. Occurrences: 1993. Action Sequence: [2.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
7. Occurrences: 1944. Action Sequence: [1.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
8. Occurrences: 1932. Action Sequence: [3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0)]
9. Occurrences: 1822. Action Sequence: [0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
10. Occurrences: 1765. Action Sequence: [3.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
11. Occurrences: 1758. Action Sequence: [2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0)]
12. Occurrences: 1742. Action Sequence: [1.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0)]
13. Occurrences: 1725. Action Sequence: [1.,2.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
14. Occurrences: 1689. Action Sequence: [2.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
15. Occurrences: 1648. Action Sequence: [0.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0)]
16. Occurrences: 1639. Action Sequence: [0.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
17. Occurrences: 1575. Action Sequence: [1.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
18. Occurrences: 1558. Action Sequence: [3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0)]
19. Occurrences: 1463. Action Sequence: [2.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
20. Occurrences: 1410. Action Sequence: [0.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
21. Occurrences: 374. Action Sequence: [1.,2.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
22. Occurrences: 366. Action Sequence: [1.,3.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
23. Occurrences: 364. Action Sequence: [3.,2.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
24. Occurrences: 358. Action Sequence: [3.,3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0)]
25. Occurrences: 348. Action Sequence: [3.,3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
26. Occurrences: 338. Action Sequence: [1.,1.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
27. Occurrences: 334. Action Sequence: [3.,1.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
28. Occurrences: 332. Action Sequence: [1.,3.,2.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
29. Occurrences: 325. Action Sequence: [2.,3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
30. Occurrences: 324. Action Sequence: [2.,2.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
31. Occurrences: 321. Action Sequence: [3.,2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
32. Occurrences: 316. Action Sequence: [2.,3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0)]
33. Occurrences: 311. Action Sequence: [2.,1.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
34. Occurrences: 310. Action Sequence: [0.,3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
35. Occurrences: 308. Action Sequence: [0.,3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0)]
36. Occurrences: 305. Action Sequence: [1.,3.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 1.0)]
37. Occurrences: 305. Action Sequence: [1.,2.,2.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
38. Occurrences: 304. Action Sequence: [1.,2.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 1.0)]
39. Occurrences: 304. Action Sequence: [3.,1.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0)]
40. Occurrences: 300. Action Sequence: [3.,3.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
41. Occurrences: 293. Action Sequence: [0.,1.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
42. Occurrences: 293. Action Sequence: [0.,2.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
43. Occurrences: 289. Action Sequence: [3.,0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
44. Occurrences: 288. Action Sequence: [2.,3.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
45. Occurrences: 283. Action Sequence: [1.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 1.0)]
46. Occurrences: 279. Action Sequence: [0.,3.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
47. Occurrences: 278. Action Sequence: [2.,0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
48. Occurrences: 278. Action Sequence: [1.,1.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
49. Occurrences: 275. Action Sequence: [3.,0.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
50. Occurrences: 272. Action Sequence: [1.,0.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 3
3484 different trajectories
1. Occurrences: 15074. Action Sequence: [0.]
(pi_b, pi_e): [(0.5, 1.0)]
2. Occurrences: 13438. Action Sequence: [1.]
(pi_b, pi_e): [(0.5, 0.0)]
3. Occurrences: 7757. Action Sequence: [1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0)]
4. Occurrences: 7156. Action Sequence: [0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0)]
5. Occurrences: 6475. Action Sequence: [1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0)]
6. Occurrences: 6349. Action Sequence: [0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0)]
7. Occurrences: 2595. Action Sequence: [1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
8. Occurrences: 2525. Action Sequence: [0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
9. Occurrences: 2290. Action Sequence: [1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
10. Occurrences: 2153. Action Sequence: [0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
11. Occurrences: 2100. Action Sequence: [0.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
12. Occurrences: 2096. Action Sequence: [1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
13. Occurrences: 1994. Action Sequence: [1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
14. Occurrences: 1683. Action Sequence: [0.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
15. Occurrences: 813. Action Sequence: [1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
16. Occurrences: 795. Action Sequence: [0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
17. Occurrences: 749. Action Sequence: [1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
18. Occurrences: 749. Action Sequence: [1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
19. Occurrences: 690. Action Sequence: [0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
20. Occurrences: 689. Action Sequence: [1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
21. Occurrences: 663. Action Sequence: [0.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
22. Occurrences: 644. Action Sequence: [0.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
23. Occurrences: 638. Action Sequence: [1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
24. Occurrences: 617. Action Sequence: [1.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
25. Occurrences: 596. Action Sequence: [0.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
26. Occurrences: 575. Action Sequence: [1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
27. Occurrences: 543. Action Sequence: [0.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
28. Occurrences: 508. Action Sequence: [1.,0.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
29. Occurrences: 501. Action Sequence: [0.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
30. Occurrences: 414. Action Sequence: [0.,0.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
31. Occurrences: 251. Action Sequence: [1.,1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
32. Occurrences: 244. Action Sequence: [0.,1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
33. Occurrences: 239. Action Sequence: [1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
34. Occurrences: 235. Action Sequence: [1.,0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
35. Occurrences: 228. Action Sequence: [1.,1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
36. Occurrences: 222. Action Sequence: [0.,0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
37. Occurrences: 222. Action Sequence: [0.,1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
38. Occurrences: 219. Action Sequence: [1.,1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
39. Occurrences: 215. Action Sequence: [0.,1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
40. Occurrences: 214. Action Sequence: [1.,1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
41. Occurrences: 206. Action Sequence: [1.,0.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
42. Occurrences: 204. Action Sequence: [1.,1.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
43. Occurrences: 201. Action Sequence: [0.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
44. Occurrences: 187. Action Sequence: [1.,0.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
45. Occurrences: 186. Action Sequence: [1.,1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
46. Occurrences: 186. Action Sequence: [1.,0.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
47. Occurrences: 185. Action Sequence: [0.,1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
48. Occurrences: 184. Action Sequence: [0.,0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
49. Occurrences: 179. Action Sequence: [1.,0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
50. Occurrences: 175. Action Sequence: [0.,1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]

FACTORED SPACE 1, factorisation 3
8356 different trajectories
1. Occurrences: 14461. Action Sequence: [1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
2. Occurrences: 13501. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
3. Occurrences: 6438. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
4. Occurrences: 5200. Action Sequence: [2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
5. Occurrences: 3828. Action Sequence: [2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
6. Occurrences: 3652. Action Sequence: [0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
7. Occurrences: 2077. Action Sequence: [3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
8. Occurrences: 1929. Action Sequence: [1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
9. Occurrences: 1668. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
10. Occurrences: 1628. Action Sequence: [2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
11. Occurrences: 1580. Action Sequence: [3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
12. Occurrences: 1549. Action Sequence: [0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
13. Occurrences: 1468. Action Sequence: [1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
14. Occurrences: 1411. Action Sequence: [1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
15. Occurrences: 1296. Action Sequence: [2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
16. Occurrences: 1268. Action Sequence: [0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
17. Occurrences: 1009. Action Sequence: [2.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
18. Occurrences: 924. Action Sequence: [2.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
19. Occurrences: 871. Action Sequence: [0.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
20. Occurrences: 813. Action Sequence: [0.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
21. Occurrences: 402. Action Sequence: [2.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
22. Occurrences: 373. Action Sequence: [2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
23. Occurrences: 369. Action Sequence: [2.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
24. Occurrences: 368. Action Sequence: [0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
25. Occurrences: 365. Action Sequence: [2.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
26. Occurrences: 362. Action Sequence: [0.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
27. Occurrences: 360. Action Sequence: [0.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
28. Occurrences: 346. Action Sequence: [3.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
29. Occurrences: 314. Action Sequence: [1.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
30. Occurrences: 306. Action Sequence: [2.,0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
31. Occurrences: 303. Action Sequence: [0.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
32. Occurrences: 301. Action Sequence: [2.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
33. Occurrences: 297. Action Sequence: [2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
34. Occurrences: 289. Action Sequence: [2.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
35. Occurrences: 287. Action Sequence: [0.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
36. Occurrences: 286. Action Sequence: [3.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
37. Occurrences: 276. Action Sequence: [2.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
38. Occurrences: 274. Action Sequence: [0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
39. Occurrences: 272. Action Sequence: [1.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
40. Occurrences: 272. Action Sequence: [0.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
41. Occurrences: 251. Action Sequence: [0.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
42. Occurrences: 248. Action Sequence: [0.,0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
43. Occurrences: 237. Action Sequence: [0.,1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
44. Occurrences: 233. Action Sequence: [3.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
45. Occurrences: 232. Action Sequence: [2.,2.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
46. Occurrences: 230. Action Sequence: [3.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
47. Occurrences: 230. Action Sequence: [3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
48. Occurrences: 220. Action Sequence: [2.,1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
49. Occurrences: 218. Action Sequence: [0.,2.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
50. Occurrences: 217. Action Sequence: [1.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 4
3484 different trajectories
1. Occurrences: 15074. Action Sequence: [0.]
(pi_b, pi_e): [(0.5, 1.0)]
2. Occurrences: 13438. Action Sequence: [1.]
(pi_b, pi_e): [(0.5, 0.0)]
3. Occurrences: 7757. Action Sequence: [1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0)]
4. Occurrences: 7156. Action Sequence: [0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0)]
5. Occurrences: 6475. Action Sequence: [1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0)]
6. Occurrences: 6349. Action Sequence: [0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0)]
7. Occurrences: 2595. Action Sequence: [1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
8. Occurrences: 2525. Action Sequence: [0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
9. Occurrences: 2290. Action Sequence: [1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
10. Occurrences: 2153. Action Sequence: [0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
11. Occurrences: 2100. Action Sequence: [0.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
12. Occurrences: 2096. Action Sequence: [1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
13. Occurrences: 1994. Action Sequence: [1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
14. Occurrences: 1683. Action Sequence: [0.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
15. Occurrences: 813. Action Sequence: [1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
16. Occurrences: 795. Action Sequence: [0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
17. Occurrences: 749. Action Sequence: [1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
18. Occurrences: 749. Action Sequence: [1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
19. Occurrences: 690. Action Sequence: [0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
20. Occurrences: 689. Action Sequence: [1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
21. Occurrences: 663. Action Sequence: [0.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
22. Occurrences: 644. Action Sequence: [0.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
23. Occurrences: 638. Action Sequence: [1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
24. Occurrences: 617. Action Sequence: [1.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
25. Occurrences: 596. Action Sequence: [0.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
26. Occurrences: 575. Action Sequence: [1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
27. Occurrences: 543. Action Sequence: [0.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
28. Occurrences: 508. Action Sequence: [1.,0.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
29. Occurrences: 501. Action Sequence: [0.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
30. Occurrences: 414. Action Sequence: [0.,0.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
31. Occurrences: 251. Action Sequence: [1.,1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
32. Occurrences: 244. Action Sequence: [0.,1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
33. Occurrences: 239. Action Sequence: [1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
34. Occurrences: 235. Action Sequence: [1.,0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
35. Occurrences: 228. Action Sequence: [1.,1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
36. Occurrences: 222. Action Sequence: [0.,0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
37. Occurrences: 222. Action Sequence: [0.,1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
38. Occurrences: 219. Action Sequence: [1.,1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
39. Occurrences: 215. Action Sequence: [0.,1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
40. Occurrences: 214. Action Sequence: [1.,1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
41. Occurrences: 206. Action Sequence: [1.,0.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
42. Occurrences: 204. Action Sequence: [1.,1.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
43. Occurrences: 201. Action Sequence: [0.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
44. Occurrences: 187. Action Sequence: [1.,0.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
45. Occurrences: 186. Action Sequence: [1.,1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
46. Occurrences: 186. Action Sequence: [1.,0.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
47. Occurrences: 185. Action Sequence: [0.,1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
48. Occurrences: 184. Action Sequence: [0.,0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
49. Occurrences: 179. Action Sequence: [1.,0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
50. Occurrences: 175. Action Sequence: [0.,1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]

FACTORED SPACE 1, factorisation 4
2706 different trajectories
1. Occurrences: 24864. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
2. Occurrences: 22870. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
3. Occurrences: 10364. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
4. Occurrences: 9710. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
5. Occurrences: 3425. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
6. Occurrences: 3221. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
7. Occurrences: 3111. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
8. Occurrences: 2858. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
9. Occurrences: 1028. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
10. Occurrences: 989. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
11. Occurrences: 984. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
12. Occurrences: 955. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
13. Occurrences: 897. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
14. Occurrences: 883. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
15. Occurrences: 812. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
16. Occurrences: 762. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
17. Occurrences: 324. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
18. Occurrences: 301. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
19. Occurrences: 298. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
20. Occurrences: 288. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
21. Occurrences: 280. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
22. Occurrences: 276. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
23. Occurrences: 273. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
24. Occurrences: 269. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
25. Occurrences: 268. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
26. Occurrences: 267. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
27. Occurrences: 253. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
28. Occurrences: 253. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
29. Occurrences: 238. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
30. Occurrences: 235. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
31. Occurrences: 220. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
32. Occurrences: 218. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
33. Occurrences: 97. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
34. Occurrences: 96. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
35. Occurrences: 95. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
36. Occurrences: 94. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
37. Occurrences: 93. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
38. Occurrences: 93. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
39. Occurrences: 92. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
40. Occurrences: 89. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
41. Occurrences: 85. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
42. Occurrences: 84. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
43. Occurrences: 80. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
44. Occurrences: 80. Action Sequence: [0.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
45. Occurrences: 80. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
46. Occurrences: 79. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
47. Occurrences: 78. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
48. Occurrences: 78. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
49. Occurrences: 77. Action Sequence: [0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
50. Occurrences: 77. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]

FACTORED SPACE 2, factorisation 4
2482 different trajectories
1. Occurrences: 33146. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
2. Occurrences: 20970. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
3. Occurrences: 10039. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
4. Occurrences: 7698. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
5. Occurrences: 4525. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
6. Occurrences: 2707. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
7. Occurrences: 1958. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
8. Occurrences: 1937. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
9. Occurrences: 1835. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
10. Occurrences: 1047. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
11. Occurrences: 870. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
12. Occurrences: 702. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
13. Occurrences: 671. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
14. Occurrences: 645. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
15. Occurrences: 466. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
16. Occurrences: 462. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
17. Occurrences: 417. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
18. Occurrences: 363. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
19. Occurrences: 356. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
20. Occurrences: 291. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
21. Occurrences: 283. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
22. Occurrences: 245. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
23. Occurrences: 243. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
24. Occurrences: 205. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
25. Occurrences: 190. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
26. Occurrences: 167. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
27. Occurrences: 158. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
28. Occurrences: 148. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
29. Occurrences: 147. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
30. Occurrences: 124. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
31. Occurrences: 119. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
32. Occurrences: 116. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
33. Occurrences: 113. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
34. Occurrences: 113. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
35. Occurrences: 110. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
36. Occurrences: 109. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
37. Occurrences: 109. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
38. Occurrences: 101. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
39. Occurrences: 96. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
40. Occurrences: 82. Action Sequence: [0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
41. Occurrences: 79. Action Sequence: [0.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
42. Occurrences: 73. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
43. Occurrences: 73. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
44. Occurrences: 73. Action Sequence: [0.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
45. Occurrences: 71. Action Sequence: [0.,0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
46. Occurrences: 66. Action Sequence: [0.,0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
47. Occurrences: 65. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
48. Occurrences: 61. Action Sequence: [0.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
49. Occurrences: 58. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
50. Occurrences: 50. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]

--------------------------------------------------------
'''

### State 63, Diabetes

In [ ]:
'''
NON FACTORED SPACE
14720 different trajectories
1. Occurrences: 8101. Action Sequence: [4.]
2. Occurrences: 7851. Action Sequence: [0.]
3. Occurrences: 7502. Action Sequence: [2.]
4. Occurrences: 7408. Action Sequence: [6.]
5. Occurrences: 3850. Action Sequence: [1.]
6. Occurrences: 3768. Action Sequence: [5.]
7. Occurrences: 3138. Action Sequence: [7.]
8. Occurrences: 3122. Action Sequence: [3.]
9. Occurrences: 756. Action Sequence: [7.,3.]
10. Occurrences: 737. Action Sequence: [3.,5.]
11. Occurrences: 736. Action Sequence: [7.,1.]
12. Occurrences: 734. Action Sequence: [1.,5.]
13. Occurrences: 712. Action Sequence: [3.,7.]
14. Occurrences: 709. Action Sequence: [7.,5.]
15. Occurrences: 701. Action Sequence: [3.,1.]
16. Occurrences: 699. Action Sequence: [5.,1.]
17. Occurrences: 698. Action Sequence: [1.,1.]
18. Occurrences: 695. Action Sequence: [5.,5.]
19. Occurrences: 693. Action Sequence: [5.,3.]
20. Occurrences: 672. Action Sequence: [7.,7.]
21. Occurrences: 655. Action Sequence: [5.,7.]
22. Occurrences: 652. Action Sequence: [1.,7.]
23. Occurrences: 610. Action Sequence: [1.,3.]
24. Occurrences: 592. Action Sequence: [3.,3.]
25. Occurrences: 414. Action Sequence: [6.,0.]
26. Occurrences: 383. Action Sequence: [6.,4.]
27. Occurrences: 378. Action Sequence: [2.,4.]
28. Occurrences: 372. Action Sequence: [4.,2.]
29. Occurrences: 371. Action Sequence: [7.,0.]
30. Occurrences: 370. Action Sequence: [4.,4.]
31. Occurrences: 365. Action Sequence: [5.,0.]
32. Occurrences: 352. Action Sequence: [6.,2.]
33. Occurrences: 351. Action Sequence: [2.,6.]
34. Occurrences: 346. Action Sequence: [6.,6.]
35. Occurrences: 345. Action Sequence: [0.,4.]
36. Occurrences: 343. Action Sequence: [0.,6.]
37. Occurrences: 336. Action Sequence: [2.,0.]
38. Occurrences: 334. Action Sequence: [4.,6.]
39. Occurrences: 332. Action Sequence: [4.,0.]
40. Occurrences: 309. Action Sequence: [0.,0.]
41. Occurrences: 299. Action Sequence: [3.,0.]
42. Occurrences: 298. Action Sequence: [2.,2.]
43. Occurrences: 291. Action Sequence: [7.,2.]
44. Occurrences: 289. Action Sequence: [1.,0.]
45. Occurrences: 266. Action Sequence: [0.,2.]
46. Occurrences: 258. Action Sequence: [3.,2.]
47. Occurrences: 243. Action Sequence: [7.,4.]
48. Occurrences: 239. Action Sequence: [5.,2.]
49. Occurrences: 236. Action Sequence: [1.,2.]
50. Occurrences: 226. Action Sequence: [3.,4.]

FACTORED SPACE 0, factorisation 1
2177 different trajectories
1. Occurrences: 30880. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
2. Occurrences: 29271. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
3. Occurrences: 8372. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
4. Occurrences: 7896. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
5. Occurrences: 2454. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
6. Occurrences: 2301. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
7. Occurrences: 2284. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
8. Occurrences: 2181. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
9. Occurrences: 794. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
10. Occurrences: 702. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
11. Occurrences: 685. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
12. Occurrences: 655. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
13. Occurrences: 635. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
14. Occurrences: 628. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
15. Occurrences: 604. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
16. Occurrences: 553. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
17. Occurrences: 233. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
18. Occurrences: 217. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
19. Occurrences: 210. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
20. Occurrences: 209. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
21. Occurrences: 208. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
22. Occurrences: 206. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
23. Occurrences: 203. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
24. Occurrences: 202. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
25. Occurrences: 190. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
26. Occurrences: 184. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
27. Occurrences: 180. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
28. Occurrences: 180. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
29. Occurrences: 178. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
30. Occurrences: 173. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
31. Occurrences: 169. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
32. Occurrences: 152. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
33. Occurrences: 83. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
34. Occurrences: 80. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
35. Occurrences: 71. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
36. Occurrences: 67. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
37. Occurrences: 67. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
38. Occurrences: 67. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
39. Occurrences: 66. Action Sequence: [0.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
40. Occurrences: 66. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
41. Occurrences: 65. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
42. Occurrences: 64. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
43. Occurrences: 63. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
44. Occurrences: 62. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
45. Occurrences: 62. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
46. Occurrences: 62. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
47. Occurrences: 61. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
48. Occurrences: 60. Action Sequence: [0.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
49. Occurrences: 60. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
50. Occurrences: 60. Action Sequence: [0.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]

FACTORED SPACE 1, factorisation 1
7627 different trajectories
1. Occurrences: 15509. Action Sequence: [2.]
(pi_b, pi_e): [(0.25, 0.0)]
2. Occurrences: 15353. Action Sequence: [0.]
(pi_b, pi_e): [(0.25, 0.0)]
3. Occurrences: 6972. Action Sequence: [1.]
(pi_b, pi_e): [(0.25, 1.0)]
4. Occurrences: 6906. Action Sequence: [3.]
(pi_b, pi_e): [(0.25, 0.0)]
5. Occurrences: 2884. Action Sequence: [3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0)]
6. Occurrences: 2835. Action Sequence: [1.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
7. Occurrences: 2731. Action Sequence: [3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
8. Occurrences: 2601. Action Sequence: [1.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0)]
9. Occurrences: 1470. Action Sequence: [2.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
10. Occurrences: 1433. Action Sequence: [2.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
11. Occurrences: 1417. Action Sequence: [0.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
12. Occurrences: 1266. Action Sequence: [3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
13. Occurrences: 1209. Action Sequence: [0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
14. Occurrences: 1082. Action Sequence: [1.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
15. Occurrences: 793. Action Sequence: [3.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
16. Occurrences: 739. Action Sequence: [1.,2.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
17. Occurrences: 640. Action Sequence: [2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0)]
18. Occurrences: 630. Action Sequence: [3.,2.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
19. Occurrences: 626. Action Sequence: [2.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
20. Occurrences: 619. Action Sequence: [0.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
21. Occurrences: 605. Action Sequence: [3.,2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
22. Occurrences: 598. Action Sequence: [0.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0)]
23. Occurrences: 590. Action Sequence: [1.,2.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
24. Occurrences: 587. Action Sequence: [1.,2.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
25. Occurrences: 552. Action Sequence: [3.,0.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
26. Occurrences: 508. Action Sequence: [1.,0.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
27. Occurrences: 489. Action Sequence: [1.,0.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 1.0)]
28. Occurrences: 462. Action Sequence: [3.,0.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0)]
29. Occurrences: 283. Action Sequence: [0.,3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
30. Occurrences: 273. Action Sequence: [1.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
31. Occurrences: 269. Action Sequence: [0.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
32. Occurrences: 267. Action Sequence: [2.,3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
33. Occurrences: 263. Action Sequence: [3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
34. Occurrences: 258. Action Sequence: [2.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
35. Occurrences: 258. Action Sequence: [1.,3.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 1.0)]
36. Occurrences: 252. Action Sequence: [2.,1.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
37. Occurrences: 250. Action Sequence: [3.,3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0)]
38. Occurrences: 241. Action Sequence: [3.,1.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
39. Occurrences: 240. Action Sequence: [0.,1.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
40. Occurrences: 232. Action Sequence: [2.,1.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 1.0)]
41. Occurrences: 228. Action Sequence: [3.,1.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
42. Occurrences: 225. Action Sequence: [1.,1.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
43. Occurrences: 221. Action Sequence: [0.,1.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 1.0)]
44. Occurrences: 212. Action Sequence: [1.,1.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 0.0)]
45. Occurrences: 200. Action Sequence: [3.,3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
46. Occurrences: 192. Action Sequence: [3.,0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
47. Occurrences: 185. Action Sequence: [1.,3.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
48. Occurrences: 185. Action Sequence: [1.,1.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 0.0)]
49. Occurrences: 171. Action Sequence: [3.,1.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
50. Occurrences: 167. Action Sequence: [1.,0.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 2
2001 different trajectories
1. Occurrences: 35807. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
2. Occurrences: 27406. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
3. Occurrences: 9136. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
4. Occurrences: 6547. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
5. Occurrences: 3262. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
6. Occurrences: 1995. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
7. Occurrences: 1759. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
8. Occurrences: 1405. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
9. Occurrences: 1299. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
10. Occurrences: 743. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
11. Occurrences: 613. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
12. Occurrences: 561. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
13. Occurrences: 544. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
14. Occurrences: 479. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
15. Occurrences: 414. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
16. Occurrences: 336. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
17. Occurrences: 303. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
18. Occurrences: 288. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
19. Occurrences: 256. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
20. Occurrences: 213. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
21. Occurrences: 196. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
22. Occurrences: 185. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
23. Occurrences: 157. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
24. Occurrences: 139. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
25. Occurrences: 131. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
26. Occurrences: 130. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
27. Occurrences: 115. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
28. Occurrences: 112. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
29. Occurrences: 110. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
30. Occurrences: 105. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
31. Occurrences: 98. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
32. Occurrences: 91. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
33. Occurrences: 90. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
34. Occurrences: 89. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
35. Occurrences: 80. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
36. Occurrences: 76. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
37. Occurrences: 73. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
38. Occurrences: 72. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
39. Occurrences: 64. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
40. Occurrences: 61. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
41. Occurrences: 55. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
42. Occurrences: 54. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
43. Occurrences: 52. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
44. Occurrences: 51. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
45. Occurrences: 50. Action Sequence: [1.,0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
46. Occurrences: 49. Action Sequence: [0.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
47. Occurrences: 48. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
48. Occurrences: 46. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
49. Occurrences: 45. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
50. Occurrences: 44. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]

FACTORED SPACE 1, factorisation 2
8185 different trajectories
1. Occurrences: 11869. Action Sequence: [2.]
(pi_b, pi_e): [(0.25, 0.0)]
2. Occurrences: 11701. Action Sequence: [0.]
(pi_b, pi_e): [(0.25, 0.0)]
3. Occurrences: 10624. Action Sequence: [1.]
(pi_b, pi_e): [(0.25, 1.0)]
4. Occurrences: 10546. Action Sequence: [3.]
(pi_b, pi_e): [(0.25, 0.0)]
5. Occurrences: 1704. Action Sequence: [3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
6. Occurrences: 1565. Action Sequence: [2.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
7. Occurrences: 1554. Action Sequence: [3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0)]
8. Occurrences: 1514. Action Sequence: [1.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
9. Occurrences: 1513. Action Sequence: [1.,2.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
10. Occurrences: 1498. Action Sequence: [3.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
11. Occurrences: 1459. Action Sequence: [2.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
12. Occurrences: 1438. Action Sequence: [0.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
13. Occurrences: 1437. Action Sequence: [2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0)]
14. Occurrences: 1431. Action Sequence: [0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
15. Occurrences: 1390. Action Sequence: [1.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
16. Occurrences: 1342. Action Sequence: [3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
17. Occurrences: 1310. Action Sequence: [1.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0)]
18. Occurrences: 1284. Action Sequence: [2.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
19. Occurrences: 1269. Action Sequence: [0.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
20. Occurrences: 1235. Action Sequence: [0.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0)]
21. Occurrences: 258. Action Sequence: [1.,3.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 1.0)]
22. Occurrences: 255. Action Sequence: [0.,2.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
23. Occurrences: 244. Action Sequence: [2.,1.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
24. Occurrences: 243. Action Sequence: [1.,1.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 0.0)]
25. Occurrences: 243. Action Sequence: [3.,2.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
26. Occurrences: 239. Action Sequence: [3.,3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
27. Occurrences: 239. Action Sequence: [1.,3.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
28. Occurrences: 238. Action Sequence: [3.,3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 1.0)]
29. Occurrences: 234. Action Sequence: [1.,2.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
30. Occurrences: 233. Action Sequence: [1.,1.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 1.0)]
31. Occurrences: 233. Action Sequence: [3.,3.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
32. Occurrences: 233. Action Sequence: [2.,3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
33. Occurrences: 230. Action Sequence: [3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
34. Occurrences: 227. Action Sequence: [3.,1.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
35. Occurrences: 226. Action Sequence: [3.,1.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
36. Occurrences: 226. Action Sequence: [2.,2.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
37. Occurrences: 223. Action Sequence: [3.,2.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
38. Occurrences: 222. Action Sequence: [1.,3.,2.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
39. Occurrences: 221. Action Sequence: [0.,0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
40. Occurrences: 219. Action Sequence: [0.,3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
41. Occurrences: 218. Action Sequence: [2.,3.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
42. Occurrences: 218. Action Sequence: [2.,2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0)]
43. Occurrences: 218. Action Sequence: [2.,0.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
44. Occurrences: 217. Action Sequence: [3.,2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0)]
45. Occurrences: 216. Action Sequence: [1.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 1.0)]
46. Occurrences: 215. Action Sequence: [3.,1.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 1.0)]
47. Occurrences: 212. Action Sequence: [0.,3.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
48. Occurrences: 209. Action Sequence: [1.,0.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
49. Occurrences: 206. Action Sequence: [1.,2.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 1.0)]
50. Occurrences: 205. Action Sequence: [3.,0.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 3
2815 different trajectories
1. Occurrences: 22415. Action Sequence: [1.]
(pi_b, pi_e): [(0.5, 0.0)]
2. Occurrences: 22325. Action Sequence: [0.]
(pi_b, pi_e): [(0.5, 1.0)]
3. Occurrences: 6260. Action Sequence: [1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0)]
4. Occurrences: 5610. Action Sequence: [0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0)]
5. Occurrences: 5583. Action Sequence: [1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0)]
6. Occurrences: 5490. Action Sequence: [0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0)]
7. Occurrences: 1815. Action Sequence: [1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
8. Occurrences: 1787. Action Sequence: [0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
9. Occurrences: 1641. Action Sequence: [1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
10. Occurrences: 1601. Action Sequence: [0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
11. Occurrences: 1585. Action Sequence: [0.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
12. Occurrences: 1566. Action Sequence: [1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
13. Occurrences: 1556. Action Sequence: [1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
14. Occurrences: 1405. Action Sequence: [0.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
15. Occurrences: 574. Action Sequence: [1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
16. Occurrences: 549. Action Sequence: [0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
17. Occurrences: 512. Action Sequence: [0.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
18. Occurrences: 509. Action Sequence: [0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
19. Occurrences: 494. Action Sequence: [1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
20. Occurrences: 493. Action Sequence: [1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
21. Occurrences: 470. Action Sequence: [1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
22. Occurrences: 470. Action Sequence: [0.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
23. Occurrences: 440. Action Sequence: [1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
24. Occurrences: 439. Action Sequence: [1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
25. Occurrences: 437. Action Sequence: [0.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
26. Occurrences: 433. Action Sequence: [1.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
27. Occurrences: 433. Action Sequence: [0.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
28. Occurrences: 422. Action Sequence: [0.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
29. Occurrences: 378. Action Sequence: [1.,0.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
30. Occurrences: 365. Action Sequence: [0.,0.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
31. Occurrences: 196. Action Sequence: [1.,1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
32. Occurrences: 193. Action Sequence: [0.,0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
33. Occurrences: 166. Action Sequence: [1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
34. Occurrences: 166. Action Sequence: [0.,0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
35. Occurrences: 159. Action Sequence: [1.,0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
36. Occurrences: 156. Action Sequence: [1.,1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
37. Occurrences: 155. Action Sequence: [1.,1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
38. Occurrences: 148. Action Sequence: [0.,1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
39. Occurrences: 144. Action Sequence: [0.,1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
40. Occurrences: 144. Action Sequence: [1.,0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
41. Occurrences: 143. Action Sequence: [0.,0.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
42. Occurrences: 139. Action Sequence: [0.,1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
43. Occurrences: 137. Action Sequence: [0.,1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
44. Occurrences: 137. Action Sequence: [1.,1.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
45. Occurrences: 137. Action Sequence: [1.,1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
46. Occurrences: 135. Action Sequence: [1.,0.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
47. Occurrences: 132. Action Sequence: [1.,0.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
48. Occurrences: 131. Action Sequence: [1.,0.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
49. Occurrences: 131. Action Sequence: [0.,1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
50. Occurrences: 128. Action Sequence: [0.,0.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]

FACTORED SPACE 1, factorisation 3
6663 different trajectories
1. Occurrences: 16723. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
2. Occurrences: 15873. Action Sequence: [2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
3. Occurrences: 10476. Action Sequence: [1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
4. Occurrences: 9312. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
5. Occurrences: 3156. Action Sequence: [3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
6. Occurrences: 3056. Action Sequence: [1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
7. Occurrences: 1695. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
8. Occurrences: 1637. Action Sequence: [1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
9. Occurrences: 1604. Action Sequence: [2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
10. Occurrences: 1576. Action Sequence: [3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
11. Occurrences: 1449. Action Sequence: [2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
12. Occurrences: 1441. Action Sequence: [0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
13. Occurrences: 1391. Action Sequence: [0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
14. Occurrences: 1318. Action Sequence: [1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
15. Occurrences: 945. Action Sequence: [2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
16. Occurrences: 855. Action Sequence: [0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
17. Occurrences: 645. Action Sequence: [3.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
18. Occurrences: 608. Action Sequence: [1.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
19. Occurrences: 590. Action Sequence: [1.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
20. Occurrences: 584. Action Sequence: [3.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 1.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
21. Occurrences: 301. Action Sequence: [2.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
22. Occurrences: 290. Action Sequence: [0.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
23. Occurrences: 290. Action Sequence: [3.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
24. Occurrences: 288. Action Sequence: [2.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
25. Occurrences: 288. Action Sequence: [3.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
26. Occurrences: 283. Action Sequence: [3.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
27. Occurrences: 278. Action Sequence: [0.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
28. Occurrences: 269. Action Sequence: [1.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
29. Occurrences: 268. Action Sequence: [1.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
30. Occurrences: 267. Action Sequence: [3.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
31. Occurrences: 267. Action Sequence: [1.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
32. Occurrences: 249. Action Sequence: [1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
33. Occurrences: 240. Action Sequence: [3.,0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
34. Occurrences: 238. Action Sequence: [3.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
35. Occurrences: 236. Action Sequence: [1.,0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
36. Occurrences: 225. Action Sequence: [3.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
37. Occurrences: 219. Action Sequence: [1.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
38. Occurrences: 215. Action Sequence: [3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
39. Occurrences: 206. Action Sequence: [1.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
40. Occurrences: 194. Action Sequence: [2.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
41. Occurrences: 192. Action Sequence: [3.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
42. Occurrences: 181. Action Sequence: [1.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
43. Occurrences: 178. Action Sequence: [1.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
44. Occurrences: 178. Action Sequence: [3.,1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
45. Occurrences: 173. Action Sequence: [2.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
46. Occurrences: 165. Action Sequence: [1.,1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
47. Occurrences: 156. Action Sequence: [2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
48. Occurrences: 155. Action Sequence: [0.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
49. Occurrences: 155. Action Sequence: [2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
50. Occurrences: 154. Action Sequence: [0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 4
2815 different trajectories
1. Occurrences: 22415. Action Sequence: [1.]
(pi_b, pi_e): [(0.5, 0.0)]
2. Occurrences: 22325. Action Sequence: [0.]
(pi_b, pi_e): [(0.5, 1.0)]
3. Occurrences: 6260. Action Sequence: [1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0)]
4. Occurrences: 5610. Action Sequence: [0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0)]
5. Occurrences: 5583. Action Sequence: [1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0)]
6. Occurrences: 5490. Action Sequence: [0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0)]
7. Occurrences: 1815. Action Sequence: [1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
8. Occurrences: 1787. Action Sequence: [0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
9. Occurrences: 1641. Action Sequence: [1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
10. Occurrences: 1601. Action Sequence: [0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
11. Occurrences: 1585. Action Sequence: [0.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
12. Occurrences: 1566. Action Sequence: [1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
13. Occurrences: 1556. Action Sequence: [1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
14. Occurrences: 1405. Action Sequence: [0.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
15. Occurrences: 574. Action Sequence: [1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
16. Occurrences: 549. Action Sequence: [0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
17. Occurrences: 512. Action Sequence: [0.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
18. Occurrences: 509. Action Sequence: [0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
19. Occurrences: 494. Action Sequence: [1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
20. Occurrences: 493. Action Sequence: [1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
21. Occurrences: 470. Action Sequence: [1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
22. Occurrences: 470. Action Sequence: [0.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
23. Occurrences: 440. Action Sequence: [1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
24. Occurrences: 439. Action Sequence: [1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
25. Occurrences: 437. Action Sequence: [0.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
26. Occurrences: 433. Action Sequence: [1.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
27. Occurrences: 433. Action Sequence: [0.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
28. Occurrences: 422. Action Sequence: [0.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
29. Occurrences: 378. Action Sequence: [1.,0.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
30. Occurrences: 365. Action Sequence: [0.,0.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
31. Occurrences: 196. Action Sequence: [1.,1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
32. Occurrences: 193. Action Sequence: [0.,0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
33. Occurrences: 166. Action Sequence: [1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
34. Occurrences: 166. Action Sequence: [0.,0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
35. Occurrences: 159. Action Sequence: [1.,0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
36. Occurrences: 156. Action Sequence: [1.,1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
37. Occurrences: 155. Action Sequence: [1.,1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
38. Occurrences: 148. Action Sequence: [0.,1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
39. Occurrences: 144. Action Sequence: [0.,1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
40. Occurrences: 144. Action Sequence: [1.,0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
41. Occurrences: 143. Action Sequence: [0.,0.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
42. Occurrences: 139. Action Sequence: [0.,1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
43. Occurrences: 137. Action Sequence: [0.,1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
44. Occurrences: 137. Action Sequence: [1.,1.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
45. Occurrences: 137. Action Sequence: [1.,1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
46. Occurrences: 135. Action Sequence: [1.,0.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
47. Occurrences: 132. Action Sequence: [1.,0.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
48. Occurrences: 131. Action Sequence: [1.,0.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
49. Occurrences: 131. Action Sequence: [0.,1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
50. Occurrences: 128. Action Sequence: [0.,0.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]

FACTORED SPACE 1, factorisation 4
2177 different trajectories
1. Occurrences: 30880. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
2. Occurrences: 29271. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
3. Occurrences: 8372. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
4. Occurrences: 7896. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
5. Occurrences: 2454. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
6. Occurrences: 2301. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
7. Occurrences: 2284. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
8. Occurrences: 2181. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
9. Occurrences: 794. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
10. Occurrences: 702. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
11. Occurrences: 685. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
12. Occurrences: 655. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
13. Occurrences: 635. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
14. Occurrences: 628. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
15. Occurrences: 604. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
16. Occurrences: 553. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
17. Occurrences: 233. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
18. Occurrences: 217. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
19. Occurrences: 210. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
20. Occurrences: 209. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
21. Occurrences: 208. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
22. Occurrences: 206. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
23. Occurrences: 203. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
24. Occurrences: 202. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
25. Occurrences: 190. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
26. Occurrences: 184. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
27. Occurrences: 180. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
28. Occurrences: 180. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
29. Occurrences: 178. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
30. Occurrences: 173. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
31. Occurrences: 169. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
32. Occurrences: 152. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
33. Occurrences: 83. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
34. Occurrences: 80. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
35. Occurrences: 71. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
36. Occurrences: 67. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
37. Occurrences: 67. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
38. Occurrences: 67. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
39. Occurrences: 66. Action Sequence: [0.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
40. Occurrences: 66. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
41. Occurrences: 65. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
42. Occurrences: 64. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
43. Occurrences: 63. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
44. Occurrences: 62. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
45. Occurrences: 62. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
46. Occurrences: 62. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
47. Occurrences: 61. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
48. Occurrences: 60. Action Sequence: [0.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
49. Occurrences: 60. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
50. Occurrences: 60. Action Sequence: [0.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]

FACTORED SPACE 2, factorisation 4
2001 different trajectories
1. Occurrences: 35807. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
2. Occurrences: 27406. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
3. Occurrences: 9136. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
4. Occurrences: 6547. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
5. Occurrences: 3262. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
6. Occurrences: 1995. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
7. Occurrences: 1759. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
8. Occurrences: 1405. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
9. Occurrences: 1299. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
10. Occurrences: 743. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
11. Occurrences: 613. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
12. Occurrences: 561. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
13. Occurrences: 544. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
14. Occurrences: 479. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
15. Occurrences: 414. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
16. Occurrences: 336. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
17. Occurrences: 303. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
18. Occurrences: 288. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
19. Occurrences: 256. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
20. Occurrences: 213. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
21. Occurrences: 196. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
22. Occurrences: 185. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
23. Occurrences: 157. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
24. Occurrences: 139. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
25. Occurrences: 131. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
26. Occurrences: 130. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
27. Occurrences: 115. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
28. Occurrences: 112. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
29. Occurrences: 110. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
30. Occurrences: 105. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
31. Occurrences: 98. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
32. Occurrences: 91. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
33. Occurrences: 90. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
34. Occurrences: 89. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
35. Occurrences: 80. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
36. Occurrences: 76. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
37. Occurrences: 73. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
38. Occurrences: 72. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
39. Occurrences: 64. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
40. Occurrences: 61. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
41. Occurrences: 55. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
42. Occurrences: 54. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
43. Occurrences: 52. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
44. Occurrences: 51. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
45. Occurrences: 50. Action Sequence: [1.,0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
46. Occurrences: 49. Action Sequence: [0.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
47. Occurrences: 48. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
48. Occurrences: 46. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
49. Occurrences: 45. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
50. Occurrences: 44. Action Sequence: [1.,1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]

--------------------------------------------------------
'''

### State 56, Diabetes

In [ ]:
'''
NON FACTORED SPACE
18144 different trajectories
1. Occurrences: 7992. Action Sequence: [4.]
2. Occurrences: 7483. Action Sequence: [6.]
3. Occurrences: 6657. Action Sequence: [0.]
4. Occurrences: 6052. Action Sequence: [2.]
5. Occurrences: 3771. Action Sequence: [5.]
6. Occurrences: 3367. Action Sequence: [1.]
7. Occurrences: 3057. Action Sequence: [7.]
8. Occurrences: 2903. Action Sequence: [3.]
9. Occurrences: 736. Action Sequence: [7.,1.]
10. Occurrences: 736. Action Sequence: [5.,5.]
11. Occurrences: 729. Action Sequence: [7.,5.]
12. Occurrences: 692. Action Sequence: [5.,1.]
13. Occurrences: 687. Action Sequence: [5.,7.]
14. Occurrences: 672. Action Sequence: [7.,3.]
15. Occurrences: 668. Action Sequence: [5.,3.]
16. Occurrences: 659. Action Sequence: [7.,7.]
17. Occurrences: 651. Action Sequence: [1.,5.]
18. Occurrences: 650. Action Sequence: [3.,1.]
19. Occurrences: 621. Action Sequence: [3.,5.]
20. Occurrences: 601. Action Sequence: [1.,1.]
21. Occurrences: 582. Action Sequence: [3.,7.]
22. Occurrences: 577. Action Sequence: [3.,3.]
23. Occurrences: 574. Action Sequence: [1.,7.]
24. Occurrences: 560. Action Sequence: [1.,3.]
25. Occurrences: 447. Action Sequence: [6.,0.]
26. Occurrences: 403. Action Sequence: [6.,4.]
27. Occurrences: 384. Action Sequence: [4.,6.]
28. Occurrences: 370. Action Sequence: [4.,4.]
29. Occurrences: 362. Action Sequence: [7.,0.]
30. Occurrences: 351. Action Sequence: [6.,2.]
31. Occurrences: 348. Action Sequence: [4.,2.]
32. Occurrences: 345. Action Sequence: [6.,6.]
33. Occurrences: 341. Action Sequence: [4.,0.]
34. Occurrences: 322. Action Sequence: [3.,0.]
35. Occurrences: 321. Action Sequence: [2.,4.]
36. Occurrences: 319. Action Sequence: [0.,4.]
37. Occurrences: 318. Action Sequence: [2.,0.]
38. Occurrences: 306. Action Sequence: [5.,0.]
39. Occurrences: 283. Action Sequence: [1.,0.]
40. Occurrences: 280. Action Sequence: [7.,2.]
41. Occurrences: 279. Action Sequence: [2.,6.]
42. Occurrences: 279. Action Sequence: [2.,2.]
43. Occurrences: 267. Action Sequence: [5.,2.]
44. Occurrences: 267. Action Sequence: [0.,0.]
45. Occurrences: 266. Action Sequence: [0.,6.]
46. Occurrences: 247. Action Sequence: [0.,5.]
47. Occurrences: 245. Action Sequence: [0.,2.]
48. Occurrences: 242. Action Sequence: [7.,4.]
49. Occurrences: 241. Action Sequence: [5.,4.]
50. Occurrences: 236. Action Sequence: [0.,1.]

FACTORED SPACE 0, factorisation 1
3682 different trajectories
1. Occurrences: 29152. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
2. Occurrences: 27092. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
3. Occurrences: 8481. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
4. Occurrences: 7896. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
5. Occurrences: 2564. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
6. Occurrences: 2443. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
7. Occurrences: 2373. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
8. Occurrences: 2210. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
9. Occurrences: 785. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
10. Occurrences: 778. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
11. Occurrences: 716. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
12. Occurrences: 697. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
13. Occurrences: 696. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
14. Occurrences: 695. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
15. Occurrences: 648. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
16. Occurrences: 643. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
17. Occurrences: 262. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
18. Occurrences: 238. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
19. Occurrences: 238. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
20. Occurrences: 235. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
21. Occurrences: 230. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
22. Occurrences: 229. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
23. Occurrences: 227. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
24. Occurrences: 222. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
25. Occurrences: 215. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
26. Occurrences: 214. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
27. Occurrences: 209. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
28. Occurrences: 205. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
29. Occurrences: 200. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
30. Occurrences: 187. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
31. Occurrences: 176. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
32. Occurrences: 175. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
33. Occurrences: 98. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
34. Occurrences: 86. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
35. Occurrences: 85. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
36. Occurrences: 85. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
37. Occurrences: 79. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
38. Occurrences: 76. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
39. Occurrences: 76. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
40. Occurrences: 74. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
41. Occurrences: 72. Action Sequence: [0.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
42. Occurrences: 71. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
43. Occurrences: 71. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
44. Occurrences: 71. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
45. Occurrences: 70. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
46. Occurrences: 70. Action Sequence: [0.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
47. Occurrences: 69. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
48. Occurrences: 68. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
49. Occurrences: 66. Action Sequence: [0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
50. Occurrences: 65. Action Sequence: [0.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]

FACTORED SPACE 1, factorisation 1
10364 different trajectories
1. Occurrences: 15475. Action Sequence: [2.]
(pi_b, pi_e): [(0.25, 0.0)]
2. Occurrences: 12709. Action Sequence: [0.]
(pi_b, pi_e): [(0.25, 0.0)]
3. Occurrences: 6828. Action Sequence: [3.]
(pi_b, pi_e): [(0.25, 0.0)]
4. Occurrences: 6270. Action Sequence: [1.]
(pi_b, pi_e): [(0.25, 1.0)]
5. Occurrences: 2811. Action Sequence: [3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
6. Occurrences: 2768. Action Sequence: [3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0)]
7. Occurrences: 2428. Action Sequence: [1.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
8. Occurrences: 2388. Action Sequence: [1.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
9. Occurrences: 1502. Action Sequence: [2.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
10. Occurrences: 1487. Action Sequence: [2.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
11. Occurrences: 1215. Action Sequence: [3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
12. Occurrences: 1185. Action Sequence: [0.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
13. Occurrences: 1109. Action Sequence: [0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
14. Occurrences: 1011. Action Sequence: [1.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
15. Occurrences: 868. Action Sequence: [0.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
16. Occurrences: 867. Action Sequence: [3.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
17. Occurrences: 854. Action Sequence: [0.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0)]
18. Occurrences: 735. Action Sequence: [1.,2.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
19. Occurrences: 688. Action Sequence: [2.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
20. Occurrences: 645. Action Sequence: [2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0)]
21. Occurrences: 634. Action Sequence: [3.,2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0)]
22. Occurrences: 593. Action Sequence: [1.,2.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
23. Occurrences: 590. Action Sequence: [3.,2.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
24. Occurrences: 566. Action Sequence: [1.,2.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 1.0)]
25. Occurrences: 531. Action Sequence: [3.,0.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
26. Occurrences: 505. Action Sequence: [1.,0.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
27. Occurrences: 479. Action Sequence: [1.,0.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 1.0)]
28. Occurrences: 454. Action Sequence: [3.,0.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
29. Occurrences: 280. Action Sequence: [2.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
30. Occurrences: 280. Action Sequence: [3.,3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
31. Occurrences: 278. Action Sequence: [0.,3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
32. Occurrences: 270. Action Sequence: [2.,1.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
33. Occurrences: 258. Action Sequence: [0.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
34. Occurrences: 251. Action Sequence: [1.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
35. Occurrences: 250. Action Sequence: [1.,3.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 1.0)]
36. Occurrences: 250. Action Sequence: [0.,1.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 1.0)]
37. Occurrences: 249. Action Sequence: [1.,1.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 1.0)]
38. Occurrences: 246. Action Sequence: [0.,1.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
39. Occurrences: 243. Action Sequence: [2.,1.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
40. Occurrences: 242. Action Sequence: [3.,1.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
41. Occurrences: 237. Action Sequence: [3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
42. Occurrences: 224. Action Sequence: [2.,3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
43. Occurrences: 205. Action Sequence: [3.,1.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 1.0)]
44. Occurrences: 200. Action Sequence: [1.,0.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
45. Occurrences: 200. Action Sequence: [1.,1.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 0.0)]
46. Occurrences: 198. Action Sequence: [1.,3.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 0.0)]
47. Occurrences: 196. Action Sequence: [3.,3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
48. Occurrences: 193. Action Sequence: [1.,1.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 0.0)]
49. Occurrences: 190. Action Sequence: [3.,0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
50. Occurrences: 177. Action Sequence: [0.,2.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 2
3482 different trajectories
1. Occurrences: 33749. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
2. Occurrences: 25943. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
3. Occurrences: 9088. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
4. Occurrences: 6662. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
5. Occurrences: 3316. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
6. Occurrences: 2007. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
7. Occurrences: 1728. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
8. Occurrences: 1501. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
9. Occurrences: 1415. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
10. Occurrences: 731. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
11. Occurrences: 693. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
12. Occurrences: 569. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
13. Occurrences: 560. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
14. Occurrences: 505. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
15. Occurrences: 440. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
16. Occurrences: 397. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
17. Occurrences: 380. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
18. Occurrences: 269. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
19. Occurrences: 267. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
20. Occurrences: 263. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
21. Occurrences: 247. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
22. Occurrences: 227. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
23. Occurrences: 166. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
24. Occurrences: 152. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
25. Occurrences: 147. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
26. Occurrences: 143. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
27. Occurrences: 143. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
28. Occurrences: 136. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
29. Occurrences: 127. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
30. Occurrences: 126. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
31. Occurrences: 116. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
32. Occurrences: 115. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
33. Occurrences: 114. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
34. Occurrences: 108. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
35. Occurrences: 103. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
36. Occurrences: 94. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
37. Occurrences: 90. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
38. Occurrences: 79. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
39. Occurrences: 76. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
40. Occurrences: 69. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
41. Occurrences: 65. Action Sequence: [0.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
42. Occurrences: 64. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
43. Occurrences: 64. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
44. Occurrences: 59. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
45. Occurrences: 57. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
46. Occurrences: 54. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
47. Occurrences: 53. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
48. Occurrences: 51. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
49. Occurrences: 50. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
50. Occurrences: 50. Action Sequence: [1.,0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]

FACTORED SPACE 1, factorisation 2
10887 different trajectories
1. Occurrences: 11763. Action Sequence: [2.]
(pi_b, pi_e): [(0.25, 0.0)]
2. Occurrences: 10540. Action Sequence: [3.]
(pi_b, pi_e): [(0.25, 0.0)]
3. Occurrences: 10024. Action Sequence: [0.]
(pi_b, pi_e): [(0.25, 0.0)]
4. Occurrences: 8955. Action Sequence: [1.]
(pi_b, pi_e): [(0.25, 1.0)]
5. Occurrences: 1741. Action Sequence: [3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
6. Occurrences: 1581. Action Sequence: [3.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
7. Occurrences: 1537. Action Sequence: [2.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
8. Occurrences: 1514. Action Sequence: [1.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
9. Occurrences: 1504. Action Sequence: [2.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
10. Occurrences: 1461. Action Sequence: [3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0)]
11. Occurrences: 1436. Action Sequence: [0.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
12. Occurrences: 1409. Action Sequence: [2.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0)]
13. Occurrences: 1394. Action Sequence: [1.,2.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
14. Occurrences: 1389. Action Sequence: [2.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
15. Occurrences: 1387. Action Sequence: [0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
16. Occurrences: 1361. Action Sequence: [3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
17. Occurrences: 1257. Action Sequence: [1.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0)]
18. Occurrences: 1206. Action Sequence: [1.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0)]
19. Occurrences: 1204. Action Sequence: [0.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
20. Occurrences: 1180. Action Sequence: [0.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0)]
21. Occurrences: 281. Action Sequence: [3.,3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
22. Occurrences: 262. Action Sequence: [1.,3.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
23. Occurrences: 261. Action Sequence: [1.,1.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 0.0)]
24. Occurrences: 255. Action Sequence: [0.,3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
25. Occurrences: 255. Action Sequence: [1.,2.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
26. Occurrences: 245. Action Sequence: [3.,2.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
27. Occurrences: 243. Action Sequence: [0.,2.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
28. Occurrences: 242. Action Sequence: [2.,2.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
29. Occurrences: 239. Action Sequence: [0.,3.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
30. Occurrences: 237. Action Sequence: [1.,3.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 1.0)]
31. Occurrences: 236. Action Sequence: [3.,3.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
32. Occurrences: 236. Action Sequence: [0.,0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
33. Occurrences: 235. Action Sequence: [0.,1.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
34. Occurrences: 235. Action Sequence: [0.,3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
35. Occurrences: 225. Action Sequence: [1.,0.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 1.0)]
36. Occurrences: 225. Action Sequence: [1.,3.,2.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
37. Occurrences: 224. Action Sequence: [1.,2.,2.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
38. Occurrences: 224. Action Sequence: [3.,2.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
39. Occurrences: 223. Action Sequence: [3.,3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0)]
40. Occurrences: 223. Action Sequence: [1.,2.,1.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
41. Occurrences: 220. Action Sequence: [1.,0.,0.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0)]
42. Occurrences: 219. Action Sequence: [0.,1.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
43. Occurrences: 214. Action Sequence: [3.,1.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
44. Occurrences: 213. Action Sequence: [3.,0.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
45. Occurrences: 211. Action Sequence: [3.,1.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0)]
46. Occurrences: 210. Action Sequence: [2.,3.,2.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
47. Occurrences: 210. Action Sequence: [2.,3.,1.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0)]
48. Occurrences: 209. Action Sequence: [2.,3.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
49. Occurrences: 209. Action Sequence: [3.,0.,0.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
50. Occurrences: 208. Action Sequence: [1.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 1.0)]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 3
4459 different trajectories
1. Occurrences: 22303. Action Sequence: [1.]
(pi_b, pi_e): [(0.5, 0.0)]
2. Occurrences: 18979. Action Sequence: [0.]
(pi_b, pi_e): [(0.5, 1.0)]
3. Occurrences: 6115. Action Sequence: [1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0)]
4. Occurrences: 5868. Action Sequence: [1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0)]
5. Occurrences: 5362. Action Sequence: [0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0)]
6. Occurrences: 5216. Action Sequence: [0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0)]
7. Occurrences: 1894. Action Sequence: [0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
8. Occurrences: 1807. Action Sequence: [1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
9. Occurrences: 1791. Action Sequence: [0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
10. Occurrences: 1642. Action Sequence: [0.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
11. Occurrences: 1584. Action Sequence: [1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
12. Occurrences: 1549. Action Sequence: [1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
13. Occurrences: 1529. Action Sequence: [1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
14. Occurrences: 1400. Action Sequence: [0.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
15. Occurrences: 588. Action Sequence: [0.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
16. Occurrences: 578. Action Sequence: [0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
17. Occurrences: 562. Action Sequence: [0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
18. Occurrences: 558. Action Sequence: [1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
19. Occurrences: 546. Action Sequence: [1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
20. Occurrences: 528. Action Sequence: [0.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
21. Occurrences: 491. Action Sequence: [1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
22. Occurrences: 475. Action Sequence: [1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
23. Occurrences: 468. Action Sequence: [1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
24. Occurrences: 465. Action Sequence: [0.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
25. Occurrences: 458. Action Sequence: [0.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
26. Occurrences: 435. Action Sequence: [1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
27. Occurrences: 421. Action Sequence: [0.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
28. Occurrences: 403. Action Sequence: [1.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
29. Occurrences: 383. Action Sequence: [1.,0.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
30. Occurrences: 378. Action Sequence: [0.,0.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
31. Occurrences: 220. Action Sequence: [0.,1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
32. Occurrences: 205. Action Sequence: [0.,1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
33. Occurrences: 199. Action Sequence: [0.,0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
34. Occurrences: 190. Action Sequence: [0.,1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
35. Occurrences: 184. Action Sequence: [0.,0.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
36. Occurrences: 180. Action Sequence: [1.,1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
37. Occurrences: 176. Action Sequence: [0.,0.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
38. Occurrences: 171. Action Sequence: [0.,0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
39. Occurrences: 170. Action Sequence: [0.,1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
40. Occurrences: 166. Action Sequence: [0.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
41. Occurrences: 163. Action Sequence: [1.,0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
42. Occurrences: 160. Action Sequence: [1.,1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
43. Occurrences: 156. Action Sequence: [0.,0.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
44. Occurrences: 147. Action Sequence: [1.,0.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
45. Occurrences: 144. Action Sequence: [1.,1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
46. Occurrences: 142. Action Sequence: [0.,1.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
47. Occurrences: 141. Action Sequence: [1.,1.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
48. Occurrences: 140. Action Sequence: [1.,0.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
49. Occurrences: 138. Action Sequence: [1.,0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
50. Occurrences: 135. Action Sequence: [1.,1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]

FACTORED SPACE 1, factorisation 3
9267 different trajectories
1. Occurrences: 15573. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
2. Occurrences: 14521. Action Sequence: [2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
3. Occurrences: 9898. Action Sequence: [1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
4. Occurrences: 8718. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
5. Occurrences: 2996. Action Sequence: [3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
6. Occurrences: 2908. Action Sequence: [1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
7. Occurrences: 1781. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 1.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
8. Occurrences: 1598. Action Sequence: [2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
9. Occurrences: 1569. Action Sequence: [1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
10. Occurrences: 1444. Action Sequence: [3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
11. Occurrences: 1407. Action Sequence: [0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
12. Occurrences: 1393. Action Sequence: [2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
13. Occurrences: 1371. Action Sequence: [0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
14. Occurrences: 1360. Action Sequence: [1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
15. Occurrences: 1153. Action Sequence: [2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
16. Occurrences: 1084. Action Sequence: [0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
17. Occurrences: 621. Action Sequence: [3.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
18. Occurrences: 615. Action Sequence: [1.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
19. Occurrences: 594. Action Sequence: [3.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
20. Occurrences: 556. Action Sequence: [1.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
21. Occurrences: 337. Action Sequence: [3.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
22. Occurrences: 309. Action Sequence: [2.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
23. Occurrences: 306. Action Sequence: [3.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
24. Occurrences: 302. Action Sequence: [3.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
25. Occurrences: 297. Action Sequence: [0.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
26. Occurrences: 296. Action Sequence: [2.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
27. Occurrences: 287. Action Sequence: [3.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
28. Occurrences: 279. Action Sequence: [1.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
29. Occurrences: 273. Action Sequence: [1.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
30. Occurrences: 271. Action Sequence: [1.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
31. Occurrences: 248. Action Sequence: [0.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
32. Occurrences: 229. Action Sequence: [1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
33. Occurrences: 226. Action Sequence: [3.,0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
34. Occurrences: 224. Action Sequence: [1.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
35. Occurrences: 218. Action Sequence: [3.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
36. Occurrences: 204. Action Sequence: [1.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
37. Occurrences: 199. Action Sequence: [1.,0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
38. Occurrences: 197. Action Sequence: [3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
39. Occurrences: 195. Action Sequence: [3.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 1.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
40. Occurrences: 193. Action Sequence: [3.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
41. Occurrences: 193. Action Sequence: [1.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
42. Occurrences: 178. Action Sequence: [1.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
43. Occurrences: 176. Action Sequence: [2.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
44. Occurrences: 175. Action Sequence: [2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
45. Occurrences: 172. Action Sequence: [0.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
46. Occurrences: 170. Action Sequence: [1.,1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 1.0), (0.25, 1.0), (0.25, 1.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
47. Occurrences: 170. Action Sequence: [2.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
48. Occurrences: 168. Action Sequence: [0.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
49. Occurrences: 167. Action Sequence: [3.,1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 1.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]
50. Occurrences: 162. Action Sequence: [2.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
(pi_b, pi_e): [(0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0), (0.25, 0.0)]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 4
4459 different trajectories
1. Occurrences: 22303. Action Sequence: [1.]
(pi_b, pi_e): [(0.5, 0.0)]
2. Occurrences: 18979. Action Sequence: [0.]
(pi_b, pi_e): [(0.5, 1.0)]
3. Occurrences: 6115. Action Sequence: [1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0)]
4. Occurrences: 5868. Action Sequence: [1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0)]
5. Occurrences: 5362. Action Sequence: [0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0)]
6. Occurrences: 5216. Action Sequence: [0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0)]
7. Occurrences: 1894. Action Sequence: [0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
8. Occurrences: 1807. Action Sequence: [1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
9. Occurrences: 1791. Action Sequence: [0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
10. Occurrences: 1642. Action Sequence: [0.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
11. Occurrences: 1584. Action Sequence: [1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
12. Occurrences: 1549. Action Sequence: [1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
13. Occurrences: 1529. Action Sequence: [1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
14. Occurrences: 1400. Action Sequence: [0.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
15. Occurrences: 588. Action Sequence: [0.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
16. Occurrences: 578. Action Sequence: [0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
17. Occurrences: 562. Action Sequence: [0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
18. Occurrences: 558. Action Sequence: [1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
19. Occurrences: 546. Action Sequence: [1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
20. Occurrences: 528. Action Sequence: [0.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
21. Occurrences: 491. Action Sequence: [1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
22. Occurrences: 475. Action Sequence: [1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
23. Occurrences: 468. Action Sequence: [1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
24. Occurrences: 465. Action Sequence: [0.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
25. Occurrences: 458. Action Sequence: [0.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
26. Occurrences: 435. Action Sequence: [1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
27. Occurrences: 421. Action Sequence: [0.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
28. Occurrences: 403. Action Sequence: [1.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
29. Occurrences: 383. Action Sequence: [1.,0.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
30. Occurrences: 378. Action Sequence: [0.,0.,0.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
31. Occurrences: 220. Action Sequence: [0.,1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
32. Occurrences: 205. Action Sequence: [0.,1.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
33. Occurrences: 199. Action Sequence: [0.,0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
34. Occurrences: 190. Action Sequence: [0.,1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
35. Occurrences: 184. Action Sequence: [0.,0.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
36. Occurrences: 180. Action Sequence: [1.,1.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
37. Occurrences: 176. Action Sequence: [0.,0.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
38. Occurrences: 171. Action Sequence: [0.,0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0)]
39. Occurrences: 170. Action Sequence: [0.,1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
40. Occurrences: 166. Action Sequence: [0.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
41. Occurrences: 163. Action Sequence: [1.,0.,1.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
42. Occurrences: 160. Action Sequence: [1.,1.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
43. Occurrences: 156. Action Sequence: [0.,0.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0)]
44. Occurrences: 147. Action Sequence: [1.,0.,0.,1.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
45. Occurrences: 144. Action Sequence: [1.,1.,1.,0.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0)]
46. Occurrences: 142. Action Sequence: [0.,1.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0)]
47. Occurrences: 141. Action Sequence: [1.,1.,0.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
48. Occurrences: 140. Action Sequence: [1.,0.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0)]
49. Occurrences: 138. Action Sequence: [1.,0.,1.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0)]
50. Occurrences: 135. Action Sequence: [1.,1.,0.,0.,0.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]

FACTORED SPACE 1, factorisation 4
3682 different trajectories
1. Occurrences: 29152. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
2. Occurrences: 27092. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
3. Occurrences: 8481. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
4. Occurrences: 7896. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
5. Occurrences: 2564. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
6. Occurrences: 2443. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
7. Occurrences: 2373. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
8. Occurrences: 2210. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
9. Occurrences: 785. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
10. Occurrences: 778. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
11. Occurrences: 716. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
12. Occurrences: 697. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
13. Occurrences: 696. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
14. Occurrences: 695. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
15. Occurrences: 648. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
16. Occurrences: 643. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
17. Occurrences: 262. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
18. Occurrences: 238. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
19. Occurrences: 238. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
20. Occurrences: 235. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
21. Occurrences: 230. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
22. Occurrences: 229. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
23. Occurrences: 227. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
24. Occurrences: 222. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
25. Occurrences: 215. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
26. Occurrences: 214. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
27. Occurrences: 209. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
28. Occurrences: 205. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
29. Occurrences: 200. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
30. Occurrences: 187. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
31. Occurrences: 176. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
32. Occurrences: 175. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
33. Occurrences: 98. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
34. Occurrences: 86. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
35. Occurrences: 85. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
36. Occurrences: 85. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
37. Occurrences: 79. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
38. Occurrences: 76. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
39. Occurrences: 76. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
40. Occurrences: 74. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
41. Occurrences: 72. Action Sequence: [0.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
42. Occurrences: 71. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
43. Occurrences: 71. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
44. Occurrences: 71. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
45. Occurrences: 70. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
46. Occurrences: 70. Action Sequence: [0.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
47. Occurrences: 69. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
48. Occurrences: 68. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
49. Occurrences: 66. Action Sequence: [0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]
50. Occurrences: 65. Action Sequence: [0.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]

FACTORED SPACE 2, factorisation 4
3482 different trajectories
1. Occurrences: 33749. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
2. Occurrences: 25943. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
3. Occurrences: 9088. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
4. Occurrences: 6662. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
5. Occurrences: 3316. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
6. Occurrences: 2007. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
7. Occurrences: 1728. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
8. Occurrences: 1501. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
9. Occurrences: 1415. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
10. Occurrences: 731. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
11. Occurrences: 693. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
12. Occurrences: 569. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
13. Occurrences: 560. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
14. Occurrences: 505. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
15. Occurrences: 440. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
16. Occurrences: 397. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
17. Occurrences: 380. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
18. Occurrences: 269. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
19. Occurrences: 267. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
20. Occurrences: 263. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
21. Occurrences: 247. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
22. Occurrences: 227. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
23. Occurrences: 166. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
24. Occurrences: 152. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
25. Occurrences: 147. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
26. Occurrences: 143. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
27. Occurrences: 143. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
28. Occurrences: 136. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
29. Occurrences: 127. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
30. Occurrences: 126. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
31. Occurrences: 116. Action Sequence: [1.,0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
32. Occurrences: 115. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
33. Occurrences: 114. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
34. Occurrences: 108. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
35. Occurrences: 103. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
36. Occurrences: 94. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
37. Occurrences: 90. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
38. Occurrences: 79. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
39. Occurrences: 76. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
40. Occurrences: 69. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
41. Occurrences: 65. Action Sequence: [0.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
42. Occurrences: 64. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
43. Occurrences: 64. Action Sequence: [0.,1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
44. Occurrences: 59. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
45. Occurrences: 57. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
46. Occurrences: 54. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
47. Occurrences: 53. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
48. Occurrences: 51. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
49. Occurrences: 50. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]
50. Occurrences: 50. Action Sequence: [1.,0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
(pi_b, pi_e): [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 1.0), (0.5, 1.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0), (0.5, 0.0)]

--------------------------------------------------------
'''

### State 56, Non Diabetes

In [ ]:
'''
NON FACTORED SPACE
67227 different trajectories
1. Occurrences: 3404. Action Sequence: [4.]
2. Occurrences: 2967. Action Sequence: [0.]
3. Occurrences: 2460. Action Sequence: [6.]
4. Occurrences: 2086. Action Sequence: [2.]
5. Occurrences: 392. Action Sequence: [5.]
6. Occurrences: 346. Action Sequence: [6.,4.]
7. Occurrences: 327. Action Sequence: [4.,0.]
8. Occurrences: 324. Action Sequence: [1.]
9. Occurrences: 321. Action Sequence: [6.,0.]
10. Occurrences: 309. Action Sequence: [4.,4.]
11. Occurrences: 297. Action Sequence: [2.,4.]
12. Occurrences: 295. Action Sequence: [2.,0.]
13. Occurrences: 291. Action Sequence: [1.,0.]
14. Occurrences: 268. Action Sequence: [0.,4.]
15. Occurrences: 263. Action Sequence: [0.,0.]
16. Occurrences: 254. Action Sequence: [6.,6.]
17. Occurrences: 240. Action Sequence: [6.,2.]
18. Occurrences: 240. Action Sequence: [3.,0.]
19. Occurrences: 220. Action Sequence: [4.,2.]
20. Occurrences: 211. Action Sequence: [4.,6.]
21. Occurrences: 208. Action Sequence: [5.,0.]
22. Occurrences: 200. Action Sequence: [2.,6.]
23. Occurrences: 196. Action Sequence: [5.,4.]
24. Occurrences: 196. Action Sequence: [1.,4.]
25. Occurrences: 186. Action Sequence: [7.,0.]
26. Occurrences: 184. Action Sequence: [2.,2.]
27. Occurrences: 180. Action Sequence: [0.,6.]
28. Occurrences: 173. Action Sequence: [7.,4.]
29. Occurrences: 173. Action Sequence: [0.,2.]
30. Occurrences: 154. Action Sequence: [5.,2.]
31. Occurrences: 153. Action Sequence: [3.,4.]
32. Occurrences: 149. Action Sequence: [1.,5.]
33. Occurrences: 148. Action Sequence: [1.,1.]
34. Occurrences: 140. Action Sequence: [5.,1.]
35. Occurrences: 140. Action Sequence: [7.,2.]
36. Occurrences: 134. Action Sequence: [5.,5.]
37. Occurrences: 113. Action Sequence: [7.,1.]
38. Occurrences: 111. Action Sequence: [3.,6.]
39. Occurrences: 109. Action Sequence: [5.,6.]
40. Occurrences: 106. Action Sequence: [7.,6.]
41. Occurrences: 102. Action Sequence: [1.,2.]
42. Occurrences: 95. Action Sequence: [1.,6.]
43. Occurrences: 90. Action Sequence: [7.,5.]
44. Occurrences: 84. Action Sequence: [3.,1.]
45. Occurrences: 84. Action Sequence: [3.,2.]
46. Occurrences: 80. Action Sequence: [3.,5.]
47. Occurrences: 72. Action Sequence: [0.,1.]
48. Occurrences: 64. Action Sequence: [0.,5.]
49. Occurrences: 50. Action Sequence: [7.,1.,0.]
50. Occurrences: 48. Action Sequence: [2.,5.]

FACTORED SPACE 0, factorisation 1
25045 different trajectories
1. Occurrences: 9671. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 7228. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 4002. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 3844. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1984. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1916. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1873. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1854. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 1001. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 966. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 955. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 920. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 920. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 901. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 864. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 852. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 467. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 461. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 459. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 456. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 454. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 452. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 427. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 425. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 423. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 417. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 409. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 408. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 398. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 389. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 370. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 359. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 244. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 238. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 232. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 228. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 217. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 215. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 215. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 209. Action Sequence: [0.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 205. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 205. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 202. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 191. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 190. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 190. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 188. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 187. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 186. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 185. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 1
53898 different trajectories
1. Occurrences: 5864. Action Sequence: [2.]
2. Occurrences: 5053. Action Sequence: [0.]
3. Occurrences: 1120. Action Sequence: [2.,2.]
4. Occurrences: 1108. Action Sequence: [2.,0.]
5. Occurrences: 945. Action Sequence: [0.,2.]
6. Occurrences: 915. Action Sequence: [0.,0.]
7. Occurrences: 717. Action Sequence: [1.,0.]
8. Occurrences: 688. Action Sequence: [3.,0.]
9. Occurrences: 584. Action Sequence: [3.,2.]
10. Occurrences: 555. Action Sequence: [1.,2.]
11. Occurrences: 392. Action Sequence: [3.]
12. Occurrences: 324. Action Sequence: [1.]
13. Occurrences: 283. Action Sequence: [3.,1.]
14. Occurrences: 248. Action Sequence: [1.,3.]
15. Occurrences: 247. Action Sequence: [1.,1.]
16. Occurrences: 243. Action Sequence: [3.,1.,0.]
17. Occurrences: 240. Action Sequence: [3.,3.]
18. Occurrences: 240. Action Sequence: [3.,3.,0.]
19. Occurrences: 233. Action Sequence: [1.,3.,0.]
20. Occurrences: 220. Action Sequence: [1.,1.,0.]
21. Occurrences: 217. Action Sequence: [2.,2.,2.]
22. Occurrences: 215. Action Sequence: [3.,2.,1.]
23. Occurrences: 215. Action Sequence: [2.,0.,2.]
24. Occurrences: 205. Action Sequence: [2.,2.,0.]
25. Occurrences: 204. Action Sequence: [1.,2.,1.]
26. Occurrences: 202. Action Sequence: [0.,0.,0.]
27. Occurrences: 195. Action Sequence: [3.,0.,0.]
28. Occurrences: 194. Action Sequence: [0.,2.,2.]
29. Occurrences: 192. Action Sequence: [0.,2.,0.]
30. Occurrences: 185. Action Sequence: [1.,0.,3.]
31. Occurrences: 181. Action Sequence: [2.,0.,0.]
32. Occurrences: 181. Action Sequence: [1.,2.,3.]
33. Occurrences: 175. Action Sequence: [0.,0.,2.]
34. Occurrences: 171. Action Sequence: [3.,2.,3.]
35. Occurrences: 169. Action Sequence: [3.,0.,3.]
36. Occurrences: 169. Action Sequence: [0.,1.,0.]
37. Occurrences: 169. Action Sequence: [1.,0.,0.]
38. Occurrences: 163. Action Sequence: [0.,3.,0.]
39. Occurrences: 153. Action Sequence: [3.,2.,0.]
40. Occurrences: 153. Action Sequence: [3.,1.,2.]
41. Occurrences: 151. Action Sequence: [1.,2.,0.]
42. Occurrences: 151. Action Sequence: [0.,1.]
43. Occurrences: 149. Action Sequence: [3.,0.,1.]
44. Occurrences: 147. Action Sequence: [0.,3.]
45. Occurrences: 137. Action Sequence: [3.,3.,2.]
46. Occurrences: 133. Action Sequence: [1.,3.,2.]
47. Occurrences: 133. Action Sequence: [3.,0.,2.]
48. Occurrences: 131. Action Sequence: [1.,0.,1.]
49. Occurrences: 130. Action Sequence: [2.,3.,2.]
50. Occurrences: 117. Action Sequence: [2.,1.,0.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 2
24656 different trajectories
1. Occurrences: 12185. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 4667. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 4479. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2744. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 2354. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 2338. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1874. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1766. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 1198. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 1142. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 1115. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 983. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 917. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 897. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 728. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 658. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 557. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 556. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 531. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 485. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 463. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 462. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 453. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 437. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 435. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 432. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 429. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 365. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 348. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 320. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 283. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 273. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 267. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 259. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 256. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 238. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 235. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 232. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 229. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 222. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 218. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 213. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 209. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 209. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 207. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 206. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 200. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 199. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 195. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 192. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 2
54219 different trajectories
1. Occurrences: 3796. Action Sequence: [2.]
2. Occurrences: 3291. Action Sequence: [0.]
3. Occurrences: 2460. Action Sequence: [3.]
4. Occurrences: 2086. Action Sequence: [1.]
5. Occurrences: 774. Action Sequence: [0.,0.]
6. Occurrences: 698. Action Sequence: [2.,0.]
7. Occurrences: 677. Action Sequence: [0.,2.]
8. Occurrences: 671. Action Sequence: [2.,2.]
9. Occurrences: 656. Action Sequence: [1.,0.]
10. Occurrences: 652. Action Sequence: [3.,0.]
11. Occurrences: 648. Action Sequence: [3.,2.]
12. Occurrences: 578. Action Sequence: [1.,2.]
13. Occurrences: 404. Action Sequence: [2.,1.]
14. Occurrences: 380. Action Sequence: [3.,1.]
15. Occurrences: 360. Action Sequence: [3.,3.]
16. Occurrences: 336. Action Sequence: [2.,3.]
17. Occurrences: 335. Action Sequence: [1.,3.]
18. Occurrences: 314. Action Sequence: [0.,1.]
19. Occurrences: 305. Action Sequence: [0.,3.]
20. Occurrences: 286. Action Sequence: [1.,1.]
21. Occurrences: 211. Action Sequence: [0.,2.,0.]
22. Occurrences: 210. Action Sequence: [3.,2.,0.]
23. Occurrences: 190. Action Sequence: [0.,1.,0.]
24. Occurrences: 190. Action Sequence: [1.,1.,0.]
25. Occurrences: 188. Action Sequence: [0.,0.,0.]
26. Occurrences: 187. Action Sequence: [3.,0.,0.]
27. Occurrences: 180. Action Sequence: [1.,2.,0.]
28. Occurrences: 177. Action Sequence: [1.,3.,0.]
29. Occurrences: 176. Action Sequence: [2.,2.,0.]
30. Occurrences: 174. Action Sequence: [3.,0.,2.]
31. Occurrences: 173. Action Sequence: [3.,3.,0.]
32. Occurrences: 170. Action Sequence: [2.,3.,0.]
33. Occurrences: 167. Action Sequence: [1.,0.,0.]
34. Occurrences: 166. Action Sequence: [3.,3.,2.]
35. Occurrences: 163. Action Sequence: [3.,1.,0.]
36. Occurrences: 161. Action Sequence: [3.,2.,2.]
37. Occurrences: 161. Action Sequence: [0.,3.,0.]
38. Occurrences: 161. Action Sequence: [2.,1.,0.]
39. Occurrences: 160. Action Sequence: [1.,2.,2.]
40. Occurrences: 159. Action Sequence: [2.,2.,2.]
41. Occurrences: 156. Action Sequence: [2.,0.,0.]
42. Occurrences: 155. Action Sequence: [2.,1.,2.]
43. Occurrences: 152. Action Sequence: [0.,2.,2.]
44. Occurrences: 148. Action Sequence: [0.,0.,2.]
45. Occurrences: 147. Action Sequence: [3.,1.,2.]
46. Occurrences: 147. Action Sequence: [2.,0.,2.]
47. Occurrences: 146. Action Sequence: [0.,1.,2.]
48. Occurrences: 140. Action Sequence: [1.,3.,2.]
49. Occurrences: 140. Action Sequence: [0.,3.,2.]
50. Occurrences: 138. Action Sequence: [1.,0.,2.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 3
29540 different trajectories
1. Occurrences: 6256. Action Sequence: [1.]
2. Occurrences: 5377. Action Sequence: [0.]
3. Occurrences: 2134. Action Sequence: [1.,0.]
4. Occurrences: 2030. Action Sequence: [0.,0.]
5. Occurrences: 2015. Action Sequence: [1.,1.]
6. Occurrences: 1895. Action Sequence: [0.,1.]
7. Occurrences: 1117. Action Sequence: [0.,1.,0.]
8. Occurrences: 1074. Action Sequence: [1.,1.,0.]
9. Occurrences: 1039. Action Sequence: [0.,0.,0.]
10. Occurrences: 1010. Action Sequence: [1.,0.,0.]
11. Occurrences: 919. Action Sequence: [1.,0.,1.]
12. Occurrences: 918. Action Sequence: [1.,1.,1.]
13. Occurrences: 876. Action Sequence: [0.,1.,1.]
14. Occurrences: 833. Action Sequence: [0.,0.,1.]
15. Occurrences: 599. Action Sequence: [0.,1.,1.,0.]
16. Occurrences: 583. Action Sequence: [0.,0.,1.,0.]
17. Occurrences: 579. Action Sequence: [1.,0.,1.,0.]
18. Occurrences: 539. Action Sequence: [0.,0.,0.,0.]
19. Occurrences: 538. Action Sequence: [1.,1.,1.,0.]
20. Occurrences: 536. Action Sequence: [0.,1.,0.,0.]
21. Occurrences: 519. Action Sequence: [1.,0.,0.,0.]
22. Occurrences: 499. Action Sequence: [1.,1.,0.,0.]
23. Occurrences: 463. Action Sequence: [1.,0.,1.,1.]
24. Occurrences: 449. Action Sequence: [1.,1.,1.,1.]
25. Occurrences: 443. Action Sequence: [1.,1.,0.,1.]
26. Occurrences: 414. Action Sequence: [0.,1.,0.,1.]
27. Occurrences: 402. Action Sequence: [0.,1.,1.,1.]
28. Occurrences: 390. Action Sequence: [1.,0.,0.,1.]
29. Occurrences: 388. Action Sequence: [0.,0.,1.,1.]
30. Occurrences: 369. Action Sequence: [0.,0.,0.,1.]
31. Occurrences: 290. Action Sequence: [0.,1.,1.,1.,0.]
32. Occurrences: 284. Action Sequence: [0.,0.,1.,1.,0.]
33. Occurrences: 279. Action Sequence: [0.,1.,1.,0.,0.]
34. Occurrences: 276. Action Sequence: [1.,1.,0.,1.,0.]
35. Occurrences: 274. Action Sequence: [0.,1.,0.,1.,0.]
36. Occurrences: 267. Action Sequence: [1.,1.,1.,1.,0.]
37. Occurrences: 259. Action Sequence: [1.,0.,1.,1.,0.]
38. Occurrences: 258. Action Sequence: [1.,0.,1.,0.,0.]
39. Occurrences: 258. Action Sequence: [0.,1.,0.,0.,0.]
40. Occurrences: 256. Action Sequence: [1.,0.,0.,1.,0.]
41. Occurrences: 252. Action Sequence: [0.,0.,0.,1.,0.]
42. Occurrences: 251. Action Sequence: [0.,0.,0.,0.,0.]
43. Occurrences: 248. Action Sequence: [1.,1.,1.,0.,0.]
44. Occurrences: 248. Action Sequence: [1.,1.,0.,0.,0.]
45. Occurrences: 247. Action Sequence: [0.,0.,1.,0.,0.]
46. Occurrences: 219. Action Sequence: [1.,1.,1.,0.,1.]
47. Occurrences: 219. Action Sequence: [1.,0.,0.,0.,0.]
48. Occurrences: 217. Action Sequence: [1.,0.,1.,0.,1.]
49. Occurrences: 213. Action Sequence: [1.,0.,1.,1.,1.]
50. Occurrences: 208. Action Sequence: [1.,1.,0.,1.,1.]

FACTORED SPACE 1, factorisation 3
51908 different trajectories
1. Occurrences: 6409. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
2. Occurrences: 4590. Action Sequence: [2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
3. Occurrences: 1282. Action Sequence: [2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
4. Occurrences: 1186. Action Sequence: [0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
5. Occurrences: 1016. Action Sequence: [1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
6. Occurrences: 907. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
7. Occurrences: 896. Action Sequence: [2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
8. Occurrences: 808. Action Sequence: [0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
9. Occurrences: 796. Action Sequence: [1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
10. Occurrences: 599. Action Sequence: [1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
11. Occurrences: 584. Action Sequence: [1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
12. Occurrences: 570. Action Sequence: [3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
13. Occurrences: 379. Action Sequence: [3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
14. Occurrences: 275. Action Sequence: [1.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
15. Occurrences: 271. Action Sequence: [3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
16. Occurrences: 267. Action Sequence: [1.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
17. Occurrences: 267. Action Sequence: [3.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
18. Occurrences: 264. Action Sequence: [2.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
19. Occurrences: 260. Action Sequence: [2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
20. Occurrences: 250. Action Sequence: [3.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
21. Occurrences: 246. Action Sequence: [1.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
22. Occurrences: 241. Action Sequence: [0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
23. Occurrences: 238. Action Sequence: [1.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
24. Occurrences: 236. Action Sequence: [3.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
25. Occurrences: 226. Action Sequence: [1.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
26. Occurrences: 222. Action Sequence: [1.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
27. Occurrences: 212. Action Sequence: [0.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
28. Occurrences: 211. Action Sequence: [3.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
29. Occurrences: 208. Action Sequence: [0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
30. Occurrences: 207. Action Sequence: [2.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
31. Occurrences: 205. Action Sequence: [2.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
32. Occurrences: 201. Action Sequence: [0.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
33. Occurrences: 192. Action Sequence: [3.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
34. Occurrences: 179. Action Sequence: [2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
35. Occurrences: 178. Action Sequence: [3.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
36. Occurrences: 171. Action Sequence: [0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
37. Occurrences: 168. Action Sequence: [1.,1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
38. Occurrences: 168. Action Sequence: [3.,0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
39. Occurrences: 167. Action Sequence: [2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
40. Occurrences: 167. Action Sequence: [2.,0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
41. Occurrences: 164. Action Sequence: [1.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
42. Occurrences: 164. Action Sequence: [0.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
43. Occurrences: 163. Action Sequence: [3.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
44. Occurrences: 156. Action Sequence: [1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
45. Occurrences: 153. Action Sequence: [0.,0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
46. Occurrences: 151. Action Sequence: [1.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
47. Occurrences: 144. Action Sequence: [2.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
48. Occurrences: 141. Action Sequence: [3.,1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
49. Occurrences: 135. Action Sequence: [1.,0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
50. Occurrences: 134. Action Sequence: [3.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 4
29540 different trajectories
1. Occurrences: 6256. Action Sequence: [1.]
2. Occurrences: 5377. Action Sequence: [0.]
3. Occurrences: 2134. Action Sequence: [1.,0.]
4. Occurrences: 2030. Action Sequence: [0.,0.]
5. Occurrences: 2015. Action Sequence: [1.,1.]
6. Occurrences: 1895. Action Sequence: [0.,1.]
7. Occurrences: 1117. Action Sequence: [0.,1.,0.]
8. Occurrences: 1074. Action Sequence: [1.,1.,0.]
9. Occurrences: 1039. Action Sequence: [0.,0.,0.]
10. Occurrences: 1010. Action Sequence: [1.,0.,0.]
11. Occurrences: 919. Action Sequence: [1.,0.,1.]
12. Occurrences: 918. Action Sequence: [1.,1.,1.]
13. Occurrences: 876. Action Sequence: [0.,1.,1.]
14. Occurrences: 833. Action Sequence: [0.,0.,1.]
15. Occurrences: 599. Action Sequence: [0.,1.,1.,0.]
16. Occurrences: 583. Action Sequence: [0.,0.,1.,0.]
17. Occurrences: 579. Action Sequence: [1.,0.,1.,0.]
18. Occurrences: 539. Action Sequence: [0.,0.,0.,0.]
19. Occurrences: 538. Action Sequence: [1.,1.,1.,0.]
20. Occurrences: 536. Action Sequence: [0.,1.,0.,0.]
21. Occurrences: 519. Action Sequence: [1.,0.,0.,0.]
22. Occurrences: 499. Action Sequence: [1.,1.,0.,0.]
23. Occurrences: 463. Action Sequence: [1.,0.,1.,1.]
24. Occurrences: 449. Action Sequence: [1.,1.,1.,1.]
25. Occurrences: 443. Action Sequence: [1.,1.,0.,1.]
26. Occurrences: 414. Action Sequence: [0.,1.,0.,1.]
27. Occurrences: 402. Action Sequence: [0.,1.,1.,1.]
28. Occurrences: 390. Action Sequence: [1.,0.,0.,1.]
29. Occurrences: 388. Action Sequence: [0.,0.,1.,1.]
30. Occurrences: 369. Action Sequence: [0.,0.,0.,1.]
31. Occurrences: 290. Action Sequence: [0.,1.,1.,1.,0.]
32. Occurrences: 284. Action Sequence: [0.,0.,1.,1.,0.]
33. Occurrences: 279. Action Sequence: [0.,1.,1.,0.,0.]
34. Occurrences: 276. Action Sequence: [1.,1.,0.,1.,0.]
35. Occurrences: 274. Action Sequence: [0.,1.,0.,1.,0.]
36. Occurrences: 267. Action Sequence: [1.,1.,1.,1.,0.]
37. Occurrences: 259. Action Sequence: [1.,0.,1.,1.,0.]
38. Occurrences: 258. Action Sequence: [1.,0.,1.,0.,0.]
39. Occurrences: 258. Action Sequence: [0.,1.,0.,0.,0.]
40. Occurrences: 256. Action Sequence: [1.,0.,0.,1.,0.]
41. Occurrences: 252. Action Sequence: [0.,0.,0.,1.,0.]
42. Occurrences: 251. Action Sequence: [0.,0.,0.,0.,0.]
43. Occurrences: 248. Action Sequence: [1.,1.,1.,0.,0.]
44. Occurrences: 248. Action Sequence: [1.,1.,0.,0.,0.]
45. Occurrences: 247. Action Sequence: [0.,0.,1.,0.,0.]
46. Occurrences: 219. Action Sequence: [1.,1.,1.,0.,1.]
47. Occurrences: 219. Action Sequence: [1.,0.,0.,0.,0.]
48. Occurrences: 217. Action Sequence: [1.,0.,1.,0.,1.]
49. Occurrences: 213. Action Sequence: [1.,0.,1.,1.,1.]
50. Occurrences: 208. Action Sequence: [1.,1.,0.,1.,1.]

FACTORED SPACE 1, factorisation 4
25045 different trajectories
1. Occurrences: 9671. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 7228. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 4002. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 3844. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1984. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1916. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1873. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1854. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 1001. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 966. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 955. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 920. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 920. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 901. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 864. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 852. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 467. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 461. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 459. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 456. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 454. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 452. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 427. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 425. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 423. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 417. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 409. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 408. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 398. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 389. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 370. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 359. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 244. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 238. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 232. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 228. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 217. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 215. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 215. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 209. Action Sequence: [0.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 205. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 205. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 202. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 191. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 190. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 190. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 188. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 187. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 186. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 185. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 2, factorisation 4
24656 different trajectories
1. Occurrences: 12185. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 4667. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 4479. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2744. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 2354. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 2338. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1874. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1766. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 1198. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 1142. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 1115. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 983. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 917. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 897. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 728. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 658. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 557. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 556. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 531. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 485. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 463. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 462. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 453. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 437. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 435. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 432. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 429. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 365. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 348. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 320. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 283. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 273. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 267. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 259. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 256. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 238. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 235. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 232. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 229. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 222. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 218. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 213. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 209. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 209. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 207. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 206. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 200. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 199. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 195. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 192. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

--------------------------------------------------------
'''

### State 377, Diabetes

In [ ]:
'''
NON FACTORED SPACE
68277 different trajectories
1. Occurrences: 3537. Action Sequence: [0.]
2. Occurrences: 1675. Action Sequence: [1.]
3. Occurrences: 1111. Action Sequence: [3.]
4. Occurrences: 584. Action Sequence: [5.]
5. Occurrences: 436. Action Sequence: [6.,0.]
6. Occurrences: 394. Action Sequence: [4.,0.]
7. Occurrences: 387. Action Sequence: [3.,1.]
8. Occurrences: 384. Action Sequence: [1.,1.]
9. Occurrences: 345. Action Sequence: [3.,0.]
10. Occurrences: 343. Action Sequence: [0.,1.]
11. Occurrences: 342. Action Sequence: [2.,1.]
12. Occurrences: 338. Action Sequence: [2.,0.]
13. Occurrences: 331. Action Sequence: [2.]
14. Occurrences: 317. Action Sequence: [1.,0.]
15. Occurrences: 273. Action Sequence: [2.,3.]
16. Occurrences: 270. Action Sequence: [3.,3.]
17. Occurrences: 258. Action Sequence: [5.,0.]
18. Occurrences: 258. Action Sequence: [4.,1.]
19. Occurrences: 256. Action Sequence: [7.,0.]
20. Occurrences: 254. Action Sequence: [5.,1.]
21. Occurrences: 253. Action Sequence: [0.,3.]
22. Occurrences: 249. Action Sequence: [0.,0.]
23. Occurrences: 246. Action Sequence: [0.,5.]
24. Occurrences: 237. Action Sequence: [1.,3.]
25. Occurrences: 218. Action Sequence: [1.,2.]
26. Occurrences: 202. Action Sequence: [7.,1.]
27. Occurrences: 198. Action Sequence: [2.,5.]
28. Occurrences: 189. Action Sequence: [3.,2.]
29. Occurrences: 177. Action Sequence: [1.,5.]
30. Occurrences: 174. Action Sequence: [6.,1.]
31. Occurrences: 164. Action Sequence: [3.,5.]
32. Occurrences: 146. Action Sequence: [4.,5.]
33. Occurrences: 138. Action Sequence: [0.,7.]
34. Occurrences: 131. Action Sequence: [5.,5.]
35. Occurrences: 122. Action Sequence: [5.,3.]
36. Occurrences: 119. Action Sequence: [4.,3.]
37. Occurrences: 116. Action Sequence: [2.,7.]
38. Occurrences: 106. Action Sequence: [7.,3.]
39. Occurrences: 102. Action Sequence: [2.,2.]
40. Occurrences: 95. Action Sequence: [7.,5.]
41. Occurrences: 91. Action Sequence: [1.,4.]
42. Occurrences: 90. Action Sequence: [6.,5.]
43. Occurrences: 87. Action Sequence: [0.,2.]
44. Occurrences: 86. Action Sequence: [5.,2.]
45. Occurrences: 83. Action Sequence: [3.,4.]
46. Occurrences: 82. Action Sequence: [6.,3.]
47. Occurrences: 74. Action Sequence: [1.,7.]
48. Occurrences: 69. Action Sequence: [3.,7.]
49. Occurrences: 66. Action Sequence: [7.,2.]
50. Occurrences: 57. Action Sequence: [2.,2.,3.]

FACTORED SPACE 0, factorisation 1
26687 different trajectories
1. Occurrences: 8534. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 4642. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 4551. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 4197. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 2264. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 2188. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 2086. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 2056. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 1061. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 1004. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 968. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 938. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 938. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 921. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 909. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 908. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 490. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 484. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 477. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 449. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 448. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 440. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 419. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 419. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 417. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 415. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 412. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 408. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 379. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 362. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 338. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 334. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 239. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 231. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 228. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 217. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 212. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 209. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 206. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 205. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 203. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 201. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 200. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 197. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 197. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 190. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 187. Action Sequence: [0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 185. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 184. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 184. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 1
54921 different trajectories
1. Occurrences: 3868. Action Sequence: [0.]
2. Occurrences: 2786. Action Sequence: [1.]
3. Occurrences: 1278. Action Sequence: [1.,1.]
4. Occurrences: 1211. Action Sequence: [0.,1.]
5. Occurrences: 1069. Action Sequence: [1.,0.]
6. Occurrences: 858. Action Sequence: [2.,0.]
7. Occurrences: 776. Action Sequence: [0.,0.]
8. Occurrences: 698. Action Sequence: [0.,3.]
9. Occurrences: 684. Action Sequence: [3.,1.]
10. Occurrences: 666. Action Sequence: [3.,0.]
11. Occurrences: 633. Action Sequence: [2.,1.]
12. Occurrences: 584. Action Sequence: [3.]
13. Occurrences: 484. Action Sequence: [1.,3.]
14. Occurrences: 334. Action Sequence: [0.,0.,1.]
15. Occurrences: 321. Action Sequence: [3.,1.,1.]
16. Occurrences: 308. Action Sequence: [2.,0.,1.]
17. Occurrences: 293. Action Sequence: [1.,0.,1.]
18. Occurrences: 287. Action Sequence: [2.,1.,1.]
19. Occurrences: 284. Action Sequence: [3.,0.,1.]
20. Occurrences: 279. Action Sequence: [2.,1.,0.]
21. Occurrences: 267. Action Sequence: [1.,1.,1.]
22. Occurrences: 263. Action Sequence: [0.,1.,1.]
23. Occurrences: 258. Action Sequence: [2.,3.]
24. Occurrences: 249. Action Sequence: [1.,2.]
25. Occurrences: 248. Action Sequence: [0.,2.,1.]
26. Occurrences: 244. Action Sequence: [3.,0.,0.]
27. Occurrences: 243. Action Sequence: [3.,3.]
28. Occurrences: 241. Action Sequence: [3.,1.,0.]
29. Occurrences: 232. Action Sequence: [1.,0.,0.]
30. Occurrences: 215. Action Sequence: [1.,1.,0.]
31. Occurrences: 209. Action Sequence: [0.,1.,0.]
32. Occurrences: 206. Action Sequence: [3.,3.,1.]
33. Occurrences: 203. Action Sequence: [1.,2.,1.]
34. Occurrences: 196. Action Sequence: [2.,2.,0.]
35. Occurrences: 191. Action Sequence: [2.,2.,1.]
36. Occurrences: 189. Action Sequence: [1.,3.,1.]
37. Occurrences: 187. Action Sequence: [2.,3.,1.]
38. Occurrences: 186. Action Sequence: [3.,2.,1.]
39. Occurrences: 184. Action Sequence: [0.,0.,3.]
40. Occurrences: 181. Action Sequence: [1.,3.,0.]
41. Occurrences: 181. Action Sequence: [3.,2.,0.]
42. Occurrences: 177. Action Sequence: [0.,0.,0.]
43. Occurrences: 175. Action Sequence: [0.,3.,0.]
44. Occurrences: 164. Action Sequence: [0.,3.,1.]
45. Occurrences: 164. Action Sequence: [3.,3.,0.]
46. Occurrences: 160. Action Sequence: [0.,2.,3.]
47. Occurrences: 159. Action Sequence: [1.,2.,0.]
48. Occurrences: 155. Action Sequence: [0.,2.,0.]
49. Occurrences: 155. Action Sequence: [2.,3.,0.]
50. Occurrences: 153. Action Sequence: [2.,0.,0.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 2
25596 different trajectories
1. Occurrences: 8836. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 8381. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 4382. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 4347. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 2014. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 2003. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1991. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1986. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 945. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 942. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 934. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 932. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 932. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 931. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 880. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 854. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 440. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 426. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 421. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 420. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 413. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 413. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 408. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 403. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 401. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 388. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 383. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 382. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 370. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 363. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 354. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 329. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 219. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 208. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 205. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 199. Action Sequence: [0.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 198. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 198. Action Sequence: [0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 190. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 189. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 185. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 185. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 183. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 182. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 182. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 182. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 181. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 181. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 181. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 178. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 2
54838 different trajectories
1. Occurrences: 5212. Action Sequence: [0.]
2. Occurrences: 1442. Action Sequence: [1.]
3. Occurrences: 1412. Action Sequence: [1.,0.]
4. Occurrences: 1293. Action Sequence: [0.,0.]
5. Occurrences: 1164. Action Sequence: [2.,0.]
6. Occurrences: 1068. Action Sequence: [3.,0.]
7. Occurrences: 834. Action Sequence: [1.,1.]
8. Occurrences: 795. Action Sequence: [0.,1.]
9. Occurrences: 584. Action Sequence: [2.]
10. Occurrences: 560. Action Sequence: [0.,2.]
11. Occurrences: 476. Action Sequence: [1.,2.]
12. Occurrences: 367. Action Sequence: [2.,1.,0.]
13. Occurrences: 346. Action Sequence: [1.,1.,0.]
14. Occurrences: 345. Action Sequence: [2.,1.]
15. Occurrences: 320. Action Sequence: [3.,1.,0.]
16. Occurrences: 318. Action Sequence: [3.,0.,0.]
17. Occurrences: 317. Action Sequence: [2.,0.,0.]
18. Occurrences: 311. Action Sequence: [2.,2.]
19. Occurrences: 293. Action Sequence: [1.,0.,0.]
20. Occurrences: 287. Action Sequence: [2.,2.,0.]
21. Occurrences: 273. Action Sequence: [1.,2.,0.]
22. Occurrences: 273. Action Sequence: [0.,0.,0.]
23. Occurrences: 270. Action Sequence: [3.,3.,0.]
24. Occurrences: 268. Action Sequence: [0.,3.]
25. Occurrences: 264. Action Sequence: [3.,1.]
26. Occurrences: 264. Action Sequence: [2.,3.,0.]
27. Occurrences: 258. Action Sequence: [0.,1.,0.]
28. Occurrences: 255. Action Sequence: [1.,3.,0.]
29. Occurrences: 254. Action Sequence: [3.,2.,0.]
30. Occurrences: 253. Action Sequence: [1.,1.,1.]
31. Occurrences: 228. Action Sequence: [1.,3.]
32. Occurrences: 228. Action Sequence: [3.,2.]
33. Occurrences: 225. Action Sequence: [0.,2.,0.]
34. Occurrences: 225. Action Sequence: [0.,3.,0.]
35. Occurrences: 207. Action Sequence: [3.,0.,1.]
36. Occurrences: 204. Action Sequence: [1.,0.,1.]
37. Occurrences: 202. Action Sequence: [3.,1.,1.]
38. Occurrences: 197. Action Sequence: [2.,0.,1.]
39. Occurrences: 189. Action Sequence: [2.,1.,1.]
40. Occurrences: 186. Action Sequence: [0.,0.,1.]
41. Occurrences: 177. Action Sequence: [0.,1.,1.]
42. Occurrences: 137. Action Sequence: [1.,0.,2.]
43. Occurrences: 134. Action Sequence: [1.,3.,1.]
44. Occurrences: 129. Action Sequence: [0.,3.,1.]
45. Occurrences: 123. Action Sequence: [1.,2.,1.]
46. Occurrences: 113. Action Sequence: [0.,2.,2.]
47. Occurrences: 110. Action Sequence: [0.,2.,1.]
48. Occurrences: 109. Action Sequence: [1.,1.,2.]
49. Occurrences: 108. Action Sequence: [3.,2.,2.]
50. Occurrences: 107. Action Sequence: [2.,2.,2.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 3
30465 different trajectories
1. Occurrences: 6654. Action Sequence: [0.]
2. Occurrences: 4334. Action Sequence: [0.,0.]
3. Occurrences: 2841. Action Sequence: [1.,0.]
4. Occurrences: 2117. Action Sequence: [1.,0.,0.]
5. Occurrences: 1990. Action Sequence: [0.,0.,0.]
6. Occurrences: 1532. Action Sequence: [0.,1.]
7. Occurrences: 1474. Action Sequence: [0.,1.,0.]
8. Occurrences: 1466. Action Sequence: [1.,1.,0.]
9. Occurrences: 1017. Action Sequence: [1.,1.,0.,0.]
10. Occurrences: 983. Action Sequence: [1.,0.,0.,0.]
11. Occurrences: 932. Action Sequence: [0.,1.,0.,0.]
12. Occurrences: 828. Action Sequence: [0.,0.,0.,0.]
13. Occurrences: 700. Action Sequence: [1.,1.,1.,0.]
14. Occurrences: 659. Action Sequence: [0.,0.,1.]
15. Occurrences: 659. Action Sequence: [1.,0.,1.,0.]
16. Occurrences: 655. Action Sequence: [0.,1.,1.,0.]
17. Occurrences: 584. Action Sequence: [1.]
18. Occurrences: 582. Action Sequence: [0.,0.,1.,0.]
19. Occurrences: 581. Action Sequence: [1.,1.]
20. Occurrences: 565. Action Sequence: [0.,1.,1.]
21. Occurrences: 518. Action Sequence: [1.,0.,1.,0.,0.]
22. Occurrences: 491. Action Sequence: [1.,1.,1.,0.,0.]
23. Occurrences: 444. Action Sequence: [0.,1.,1.,0.,0.]
24. Occurrences: 429. Action Sequence: [1.,1.,0.,0.,0.]
25. Occurrences: 421. Action Sequence: [1.,0.,1.]
26. Occurrences: 414. Action Sequence: [0.,1.,0.,0.,0.]
27. Occurrences: 389. Action Sequence: [1.,1.,1.]
28. Occurrences: 357. Action Sequence: [1.,0.,0.,0.,0.]
29. Occurrences: 350. Action Sequence: [0.,0.,1.,0.,0.]
30. Occurrences: 326. Action Sequence: [1.,1.,1.,1.,0.]
31. Occurrences: 308. Action Sequence: [1.,0.,1.,1.,0.]
32. Occurrences: 308. Action Sequence: [0.,0.,0.,0.,0.]
33. Occurrences: 307. Action Sequence: [1.,1.,0.,1.,0.]
34. Occurrences: 301. Action Sequence: [0.,1.,1.,1.,0.]
35. Occurrences: 286. Action Sequence: [0.,1.,0.,1.,0.]
36. Occurrences: 281. Action Sequence: [1.,0.,0.,1.]
37. Occurrences: 278. Action Sequence: [1.,0.,0.,1.,0.]
38. Occurrences: 264. Action Sequence: [0.,0.,0.,1.]
39. Occurrences: 258. Action Sequence: [0.,0.,1.,1.,0.]
40. Occurrences: 253. Action Sequence: [0.,0.,1.,1.]
41. Occurrences: 247. Action Sequence: [1.,1.,0.,1.,0.,0.]
42. Occurrences: 236. Action Sequence: [1.,1.,1.,1.,0.,0.]
43. Occurrences: 233. Action Sequence: [1.,1.,0.,1.]
44. Occurrences: 231. Action Sequence: [0.,1.,0.,1.]
45. Occurrences: 225. Action Sequence: [0.,1.,1.,1.]
46. Occurrences: 221. Action Sequence: [0.,0.,0.,1.,0.]
47. Occurrences: 217. Action Sequence: [1.,1.,1.,1.]
48. Occurrences: 215. Action Sequence: [1.,0.,1.,1.,0.,0.]
49. Occurrences: 213. Action Sequence: [0.,1.,1.,1.,0.,0.]
50. Occurrences: 202. Action Sequence: [1.,1.,1.,0.,0.,0.]

FACTORED SPACE 1, factorisation 3
53136 different trajectories
1. Occurrences: 4203. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
2. Occurrences: 2850. Action Sequence: [1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
3. Occurrences: 1718. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
4. Occurrences: 1121. Action Sequence: [0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
5. Occurrences: 1090. Action Sequence: [1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
6. Occurrences: 1003. Action Sequence: [2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
7. Occurrences: 998. Action Sequence: [3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
8. Occurrences: 949. Action Sequence: [2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
9. Occurrences: 945. Action Sequence: [2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
10. Occurrences: 876. Action Sequence: [0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
11. Occurrences: 866. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
12. Occurrences: 862. Action Sequence: [1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
13. Occurrences: 484. Action Sequence: [1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
14. Occurrences: 430. Action Sequence: [3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
15. Occurrences: 317. Action Sequence: [2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
16. Occurrences: 317. Action Sequence: [2.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
17. Occurrences: 309. Action Sequence: [0.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
18. Occurrences: 285. Action Sequence: [2.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
19. Occurrences: 285. Action Sequence: [0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
20. Occurrences: 258. Action Sequence: [1.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
21. Occurrences: 253. Action Sequence: [3.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
22. Occurrences: 252. Action Sequence: [3.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
23. Occurrences: 251. Action Sequence: [3.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
24. Occurrences: 243. Action Sequence: [2.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
25. Occurrences: 242. Action Sequence: [1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
26. Occurrences: 240. Action Sequence: [1.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
27. Occurrences: 234. Action Sequence: [3.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
28. Occurrences: 232. Action Sequence: [0.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
29. Occurrences: 226. Action Sequence: [2.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
30. Occurrences: 225. Action Sequence: [1.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
31. Occurrences: 221. Action Sequence: [0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
32. Occurrences: 219. Action Sequence: [2.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
33. Occurrences: 219. Action Sequence: [2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
34. Occurrences: 206. Action Sequence: [1.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
35. Occurrences: 206. Action Sequence: [3.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
36. Occurrences: 204. Action Sequence: [2.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
37. Occurrences: 202. Action Sequence: [1.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
38. Occurrences: 202. Action Sequence: [0.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
39. Occurrences: 200. Action Sequence: [2.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
40. Occurrences: 200. Action Sequence: [3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
41. Occurrences: 199. Action Sequence: [3.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
42. Occurrences: 199. Action Sequence: [0.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
43. Occurrences: 198. Action Sequence: [3.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
44. Occurrences: 192. Action Sequence: [1.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
45. Occurrences: 185. Action Sequence: [0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
46. Occurrences: 185. Action Sequence: [0.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
47. Occurrences: 174. Action Sequence: [1.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
48. Occurrences: 173. Action Sequence: [0.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
49. Occurrences: 133. Action Sequence: [2.,1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
50. Occurrences: 125. Action Sequence: [3.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 4
30465 different trajectories
1. Occurrences: 6654. Action Sequence: [0.]
2. Occurrences: 4334. Action Sequence: [0.,0.]
3. Occurrences: 2841. Action Sequence: [1.,0.]
4. Occurrences: 2117. Action Sequence: [1.,0.,0.]
5. Occurrences: 1990. Action Sequence: [0.,0.,0.]
6. Occurrences: 1532. Action Sequence: [0.,1.]
7. Occurrences: 1474. Action Sequence: [0.,1.,0.]
8. Occurrences: 1466. Action Sequence: [1.,1.,0.]
9. Occurrences: 1017. Action Sequence: [1.,1.,0.,0.]
10. Occurrences: 983. Action Sequence: [1.,0.,0.,0.]
11. Occurrences: 932. Action Sequence: [0.,1.,0.,0.]
12. Occurrences: 828. Action Sequence: [0.,0.,0.,0.]
13. Occurrences: 700. Action Sequence: [1.,1.,1.,0.]
14. Occurrences: 659. Action Sequence: [0.,0.,1.]
15. Occurrences: 659. Action Sequence: [1.,0.,1.,0.]
16. Occurrences: 655. Action Sequence: [0.,1.,1.,0.]
17. Occurrences: 584. Action Sequence: [1.]
18. Occurrences: 582. Action Sequence: [0.,0.,1.,0.]
19. Occurrences: 581. Action Sequence: [1.,1.]
20. Occurrences: 565. Action Sequence: [0.,1.,1.]
21. Occurrences: 518. Action Sequence: [1.,0.,1.,0.,0.]
22. Occurrences: 491. Action Sequence: [1.,1.,1.,0.,0.]
23. Occurrences: 444. Action Sequence: [0.,1.,1.,0.,0.]
24. Occurrences: 429. Action Sequence: [1.,1.,0.,0.,0.]
25. Occurrences: 421. Action Sequence: [1.,0.,1.]
26. Occurrences: 414. Action Sequence: [0.,1.,0.,0.,0.]
27. Occurrences: 389. Action Sequence: [1.,1.,1.]
28. Occurrences: 357. Action Sequence: [1.,0.,0.,0.,0.]
29. Occurrences: 350. Action Sequence: [0.,0.,1.,0.,0.]
30. Occurrences: 326. Action Sequence: [1.,1.,1.,1.,0.]
31. Occurrences: 308. Action Sequence: [1.,0.,1.,1.,0.]
32. Occurrences: 308. Action Sequence: [0.,0.,0.,0.,0.]
33. Occurrences: 307. Action Sequence: [1.,1.,0.,1.,0.]
34. Occurrences: 301. Action Sequence: [0.,1.,1.,1.,0.]
35. Occurrences: 286. Action Sequence: [0.,1.,0.,1.,0.]
36. Occurrences: 281. Action Sequence: [1.,0.,0.,1.]
37. Occurrences: 278. Action Sequence: [1.,0.,0.,1.,0.]
38. Occurrences: 264. Action Sequence: [0.,0.,0.,1.]
39. Occurrences: 258. Action Sequence: [0.,0.,1.,1.,0.]
40. Occurrences: 253. Action Sequence: [0.,0.,1.,1.]
41. Occurrences: 247. Action Sequence: [1.,1.,0.,1.,0.,0.]
42. Occurrences: 236. Action Sequence: [1.,1.,1.,1.,0.,0.]
43. Occurrences: 233. Action Sequence: [1.,1.,0.,1.]
44. Occurrences: 231. Action Sequence: [0.,1.,0.,1.]
45. Occurrences: 225. Action Sequence: [0.,1.,1.,1.]
46. Occurrences: 221. Action Sequence: [0.,0.,0.,1.,0.]
47. Occurrences: 217. Action Sequence: [1.,1.,1.,1.]
48. Occurrences: 215. Action Sequence: [1.,0.,1.,1.,0.,0.]
49. Occurrences: 213. Action Sequence: [0.,1.,1.,1.,0.,0.]
50. Occurrences: 202. Action Sequence: [1.,1.,1.,0.,0.,0.]

FACTORED SPACE 1, factorisation 4
26687 different trajectories
1. Occurrences: 8534. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 4642. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 4551. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 4197. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 2264. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 2188. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 2086. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 2056. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 1061. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 1004. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 968. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 938. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 938. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 921. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 909. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 908. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 490. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 484. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 477. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 449. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 448. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 440. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 419. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 419. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 417. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 415. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 412. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 408. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 379. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 362. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 338. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 334. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 239. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 231. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 228. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 217. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 212. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 209. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 206. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 205. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 203. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 201. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 200. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 197. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 197. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 190. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 187. Action Sequence: [0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 185. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 184. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 184. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 2, factorisation 4
25596 different trajectories
1. Occurrences: 8836. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 8381. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 4382. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 4347. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 2014. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 2003. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1991. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1986. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 945. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 942. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 934. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 932. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 932. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 931. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 880. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 854. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 440. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 426. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 421. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 420. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 413. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 413. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 408. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 403. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 401. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 388. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 383. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 382. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 370. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 363. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 354. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 329. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 219. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 208. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 205. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 199. Action Sequence: [0.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 198. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 198. Action Sequence: [0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 190. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 189. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 185. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 185. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 183. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 182. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 182. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 182. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 181. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 181. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 181. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 178. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

--------------------------------------------------------
'''

### State 377, No Diabetes

In [ ]:
'''
NON FACTORED SPACE
80426 different trajectories
1. Occurrences: 5973. Action Sequence: [0.]
2. Occurrences: 730. Action Sequence: [6.,0.]
3. Occurrences: 666. Action Sequence: [4.,0.]
4. Occurrences: 447. Action Sequence: [2.,0.]
5. Occurrences: 378. Action Sequence: [3.,0.]
6. Occurrences: 371. Action Sequence: [5.,0.]
7. Occurrences: 366. Action Sequence: [7.,0.]
8. Occurrences: 361. Action Sequence: [1.,0.]
9. Occurrences: 168. Action Sequence: [1.]
10. Occurrences: 135. Action Sequence: [0.,1.]
11. Occurrences: 113. Action Sequence: [2.,1.]
12. Occurrences: 112. Action Sequence: [0.,0.]
13. Occurrences: 89. Action Sequence: [4.,1.]
14. Occurrences: 82. Action Sequence: [2.]
15. Occurrences: 79. Action Sequence: [3.,2.]
16. Occurrences: 76. Action Sequence: [0.,5.]
17. Occurrences: 75. Action Sequence: [4.,6.,0.]
18. Occurrences: 74. Action Sequence: [6.,4.,0.]
19. Occurrences: 73. Action Sequence: [1.,1.]
20. Occurrences: 73. Action Sequence: [0.,3.]
21. Occurrences: 72. Action Sequence: [4.,4.,0.]
22. Occurrences: 72. Action Sequence: [6.,6.,0.]
23. Occurrences: 65. Action Sequence: [2.,2.]
24. Occurrences: 65. Action Sequence: [2.,6.,0.]
25. Occurrences: 63. Action Sequence: [5.,6.,0.]
26. Occurrences: 62. Action Sequence: [1.,2.]
27. Occurrences: 62. Action Sequence: [7.,6.,0.]
28. Occurrences: 62. Action Sequence: [2.,3.]
29. Occurrences: 60. Action Sequence: [2.,5.]
30. Occurrences: 60. Action Sequence: [1.,4.,0.]
31. Occurrences: 60. Action Sequence: [7.,4.,0.]
32. Occurrences: 59. Action Sequence: [6.,3.,0.]
33. Occurrences: 56. Action Sequence: [3.,6.,0.]
34. Occurrences: 54. Action Sequence: [2.,4.,0.]
35. Occurrences: 52. Action Sequence: [7.,2.,0.]
36. Occurrences: 51. Action Sequence: [2.,3.,0.]
37. Occurrences: 51. Action Sequence: [4.,2.,0.]
38. Occurrences: 50. Action Sequence: [0.,2.]
39. Occurrences: 49. Action Sequence: [6.,2.,0.]
40. Occurrences: 49. Action Sequence: [5.,4.,0.]
41. Occurrences: 49. Action Sequence: [1.,5.,0.]
42. Occurrences: 48. Action Sequence: [1.,2.,0.]
43. Occurrences: 48. Action Sequence: [6.,1.]
44. Occurrences: 48. Action Sequence: [1.,4.]
45. Occurrences: 47. Action Sequence: [6.,1.,0.]
46. Occurrences: 46. Action Sequence: [3.,1.]
47. Occurrences: 46. Action Sequence: [2.,5.,0.]
48. Occurrences: 45. Action Sequence: [3.,1.,0.]
49. Occurrences: 44. Action Sequence: [4.,5.]
50. Occurrences: 43. Action Sequence: [1.,6.,0.]

FACTORED SPACE 0, factorisation 1
40943 different trajectories
1. Occurrences: 7034. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 2983. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 2642. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 1407. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1306. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1268. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1263. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1012. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 701. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 673. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 652. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 650. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 649. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 645. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 619. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 583. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 388. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 375. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 370. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 365. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 356. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 347. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 340. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 336. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 332. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 309. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 306. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 306. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 305. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 302. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 280. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 262. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 206. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 201. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 186. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 179. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 175. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 170. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 170. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 170. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 170. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 170. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 169. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 166. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 165. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 164. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 161. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 159. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 158. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 157. Action Sequence: [0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 1
70725 different trajectories
1. Occurrences: 6055. Action Sequence: [0.]
2. Occurrences: 1407. Action Sequence: [2.,0.]
3. Occurrences: 880. Action Sequence: [1.,0.]
4. Occurrences: 792. Action Sequence: [3.,0.]
5. Occurrences: 674. Action Sequence: [0.,0.]
6. Occurrences: 383. Action Sequence: [0.,1.]
7. Occurrences: 298. Action Sequence: [2.,2.,0.]
8. Occurrences: 248. Action Sequence: [3.,2.,0.]
9. Occurrences: 233. Action Sequence: [2.,1.,0.]
10. Occurrences: 219. Action Sequence: [1.,2.,0.]
11. Occurrences: 193. Action Sequence: [1.,3.,0.]
12. Occurrences: 192. Action Sequence: [0.,3.]
13. Occurrences: 191. Action Sequence: [2.,1.]
14. Occurrences: 189. Action Sequence: [1.,1.,0.]
15. Occurrences: 182. Action Sequence: [3.,0.,0.]
16. Occurrences: 177. Action Sequence: [1.,0.,0.]
17. Occurrences: 168. Action Sequence: [2.,3.,0.]
18. Occurrences: 168. Action Sequence: [1.]
19. Occurrences: 168. Action Sequence: [2.,0.,0.]
20. Occurrences: 161. Action Sequence: [2.,0.,1.]
21. Occurrences: 161. Action Sequence: [0.,1.,0.]
22. Occurrences: 160. Action Sequence: [0.,2.,0.]
23. Occurrences: 158. Action Sequence: [3.,1.,0.]
24. Occurrences: 152. Action Sequence: [0.,0.,1.]
25. Occurrences: 152. Action Sequence: [0.,3.,0.]
26. Occurrences: 140. Action Sequence: [3.,3.,0.]
27. Occurrences: 124. Action Sequence: [0.,0.,0.]
28. Occurrences: 123. Action Sequence: [1.,2.]
29. Occurrences: 123. Action Sequence: [1.,1.]
30. Occurrences: 102. Action Sequence: [0.,2.,1.]
31. Occurrences: 98. Action Sequence: [0.,0.,3.]
32. Occurrences: 92. Action Sequence: [2.,2.,1.]
33. Occurrences: 90. Action Sequence: [2.,1.,1.]
34. Occurrences: 90. Action Sequence: [3.,0.,1.]
35. Occurrences: 89. Action Sequence: [1.,0.,1.]
36. Occurrences: 83. Action Sequence: [2.,3.]
37. Occurrences: 77. Action Sequence: [0.,1.,1.]
38. Occurrences: 76. Action Sequence: [1.,2.,1.]
39. Occurrences: 70. Action Sequence: [1.,3.,2.,0.]
40. Occurrences: 67. Action Sequence: [3.,0.,2.,0.]
41. Occurrences: 66. Action Sequence: [0.,2.]
42. Occurrences: 64. Action Sequence: [2.,1.,2.,0.]
43. Occurrences: 63. Action Sequence: [0.,2.,3.]
44. Occurrences: 60. Action Sequence: [2.,0.,3.]
45. Occurrences: 58. Action Sequence: [1.,1.,2.]
46. Occurrences: 57. Action Sequence: [1.,1.,2.,0.]
47. Occurrences: 56. Action Sequence: [2.,1.,1.,0.]
48. Occurrences: 56. Action Sequence: [3.,2.,2.,0.]
49. Occurrences: 55. Action Sequence: [1.,2.,2.,0.]
50. Occurrences: 55. Action Sequence: [2.,3.,3.,0.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 2
40013 different trajectories
1. Occurrences: 7566. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 3377. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 2592. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 1580. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1469. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1327. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1170. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 799. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 739. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 701. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 695. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 666. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 630. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 618. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 609. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 594. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 406. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 370. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 369. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 368. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 362. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 355. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 351. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 337. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 315. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 314. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 307. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 301. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 300. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 283. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 282. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 263. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 197. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 190. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 183. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 180. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 180. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 177. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 175. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 175. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 174. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 172. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 170. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 167. Action Sequence: [0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 167. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 164. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 161. Action Sequence: [0.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 160. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 160. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 159. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 2
70519 different trajectories
1. Occurrences: 6141. Action Sequence: [0.]
2. Occurrences: 1154. Action Sequence: [3.,0.]
3. Occurrences: 1153. Action Sequence: [2.,0.]
4. Occurrences: 984. Action Sequence: [1.,0.]
5. Occurrences: 681. Action Sequence: [0.,0.]
6. Occurrences: 250. Action Sequence: [2.,2.,0.]
7. Occurrences: 244. Action Sequence: [3.,1.,0.]
8. Occurrences: 234. Action Sequence: [2.,3.,0.]
9. Occurrences: 232. Action Sequence: [3.,2.,0.]
10. Occurrences: 231. Action Sequence: [1.,3.,0.]
11. Occurrences: 225. Action Sequence: [3.,3.,0.]
12. Occurrences: 217. Action Sequence: [1.,2.,0.]
13. Occurrences: 211. Action Sequence: [2.,1.,0.]
14. Occurrences: 206. Action Sequence: [1.,1.,0.]
15. Occurrences: 206. Action Sequence: [1.,1.]
16. Occurrences: 190. Action Sequence: [0.,2.,0.]
17. Occurrences: 189. Action Sequence: [0.,1.]
18. Occurrences: 187. Action Sequence: [2.,0.,0.]
19. Occurrences: 178. Action Sequence: [1.,0.,0.]
20. Occurrences: 176. Action Sequence: [3.,0.,0.]
21. Occurrences: 174. Action Sequence: [0.,2.]
22. Occurrences: 163. Action Sequence: [0.,1.,0.]
23. Occurrences: 147. Action Sequence: [0.,0.,0.]
24. Occurrences: 143. Action Sequence: [0.,3.,0.]
25. Occurrences: 143. Action Sequence: [1.,2.]
26. Occurrences: 91. Action Sequence: [1.,1.,1.]
27. Occurrences: 90. Action Sequence: [1.,0.,1.]
28. Occurrences: 82. Action Sequence: [1.]
29. Occurrences: 81. Action Sequence: [3.,1.,1.]
30. Occurrences: 77. Action Sequence: [2.,0.,1.]
31. Occurrences: 74. Action Sequence: [2.,2.]
32. Occurrences: 73. Action Sequence: [2.,1.,1.]
33. Occurrences: 73. Action Sequence: [3.,1.,2.,0.]
34. Occurrences: 73. Action Sequence: [3.,0.,1.]
35. Occurrences: 73. Action Sequence: [1.,1.,2.]
36. Occurrences: 73. Action Sequence: [0.,1.,1.]
37. Occurrences: 69. Action Sequence: [2.,1.]
38. Occurrences: 68. Action Sequence: [1.,0.,2.]
39. Occurrences: 65. Action Sequence: [2.,3.,3.,0.]
40. Occurrences: 65. Action Sequence: [1.,2.,1.,0.]
41. Occurrences: 62. Action Sequence: [2.,1.,2.,0.]
42. Occurrences: 61. Action Sequence: [2.,2.,2.,0.]
43. Occurrences: 60. Action Sequence: [1.,1.,1.,0.]
44. Occurrences: 60. Action Sequence: [0.,0.,1.]
45. Occurrences: 59. Action Sequence: [1.,2.,2.,0.]
46. Occurrences: 59. Action Sequence: [2.,1.,1.,0.]
47. Occurrences: 59. Action Sequence: [1.,3.,3.,0.]
48. Occurrences: 58. Action Sequence: [1.,2.,3.,0.]
49. Occurrences: 58. Action Sequence: [3.,2.,1.,0.]
50. Occurrences: 58. Action Sequence: [0.,0.,2.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 3
45908 different trajectories
1. Occurrences: 6223. Action Sequence: [0.]
2. Occurrences: 2433. Action Sequence: [1.,0.]
3. Occurrences: 2060. Action Sequence: [0.,0.]
4. Occurrences: 1122. Action Sequence: [1.,0.,0.]
5. Occurrences: 1039. Action Sequence: [1.,1.,0.]
6. Occurrences: 1008. Action Sequence: [0.,0.,0.]
7. Occurrences: 957. Action Sequence: [0.,1.,0.]
8. Occurrences: 578. Action Sequence: [1.,0.,0.,0.]
9. Occurrences: 572. Action Sequence: [1.,1.,0.,0.]
10. Occurrences: 539. Action Sequence: [0.,1.,0.,0.]
11. Occurrences: 536. Action Sequence: [0.,0.,0.,0.]
12. Occurrences: 516. Action Sequence: [1.,1.,1.,0.]
13. Occurrences: 499. Action Sequence: [1.,0.,1.,0.]
14. Occurrences: 482. Action Sequence: [0.,1.,1.,0.]
15. Occurrences: 428. Action Sequence: [0.,1.]
16. Occurrences: 424. Action Sequence: [0.,0.,1.,0.]
17. Occurrences: 347. Action Sequence: [0.,0.,1.]
18. Occurrences: 341. Action Sequence: [1.,1.,1.,0.,0.]
19. Occurrences: 318. Action Sequence: [1.,0.,1.,0.,0.]
20. Occurrences: 293. Action Sequence: [1.,1.,0.,0.,0.]
21. Occurrences: 285. Action Sequence: [1.,1.,1.,1.,0.]
22. Occurrences: 274. Action Sequence: [0.,1.,0.,0.,0.]
23. Occurrences: 271. Action Sequence: [1.,0.,0.,0.,0.]
24. Occurrences: 267. Action Sequence: [1.,0.,1.,1.,0.]
25. Occurrences: 259. Action Sequence: [0.,0.,1.,0.,0.]
26. Occurrences: 257. Action Sequence: [0.,1.,1.,0.,0.]
27. Occurrences: 256. Action Sequence: [0.,1.,1.,1.,0.]
28. Occurrences: 255. Action Sequence: [0.,1.,0.,1.,0.]
29. Occurrences: 248. Action Sequence: [0.,0.,1.,1.,0.]
30. Occurrences: 243. Action Sequence: [1.,1.,0.,1.,0.]
31. Occurrences: 236. Action Sequence: [0.,1.,1.]
32. Occurrences: 227. Action Sequence: [0.,0.,0.,0.,0.]
33. Occurrences: 218. Action Sequence: [0.,0.,0.,1.,0.]
34. Occurrences: 217. Action Sequence: [1.,0.,0.,1.,0.]
35. Occurrences: 191. Action Sequence: [1.,1.,0.,1.,0.,0.]
36. Occurrences: 175. Action Sequence: [0.,0.,0.,1.]
37. Occurrences: 174. Action Sequence: [1.,1.,1.,1.,0.,0.]
38. Occurrences: 169. Action Sequence: [0.,1.,1.,1.,0.,0.]
39. Occurrences: 166. Action Sequence: [1.,0.,1.]
40. Occurrences: 161. Action Sequence: [1.,1.,1.,0.,0.,0.]
41. Occurrences: 156. Action Sequence: [1.,0.,0.,1.,0.,0.]
42. Occurrences: 154. Action Sequence: [0.,0.,1.,1.]
43. Occurrences: 151. Action Sequence: [1.,0.,1.,1.,0.,0.]
44. Occurrences: 149. Action Sequence: [0.,1.,1.,0.,0.,0.]
45. Occurrences: 143. Action Sequence: [0.,1.,0.,1.,0.,0.]
46. Occurrences: 143. Action Sequence: [0.,0.,1.,1.,0.,0.]
47. Occurrences: 141. Action Sequence: [1.,0.,1.,0.,0.,0.]
48. Occurrences: 138. Action Sequence: [1.,1.,0.,0.,1.,0.]
49. Occurrences: 138. Action Sequence: [1.,1.,0.,1.,1.,0.]
50. Occurrences: 137. Action Sequence: [1.,1.,0.,0.,0.,0.]

FACTORED SPACE 1, factorisation 3
69427 different trajectories
1. Occurrences: 6145. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
2. Occurrences: 1295. Action Sequence: [2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
3. Occurrences: 889. Action Sequence: [0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
4. Occurrences: 845. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
5. Occurrences: 843. Action Sequence: [1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
6. Occurrences: 367. Action Sequence: [0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
7. Occurrences: 283. Action Sequence: [2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
8. Occurrences: 280. Action Sequence: [2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
9. Occurrences: 228. Action Sequence: [3.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
10. Occurrences: 224. Action Sequence: [2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
11. Occurrences: 221. Action Sequence: [1.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
12. Occurrences: 218. Action Sequence: [2.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
13. Occurrences: 200. Action Sequence: [2.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
14. Occurrences: 194. Action Sequence: [0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
15. Occurrences: 193. Action Sequence: [3.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
16. Occurrences: 189. Action Sequence: [2.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
17. Occurrences: 180. Action Sequence: [1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
18. Occurrences: 179. Action Sequence: [3.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
19. Occurrences: 176. Action Sequence: [1.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
20. Occurrences: 167. Action Sequence: [0.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
21. Occurrences: 167. Action Sequence: [1.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
22. Occurrences: 160. Action Sequence: [0.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
23. Occurrences: 158. Action Sequence: [3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
24. Occurrences: 157. Action Sequence: [1.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
25. Occurrences: 156. Action Sequence: [1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
26. Occurrences: 156. Action Sequence: [2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
27. Occurrences: 152. Action Sequence: [3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
28. Occurrences: 148. Action Sequence: [0.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
29. Occurrences: 146. Action Sequence: [0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
30. Occurrences: 143. Action Sequence: [2.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
31. Occurrences: 142. Action Sequence: [0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
32. Occurrences: 130. Action Sequence: [1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
33. Occurrences: 127. Action Sequence: [0.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
34. Occurrences: 105. Action Sequence: [2.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
35. Occurrences: 95. Action Sequence: [1.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
36. Occurrences: 88. Action Sequence: [0.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
37. Occurrences: 85. Action Sequence: [3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
38. Occurrences: 77. Action Sequence: [0.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
39. Occurrences: 74. Action Sequence: [3.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
40. Occurrences: 74. Action Sequence: [2.,1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
41. Occurrences: 74. Action Sequence: [3.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
42. Occurrences: 72. Action Sequence: [1.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
43. Occurrences: 72. Action Sequence: [2.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
44. Occurrences: 68. Action Sequence: [2.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
45. Occurrences: 66. Action Sequence: [2.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
46. Occurrences: 62. Action Sequence: [1.,1.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
47. Occurrences: 60. Action Sequence: [2.,3.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
48. Occurrences: 59. Action Sequence: [1.,2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
49. Occurrences: 58. Action Sequence: [3.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
50. Occurrences: 58. Action Sequence: [1.,3.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 4
45908 different trajectories
1. Occurrences: 6223. Action Sequence: [0.]
2. Occurrences: 2433. Action Sequence: [1.,0.]
3. Occurrences: 2060. Action Sequence: [0.,0.]
4. Occurrences: 1122. Action Sequence: [1.,0.,0.]
5. Occurrences: 1039. Action Sequence: [1.,1.,0.]
6. Occurrences: 1008. Action Sequence: [0.,0.,0.]
7. Occurrences: 957. Action Sequence: [0.,1.,0.]
8. Occurrences: 578. Action Sequence: [1.,0.,0.,0.]
9. Occurrences: 572. Action Sequence: [1.,1.,0.,0.]
10. Occurrences: 539. Action Sequence: [0.,1.,0.,0.]
11. Occurrences: 536. Action Sequence: [0.,0.,0.,0.]
12. Occurrences: 516. Action Sequence: [1.,1.,1.,0.]
13. Occurrences: 499. Action Sequence: [1.,0.,1.,0.]
14. Occurrences: 482. Action Sequence: [0.,1.,1.,0.]
15. Occurrences: 428. Action Sequence: [0.,1.]
16. Occurrences: 424. Action Sequence: [0.,0.,1.,0.]
17. Occurrences: 347. Action Sequence: [0.,0.,1.]
18. Occurrences: 341. Action Sequence: [1.,1.,1.,0.,0.]
19. Occurrences: 318. Action Sequence: [1.,0.,1.,0.,0.]
20. Occurrences: 293. Action Sequence: [1.,1.,0.,0.,0.]
21. Occurrences: 285. Action Sequence: [1.,1.,1.,1.,0.]
22. Occurrences: 274. Action Sequence: [0.,1.,0.,0.,0.]
23. Occurrences: 271. Action Sequence: [1.,0.,0.,0.,0.]
24. Occurrences: 267. Action Sequence: [1.,0.,1.,1.,0.]
25. Occurrences: 259. Action Sequence: [0.,0.,1.,0.,0.]
26. Occurrences: 257. Action Sequence: [0.,1.,1.,0.,0.]
27. Occurrences: 256. Action Sequence: [0.,1.,1.,1.,0.]
28. Occurrences: 255. Action Sequence: [0.,1.,0.,1.,0.]
29. Occurrences: 248. Action Sequence: [0.,0.,1.,1.,0.]
30. Occurrences: 243. Action Sequence: [1.,1.,0.,1.,0.]
31. Occurrences: 236. Action Sequence: [0.,1.,1.]
32. Occurrences: 227. Action Sequence: [0.,0.,0.,0.,0.]
33. Occurrences: 218. Action Sequence: [0.,0.,0.,1.,0.]
34. Occurrences: 217. Action Sequence: [1.,0.,0.,1.,0.]
35. Occurrences: 191. Action Sequence: [1.,1.,0.,1.,0.,0.]
36. Occurrences: 175. Action Sequence: [0.,0.,0.,1.]
37. Occurrences: 174. Action Sequence: [1.,1.,1.,1.,0.,0.]
38. Occurrences: 169. Action Sequence: [0.,1.,1.,1.,0.,0.]
39. Occurrences: 166. Action Sequence: [1.,0.,1.]
40. Occurrences: 161. Action Sequence: [1.,1.,1.,0.,0.,0.]
41. Occurrences: 156. Action Sequence: [1.,0.,0.,1.,0.,0.]
42. Occurrences: 154. Action Sequence: [0.,0.,1.,1.]
43. Occurrences: 151. Action Sequence: [1.,0.,1.,1.,0.,0.]
44. Occurrences: 149. Action Sequence: [0.,1.,1.,0.,0.,0.]
45. Occurrences: 143. Action Sequence: [0.,1.,0.,1.,0.,0.]
46. Occurrences: 143. Action Sequence: [0.,0.,1.,1.,0.,0.]
47. Occurrences: 141. Action Sequence: [1.,0.,1.,0.,0.,0.]
48. Occurrences: 138. Action Sequence: [1.,1.,0.,0.,1.,0.]
49. Occurrences: 138. Action Sequence: [1.,1.,0.,1.,1.,0.]
50. Occurrences: 137. Action Sequence: [1.,1.,0.,0.,0.,0.]

FACTORED SPACE 1, factorisation 4
40943 different trajectories
1. Occurrences: 7034. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 2983. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 2642. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 1407. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1306. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1268. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1263. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1012. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 701. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 673. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 652. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 650. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 649. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 645. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 619. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 583. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 388. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 375. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 370. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 365. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 356. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 347. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 340. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 336. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 332. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 309. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 306. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 306. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 305. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 302. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 280. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 262. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 206. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 201. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 186. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 179. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 175. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 170. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 170. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 170. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 170. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 170. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 169. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 166. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 165. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 164. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 161. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 159. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 158. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 157. Action Sequence: [0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 2, factorisation 4
40013 different trajectories
1. Occurrences: 7566. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 3377. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 2592. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 1580. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1469. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1327. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1170. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 799. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 739. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 701. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 695. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 666. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 630. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 618. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 609. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 594. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 406. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 370. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 369. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 368. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 362. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 355. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 351. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 337. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 315. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 314. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 307. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 301. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 300. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 283. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 282. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 263. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 197. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 190. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 183. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 180. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 180. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 177. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 175. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 175. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 174. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 172. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 170. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 167. Action Sequence: [0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 167. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 164. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 161. Action Sequence: [0.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 160. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 160. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 159. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

--------------------------------------------------------
'''

### State 696, Diabetes

In [ ]:
'''
NON FACTORED SPACE
39492 different trajectories
1. Occurrences: 6682. Action Sequence: [0.]
2. Occurrences: 6233. Action Sequence: [1.]
3. Occurrences: 6070. Action Sequence: [2.]
4. Occurrences: 5693. Action Sequence: [3.]
5. Occurrences: 3555. Action Sequence: [5.]
6. Occurrences: 2984. Action Sequence: [7.]
7. Occurrences: 2354. Action Sequence: [4.]
8. Occurrences: 1932. Action Sequence: [6.]
9. Occurrences: 441. Action Sequence: [6.,1.]
10. Occurrences: 436. Action Sequence: [3.,2.]
11. Occurrences: 427. Action Sequence: [3.,0.]
12. Occurrences: 422. Action Sequence: [4.,1.]
13. Occurrences: 384. Action Sequence: [6.,3.]
14. Occurrences: 379. Action Sequence: [1.,0.]
15. Occurrences: 377. Action Sequence: [2.,1.]
16. Occurrences: 367. Action Sequence: [7.,1.]
17. Occurrences: 362. Action Sequence: [3.,1.]
18. Occurrences: 352. Action Sequence: [3.,3.]
19. Occurrences: 348. Action Sequence: [7.,0.]
20. Occurrences: 342. Action Sequence: [1.,1.]
21. Occurrences: 338. Action Sequence: [4.,3.]
22. Occurrences: 333. Action Sequence: [1.,2.]
23. Occurrences: 332. Action Sequence: [0.,3.]
24. Occurrences: 330. Action Sequence: [1.,3.]
25. Occurrences: 327. Action Sequence: [0.,1.]
26. Occurrences: 322. Action Sequence: [2.,0.]
27. Occurrences: 318. Action Sequence: [4.,0.]
28. Occurrences: 316. Action Sequence: [5.,0.]
29. Occurrences: 310. Action Sequence: [6.,0.]
30. Occurrences: 309. Action Sequence: [2.,3.]
31. Occurrences: 306. Action Sequence: [5.,1.]
32. Occurrences: 302. Action Sequence: [2.,2.]
33. Occurrences: 287. Action Sequence: [0.,0.]
34. Occurrences: 267. Action Sequence: [5.,3.]
35. Occurrences: 267. Action Sequence: [0.,2.]
36. Occurrences: 263. Action Sequence: [7.,3.]
37. Occurrences: 247. Action Sequence: [7.,2.]
38. Occurrences: 236. Action Sequence: [6.,5.]
39. Occurrences: 234. Action Sequence: [5.,2.]
40. Occurrences: 220. Action Sequence: [4.,5.]
41. Occurrences: 204. Action Sequence: [2.,5.]
42. Occurrences: 190. Action Sequence: [5.,5.]
43. Occurrences: 188. Action Sequence: [3.,5.]
44. Occurrences: 182. Action Sequence: [2.,7.]
45. Occurrences: 179. Action Sequence: [1.,7.]
46. Occurrences: 178. Action Sequence: [1.,5.]
47. Occurrences: 177. Action Sequence: [3.,7.]
48. Occurrences: 168. Action Sequence: [0.,5.]
49. Occurrences: 167. Action Sequence: [7.,5.]
50. Occurrences: 151. Action Sequence: [0.,7.]

FACTORED SPACE 0, factorisation 1
16235 different trajectories
1. Occurrences: 22994. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 21291. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 5315. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 4990. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1642. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1637. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1556. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1513. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 646. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 637. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 616. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 575. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 560. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 557. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 540. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 524. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 270. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 266. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 263. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 256. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 255. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 253. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 252. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 247. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 244. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 239. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 233. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 233. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 228. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 212. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 211. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 211. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 120. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 117. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 115. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 115. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 114. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 113. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 113. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 112. Action Sequence: [0.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 112. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 111. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 111. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 109. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 107. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 106. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 106. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 105. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 104. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 104. Action Sequence: [0.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 1
31739 different trajectories
1. Occurrences: 12752. Action Sequence: [0.]
2. Occurrences: 11926. Action Sequence: [1.]
3. Occurrences: 6539. Action Sequence: [3.]
4. Occurrences: 4286. Action Sequence: [2.]
5. Occurrences: 1585. Action Sequence: [2.,1.]
6. Occurrences: 1575. Action Sequence: [1.,0.]
7. Occurrences: 1386. Action Sequence: [1.,1.]
8. Occurrences: 1345. Action Sequence: [0.,1.]
9. Occurrences: 1203. Action Sequence: [3.,1.]
10. Occurrences: 1178. Action Sequence: [0.,0.]
11. Occurrences: 1145. Action Sequence: [3.,0.]
12. Occurrences: 876. Action Sequence: [2.,0.]
13. Occurrences: 740. Action Sequence: [2.,3.]
14. Occurrences: 722. Action Sequence: [1.,3.]
15. Occurrences: 705. Action Sequence: [0.,3.]
16. Occurrences: 580. Action Sequence: [3.,3.]
17. Occurrences: 455. Action Sequence: [1.,2.]
18. Occurrences: 377. Action Sequence: [0.,2.]
19. Occurrences: 358. Action Sequence: [2.,0.,1.]
20. Occurrences: 319. Action Sequence: [3.,2.]
21. Occurrences: 237. Action Sequence: [2.,2.,1.]
22. Occurrences: 234. Action Sequence: [2.,1.,1.]
23. Occurrences: 229. Action Sequence: [2.,1.,0.]
24. Occurrences: 226. Action Sequence: [3.,1.,1.]
25. Occurrences: 217. Action Sequence: [3.,0.,1.]
26. Occurrences: 213. Action Sequence: [3.,0.,0.]
27. Occurrences: 210. Action Sequence: [0.,0.,1.]
28. Occurrences: 205. Action Sequence: [2.,0.,3.]
29. Occurrences: 202. Action Sequence: [1.,1.,0.]
30. Occurrences: 199. Action Sequence: [2.,0.,0.]
31. Occurrences: 197. Action Sequence: [3.,1.,0.]
32. Occurrences: 192. Action Sequence: [1.,2.,1.]
33. Occurrences: 192. Action Sequence: [0.,2.,1.]
34. Occurrences: 188. Action Sequence: [2.,2.,0.]
35. Occurrences: 177. Action Sequence: [3.,2.,1.]
36. Occurrences: 176. Action Sequence: [3.,3.,1.]
37. Occurrences: 173. Action Sequence: [2.,3.,0.]
38. Occurrences: 171. Action Sequence: [0.,1.,1.]
39. Occurrences: 170. Action Sequence: [1.,1.,1.]
40. Occurrences: 165. Action Sequence: [3.,2.,0.]
41. Occurrences: 162. Action Sequence: [1.,3.,1.]
42. Occurrences: 161. Action Sequence: [3.,3.,0.]
43. Occurrences: 156. Action Sequence: [0.,1.,0.]
44. Occurrences: 156. Action Sequence: [1.,0.,1.]
45. Occurrences: 150. Action Sequence: [2.,2.]
46. Occurrences: 150. Action Sequence: [2.,3.,1.]
47. Occurrences: 150. Action Sequence: [1.,0.,0.]
48. Occurrences: 147. Action Sequence: [1.,3.,0.]
49. Occurrences: 143. Action Sequence: [0.,0.,0.]
50. Occurrences: 131. Action Sequence: [0.,3.,1.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 2
15632 different trajectories
1. Occurrences: 23981. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 22952. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 5164. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 4730. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1633. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1484. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1451. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1448. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 694. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 575. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 568. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 557. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 527. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 523. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 517. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 513. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 283. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 257. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 250. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 250. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 240. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 225. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 224. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 224. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 222. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 222. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 221. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 219. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 216. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 210. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 205. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 199. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 117. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 116. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 114. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 112. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 111. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 111. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 109. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 108. Action Sequence: [0.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 108. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 103. Action Sequence: [0.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 102. Action Sequence: [0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 101. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 101. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 100. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 100. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 98. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 97. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 97. Action Sequence: [0.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 2
31662 different trajectories
1. Occurrences: 12915. Action Sequence: [0.]
2. Occurrences: 11763. Action Sequence: [1.]
3. Occurrences: 5909. Action Sequence: [2.]
4. Occurrences: 4916. Action Sequence: [3.]
5. Occurrences: 1488. Action Sequence: [1.,0.]
6. Occurrences: 1466. Action Sequence: [3.,0.]
7. Occurrences: 1399. Action Sequence: [1.,1.]
8. Occurrences: 1362. Action Sequence: [2.,0.]
9. Occurrences: 1335. Action Sequence: [0.,0.]
10. Occurrences: 1262. Action Sequence: [0.,1.]
11. Occurrences: 1036. Action Sequence: [3.,1.]
12. Occurrences: 945. Action Sequence: [2.,1.]
13. Occurrences: 623. Action Sequence: [1.,2.]
14. Occurrences: 601. Action Sequence: [0.,2.]
15. Occurrences: 575. Action Sequence: [2.,2.]
16. Occurrences: 560. Action Sequence: [3.,2.]
17. Occurrences: 546. Action Sequence: [1.,3.]
18. Occurrences: 489. Action Sequence: [0.,3.]
19. Occurrences: 339. Action Sequence: [3.,3.]
20. Occurrences: 315. Action Sequence: [2.,3.]
21. Occurrences: 283. Action Sequence: [3.,1.,0.]
22. Occurrences: 274. Action Sequence: [3.,2.,0.]
23. Occurrences: 266. Action Sequence: [3.,0.,0.]
24. Occurrences: 261. Action Sequence: [2.,0.,0.]
25. Occurrences: 257. Action Sequence: [2.,1.,0.]
26. Occurrences: 244. Action Sequence: [2.,2.,0.]
27. Occurrences: 240. Action Sequence: [2.,3.,0.]
28. Occurrences: 225. Action Sequence: [3.,3.,0.]
29. Occurrences: 205. Action Sequence: [3.,1.,1.]
30. Occurrences: 204. Action Sequence: [3.,0.,1.]
31. Occurrences: 203. Action Sequence: [1.,1.,0.]
32. Occurrences: 200. Action Sequence: [2.,0.,1.]
33. Occurrences: 197. Action Sequence: [2.,1.,1.]
34. Occurrences: 185. Action Sequence: [1.,1.,1.]
35. Occurrences: 185. Action Sequence: [1.,3.,0.]
36. Occurrences: 184. Action Sequence: [0.,2.,0.]
37. Occurrences: 178. Action Sequence: [1.,2.,0.]
38. Occurrences: 175. Action Sequence: [0.,1.,0.]
39. Occurrences: 175. Action Sequence: [1.,0.,1.]
40. Occurrences: 174. Action Sequence: [1.,0.,0.]
41. Occurrences: 160. Action Sequence: [0.,0.,0.]
42. Occurrences: 157. Action Sequence: [0.,1.,1.]
43. Occurrences: 153. Action Sequence: [0.,3.,0.]
44. Occurrences: 129. Action Sequence: [0.,0.,1.]
45. Occurrences: 128. Action Sequence: [1.,3.,1.]
46. Occurrences: 119. Action Sequence: [1.,2.,1.]
47. Occurrences: 118. Action Sequence: [2.,3.,1.]
48. Occurrences: 118. Action Sequence: [3.,1.,2.]
49. Occurrences: 117. Action Sequence: [3.,3.,1.]
50. Occurrences: 111. Action Sequence: [0.,3.,1.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 3
18296 different trajectories
1. Occurrences: 24678. Action Sequence: [0.]
2. Occurrences: 10825. Action Sequence: [1.]
3. Occurrences: 5484. Action Sequence: [0.,0.]
4. Occurrences: 4809. Action Sequence: [1.,0.]
5. Occurrences: 2259. Action Sequence: [0.,1.]
6. Occurrences: 1873. Action Sequence: [1.,0.,0.]
7. Occurrences: 1789. Action Sequence: [1.,1.]
8. Occurrences: 1427. Action Sequence: [1.,1.,0.]
9. Occurrences: 1358. Action Sequence: [0.,0.,0.]
10. Occurrences: 1158. Action Sequence: [0.,1.,0.]
11. Occurrences: 837. Action Sequence: [1.,1.,0.,0.]
12. Occurrences: 675. Action Sequence: [1.,0.,0.,0.]
13. Occurrences: 675. Action Sequence: [1.,0.,1.]
14. Occurrences: 536. Action Sequence: [0.,0.,1.]
15. Occurrences: 520. Action Sequence: [1.,1.,1.,0.]
16. Occurrences: 506. Action Sequence: [1.,0.,1.,0.]
17. Occurrences: 479. Action Sequence: [0.,1.,0.,0.]
18. Occurrences: 458. Action Sequence: [1.,1.,1.]
19. Occurrences: 413. Action Sequence: [0.,1.,1.]
20. Occurrences: 412. Action Sequence: [1.,1.,1.,0.,0.]
21. Occurrences: 381. Action Sequence: [0.,0.,0.,0.]
22. Occurrences: 349. Action Sequence: [1.,1.,0.,0.,0.]
23. Occurrences: 331. Action Sequence: [0.,1.,1.,0.]
24. Occurrences: 322. Action Sequence: [1.,0.,1.,0.,0.]
25. Occurrences: 285. Action Sequence: [0.,0.,1.,0.]
26. Occurrences: 274. Action Sequence: [1.,1.,1.,1.,0.]
27. Occurrences: 251. Action Sequence: [1.,0.,0.,0.,0.]
28. Occurrences: 244. Action Sequence: [1.,1.,0.,1.,0.]
29. Occurrences: 227. Action Sequence: [1.,0.,1.,1.,0.]
30. Occurrences: 215. Action Sequence: [1.,1.,0.,1.]
31. Occurrences: 214. Action Sequence: [0.,1.,1.,0.,0.]
32. Occurrences: 196. Action Sequence: [1.,1.,1.,1.,0.,0.]
33. Occurrences: 192. Action Sequence: [1.,0.,0.,1.,0.]
34. Occurrences: 192. Action Sequence: [1.,0.,0.,1.]
35. Occurrences: 188. Action Sequence: [0.,1.,0.,0.,0.]
36. Occurrences: 172. Action Sequence: [1.,1.,1.,0.,0.,0.]
37. Occurrences: 165. Action Sequence: [1.,1.,1.,1.]
38. Occurrences: 156. Action Sequence: [1.,0.,1.,1.]
39. Occurrences: 153. Action Sequence: [0.,1.,0.,1.]
40. Occurrences: 148. Action Sequence: [1.,0.,1.,1.,0.,0.]
41. Occurrences: 142. Action Sequence: [0.,1.,1.,1.,0.]
42. Occurrences: 140. Action Sequence: [0.,0.,0.,1.]
43. Occurrences: 139. Action Sequence: [0.,0.,1.,0.,0.]
44. Occurrences: 139. Action Sequence: [1.,0.,1.,0.,0.,0.]
45. Occurrences: 132. Action Sequence: [0.,1.,0.,1.,0.]
46. Occurrences: 130. Action Sequence: [1.,1.,0.,1.,0.,0.]
47. Occurrences: 129. Action Sequence: [1.,1.,0.,0.,0.,0.]
48. Occurrences: 128. Action Sequence: [0.,0.,0.,0.,0.]
49. Occurrences: 122. Action Sequence: [1.,1.,1.,1.,1.,0.]
50. Occurrences: 121. Action Sequence: [1.,0.,0.,1.,0.,0.]

FACTORED SPACE 1, factorisation 3
30846 different trajectories
1. Occurrences: 10806. Action Sequence: [1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
2. Occurrences: 10128. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
3. Occurrences: 9722. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
4. Occurrences: 9155. Action Sequence: [2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
5. Occurrences: 1381. Action Sequence: [2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
6. Occurrences: 1227. Action Sequence: [0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
7. Occurrences: 1208. Action Sequence: [3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
8. Occurrences: 1161. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
9. Occurrences: 1122. Action Sequence: [1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
10. Occurrences: 1040. Action Sequence: [1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
11. Occurrences: 994. Action Sequence: [3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
12. Occurrences: 982. Action Sequence: [0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
13. Occurrences: 964. Action Sequence: [2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
14. Occurrences: 822. Action Sequence: [1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
15. Occurrences: 732. Action Sequence: [2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
16. Occurrences: 641. Action Sequence: [0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
17. Occurrences: 251. Action Sequence: [2.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
18. Occurrences: 245. Action Sequence: [0.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
19. Occurrences: 242. Action Sequence: [2.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
20. Occurrences: 221. Action Sequence: [0.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
21. Occurrences: 198. Action Sequence: [3.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
22. Occurrences: 195. Action Sequence: [1.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
23. Occurrences: 192. Action Sequence: [3.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
24. Occurrences: 186. Action Sequence: [2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
25. Occurrences: 171. Action Sequence: [2.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
26. Occurrences: 168. Action Sequence: [3.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
27. Occurrences: 168. Action Sequence: [2.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
28. Occurrences: 168. Action Sequence: [2.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
29. Occurrences: 166. Action Sequence: [3.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
30. Occurrences: 166. Action Sequence: [1.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
31. Occurrences: 164. Action Sequence: [2.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
32. Occurrences: 163. Action Sequence: [1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
33. Occurrences: 163. Action Sequence: [0.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
34. Occurrences: 163. Action Sequence: [1.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
35. Occurrences: 162. Action Sequence: [0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
36. Occurrences: 160. Action Sequence: [1.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
37. Occurrences: 159. Action Sequence: [3.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
38. Occurrences: 157. Action Sequence: [3.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
39. Occurrences: 156. Action Sequence: [3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
40. Occurrences: 153. Action Sequence: [0.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
41. Occurrences: 152. Action Sequence: [1.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
42. Occurrences: 149. Action Sequence: [0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
43. Occurrences: 149. Action Sequence: [0.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
44. Occurrences: 148. Action Sequence: [3.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
45. Occurrences: 143. Action Sequence: [1.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
46. Occurrences: 141. Action Sequence: [1.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
47. Occurrences: 139. Action Sequence: [0.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
48. Occurrences: 138. Action Sequence: [2.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
49. Occurrences: 126. Action Sequence: [2.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
50. Occurrences: 112. Action Sequence: [1.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 4
18296 different trajectories
1. Occurrences: 24678. Action Sequence: [0.]
2. Occurrences: 10825. Action Sequence: [1.]
3. Occurrences: 5484. Action Sequence: [0.,0.]
4. Occurrences: 4809. Action Sequence: [1.,0.]
5. Occurrences: 2259. Action Sequence: [0.,1.]
6. Occurrences: 1873. Action Sequence: [1.,0.,0.]
7. Occurrences: 1789. Action Sequence: [1.,1.]
8. Occurrences: 1427. Action Sequence: [1.,1.,0.]
9. Occurrences: 1358. Action Sequence: [0.,0.,0.]
10. Occurrences: 1158. Action Sequence: [0.,1.,0.]
11. Occurrences: 837. Action Sequence: [1.,1.,0.,0.]
12. Occurrences: 675. Action Sequence: [1.,0.,0.,0.]
13. Occurrences: 675. Action Sequence: [1.,0.,1.]
14. Occurrences: 536. Action Sequence: [0.,0.,1.]
15. Occurrences: 520. Action Sequence: [1.,1.,1.,0.]
16. Occurrences: 506. Action Sequence: [1.,0.,1.,0.]
17. Occurrences: 479. Action Sequence: [0.,1.,0.,0.]
18. Occurrences: 458. Action Sequence: [1.,1.,1.]
19. Occurrences: 413. Action Sequence: [0.,1.,1.]
20. Occurrences: 412. Action Sequence: [1.,1.,1.,0.,0.]
21. Occurrences: 381. Action Sequence: [0.,0.,0.,0.]
22. Occurrences: 349. Action Sequence: [1.,1.,0.,0.,0.]
23. Occurrences: 331. Action Sequence: [0.,1.,1.,0.]
24. Occurrences: 322. Action Sequence: [1.,0.,1.,0.,0.]
25. Occurrences: 285. Action Sequence: [0.,0.,1.,0.]
26. Occurrences: 274. Action Sequence: [1.,1.,1.,1.,0.]
27. Occurrences: 251. Action Sequence: [1.,0.,0.,0.,0.]
28. Occurrences: 244. Action Sequence: [1.,1.,0.,1.,0.]
29. Occurrences: 227. Action Sequence: [1.,0.,1.,1.,0.]
30. Occurrences: 215. Action Sequence: [1.,1.,0.,1.]
31. Occurrences: 214. Action Sequence: [0.,1.,1.,0.,0.]
32. Occurrences: 196. Action Sequence: [1.,1.,1.,1.,0.,0.]
33. Occurrences: 192. Action Sequence: [1.,0.,0.,1.,0.]
34. Occurrences: 192. Action Sequence: [1.,0.,0.,1.]
35. Occurrences: 188. Action Sequence: [0.,1.,0.,0.,0.]
36. Occurrences: 172. Action Sequence: [1.,1.,1.,0.,0.,0.]
37. Occurrences: 165. Action Sequence: [1.,1.,1.,1.]
38. Occurrences: 156. Action Sequence: [1.,0.,1.,1.]
39. Occurrences: 153. Action Sequence: [0.,1.,0.,1.]
40. Occurrences: 148. Action Sequence: [1.,0.,1.,1.,0.,0.]
41. Occurrences: 142. Action Sequence: [0.,1.,1.,1.,0.]
42. Occurrences: 140. Action Sequence: [0.,0.,0.,1.]
43. Occurrences: 139. Action Sequence: [0.,0.,1.,0.,0.]
44. Occurrences: 139. Action Sequence: [1.,0.,1.,0.,0.,0.]
45. Occurrences: 132. Action Sequence: [0.,1.,0.,1.,0.]
46. Occurrences: 130. Action Sequence: [1.,1.,0.,1.,0.,0.]
47. Occurrences: 129. Action Sequence: [1.,1.,0.,0.,0.,0.]
48. Occurrences: 128. Action Sequence: [0.,0.,0.,0.,0.]
49. Occurrences: 122. Action Sequence: [1.,1.,1.,1.,1.,0.]
50. Occurrences: 121. Action Sequence: [1.,0.,0.,1.,0.,0.]

FACTORED SPACE 1, factorisation 4
16235 different trajectories
1. Occurrences: 22994. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 21291. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 5315. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 4990. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1642. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1637. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1556. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1513. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 646. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 637. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 616. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 575. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 560. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 557. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 540. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 524. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 270. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 266. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 263. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 256. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 255. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 253. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 252. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 247. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 244. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 239. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 233. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 233. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 228. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 212. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 211. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 211. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 120. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 117. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 115. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 115. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 114. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 113. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 113. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 112. Action Sequence: [0.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 112. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 111. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 111. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 109. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 107. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 106. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 106. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 105. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 104. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 104. Action Sequence: [0.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 2, factorisation 4
15632 different trajectories
1. Occurrences: 23981. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 22952. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 5164. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 4730. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1633. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1484. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1451. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1448. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 694. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 575. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 568. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 557. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 527. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 523. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 517. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 513. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 283. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 257. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 250. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 250. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 240. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 225. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 224. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 224. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 222. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 222. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 221. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 219. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 216. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 210. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 205. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 199. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 117. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 116. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 114. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 112. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 111. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 111. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 109. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 108. Action Sequence: [0.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 108. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 103. Action Sequence: [0.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 102. Action Sequence: [0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 101. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 101. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 100. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 100. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 98. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 97. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 97. Action Sequence: [0.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

--------------------------------------------------------
'''


### State 696, No Diabetes

In [ ]:
'''
NON FACTORED SPACE
76502 different trajectories
1. Occurrences: 3001. Action Sequence: [0.]
2. Occurrences: 2030. Action Sequence: [2.]
3. Occurrences: 1121. Action Sequence: [1.]
4. Occurrences: 993. Action Sequence: [4.]
5. Occurrences: 648. Action Sequence: [6.]
6. Occurrences: 583. Action Sequence: [5.]
7. Occurrences: 348. Action Sequence: [6.,0.]
8. Occurrences: 346. Action Sequence: [3.,0.]
9. Occurrences: 338. Action Sequence: [1.,0.]
10. Occurrences: 326. Action Sequence: [4.,0.]
11. Occurrences: 317. Action Sequence: [7.,0.]
12. Occurrences: 295. Action Sequence: [5.,0.]
13. Occurrences: 284. Action Sequence: [2.,0.]
14. Occurrences: 271. Action Sequence: [0.,0.]
15. Occurrences: 240. Action Sequence: [3.,2.]
16. Occurrences: 196. Action Sequence: [2.,2.]
17. Occurrences: 194. Action Sequence: [1.,2.]
18. Occurrences: 190. Action Sequence: [0.,2.]
19. Occurrences: 176. Action Sequence: [4.,1.]
20. Occurrences: 146. Action Sequence: [7.,2.]
21. Occurrences: 138. Action Sequence: [0.,1.]
22. Occurrences: 129. Action Sequence: [2.,1.]
23. Occurrences: 125. Action Sequence: [6.,1.]
24. Occurrences: 121. Action Sequence: [5.,2.]
25. Occurrences: 121. Action Sequence: [1.,1.]
26. Occurrences: 119. Action Sequence: [3.,1.]
27. Occurrences: 104. Action Sequence: [3.,4.]
28. Occurrences: 97. Action Sequence: [1.,4.]
29. Occurrences: 89. Action Sequence: [2.,4.]
30. Occurrences: 86. Action Sequence: [6.,2.]
31. Occurrences: 85. Action Sequence: [7.,1.]
32. Occurrences: 85. Action Sequence: [5.,1.]
33. Occurrences: 84. Action Sequence: [3.,6.]
34. Occurrences: 77. Action Sequence: [4.,2.]
35. Occurrences: 75. Action Sequence: [6.,3.]
36. Occurrences: 75. Action Sequence: [4.,5.]
37. Occurrences: 74. Action Sequence: [4.,3.]
38. Occurrences: 68. Action Sequence: [0.,4.]
39. Occurrences: 65. Action Sequence: [3.,5.]
40. Occurrences: 64. Action Sequence: [5.,4.]
41. Occurrences: 62. Action Sequence: [2.,5.]
42. Occurrences: 58. Action Sequence: [2.,6.]
43. Occurrences: 57. Action Sequence: [1.,5.]
44. Occurrences: 52. Action Sequence: [1.,6.]
45. Occurrences: 52. Action Sequence: [4.,5.,0.]
46. Occurrences: 52. Action Sequence: [6.,4.,0.]
47. Occurrences: 50. Action Sequence: [6.,6.,0.]
48. Occurrences: 50. Action Sequence: [7.,4.]
49. Occurrences: 50. Action Sequence: [0.,6.]
50. Occurrences: 49. Action Sequence: [7.,4.,0.]

FACTORED SPACE 0, factorisation 1
40648 different trajectories
1. Occurrences: 7214. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 4414. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 2982. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2860. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1422. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1369. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1364. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1253. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 658. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 658. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 636. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 623. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 623. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 619. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 592. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 583. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 334. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 330. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 329. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 323. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 315. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 306. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 305. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 301. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 296. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 285. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 284. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 280. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 280. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 280. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 278. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 263. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 178. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 170. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 167. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 163. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 156. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 153. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 151. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 151. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 150. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 146. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 144. Action Sequence: [0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 142. Action Sequence: [0.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 142. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 142. Action Sequence: [0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 141. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 141. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 139. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 139. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 1
67577 different trajectories
1. Occurrences: 5031. Action Sequence: [0.]
2. Occurrences: 1641. Action Sequence: [2.]
3. Occurrences: 1121. Action Sequence: [1.]
4. Occurrences: 1118. Action Sequence: [1.,0.]
5. Occurrences: 941. Action Sequence: [0.,0.]
6. Occurrences: 879. Action Sequence: [3.,0.]
7. Occurrences: 837. Action Sequence: [2.,0.]
8. Occurrences: 583. Action Sequence: [3.]
9. Occurrences: 450. Action Sequence: [2.,1.]
10. Occurrences: 337. Action Sequence: [1.,2.]
11. Occurrences: 313. Action Sequence: [0.,1.]
12. Occurrences: 265. Action Sequence: [0.,2.]
13. Occurrences: 252. Action Sequence: [1.,1.,0.]
14. Occurrences: 241. Action Sequence: [1.,1.]
15. Occurrences: 225. Action Sequence: [0.,1.,0.]
16. Occurrences: 210. Action Sequence: [3.,1.,0.]
17. Occurrences: 205. Action Sequence: [2.,2.,0.]
18. Occurrences: 200. Action Sequence: [1.,0.,0.]
19. Occurrences: 197. Action Sequence: [3.,2.,0.]
20. Occurrences: 194. Action Sequence: [1.,3.,0.]
21. Occurrences: 193. Action Sequence: [3.,0.,0.]
22. Occurrences: 193. Action Sequence: [2.,1.,0.]
23. Occurrences: 190. Action Sequence: [0.,3.,0.]
24. Occurrences: 190. Action Sequence: [2.,3.,0.]
25. Occurrences: 179. Action Sequence: [1.,2.,0.]
26. Occurrences: 179. Action Sequence: [3.,3.,0.]
27. Occurrences: 176. Action Sequence: [3.,1.]
28. Occurrences: 176. Action Sequence: [2.,3.]
29. Occurrences: 173. Action Sequence: [3.,2.]
30. Occurrences: 172. Action Sequence: [2.,0.,1.]
31. Occurrences: 170. Action Sequence: [2.,0.,0.]
32. Occurrences: 163. Action Sequence: [0.,0.,0.]
33. Occurrences: 158. Action Sequence: [0.,2.,0.]
34. Occurrences: 149. Action Sequence: [2.,2.,1.]
35. Occurrences: 132. Action Sequence: [0.,3.]
36. Occurrences: 122. Action Sequence: [1.,3.]
37. Occurrences: 122. Action Sequence: [2.,2.]
38. Occurrences: 98. Action Sequence: [0.,0.,1.]
39. Occurrences: 97. Action Sequence: [1.,2.,1.]
40. Occurrences: 94. Action Sequence: [3.,0.,1.]
41. Occurrences: 91. Action Sequence: [0.,2.,1.]
42. Occurrences: 79. Action Sequence: [2.,1.,1.]
43. Occurrences: 79. Action Sequence: [2.,2.,3.]
44. Occurrences: 77. Action Sequence: [1.,1.,2.]
45. Occurrences: 75. Action Sequence: [3.,3.]
46. Occurrences: 74. Action Sequence: [2.,0.,3.]
47. Occurrences: 70. Action Sequence: [0.,1.,2.]
48. Occurrences: 64. Action Sequence: [1.,0.,1.]
49. Occurrences: 62. Action Sequence: [3.,2.,1.]
50. Occurrences: 62. Action Sequence: [3.,2.,2.,0.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 2
39754 different trajectories
1. Occurrences: 8276. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 3285. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 3269. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2724. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1436. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1410. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1409. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1405. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 732. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 725. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 709. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 681. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 654. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 601. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 598. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 584. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 367. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 362. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 347. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 330. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 319. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 316. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 316. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 311. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 303. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 296. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 288. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 286. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 283. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 274. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 273. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 258. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 195. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 184. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 172. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 164. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 157. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 157. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 156. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 149. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 148. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 147. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 147. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 147. Action Sequence: [0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 146. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 146. Action Sequence: [0.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 146. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 145. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 144. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 142. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 2
67415 different trajectories
1. Occurrences: 4122. Action Sequence: [0.]
2. Occurrences: 2030. Action Sequence: [1.]
3. Occurrences: 1576. Action Sequence: [2.]
4. Occurrences: 882. Action Sequence: [2.,0.]
5. Occurrences: 878. Action Sequence: [1.,0.]
6. Occurrences: 875. Action Sequence: [3.,0.]
7. Occurrences: 868. Action Sequence: [0.,0.]
8. Occurrences: 648. Action Sequence: [3.]
9. Occurrences: 453. Action Sequence: [1.,1.]
10. Occurrences: 414. Action Sequence: [0.,1.]
11. Occurrences: 320. Action Sequence: [1.,2.]
12. Occurrences: 307. Action Sequence: [3.,1.]
13. Occurrences: 278. Action Sequence: [2.,1.]
14. Occurrences: 271. Action Sequence: [0.,2.]
15. Occurrences: 229. Action Sequence: [2.,2.,0.]
16. Occurrences: 228. Action Sequence: [3.,2.,0.]
17. Occurrences: 227. Action Sequence: [3.,3.,0.]
18. Occurrences: 225. Action Sequence: [2.,2.]
19. Occurrences: 224. Action Sequence: [3.,1.,0.]
20. Occurrences: 213. Action Sequence: [2.,1.,0.]
21. Occurrences: 208. Action Sequence: [0.,1.,0.]
22. Occurrences: 202. Action Sequence: [3.,0.,0.]
23. Occurrences: 202. Action Sequence: [0.,2.,0.]
24. Occurrences: 196. Action Sequence: [2.,0.,0.]
25. Occurrences: 187. Action Sequence: [1.,2.,0.]
26. Occurrences: 184. Action Sequence: [1.,3.,0.]
27. Occurrences: 184. Action Sequence: [1.,0.,0.]
28. Occurrences: 182. Action Sequence: [1.,1.,0.]
29. Occurrences: 180. Action Sequence: [0.,0.,0.]
30. Occurrences: 179. Action Sequence: [2.,3.,0.]
31. Occurrences: 173. Action Sequence: [3.,2.]
32. Occurrences: 164. Action Sequence: [0.,3.,0.]
33. Occurrences: 150. Action Sequence: [1.,3.]
34. Occurrences: 115. Action Sequence: [0.,3.]
35. Occurrences: 95. Action Sequence: [1.,0.,1.]
36. Occurrences: 93. Action Sequence: [1.,1.,1.]
37. Occurrences: 90. Action Sequence: [3.,1.,1.]
38. Occurrences: 88. Action Sequence: [2.,1.,1.]
39. Occurrences: 87. Action Sequence: [0.,1.,1.]
40. Occurrences: 86. Action Sequence: [2.,3.]
41. Occurrences: 82. Action Sequence: [3.,0.,1.]
42. Occurrences: 82. Action Sequence: [0.,0.,1.]
43. Occurrences: 77. Action Sequence: [1.,1.,2.]
44. Occurrences: 76. Action Sequence: [2.,0.,1.]
45. Occurrences: 75. Action Sequence: [1.,0.,2.]
46. Occurrences: 73. Action Sequence: [1.,2.,1.]
47. Occurrences: 71. Action Sequence: [2.,0.,2.]
48. Occurrences: 68. Action Sequence: [1.,3.,1.]
49. Occurrences: 66. Action Sequence: [0.,0.,2.]
50. Occurrences: 64. Action Sequence: [2.,2.,3.,0.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 3
45196 different trajectories
1. Occurrences: 6152. Action Sequence: [0.]
2. Occurrences: 2613. Action Sequence: [0.,0.]
3. Occurrences: 2342. Action Sequence: [1.,0.]
4. Occurrences: 2224. Action Sequence: [1.]
5. Occurrences: 1171. Action Sequence: [1.,0.,0.]
6. Occurrences: 1111. Action Sequence: [0.,0.,0.]
7. Occurrences: 1067. Action Sequence: [1.,1.,0.]
8. Occurrences: 982. Action Sequence: [0.,1.,0.]
9. Occurrences: 856. Action Sequence: [0.,1.]
10. Occurrences: 607. Action Sequence: [1.,1.,0.,0.]
11. Occurrences: 573. Action Sequence: [1.,0.,0.,0.]
12. Occurrences: 556. Action Sequence: [0.,1.,0.,0.]
13. Occurrences: 546. Action Sequence: [1.,1.]
14. Occurrences: 528. Action Sequence: [1.,0.,1.,0.]
15. Occurrences: 519. Action Sequence: [1.,1.,1.,0.]
16. Occurrences: 492. Action Sequence: [0.,0.,0.,0.]
17. Occurrences: 458. Action Sequence: [0.,0.,1.,0.]
18. Occurrences: 449. Action Sequence: [0.,1.,1.,0.]
19. Occurrences: 374. Action Sequence: [0.,0.,1.]
20. Occurrences: 327. Action Sequence: [1.,0.,1.]
21. Occurrences: 325. Action Sequence: [1.,1.,1.,0.,0.]
22. Occurrences: 304. Action Sequence: [1.,1.,0.,0.,0.]
23. Occurrences: 290. Action Sequence: [0.,1.,1.,0.,0.]
24. Occurrences: 266. Action Sequence: [1.,0.,0.,0.,0.]
25. Occurrences: 265. Action Sequence: [0.,1.,1.,1.,0.]
26. Occurrences: 262. Action Sequence: [1.,0.,1.,0.,0.]
27. Occurrences: 251. Action Sequence: [0.,1.,0.,0.,0.]
28. Occurrences: 247. Action Sequence: [1.,1.,1.,1.,0.]
29. Occurrences: 246. Action Sequence: [0.,1.,1.]
30. Occurrences: 245. Action Sequence: [0.,0.,1.,0.,0.]
31. Occurrences: 243. Action Sequence: [1.,1.,0.,1.,0.]
32. Occurrences: 232. Action Sequence: [1.,1.,1.]
33. Occurrences: 230. Action Sequence: [1.,0.,1.,1.,0.]
34. Occurrences: 230. Action Sequence: [0.,0.,0.,0.,0.]
35. Occurrences: 219. Action Sequence: [1.,0.,0.,1.,0.]
36. Occurrences: 207. Action Sequence: [0.,1.,0.,1.,0.]
37. Occurrences: 198. Action Sequence: [0.,0.,1.,1.,0.]
38. Occurrences: 196. Action Sequence: [0.,0.,0.,1.,0.]
39. Occurrences: 180. Action Sequence: [1.,0.,0.,1.]
40. Occurrences: 179. Action Sequence: [0.,0.,0.,1.]
41. Occurrences: 156. Action Sequence: [1.,1.,1.,1.,0.,0.]
42. Occurrences: 152. Action Sequence: [1.,1.,0.,0.,0.,0.]
43. Occurrences: 151. Action Sequence: [1.,1.,1.,0.,0.,0.]
44. Occurrences: 150. Action Sequence: [0.,1.,1.,1.,0.,0.]
45. Occurrences: 146. Action Sequence: [1.,1.,0.,1.]
46. Occurrences: 140. Action Sequence: [1.,0.,1.,1.,0.,0.]
47. Occurrences: 139. Action Sequence: [1.,1.,0.,1.,0.,0.]
48. Occurrences: 137. Action Sequence: [1.,0.,1.,0.,0.,0.]
49. Occurrences: 130. Action Sequence: [1.,1.,1.,0.,1.,0.]
50. Occurrences: 128. Action Sequence: [0.,0.,1.,1.]

FACTORED SPACE 1, factorisation 3
66596 different trajectories
1. Occurrences: 4159. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
2. Occurrences: 2810. Action Sequence: [2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
3. Occurrences: 1712. Action Sequence: [1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
4. Occurrences: 851. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
5. Occurrences: 830. Action Sequence: [2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
6. Occurrences: 829. Action Sequence: [1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
7. Occurrences: 769. Action Sequence: [0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
8. Occurrences: 537. Action Sequence: [3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
9. Occurrences: 448. Action Sequence: [0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
10. Occurrences: 441. Action Sequence: [2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
11. Occurrences: 431. Action Sequence: [1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
12. Occurrences: 399. Action Sequence: [0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
13. Occurrences: 377. Action Sequence: [2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
14. Occurrences: 307. Action Sequence: [3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
15. Occurrences: 304. Action Sequence: [1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
16. Occurrences: 217. Action Sequence: [0.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
17. Occurrences: 206. Action Sequence: [3.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
18. Occurrences: 199. Action Sequence: [3.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
19. Occurrences: 191. Action Sequence: [3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
20. Occurrences: 190. Action Sequence: [2.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
21. Occurrences: 188. Action Sequence: [0.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
22. Occurrences: 186. Action Sequence: [1.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
23. Occurrences: 184. Action Sequence: [2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
24. Occurrences: 182. Action Sequence: [2.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
25. Occurrences: 179. Action Sequence: [1.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
26. Occurrences: 178. Action Sequence: [3.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
27. Occurrences: 174. Action Sequence: [0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
28. Occurrences: 172. Action Sequence: [2.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
29. Occurrences: 170. Action Sequence: [1.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
30. Occurrences: 164. Action Sequence: [1.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
31. Occurrences: 158. Action Sequence: [0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
32. Occurrences: 150. Action Sequence: [0.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
33. Occurrences: 143. Action Sequence: [2.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
34. Occurrences: 133. Action Sequence: [2.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
35. Occurrences: 115. Action Sequence: [0.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
36. Occurrences: 105. Action Sequence: [3.,1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
37. Occurrences: 104. Action Sequence: [1.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
38. Occurrences: 101. Action Sequence: [3.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
39. Occurrences: 98. Action Sequence: [3.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
40. Occurrences: 97. Action Sequence: [3.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
41. Occurrences: 93. Action Sequence: [2.,1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
42. Occurrences: 90. Action Sequence: [2.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
43. Occurrences: 90. Action Sequence: [1.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
44. Occurrences: 87. Action Sequence: [2.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
45. Occurrences: 84. Action Sequence: [0.,1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
46. Occurrences: 83. Action Sequence: [3.,0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
47. Occurrences: 80. Action Sequence: [1.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
48. Occurrences: 80. Action Sequence: [1.,1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
49. Occurrences: 79. Action Sequence: [0.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
50. Occurrences: 78. Action Sequence: [3.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 4
45196 different trajectories
1. Occurrences: 6152. Action Sequence: [0.]
2. Occurrences: 2613. Action Sequence: [0.,0.]
3. Occurrences: 2342. Action Sequence: [1.,0.]
4. Occurrences: 2224. Action Sequence: [1.]
5. Occurrences: 1171. Action Sequence: [1.,0.,0.]
6. Occurrences: 1111. Action Sequence: [0.,0.,0.]
7. Occurrences: 1067. Action Sequence: [1.,1.,0.]
8. Occurrences: 982. Action Sequence: [0.,1.,0.]
9. Occurrences: 856. Action Sequence: [0.,1.]
10. Occurrences: 607. Action Sequence: [1.,1.,0.,0.]
11. Occurrences: 573. Action Sequence: [1.,0.,0.,0.]
12. Occurrences: 556. Action Sequence: [0.,1.,0.,0.]
13. Occurrences: 546. Action Sequence: [1.,1.]
14. Occurrences: 528. Action Sequence: [1.,0.,1.,0.]
15. Occurrences: 519. Action Sequence: [1.,1.,1.,0.]
16. Occurrences: 492. Action Sequence: [0.,0.,0.,0.]
17. Occurrences: 458. Action Sequence: [0.,0.,1.,0.]
18. Occurrences: 449. Action Sequence: [0.,1.,1.,0.]
19. Occurrences: 374. Action Sequence: [0.,0.,1.]
20. Occurrences: 327. Action Sequence: [1.,0.,1.]
21. Occurrences: 325. Action Sequence: [1.,1.,1.,0.,0.]
22. Occurrences: 304. Action Sequence: [1.,1.,0.,0.,0.]
23. Occurrences: 290. Action Sequence: [0.,1.,1.,0.,0.]
24. Occurrences: 266. Action Sequence: [1.,0.,0.,0.,0.]
25. Occurrences: 265. Action Sequence: [0.,1.,1.,1.,0.]
26. Occurrences: 262. Action Sequence: [1.,0.,1.,0.,0.]
27. Occurrences: 251. Action Sequence: [0.,1.,0.,0.,0.]
28. Occurrences: 247. Action Sequence: [1.,1.,1.,1.,0.]
29. Occurrences: 246. Action Sequence: [0.,1.,1.]
30. Occurrences: 245. Action Sequence: [0.,0.,1.,0.,0.]
31. Occurrences: 243. Action Sequence: [1.,1.,0.,1.,0.]
32. Occurrences: 232. Action Sequence: [1.,1.,1.]
33. Occurrences: 230. Action Sequence: [1.,0.,1.,1.,0.]
34. Occurrences: 230. Action Sequence: [0.,0.,0.,0.,0.]
35. Occurrences: 219. Action Sequence: [1.,0.,0.,1.,0.]
36. Occurrences: 207. Action Sequence: [0.,1.,0.,1.,0.]
37. Occurrences: 198. Action Sequence: [0.,0.,1.,1.,0.]
38. Occurrences: 196. Action Sequence: [0.,0.,0.,1.,0.]
39. Occurrences: 180. Action Sequence: [1.,0.,0.,1.]
40. Occurrences: 179. Action Sequence: [0.,0.,0.,1.]
41. Occurrences: 156. Action Sequence: [1.,1.,1.,1.,0.,0.]
42. Occurrences: 152. Action Sequence: [1.,1.,0.,0.,0.,0.]
43. Occurrences: 151. Action Sequence: [1.,1.,1.,0.,0.,0.]
44. Occurrences: 150. Action Sequence: [0.,1.,1.,1.,0.,0.]
45. Occurrences: 146. Action Sequence: [1.,1.,0.,1.]
46. Occurrences: 140. Action Sequence: [1.,0.,1.,1.,0.,0.]
47. Occurrences: 139. Action Sequence: [1.,1.,0.,1.,0.,0.]
48. Occurrences: 137. Action Sequence: [1.,0.,1.,0.,0.,0.]
49. Occurrences: 130. Action Sequence: [1.,1.,1.,0.,1.,0.]
50. Occurrences: 128. Action Sequence: [0.,0.,1.,1.]

FACTORED SPACE 1, factorisation 4
40648 different trajectories
1. Occurrences: 7214. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 4414. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 2982. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2860. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1422. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1369. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1364. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1253. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 658. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 658. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 636. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 623. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 623. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 619. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 592. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 583. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 334. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 330. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 329. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 323. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 315. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 306. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 305. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 301. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 296. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 285. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 284. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 280. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 280. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 280. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 278. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 263. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 178. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 170. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 167. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 163. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 156. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 153. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 151. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 151. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 150. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 146. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 144. Action Sequence: [0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 142. Action Sequence: [0.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 142. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 142. Action Sequence: [0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 141. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 141. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 139. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 139. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 2, factorisation 4
39754 different trajectories
1. Occurrences: 8276. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 3285. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 3269. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2724. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1436. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1410. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1409. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1405. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 732. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 725. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 709. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 681. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 654. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 601. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 598. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 584. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 367. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 362. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 347. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 330. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 319. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 316. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 316. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 311. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 303. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 296. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 288. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 286. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 283. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 274. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 273. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 258. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 195. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 184. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 172. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 164. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 157. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 157. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 156. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 149. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 148. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 147. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 147. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 147. Action Sequence: [0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 146. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 146. Action Sequence: [0.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 146. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 145. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 144. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 142. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

--------------------------------------------------------
'''

### State 703, Diabetes

In [ ]:
'''
NON FACTORED SPACE
36799 different trajectories
1. Occurrences: 7545. Action Sequence: [0.]
2. Occurrences: 7157. Action Sequence: [2.]
3. Occurrences: 6886. Action Sequence: [1.]
4. Occurrences: 6300. Action Sequence: [3.]
5. Occurrences: 3471. Action Sequence: [5.]
6. Occurrences: 2862. Action Sequence: [7.]
7. Occurrences: 2331. Action Sequence: [4.]
8. Occurrences: 1901. Action Sequence: [6.]
9. Occurrences: 455. Action Sequence: [4.,1.]
10. Occurrences: 429. Action Sequence: [3.,0.]
11. Occurrences: 415. Action Sequence: [3.,2.]
12. Occurrences: 408. Action Sequence: [6.,1.]
13. Occurrences: 391. Action Sequence: [3.,1.]
14. Occurrences: 367. Action Sequence: [4.,3.]
15. Occurrences: 360. Action Sequence: [1.,0.]
16. Occurrences: 353. Action Sequence: [1.,2.]
17. Occurrences: 348. Action Sequence: [7.,0.]
18. Occurrences: 345. Action Sequence: [7.,1.]
19. Occurrences: 342. Action Sequence: [1.,3.]
20. Occurrences: 340. Action Sequence: [5.,0.]
21. Occurrences: 337. Action Sequence: [6.,3.]
22. Occurrences: 336. Action Sequence: [2.,0.]
23. Occurrences: 335. Action Sequence: [1.,1.]
24. Occurrences: 329. Action Sequence: [2.,1.]
25. Occurrences: 328. Action Sequence: [3.,3.]
26. Occurrences: 317. Action Sequence: [0.,1.]
27. Occurrences: 301. Action Sequence: [5.,1.]
28. Occurrences: 299. Action Sequence: [2.,3.]
29. Occurrences: 298. Action Sequence: [4.,0.]
30. Occurrences: 298. Action Sequence: [6.,0.]
31. Occurrences: 293. Action Sequence: [0.,0.]
32. Occurrences: 288. Action Sequence: [2.,2.]
33. Occurrences: 284. Action Sequence: [7.,3.]
34. Occurrences: 272. Action Sequence: [0.,3.]
35. Occurrences: 264. Action Sequence: [7.,2.]
36. Occurrences: 263. Action Sequence: [0.,2.]
37. Occurrences: 249. Action Sequence: [4.,5.]
38. Occurrences: 245. Action Sequence: [5.,2.]
39. Occurrences: 230. Action Sequence: [3.,5.]
40. Occurrences: 223. Action Sequence: [5.,3.]
41. Occurrences: 223. Action Sequence: [6.,5.]
42. Occurrences: 221. Action Sequence: [1.,5.]
43. Occurrences: 201. Action Sequence: [2.,5.]
44. Occurrences: 187. Action Sequence: [1.,7.]
45. Occurrences: 178. Action Sequence: [5.,5.]
46. Occurrences: 171. Action Sequence: [3.,7.]
47. Occurrences: 170. Action Sequence: [6.,7.]
48. Occurrences: 164. Action Sequence: [7.,5.]
49. Occurrences: 163. Action Sequence: [0.,5.]
50. Occurrences: 162. Action Sequence: [2.,7.]

FACTORED SPACE 0, factorisation 1
15305 different trajectories
1. Occurrences: 24414. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 22746. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 5308. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 5018. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1631. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1546. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1461. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1438. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 641. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 596. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 582. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 563. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 562. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 532. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 530. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 526. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 280. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 242. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 231. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 231. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 231. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 230. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 230. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 229. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 216. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 212. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 212. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 211. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 209. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 202. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 201. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 176. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 143. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 118. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 114. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 113. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 113. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 111. Action Sequence: [0.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 109. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 108. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 107. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 107. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 106. Action Sequence: [0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 105. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 103. Action Sequence: [0.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 102. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 100. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 100. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 100. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 99. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 1
29462 different trajectories
1. Occurrences: 14702. Action Sequence: [0.]
2. Occurrences: 13186. Action Sequence: [1.]
3. Occurrences: 6333. Action Sequence: [3.]
4. Occurrences: 4232. Action Sequence: [2.]
5. Occurrences: 1567. Action Sequence: [2.,1.]
6. Occurrences: 1557. Action Sequence: [1.,0.]
7. Occurrences: 1396. Action Sequence: [1.,1.]
8. Occurrences: 1217. Action Sequence: [0.,1.]
9. Occurrences: 1197. Action Sequence: [3.,0.]
10. Occurrences: 1180. Action Sequence: [0.,0.]
11. Occurrences: 1153. Action Sequence: [3.,1.]
12. Occurrences: 885. Action Sequence: [2.,0.]
13. Occurrences: 809. Action Sequence: [1.,3.]
14. Occurrences: 798. Action Sequence: [2.,3.]
15. Occurrences: 674. Action Sequence: [0.,3.]
16. Occurrences: 544. Action Sequence: [3.,3.]
17. Occurrences: 478. Action Sequence: [1.,2.]
18. Occurrences: 388. Action Sequence: [0.,2.]
19. Occurrences: 380. Action Sequence: [2.,0.,1.]
20. Occurrences: 330. Action Sequence: [3.,2.]
21. Occurrences: 270. Action Sequence: [2.,1.,1.]
22. Occurrences: 252. Action Sequence: [3.,1.,1.]
23. Occurrences: 238. Action Sequence: [2.,2.,1.]
24. Occurrences: 231. Action Sequence: [3.,0.,1.]
25. Occurrences: 226. Action Sequence: [3.,1.,0.]
26. Occurrences: 220. Action Sequence: [2.,1.,0.]
27. Occurrences: 214. Action Sequence: [2.,0.,0.]
28. Occurrences: 205. Action Sequence: [3.,0.,0.]
29. Occurrences: 201. Action Sequence: [3.,2.,1.]
30. Occurrences: 194. Action Sequence: [1.,1.,1.]
31. Occurrences: 192. Action Sequence: [2.,3.,1.]
32. Occurrences: 188. Action Sequence: [3.,3.,1.]
33. Occurrences: 186. Action Sequence: [1.,1.,0.]
34. Occurrences: 184. Action Sequence: [1.,2.,1.]
35. Occurrences: 183. Action Sequence: [0.,2.,1.]
36. Occurrences: 172. Action Sequence: [0.,1.,0.]
37. Occurrences: 169. Action Sequence: [2.,3.,0.]
38. Occurrences: 158. Action Sequence: [2.,0.,3.]
39. Occurrences: 158. Action Sequence: [1.,0.,1.]
40. Occurrences: 156. Action Sequence: [3.,3.,0.]
41. Occurrences: 155. Action Sequence: [2.,2.,0.]
42. Occurrences: 155. Action Sequence: [2.,2.]
43. Occurrences: 149. Action Sequence: [1.,3.,0.]
44. Occurrences: 148. Action Sequence: [0.,0.,1.]
45. Occurrences: 148. Action Sequence: [0.,1.,1.]
46. Occurrences: 146. Action Sequence: [3.,2.,0.]
47. Occurrences: 136. Action Sequence: [1.,3.,1.]
48. Occurrences: 127. Action Sequence: [1.,0.,0.]
49. Occurrences: 127. Action Sequence: [0.,3.,1.]
50. Occurrences: 124. Action Sequence: [0.,0.,0.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 2
14636 different trajectories
1. Occurrences: 25043. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 24769. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 5218. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 4615. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1496. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1447. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1442. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1409. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 587. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 547. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 544. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 541. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 496. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 496. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 481. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 467. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 278. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 243. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 229. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 219. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 216. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 215. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 212. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 211. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 209. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 206. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 204. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 204. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 202. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 193. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 191. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 181. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 131. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 121. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 117. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 109. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 108. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 106. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 105. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 105. Action Sequence: [0.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 104. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 103. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 102. Action Sequence: [0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 99. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 98. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 98. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 92. Action Sequence: [0.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 92. Action Sequence: [0.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 90. Action Sequence: [0.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 89. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 2
29467 different trajectories
1. Occurrences: 14431. Action Sequence: [0.]
2. Occurrences: 13457. Action Sequence: [1.]
3. Occurrences: 5802. Action Sequence: [2.]
4. Occurrences: 4763. Action Sequence: [3.]
5. Occurrences: 1485. Action Sequence: [1.,0.]
6. Occurrences: 1399. Action Sequence: [3.,0.]
7. Occurrences: 1394. Action Sequence: [2.,0.]
8. Occurrences: 1330. Action Sequence: [1.,1.]
9. Occurrences: 1305. Action Sequence: [0.,0.]
10. Occurrences: 1230. Action Sequence: [0.,1.]
11. Occurrences: 1033. Action Sequence: [3.,1.]
12. Occurrences: 976. Action Sequence: [2.,1.]
13. Occurrences: 679. Action Sequence: [1.,2.]
14. Occurrences: 603. Action Sequence: [0.,2.]
15. Occurrences: 603. Action Sequence: [2.,2.]
16. Occurrences: 547. Action Sequence: [1.,3.]
17. Occurrences: 539. Action Sequence: [3.,2.]
18. Occurrences: 520. Action Sequence: [0.,3.]
19. Occurrences: 348. Action Sequence: [3.,3.]
20. Occurrences: 337. Action Sequence: [2.,3.]
21. Occurrences: 327. Action Sequence: [3.,1.,0.]
22. Occurrences: 274. Action Sequence: [3.,0.,0.]
23. Occurrences: 271. Action Sequence: [3.,2.,0.]
24. Occurrences: 264. Action Sequence: [2.,1.,0.]
25. Occurrences: 257. Action Sequence: [2.,0.,0.]
26. Occurrences: 234. Action Sequence: [2.,3.,0.]
27. Occurrences: 234. Action Sequence: [3.,1.,1.]
28. Occurrences: 230. Action Sequence: [3.,0.,1.]
29. Occurrences: 220. Action Sequence: [2.,0.,1.]
30. Occurrences: 205. Action Sequence: [2.,2.,0.]
31. Occurrences: 203. Action Sequence: [3.,3.,0.]
32. Occurrences: 196. Action Sequence: [1.,3.,0.]
33. Occurrences: 192. Action Sequence: [2.,1.,1.]
34. Occurrences: 191. Action Sequence: [1.,1.,0.]
35. Occurrences: 173. Action Sequence: [1.,0.,0.]
36. Occurrences: 165. Action Sequence: [0.,3.,0.]
37. Occurrences: 164. Action Sequence: [1.,0.,1.]
38. Occurrences: 158. Action Sequence: [1.,1.,1.]
39. Occurrences: 150. Action Sequence: [0.,0.,0.]
40. Occurrences: 146. Action Sequence: [0.,1.,0.]
41. Occurrences: 145. Action Sequence: [1.,2.,0.]
42. Occurrences: 141. Action Sequence: [0.,0.,1.]
43. Occurrences: 140. Action Sequence: [3.,2.,1.]
44. Occurrences: 137. Action Sequence: [2.,3.,1.]
45. Occurrences: 136. Action Sequence: [0.,2.,0.]
46. Occurrences: 134. Action Sequence: [0.,1.,1.]
47. Occurrences: 129. Action Sequence: [3.,3.,1.]
48. Occurrences: 126. Action Sequence: [2.,2.,1.]
49. Occurrences: 122. Action Sequence: [1.,2.,1.]
50. Occurrences: 110. Action Sequence: [2.,0.,2.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 3
17095 different trajectories
1. Occurrences: 27888. Action Sequence: [0.]
2. Occurrences: 10565. Action Sequence: [1.]
3. Occurrences: 5350. Action Sequence: [0.,0.]
4. Occurrences: 4802. Action Sequence: [1.,0.]
5. Occurrences: 2349. Action Sequence: [0.,1.]
6. Occurrences: 1998. Action Sequence: [1.,0.,0.]
7. Occurrences: 1827. Action Sequence: [1.,1.]
8. Occurrences: 1445. Action Sequence: [1.,1.,0.]
9. Occurrences: 1257. Action Sequence: [0.,0.,0.]
10. Occurrences: 1074. Action Sequence: [0.,1.,0.]
11. Occurrences: 845. Action Sequence: [1.,1.,0.,0.]
12. Occurrences: 676. Action Sequence: [1.,0.,0.,0.]
13. Occurrences: 626. Action Sequence: [1.,0.,1.]
14. Occurrences: 548. Action Sequence: [1.,1.,1.,0.]
15. Occurrences: 514. Action Sequence: [1.,0.,1.,0.]
16. Occurrences: 479. Action Sequence: [0.,0.,1.]
17. Occurrences: 468. Action Sequence: [1.,1.,1.]
18. Occurrences: 422. Action Sequence: [1.,1.,1.,0.,0.]
19. Occurrences: 414. Action Sequence: [0.,1.,1.]
20. Occurrences: 389. Action Sequence: [0.,1.,0.,0.]
21. Occurrences: 331. Action Sequence: [0.,1.,1.,0.]
22. Occurrences: 316. Action Sequence: [1.,1.,0.,0.,0.]
23. Occurrences: 304. Action Sequence: [1.,0.,1.,0.,0.]
24. Occurrences: 294. Action Sequence: [0.,0.,0.,0.]
25. Occurrences: 263. Action Sequence: [1.,0.,0.,0.,0.]
26. Occurrences: 262. Action Sequence: [0.,0.,1.,0.]
27. Occurrences: 258. Action Sequence: [1.,1.,0.,1.,0.]
28. Occurrences: 255. Action Sequence: [1.,1.,1.,1.,0.]
29. Occurrences: 223. Action Sequence: [1.,1.,0.,1.]
30. Occurrences: 222. Action Sequence: [1.,0.,0.,1.]
31. Occurrences: 202. Action Sequence: [1.,0.,0.,1.,0.]
32. Occurrences: 195. Action Sequence: [1.,0.,1.,1.,0.]
33. Occurrences: 182. Action Sequence: [1.,1.,1.,1.,0.,0.]
34. Occurrences: 179. Action Sequence: [0.,1.,1.,0.,0.]
35. Occurrences: 178. Action Sequence: [1.,1.,1.,0.,0.,0.]
36. Occurrences: 173. Action Sequence: [1.,1.,0.,1.,0.,0.]
37. Occurrences: 173. Action Sequence: [1.,0.,1.,1.]
38. Occurrences: 162. Action Sequence: [1.,1.,1.,1.]
39. Occurrences: 161. Action Sequence: [0.,1.,0.,0.,0.]
40. Occurrences: 145. Action Sequence: [0.,1.,0.,1.]
41. Occurrences: 137. Action Sequence: [1.,0.,1.,1.,0.,0.]
42. Occurrences: 136. Action Sequence: [1.,1.,0.,0.,0.,0.]
43. Occurrences: 129. Action Sequence: [0.,1.,1.,1.]
44. Occurrences: 124. Action Sequence: [1.,0.,1.,0.,0.,0.]
45. Occurrences: 124. Action Sequence: [1.,0.,0.,1.,0.,0.]
46. Occurrences: 122. Action Sequence: [1.,1.,1.,0.,1.,0.]
47. Occurrences: 120. Action Sequence: [0.,1.,0.,1.,0.]
48. Occurrences: 118. Action Sequence: [0.,1.,1.,1.,0.]
49. Occurrences: 116. Action Sequence: [0.,0.,0.,1.]
50. Occurrences: 114. Action Sequence: [1.,0.,1.,1.,1.,0.]

FACTORED SPACE 1, factorisation 3
28716 different trajectories
1. Occurrences: 11322. Action Sequence: [1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
2. Occurrences: 10933. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
3. Occurrences: 10186. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
4. Occurrences: 10148. Action Sequence: [2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
5. Occurrences: 1318. Action Sequence: [2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
6. Occurrences: 1295. Action Sequence: [0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
7. Occurrences: 1262. Action Sequence: [3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
8. Occurrences: 1193. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
9. Occurrences: 1158. Action Sequence: [1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
10. Occurrences: 1055. Action Sequence: [1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
11. Occurrences: 1017. Action Sequence: [3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
12. Occurrences: 945. Action Sequence: [2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
13. Occurrences: 942. Action Sequence: [0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
14. Occurrences: 886. Action Sequence: [1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
15. Occurrences: 738. Action Sequence: [2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
16. Occurrences: 674. Action Sequence: [0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
17. Occurrences: 232. Action Sequence: [0.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
18. Occurrences: 229. Action Sequence: [2.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
19. Occurrences: 209. Action Sequence: [3.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
20. Occurrences: 201. Action Sequence: [2.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
21. Occurrences: 200. Action Sequence: [2.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
22. Occurrences: 197. Action Sequence: [0.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
23. Occurrences: 181. Action Sequence: [1.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
24. Occurrences: 179. Action Sequence: [1.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
25. Occurrences: 179. Action Sequence: [3.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
26. Occurrences: 179. Action Sequence: [3.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
27. Occurrences: 179. Action Sequence: [2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
28. Occurrences: 177. Action Sequence: [3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
29. Occurrences: 173. Action Sequence: [2.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
30. Occurrences: 170. Action Sequence: [0.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
31. Occurrences: 169. Action Sequence: [2.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
32. Occurrences: 169. Action Sequence: [3.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
33. Occurrences: 157. Action Sequence: [0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
34. Occurrences: 155. Action Sequence: [3.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
35. Occurrences: 154. Action Sequence: [1.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
36. Occurrences: 154. Action Sequence: [1.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
37. Occurrences: 150. Action Sequence: [3.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
38. Occurrences: 149. Action Sequence: [1.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
39. Occurrences: 148. Action Sequence: [2.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
40. Occurrences: 146. Action Sequence: [0.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
41. Occurrences: 146. Action Sequence: [0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
42. Occurrences: 143. Action Sequence: [2.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
43. Occurrences: 138. Action Sequence: [3.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
44. Occurrences: 135. Action Sequence: [0.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
45. Occurrences: 135. Action Sequence: [1.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
46. Occurrences: 131. Action Sequence: [1.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
47. Occurrences: 131. Action Sequence: [1.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
48. Occurrences: 130. Action Sequence: [0.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
49. Occurrences: 120. Action Sequence: [2.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
50. Occurrences: 110. Action Sequence: [3.,1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 4
17095 different trajectories
1. Occurrences: 27888. Action Sequence: [0.]
2. Occurrences: 10565. Action Sequence: [1.]
3. Occurrences: 5350. Action Sequence: [0.,0.]
4. Occurrences: 4802. Action Sequence: [1.,0.]
5. Occurrences: 2349. Action Sequence: [0.,1.]
6. Occurrences: 1998. Action Sequence: [1.,0.,0.]
7. Occurrences: 1827. Action Sequence: [1.,1.]
8. Occurrences: 1445. Action Sequence: [1.,1.,0.]
9. Occurrences: 1257. Action Sequence: [0.,0.,0.]
10. Occurrences: 1074. Action Sequence: [0.,1.,0.]
11. Occurrences: 845. Action Sequence: [1.,1.,0.,0.]
12. Occurrences: 676. Action Sequence: [1.,0.,0.,0.]
13. Occurrences: 626. Action Sequence: [1.,0.,1.]
14. Occurrences: 548. Action Sequence: [1.,1.,1.,0.]
15. Occurrences: 514. Action Sequence: [1.,0.,1.,0.]
16. Occurrences: 479. Action Sequence: [0.,0.,1.]
17. Occurrences: 468. Action Sequence: [1.,1.,1.]
18. Occurrences: 422. Action Sequence: [1.,1.,1.,0.,0.]
19. Occurrences: 414. Action Sequence: [0.,1.,1.]
20. Occurrences: 389. Action Sequence: [0.,1.,0.,0.]
21. Occurrences: 331. Action Sequence: [0.,1.,1.,0.]
22. Occurrences: 316. Action Sequence: [1.,1.,0.,0.,0.]
23. Occurrences: 304. Action Sequence: [1.,0.,1.,0.,0.]
24. Occurrences: 294. Action Sequence: [0.,0.,0.,0.]
25. Occurrences: 263. Action Sequence: [1.,0.,0.,0.,0.]
26. Occurrences: 262. Action Sequence: [0.,0.,1.,0.]
27. Occurrences: 258. Action Sequence: [1.,1.,0.,1.,0.]
28. Occurrences: 255. Action Sequence: [1.,1.,1.,1.,0.]
29. Occurrences: 223. Action Sequence: [1.,1.,0.,1.]
30. Occurrences: 222. Action Sequence: [1.,0.,0.,1.]
31. Occurrences: 202. Action Sequence: [1.,0.,0.,1.,0.]
32. Occurrences: 195. Action Sequence: [1.,0.,1.,1.,0.]
33. Occurrences: 182. Action Sequence: [1.,1.,1.,1.,0.,0.]
34. Occurrences: 179. Action Sequence: [0.,1.,1.,0.,0.]
35. Occurrences: 178. Action Sequence: [1.,1.,1.,0.,0.,0.]
36. Occurrences: 173. Action Sequence: [1.,1.,0.,1.,0.,0.]
37. Occurrences: 173. Action Sequence: [1.,0.,1.,1.]
38. Occurrences: 162. Action Sequence: [1.,1.,1.,1.]
39. Occurrences: 161. Action Sequence: [0.,1.,0.,0.,0.]
40. Occurrences: 145. Action Sequence: [0.,1.,0.,1.]
41. Occurrences: 137. Action Sequence: [1.,0.,1.,1.,0.,0.]
42. Occurrences: 136. Action Sequence: [1.,1.,0.,0.,0.,0.]
43. Occurrences: 129. Action Sequence: [0.,1.,1.,1.]
44. Occurrences: 124. Action Sequence: [1.,0.,1.,0.,0.,0.]
45. Occurrences: 124. Action Sequence: [1.,0.,0.,1.,0.,0.]
46. Occurrences: 122. Action Sequence: [1.,1.,1.,0.,1.,0.]
47. Occurrences: 120. Action Sequence: [0.,1.,0.,1.,0.]
48. Occurrences: 118. Action Sequence: [0.,1.,1.,1.,0.]
49. Occurrences: 116. Action Sequence: [0.,0.,0.,1.]
50. Occurrences: 114. Action Sequence: [1.,0.,1.,1.,1.,0.]

FACTORED SPACE 1, factorisation 4
15305 different trajectories
1. Occurrences: 24414. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 22746. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 5308. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 5018. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1631. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1546. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1461. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1438. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 641. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 596. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 582. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 563. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 562. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 532. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 530. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 526. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 280. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 242. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 231. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 231. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 231. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 230. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 230. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 229. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 216. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 212. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 212. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 211. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 209. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 202. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 201. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 176. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 143. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 118. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 114. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 113. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 113. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 111. Action Sequence: [0.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 109. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 108. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 107. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 107. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 106. Action Sequence: [0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 105. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 103. Action Sequence: [0.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 102. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 100. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 100. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 100. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 99. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 2, factorisation 4
14636 different trajectories
1. Occurrences: 25043. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 24769. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 5218. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 4615. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1496. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1447. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1442. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1409. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 587. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 547. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 544. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 541. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 496. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 496. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 481. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 467. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 278. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 243. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 229. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 219. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 216. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 215. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 212. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 211. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 209. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 206. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 204. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 204. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 202. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 193. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 191. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 181. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 131. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 121. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 117. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 109. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 108. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 106. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 105. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 105. Action Sequence: [0.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 104. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 103. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 102. Action Sequence: [0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 99. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 98. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 98. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 92. Action Sequence: [0.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 92. Action Sequence: [0.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 90. Action Sequence: [0.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 89. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

--------------------------------------------------------
'''

### State 703, No Diabetes

In [ ]:
'''
NON FACTORED SPACE
75707 different trajectories
1. Occurrences: 3151. Action Sequence: [0.]
2. Occurrences: 2273. Action Sequence: [2.]
3. Occurrences: 1263. Action Sequence: [1.]
4. Occurrences: 968. Action Sequence: [4.]
5. Occurrences: 613. Action Sequence: [6.]
6. Occurrences: 533. Action Sequence: [5.]
7. Occurrences: 385. Action Sequence: [3.,0.]
8. Occurrences: 347. Action Sequence: [1.,0.]
9. Occurrences: 340. Action Sequence: [7.,0.]
10. Occurrences: 334. Action Sequence: [4.,0.]
11. Occurrences: 325. Action Sequence: [6.,0.]
12. Occurrences: 305. Action Sequence: [5.,0.]
13. Occurrences: 295. Action Sequence: [2.,0.]
14. Occurrences: 261. Action Sequence: [0.,0.]
15. Occurrences: 259. Action Sequence: [3.,2.]
16. Occurrences: 233. Action Sequence: [1.,2.]
17. Occurrences: 193. Action Sequence: [2.,2.]
18. Occurrences: 180. Action Sequence: [0.,2.]
19. Occurrences: 153. Action Sequence: [0.,1.]
20. Occurrences: 142. Action Sequence: [1.,1.]
21. Occurrences: 139. Action Sequence: [4.,1.]
22. Occurrences: 138. Action Sequence: [7.,2.]
23. Occurrences: 131. Action Sequence: [1.,4.]
24. Occurrences: 130. Action Sequence: [3.,4.]
25. Occurrences: 130. Action Sequence: [6.,1.]
26. Occurrences: 127. Action Sequence: [3.,1.]
27. Occurrences: 127. Action Sequence: [2.,1.]
28. Occurrences: 125. Action Sequence: [5.,2.]
29. Occurrences: 97. Action Sequence: [5.,1.]
30. Occurrences: 94. Action Sequence: [2.,4.]
31. Occurrences: 91. Action Sequence: [4.,2.]
32. Occurrences: 88. Action Sequence: [6.,2.]
33. Occurrences: 85. Action Sequence: [3.,6.]
34. Occurrences: 78. Action Sequence: [4.,5.]
35. Occurrences: 77. Action Sequence: [4.,3.]
36. Occurrences: 77. Action Sequence: [6.,3.]
37. Occurrences: 69. Action Sequence: [1.,5.]
38. Occurrences: 69. Action Sequence: [5.,4.]
39. Occurrences: 66. Action Sequence: [1.,6.]
40. Occurrences: 66. Action Sequence: [0.,5.]
41. Occurrences: 65. Action Sequence: [3.,5.]
42. Occurrences: 65. Action Sequence: [7.,1.]
43. Occurrences: 61. Action Sequence: [6.,5.]
44. Occurrences: 61. Action Sequence: [7.,4.]
45. Occurrences: 60. Action Sequence: [0.,4.]
46. Occurrences: 59. Action Sequence: [7.,6.,0.]
47. Occurrences: 54. Action Sequence: [6.,4.]
48. Occurrences: 53. Action Sequence: [5.,6.,0.]
49. Occurrences: 51. Action Sequence: [2.,6.]
50. Occurrences: 50. Action Sequence: [6.,6.,0.]

FACTORED SPACE 0, factorisation 1
40088 different trajectories
1. Occurrences: 7562. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 4652. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 3102. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 3034. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1371. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1362. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1311. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1310. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 672. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 669. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 658. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 657. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 631. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 615. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 613. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 561. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 332. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 330. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 329. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 317. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 316. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 313. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 313. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 310. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 296. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 291. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 281. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 269. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 267. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 265. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 259. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 256. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 168. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 167. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 166. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 161. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 159. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 155. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 153. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 151. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 151. Action Sequence: [0.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 150. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 149. Action Sequence: [0.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 149. Action Sequence: [0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 148. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 146. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 142. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 141. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 140. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 139. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 1
66592 different trajectories
1. Occurrences: 5424. Action Sequence: [0.]
2. Occurrences: 1581. Action Sequence: [2.]
3. Occurrences: 1263. Action Sequence: [1.]
4. Occurrences: 1224. Action Sequence: [1.,0.]
5. Occurrences: 929. Action Sequence: [0.,0.]
6. Occurrences: 908. Action Sequence: [3.,0.]
7. Occurrences: 838. Action Sequence: [2.,0.]
8. Occurrences: 533. Action Sequence: [3.]
9. Occurrences: 423. Action Sequence: [2.,1.]
10. Occurrences: 412. Action Sequence: [1.,2.]
11. Occurrences: 315. Action Sequence: [0.,1.]
12. Occurrences: 269. Action Sequence: [1.,1.]
13. Occurrences: 254. Action Sequence: [0.,2.]
14. Occurrences: 248. Action Sequence: [1.,1.,0.]
15. Occurrences: 226. Action Sequence: [3.,2.,0.]
16. Occurrences: 219. Action Sequence: [2.,1.,0.]
17. Occurrences: 215. Action Sequence: [3.,1.,0.]
18. Occurrences: 202. Action Sequence: [1.,2.,0.]
19. Occurrences: 201. Action Sequence: [1.,0.,0.]
20. Occurrences: 200. Action Sequence: [3.,2.]
21. Occurrences: 197. Action Sequence: [0.,0.,0.]
22. Occurrences: 196. Action Sequence: [2.,3.]
23. Occurrences: 194. Action Sequence: [0.,1.,0.]
24. Occurrences: 190. Action Sequence: [3.,0.,0.]
25. Occurrences: 186. Action Sequence: [2.,2.,0.]
26. Occurrences: 183. Action Sequence: [1.,3.,0.]
27. Occurrences: 183. Action Sequence: [2.,0.,1.]
28. Occurrences: 179. Action Sequence: [2.,3.,0.]
29. Occurrences: 177. Action Sequence: [0.,2.,0.]
30. Occurrences: 174. Action Sequence: [0.,3.,0.]
31. Occurrences: 174. Action Sequence: [2.,0.,0.]
32. Occurrences: 172. Action Sequence: [3.,3.,0.]
33. Occurrences: 164. Action Sequence: [3.,1.]
34. Occurrences: 137. Action Sequence: [0.,3.]
35. Occurrences: 134. Action Sequence: [1.,3.]
36. Occurrences: 131. Action Sequence: [2.,2.]
37. Occurrences: 128. Action Sequence: [2.,2.,1.]
38. Occurrences: 121. Action Sequence: [0.,2.,1.]
39. Occurrences: 102. Action Sequence: [1.,2.,1.]
40. Occurrences: 91. Action Sequence: [0.,0.,1.]
41. Occurrences: 88. Action Sequence: [3.,3.]
42. Occurrences: 87. Action Sequence: [1.,1.,2.]
43. Occurrences: 83. Action Sequence: [1.,0.,1.]
44. Occurrences: 75. Action Sequence: [3.,0.,1.]
45. Occurrences: 74. Action Sequence: [2.,1.,1.]
46. Occurrences: 72. Action Sequence: [2.,0.,3.]
47. Occurrences: 71. Action Sequence: [3.,2.,1.]
48. Occurrences: 71. Action Sequence: [1.,0.,2.]
49. Occurrences: 70. Action Sequence: [2.,3.,1.]
50. Occurrences: 65. Action Sequence: [3.,1.,2.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 2
39157 different trajectories
1. Occurrences: 8661. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 3535. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 3216. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2883. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1490. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1473. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1405. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1350. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 797. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 718. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 695. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 677. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 662. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 633. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 591. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 557. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 402. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 346. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 336. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 322. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 313. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 313. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 313. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 309. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 308. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 308. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 300. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 281. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 272. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 268. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 268. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 257. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 182. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 177. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 174. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 172. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 170. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 167. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 167. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 165. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 159. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 155. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 154. Action Sequence: [0.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 151. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 151. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 148. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 145. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 145. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 145. Action Sequence: [0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 139. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 2
66431 different trajectories
1. Occurrences: 4414. Action Sequence: [0.]
2. Occurrences: 2273. Action Sequence: [1.]
3. Occurrences: 1501. Action Sequence: [2.]
4. Occurrences: 934. Action Sequence: [1.,0.]
5. Occurrences: 903. Action Sequence: [0.,0.]
6. Occurrences: 875. Action Sequence: [2.,0.]
7. Occurrences: 860. Action Sequence: [3.,0.]
8. Occurrences: 613. Action Sequence: [3.]
9. Occurrences: 470. Action Sequence: [1.,1.]
10. Occurrences: 430. Action Sequence: [0.,1.]
11. Occurrences: 337. Action Sequence: [1.,2.]
12. Occurrences: 326. Action Sequence: [0.,2.]
13. Occurrences: 303. Action Sequence: [3.,1.]
14. Occurrences: 295. Action Sequence: [2.,1.]
15. Occurrences: 239. Action Sequence: [2.,2.]
16. Occurrences: 232. Action Sequence: [3.,3.,0.]
17. Occurrences: 214. Action Sequence: [3.,2.]
18. Occurrences: 213. Action Sequence: [2.,2.,0.]
19. Occurrences: 210. Action Sequence: [3.,2.,0.]
20. Occurrences: 207. Action Sequence: [3.,1.,0.]
21. Occurrences: 207. Action Sequence: [0.,1.,0.]
22. Occurrences: 205. Action Sequence: [2.,1.,0.]
23. Occurrences: 204. Action Sequence: [1.,3.,0.]
24. Occurrences: 203. Action Sequence: [2.,3.,0.]
25. Occurrences: 202. Action Sequence: [3.,0.,0.]
26. Occurrences: 201. Action Sequence: [0.,2.,0.]
27. Occurrences: 200. Action Sequence: [0.,3.,0.]
28. Occurrences: 197. Action Sequence: [2.,0.,0.]
29. Occurrences: 196. Action Sequence: [1.,1.,0.]
30. Occurrences: 188. Action Sequence: [0.,0.,0.]
31. Occurrences: 181. Action Sequence: [1.,2.,0.]
32. Occurrences: 172. Action Sequence: [1.,0.,0.]
33. Occurrences: 148. Action Sequence: [1.,3.]
34. Occurrences: 126. Action Sequence: [0.,3.]
35. Occurrences: 106. Action Sequence: [3.,0.,1.]
36. Occurrences: 102. Action Sequence: [0.,1.,1.]
37. Occurrences: 99. Action Sequence: [1.,0.,1.]
38. Occurrences: 91. Action Sequence: [3.,1.,1.]
39. Occurrences: 89. Action Sequence: [2.,1.,1.]
40. Occurrences: 88. Action Sequence: [1.,1.,1.]
41. Occurrences: 85. Action Sequence: [0.,0.,1.]
42. Occurrences: 85. Action Sequence: [2.,0.,1.]
43. Occurrences: 82. Action Sequence: [2.,3.]
44. Occurrences: 80. Action Sequence: [3.,3.]
45. Occurrences: 75. Action Sequence: [0.,1.,2.]
46. Occurrences: 73. Action Sequence: [1.,2.,1.]
47. Occurrences: 72. Action Sequence: [1.,1.,2.]
48. Occurrences: 72. Action Sequence: [1.,0.,2.]
49. Occurrences: 72. Action Sequence: [0.,2.,1.]
50. Occurrences: 71. Action Sequence: [2.,1.,2.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 3
44516 different trajectories
1. Occurrences: 6687. Action Sequence: [0.]
2. Occurrences: 2737. Action Sequence: [0.,0.]
3. Occurrences: 2333. Action Sequence: [1.,0.]
4. Occurrences: 2114. Action Sequence: [1.]
5. Occurrences: 1182. Action Sequence: [1.,0.,0.]
6. Occurrences: 1137. Action Sequence: [0.,0.,0.]
7. Occurrences: 1064. Action Sequence: [1.,1.,0.]
8. Occurrences: 1056. Action Sequence: [0.,1.,0.]
9. Occurrences: 937. Action Sequence: [0.,1.]
10. Occurrences: 615. Action Sequence: [1.,1.]
11. Occurrences: 579. Action Sequence: [1.,1.,0.,0.]
12. Occurrences: 563. Action Sequence: [0.,1.,0.,0.]
13. Occurrences: 545. Action Sequence: [1.,0.,0.,0.]
14. Occurrences: 543. Action Sequence: [1.,0.,1.,0.]
15. Occurrences: 517. Action Sequence: [1.,1.,1.,0.]
16. Occurrences: 517. Action Sequence: [0.,1.,1.,0.]
17. Occurrences: 514. Action Sequence: [0.,0.,0.,0.]
18. Occurrences: 457. Action Sequence: [0.,0.,1.,0.]
19. Occurrences: 386. Action Sequence: [0.,0.,1.]
20. Occurrences: 344. Action Sequence: [1.,0.,1.]
21. Occurrences: 309. Action Sequence: [1.,1.,1.,0.,0.]
22. Occurrences: 297. Action Sequence: [1.,1.,0.,0.,0.]
23. Occurrences: 288. Action Sequence: [1.,0.,0.,0.,0.]
24. Occurrences: 286. Action Sequence: [0.,1.,1.,0.,0.]
25. Occurrences: 284. Action Sequence: [1.,0.,1.,0.,0.]
26. Occurrences: 255. Action Sequence: [0.,1.,0.,0.,0.]
27. Occurrences: 255. Action Sequence: [1.,1.,1.,1.,0.]
28. Occurrences: 251. Action Sequence: [0.,1.,1.]
29. Occurrences: 251. Action Sequence: [1.,0.,1.,1.,0.]
30. Occurrences: 245. Action Sequence: [1.,1.,0.,1.,0.]
31. Occurrences: 228. Action Sequence: [0.,1.,1.,1.,0.]
32. Occurrences: 222. Action Sequence: [0.,0.,1.,0.,0.]
33. Occurrences: 222. Action Sequence: [0.,1.,0.,1.,0.]
34. Occurrences: 215. Action Sequence: [1.,1.,1.]
35. Occurrences: 213. Action Sequence: [1.,0.,0.,1.,0.]
36. Occurrences: 212. Action Sequence: [0.,0.,0.,0.,0.]
37. Occurrences: 209. Action Sequence: [0.,0.,0.,1.,0.]
38. Occurrences: 206. Action Sequence: [0.,0.,1.,1.,0.]
39. Occurrences: 167. Action Sequence: [1.,1.,1.,0.,0.,0.]
40. Occurrences: 162. Action Sequence: [1.,0.,0.,1.]
41. Occurrences: 158. Action Sequence: [0.,1.,0.,1.]
42. Occurrences: 156. Action Sequence: [1.,1.,0.,1.,0.,0.]
43. Occurrences: 154. Action Sequence: [0.,0.,0.,1.]
44. Occurrences: 154. Action Sequence: [1.,0.,1.,0.,0.,0.]
45. Occurrences: 149. Action Sequence: [1.,1.,1.,1.,0.,0.]
46. Occurrences: 148. Action Sequence: [1.,1.,0.,0.,0.,0.]
47. Occurrences: 143. Action Sequence: [0.,1.,1.,1.,0.,0.]
48. Occurrences: 139. Action Sequence: [1.,1.,1.,0.,1.,0.]
49. Occurrences: 135. Action Sequence: [1.,1.,0.,1.]
50. Occurrences: 130. Action Sequence: [1.,0.,0.,1.,0.,0.]

FACTORED SPACE 1, factorisation 3
65714 different trajectories
1. Occurrences: 4271. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
2. Occurrences: 3032. Action Sequence: [2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
3. Occurrences: 1798. Action Sequence: [1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
4. Occurrences: 963. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
5. Occurrences: 896. Action Sequence: [1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
6. Occurrences: 839. Action Sequence: [2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
7. Occurrences: 763. Action Sequence: [0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
8. Occurrences: 547. Action Sequence: [3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
9. Occurrences: 492. Action Sequence: [1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
10. Occurrences: 451. Action Sequence: [0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
11. Occurrences: 416. Action Sequence: [2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
12. Occurrences: 403. Action Sequence: [0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
13. Occurrences: 384. Action Sequence: [2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
14. Occurrences: 363. Action Sequence: [1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
15. Occurrences: 295. Action Sequence: [3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
16. Occurrences: 220. Action Sequence: [3.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
17. Occurrences: 208. Action Sequence: [0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
18. Occurrences: 195. Action Sequence: [3.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
19. Occurrences: 192. Action Sequence: [2.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
20. Occurrences: 192. Action Sequence: [2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
21. Occurrences: 191. Action Sequence: [3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
22. Occurrences: 189. Action Sequence: [1.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
23. Occurrences: 183. Action Sequence: [1.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
24. Occurrences: 182. Action Sequence: [1.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
25. Occurrences: 178. Action Sequence: [1.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
26. Occurrences: 174. Action Sequence: [3.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
27. Occurrences: 174. Action Sequence: [0.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
28. Occurrences: 169. Action Sequence: [0.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
29. Occurrences: 167. Action Sequence: [2.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
30. Occurrences: 167. Action Sequence: [0.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
31. Occurrences: 159. Action Sequence: [2.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
32. Occurrences: 146. Action Sequence: [0.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
33. Occurrences: 137. Action Sequence: [2.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
34. Occurrences: 133. Action Sequence: [0.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
35. Occurrences: 123. Action Sequence: [2.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
36. Occurrences: 102. Action Sequence: [3.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
37. Occurrences: 100. Action Sequence: [1.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
38. Occurrences: 98. Action Sequence: [1.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
39. Occurrences: 98. Action Sequence: [2.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
40. Occurrences: 97. Action Sequence: [3.,1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
41. Occurrences: 95. Action Sequence: [1.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
42. Occurrences: 94. Action Sequence: [1.,1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
43. Occurrences: 93. Action Sequence: [1.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
44. Occurrences: 91. Action Sequence: [3.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
45. Occurrences: 89. Action Sequence: [2.,0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
46. Occurrences: 87. Action Sequence: [3.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
47. Occurrences: 87. Action Sequence: [0.,1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
48. Occurrences: 84. Action Sequence: [0.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
49. Occurrences: 84. Action Sequence: [2.,1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
50. Occurrences: 83. Action Sequence: [0.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 4
44516 different trajectories
1. Occurrences: 6687. Action Sequence: [0.]
2. Occurrences: 2737. Action Sequence: [0.,0.]
3. Occurrences: 2333. Action Sequence: [1.,0.]
4. Occurrences: 2114. Action Sequence: [1.]
5. Occurrences: 1182. Action Sequence: [1.,0.,0.]
6. Occurrences: 1137. Action Sequence: [0.,0.,0.]
7. Occurrences: 1064. Action Sequence: [1.,1.,0.]
8. Occurrences: 1056. Action Sequence: [0.,1.,0.]
9. Occurrences: 937. Action Sequence: [0.,1.]
10. Occurrences: 615. Action Sequence: [1.,1.]
11. Occurrences: 579. Action Sequence: [1.,1.,0.,0.]
12. Occurrences: 563. Action Sequence: [0.,1.,0.,0.]
13. Occurrences: 545. Action Sequence: [1.,0.,0.,0.]
14. Occurrences: 543. Action Sequence: [1.,0.,1.,0.]
15. Occurrences: 517. Action Sequence: [1.,1.,1.,0.]
16. Occurrences: 517. Action Sequence: [0.,1.,1.,0.]
17. Occurrences: 514. Action Sequence: [0.,0.,0.,0.]
18. Occurrences: 457. Action Sequence: [0.,0.,1.,0.]
19. Occurrences: 386. Action Sequence: [0.,0.,1.]
20. Occurrences: 344. Action Sequence: [1.,0.,1.]
21. Occurrences: 309. Action Sequence: [1.,1.,1.,0.,0.]
22. Occurrences: 297. Action Sequence: [1.,1.,0.,0.,0.]
23. Occurrences: 288. Action Sequence: [1.,0.,0.,0.,0.]
24. Occurrences: 286. Action Sequence: [0.,1.,1.,0.,0.]
25. Occurrences: 284. Action Sequence: [1.,0.,1.,0.,0.]
26. Occurrences: 255. Action Sequence: [0.,1.,0.,0.,0.]
27. Occurrences: 255. Action Sequence: [1.,1.,1.,1.,0.]
28. Occurrences: 251. Action Sequence: [0.,1.,1.]
29. Occurrences: 251. Action Sequence: [1.,0.,1.,1.,0.]
30. Occurrences: 245. Action Sequence: [1.,1.,0.,1.,0.]
31. Occurrences: 228. Action Sequence: [0.,1.,1.,1.,0.]
32. Occurrences: 222. Action Sequence: [0.,0.,1.,0.,0.]
33. Occurrences: 222. Action Sequence: [0.,1.,0.,1.,0.]
34. Occurrences: 215. Action Sequence: [1.,1.,1.]
35. Occurrences: 213. Action Sequence: [1.,0.,0.,1.,0.]
36. Occurrences: 212. Action Sequence: [0.,0.,0.,0.,0.]
37. Occurrences: 209. Action Sequence: [0.,0.,0.,1.,0.]
38. Occurrences: 206. Action Sequence: [0.,0.,1.,1.,0.]
39. Occurrences: 167. Action Sequence: [1.,1.,1.,0.,0.,0.]
40. Occurrences: 162. Action Sequence: [1.,0.,0.,1.]
41. Occurrences: 158. Action Sequence: [0.,1.,0.,1.]
42. Occurrences: 156. Action Sequence: [1.,1.,0.,1.,0.,0.]
43. Occurrences: 154. Action Sequence: [0.,0.,0.,1.]
44. Occurrences: 154. Action Sequence: [1.,0.,1.,0.,0.,0.]
45. Occurrences: 149. Action Sequence: [1.,1.,1.,1.,0.,0.]
46. Occurrences: 148. Action Sequence: [1.,1.,0.,0.,0.,0.]
47. Occurrences: 143. Action Sequence: [0.,1.,1.,1.,0.,0.]
48. Occurrences: 139. Action Sequence: [1.,1.,1.,0.,1.,0.]
49. Occurrences: 135. Action Sequence: [1.,1.,0.,1.]
50. Occurrences: 130. Action Sequence: [1.,0.,0.,1.,0.,0.]

FACTORED SPACE 1, factorisation 4
40088 different trajectories
1. Occurrences: 7562. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 4652. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 3102. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 3034. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1371. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1362. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1311. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1310. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 672. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 669. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 658. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 657. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 631. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 615. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 613. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 561. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 332. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 330. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 329. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 317. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 316. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 313. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 313. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 310. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 296. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 291. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 281. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 269. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 267. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 265. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 259. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 256. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 168. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 167. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 166. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 161. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 159. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 155. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 153. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 151. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 151. Action Sequence: [0.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 150. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 149. Action Sequence: [0.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 149. Action Sequence: [0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 148. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 146. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 142. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 141. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 140. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 139. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 2, factorisation 4
39157 different trajectories
1. Occurrences: 8661. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 3535. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 3216. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2883. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1490. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1473. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 1405. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 1350. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 797. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 718. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 695. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 677. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 662. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 633. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 591. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 557. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 402. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 346. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 336. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 322. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 313. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 313. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 313. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 309. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 308. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 308. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 300. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 281. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 272. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 268. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 268. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 257. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 182. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 177. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 174. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 172. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 170. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 167. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 167. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 165. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 159. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 155. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 154. Action Sequence: [0.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 151. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 151. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 148. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 145. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 145. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 145. Action Sequence: [0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 139. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

--------------------------------------------------------
'''

### State 439, Diabetes

In [ ]:
'''
NON FACTORED SPACE
29680 different trajectories
1. Occurrences: 12508. Action Sequence: [1.]
2. Occurrences: 12058. Action Sequence: [5.]
3. Occurrences: 11905. Action Sequence: [0.]
4. Occurrences: 6248. Action Sequence: [4.]
5. Occurrences: 4595. Action Sequence: [3.]
6. Occurrences: 4390. Action Sequence: [2.]
7. Occurrences: 3677. Action Sequence: [7.]
8. Occurrences: 1873. Action Sequence: [6.]
9. Occurrences: 692. Action Sequence: [4.,1.]
10. Occurrences: 660. Action Sequence: [4.,5.]
11. Occurrences: 427. Action Sequence: [6.,1.]
12. Occurrences: 303. Action Sequence: [6.,5.]
13. Occurrences: 296. Action Sequence: [3.,1.]
14. Occurrences: 282. Action Sequence: [2.,1.]
15. Occurrences: 278. Action Sequence: [4.,3.]
16. Occurrences: 271. Action Sequence: [3.,0.]
17. Occurrences: 246. Action Sequence: [7.,1.]
18. Occurrences: 231. Action Sequence: [4.,7.]
19. Occurrences: 223. Action Sequence: [2.,0.]
20. Occurrences: 213. Action Sequence: [2.,3.]
21. Occurrences: 210. Action Sequence: [3.,3.]
22. Occurrences: 201. Action Sequence: [6.,0.]
23. Occurrences: 188. Action Sequence: [6.,3.]
24. Occurrences: 172. Action Sequence: [7.,0.]
25. Occurrences: 163. Action Sequence: [3.,2.]
26. Occurrences: 156. Action Sequence: [2.,2.]
27. Occurrences: 121. Action Sequence: [4.,0.]
28. Occurrences: 114. Action Sequence: [7.,5.]
29. Occurrences: 108. Action Sequence: [2.,5.]
30. Occurrences: 103. Action Sequence: [7.,3.]
31. Occurrences: 102. Action Sequence: [7.,2.]
32. Occurrences: 88. Action Sequence: [3.,5.]
33. Occurrences: 72. Action Sequence: [6.,2.]
34. Occurrences: 71. Action Sequence: [5.,5.]
35. Occurrences: 69. Action Sequence: [4.,4.,1.]
36. Occurrences: 68. Action Sequence: [6.,7.]
37. Occurrences: 61. Action Sequence: [5.,1.]
38. Occurrences: 61. Action Sequence: [6.,4.,1.]
39. Occurrences: 60. Action Sequence: [0.,1.]
40. Occurrences: 60. Action Sequence: [4.,0.,1.]
41. Occurrences: 58. Action Sequence: [6.,0.,1.]
42. Occurrences: 56. Action Sequence: [4.,4.,5.]
43. Occurrences: 56. Action Sequence: [4.,2.]
44. Occurrences: 56. Action Sequence: [6.,2.,1.]
45. Occurrences: 54. Action Sequence: [0.,5.]
46. Occurrences: 54. Action Sequence: [4.,0.,5.]
47. Occurrences: 52. Action Sequence: [7.,4.]
48. Occurrences: 51. Action Sequence: [6.,3.,1.]
49. Occurrences: 49. Action Sequence: [6.,4.,5.]
50. Occurrences: 45. Action Sequence: [7.,3.,1.]

FACTORED SPACE 0, factorisation 1
12416 different trajectories
1. Occurrences: 43638. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 16800. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 3826. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 1990. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1655. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1516. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 748. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 729. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 672. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 629. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 455. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 377. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 354. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 348. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 336. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 312. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 312. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 304. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 303. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 272. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 173. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 162. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 162. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 159. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 154. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 153. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 148. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 144. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 143. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 138. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 137. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 135. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 133. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 129. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 126. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 122. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 121. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 106. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 104. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 90. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 89. Action Sequence: [1.,0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 84. Action Sequence: [1.,1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 76. Action Sequence: [1.,1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 76. Action Sequence: [1.,1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 75. Action Sequence: [1.,1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 74. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 73. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 72. Action Sequence: [1.,1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 70. Action Sequence: [1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 70. Action Sequence: [1.,1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 1
24938 different trajectories
1. Occurrences: 17103. Action Sequence: [1.]
2. Occurrences: 16295. Action Sequence: [0.]
3. Occurrences: 15735. Action Sequence: [3.]
4. Occurrences: 8121. Action Sequence: [2.]
5. Occurrences: 1585. Action Sequence: [2.,1.]
6. Occurrences: 1262. Action Sequence: [2.,3.]
7. Occurrences: 585. Action Sequence: [0.,1.]
8. Occurrences: 506. Action Sequence: [1.,1.]
9. Occurrences: 450. Action Sequence: [2.,0.]
10. Occurrences: 437. Action Sequence: [3.,1.]
11. Occurrences: 434. Action Sequence: [1.,0.]
12. Occurrences: 424. Action Sequence: [0.,0.]
13. Occurrences: 353. Action Sequence: [2.,0.,1.]
14. Occurrences: 299. Action Sequence: [3.,0.]
15. Occurrences: 276. Action Sequence: [2.,2.,1.]
16. Occurrences: 219. Action Sequence: [3.,3.]
17. Occurrences: 201. Action Sequence: [2.,2.,3.]
18. Occurrences: 184. Action Sequence: [0.,3.]
19. Occurrences: 171. Action Sequence: [2.,0.,3.]
20. Occurrences: 170. Action Sequence: [2.,1.,1.]
21. Occurrences: 154. Action Sequence: [2.,0.,0.]
22. Occurrences: 147. Action Sequence: [3.,0.,1.]
23. Occurrences: 137. Action Sequence: [3.,1.,1.]
24. Occurrences: 122. Action Sequence: [2.,3.,1.]
25. Occurrences: 121. Action Sequence: [2.,1.,0.]
26. Occurrences: 117. Action Sequence: [0.,0.,1.]
27. Occurrences: 106. Action Sequence: [3.,2.,1.]
28. Occurrences: 98. Action Sequence: [0.,2.,1.]
29. Occurrences: 95. Action Sequence: [0.,1.,1.]
30. Occurrences: 95. Action Sequence: [3.,1.,0.]
31. Occurrences: 94. Action Sequence: [1.,1.,0.]
32. Occurrences: 94. Action Sequence: [0.,1.,0.]
33. Occurrences: 91. Action Sequence: [2.,3.,0.]
34. Occurrences: 88. Action Sequence: [1.,3.]
35. Occurrences: 83. Action Sequence: [1.,0.,1.]
36. Occurrences: 83. Action Sequence: [3.,0.,0.]
37. Occurrences: 82. Action Sequence: [0.,0.,0.]
38. Occurrences: 81. Action Sequence: [2.,2.,0.]
39. Occurrences: 78. Action Sequence: [1.,1.,1.]
40. Occurrences: 78. Action Sequence: [3.,3.,0.]
41. Occurrences: 73. Action Sequence: [2.,2.,0.,1.]
42. Occurrences: 72. Action Sequence: [1.,3.,0.]
43. Occurrences: 70. Action Sequence: [1.,0.,0.]
44. Occurrences: 69. Action Sequence: [3.,3.,1.]
45. Occurrences: 66. Action Sequence: [0.,3.,1.]
46. Occurrences: 65. Action Sequence: [2.,0.,0.,1.]
47. Occurrences: 65. Action Sequence: [1.,2.,1.]
48. Occurrences: 65. Action Sequence: [0.,3.,0.]
49. Occurrences: 62. Action Sequence: [1.,3.,1.]
50. Occurrences: 61. Action Sequence: [2.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 2
12380 different trajectories
1. Occurrences: 34855. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 29042. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 2754. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 1713. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1191. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 884. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 751. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 708. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 583. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 455. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 431. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 409. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 370. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 333. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 311. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 296. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 248. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 208. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 198. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 196. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 194. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 181. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 177. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 158. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 157. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 152. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 148. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 144. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 142. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 130. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 123. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 121. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 121. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 98. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 92. Action Sequence: [0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 92. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 91. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 90. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 87. Action Sequence: [0.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 80. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 79. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 78. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 78. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 78. Action Sequence: [0.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 74. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 72. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 71. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 71. Action Sequence: [0.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 70. Action Sequence: [0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 70. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 2
24288 different trajectories
1. Occurrences: 24413. Action Sequence: [0.]
2. Occurrences: 18306. Action Sequence: [2.]
3. Occurrences: 8985. Action Sequence: [1.]
4. Occurrences: 5550. Action Sequence: [3.]
5. Occurrences: 1072. Action Sequence: [1.,0.]
6. Occurrences: 1046. Action Sequence: [3.,0.]
7. Occurrences: 889. Action Sequence: [2.,0.]
8. Occurrences: 742. Action Sequence: [1.,1.]
9. Occurrences: 735. Action Sequence: [2.,2.]
10. Occurrences: 497. Action Sequence: [3.,2.]
11. Occurrences: 465. Action Sequence: [3.,1.]
12. Occurrences: 371. Action Sequence: [2.,1.]
13. Occurrences: 307. Action Sequence: [3.,1.,0.]
14. Occurrences: 267. Action Sequence: [1.,2.]
15. Occurrences: 259. Action Sequence: [2.,3.]
16. Occurrences: 244. Action Sequence: [3.,0.,0.]
17. Occurrences: 236. Action Sequence: [3.,2.,0.]
18. Occurrences: 227. Action Sequence: [1.,1.,0.]
19. Occurrences: 196. Action Sequence: [3.,3.,0.]
20. Occurrences: 190. Action Sequence: [1.,0.,0.]
21. Occurrences: 180. Action Sequence: [3.,0.,1.]
22. Occurrences: 177. Action Sequence: [3.,1.,1.]
23. Occurrences: 168. Action Sequence: [1.,3.,0.]
24. Occurrences: 146. Action Sequence: [1.,2.,0.]
25. Occurrences: 142. Action Sequence: [1.,1.,1.]
26. Occurrences: 129. Action Sequence: [1.,0.,1.]
27. Occurrences: 116. Action Sequence: [3.,2.,2.]
28. Occurrences: 112. Action Sequence: [2.,0.,0.]
29. Occurrences: 107. Action Sequence: [2.,2.,0.]
30. Occurrences: 103. Action Sequence: [2.,1.,0.]
31. Occurrences: 95. Action Sequence: [3.,2.,1.]
32. Occurrences: 95. Action Sequence: [3.,3.,1.]
33. Occurrences: 93. Action Sequence: [0.,0.]
34. Occurrences: 93. Action Sequence: [1.,3.,1.]
35. Occurrences: 87. Action Sequence: [3.,3.,2.]
36. Occurrences: 87. Action Sequence: [2.,3.,0.]
37. Occurrences: 85. Action Sequence: [3.,0.,2.]
38. Occurrences: 83. Action Sequence: [3.,1.,2.]
39. Occurrences: 82. Action Sequence: [2.,1.,1.]
40. Occurrences: 82. Action Sequence: [1.,2.,1.]
41. Occurrences: 75. Action Sequence: [3.,3.]
42. Occurrences: 75. Action Sequence: [3.,3.,0.,0.]
43. Occurrences: 71. Action Sequence: [2.,2.,2.]
44. Occurrences: 68. Action Sequence: [2.,0.,2.]
45. Occurrences: 67. Action Sequence: [3.,2.,1.,0.]
46. Occurrences: 66. Action Sequence: [1.,3.,2.]
47. Occurrences: 66. Action Sequence: [3.,2.,0.,0.]
48. Occurrences: 65. Action Sequence: [1.,3.,1.,0.]
49. Occurrences: 65. Action Sequence: [3.,3.,1.,0.]
50. Occurrences: 62. Action Sequence: [1.,0.,2.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 3
14678 different trajectories
1. Occurrences: 33398. Action Sequence: [0.]
2. Occurrences: 23856. Action Sequence: [1.]
3. Occurrences: 2771. Action Sequence: [1.,0.]
4. Occurrences: 1949. Action Sequence: [0.,0.]
5. Occurrences: 1566. Action Sequence: [1.,1.]
6. Occurrences: 1260. Action Sequence: [1.,0.,0.]
7. Occurrences: 883. Action Sequence: [1.,1.,0.]
8. Occurrences: 713. Action Sequence: [0.,0.,0.]
9. Occurrences: 557. Action Sequence: [1.,1.,0.,0.]
10. Occurrences: 521. Action Sequence: [0.,1.,0.]
11. Occurrences: 447. Action Sequence: [1.,0.,0.,0.]
12. Occurrences: 371. Action Sequence: [1.,1.,1.,0.]
13. Occurrences: 370. Action Sequence: [1.,1.,1.]
14. Occurrences: 366. Action Sequence: [0.,1.,0.,0.]
15. Occurrences: 343. Action Sequence: [0.,1.]
16. Occurrences: 333. Action Sequence: [1.,0.,1.]
17. Occurrences: 325. Action Sequence: [1.,0.,1.,0.]
18. Occurrences: 282. Action Sequence: [1.,1.,1.,0.,0.]
19. Occurrences: 280. Action Sequence: [0.,0.,0.,0.]
20. Occurrences: 279. Action Sequence: [0.,1.,1.,0.]
21. Occurrences: 251. Action Sequence: [1.,1.,0.,0.,0.]
22. Occurrences: 202. Action Sequence: [1.,0.,1.,0.,0.]
23. Occurrences: 190. Action Sequence: [0.,0.,1.,0.]
24. Occurrences: 174. Action Sequence: [0.,1.,1.,0.,0.]
25. Occurrences: 173. Action Sequence: [1.,1.,1.,1.,0.]
26. Occurrences: 168. Action Sequence: [1.,0.,1.,1.,0.]
27. Occurrences: 164. Action Sequence: [1.,1.,0.,1.,0.]
28. Occurrences: 156. Action Sequence: [1.,0.,0.,0.,0.]
29. Occurrences: 150. Action Sequence: [0.,1.,0.,0.,0.]
30. Occurrences: 145. Action Sequence: [0.,0.,1.,0.,0.]
31. Occurrences: 143. Action Sequence: [0.,1.,1.]
32. Occurrences: 141. Action Sequence: [0.,0.,1.]
33. Occurrences: 140. Action Sequence: [1.,0.,0.,1.,0.]
34. Occurrences: 132. Action Sequence: [1.,0.,0.,1.]
35. Occurrences: 125. Action Sequence: [1.,1.,1.,1.,0.,0.]
36. Occurrences: 121. Action Sequence: [1.,0.,1.,1.]
37. Occurrences: 118. Action Sequence: [1.,1.,0.,1.]
38. Occurrences: 117. Action Sequence: [0.,1.,0.,1.,0.]
39. Occurrences: 109. Action Sequence: [1.,1.,1.,1.]
40. Occurrences: 109. Action Sequence: [1.,0.,1.,1.,0.,0.]
41. Occurrences: 105. Action Sequence: [0.,1.,1.,1.,0.]
42. Occurrences: 103. Action Sequence: [1.,1.,1.,0.,0.,0.]
43. Occurrences: 100. Action Sequence: [0.,0.,1.,1.,0.]
44. Occurrences: 100. Action Sequence: [0.,0.,0.,0.,0.]
45. Occurrences: 95. Action Sequence: [1.,1.,0.,1.,0.,0.]
46. Occurrences: 92. Action Sequence: [1.,0.,1.,0.,0.,0.]
47. Occurrences: 91. Action Sequence: [1.,0.,0.,1.,0.,0.]
48. Occurrences: 80. Action Sequence: [1.,1.,1.,1.,1.,0.]
49. Occurrences: 78. Action Sequence: [1.,1.,1.,0.,1.,0.]
50. Occurrences: 76. Action Sequence: [1.,0.,1.,1.,1.,0.]

FACTORED SPACE 1, factorisation 3
23516 different trajectories
1. Occurrences: 24624. Action Sequence: [1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
2. Occurrences: 18746. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
3. Occurrences: 8685. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
4. Occurrences: 6831. Action Sequence: [2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
5. Occurrences: 1482. Action Sequence: [0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
6. Occurrences: 1210. Action Sequence: [2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
7. Occurrences: 827. Action Sequence: [3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
8. Occurrences: 633. Action Sequence: [2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
9. Occurrences: 613. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
10. Occurrences: 363. Action Sequence: [3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
11. Occurrences: 331. Action Sequence: [2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
12. Occurrences: 284. Action Sequence: [0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
13. Occurrences: 280. Action Sequence: [2.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
14. Occurrences: 266. Action Sequence: [0.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
15. Occurrences: 264. Action Sequence: [2.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
16. Occurrences: 197. Action Sequence: [2.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
17. Occurrences: 183. Action Sequence: [3.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
18. Occurrences: 174. Action Sequence: [3.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
19. Occurrences: 169. Action Sequence: [2.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
20. Occurrences: 156. Action Sequence: [2.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
21. Occurrences: 154. Action Sequence: [3.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
22. Occurrences: 152. Action Sequence: [0.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
23. Occurrences: 150. Action Sequence: [3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
24. Occurrences: 150. Action Sequence: [3.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
25. Occurrences: 149. Action Sequence: [2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
26. Occurrences: 148. Action Sequence: [2.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
27. Occurrences: 143. Action Sequence: [0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
28. Occurrences: 137. Action Sequence: [3.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
29. Occurrences: 135. Action Sequence: [3.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
30. Occurrences: 134. Action Sequence: [1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
31. Occurrences: 126. Action Sequence: [2.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
32. Occurrences: 113. Action Sequence: [3.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
33. Occurrences: 96. Action Sequence: [3.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
34. Occurrences: 93. Action Sequence: [2.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
35. Occurrences: 88. Action Sequence: [2.,0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
36. Occurrences: 81. Action Sequence: [2.,0.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
37. Occurrences: 75. Action Sequence: [0.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
38. Occurrences: 73. Action Sequence: [3.,1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
39. Occurrences: 72. Action Sequence: [0.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
40. Occurrences: 67. Action Sequence: [2.,2.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
41. Occurrences: 66. Action Sequence: [2.,1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
42. Occurrences: 64. Action Sequence: [3.,0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
43. Occurrences: 62. Action Sequence: [2.,0.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
44. Occurrences: 61. Action Sequence: [2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
45. Occurrences: 58. Action Sequence: [2.,3.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
46. Occurrences: 57. Action Sequence: [3.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
47. Occurrences: 52. Action Sequence: [0.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
48. Occurrences: 48. Action Sequence: [3.,3.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
49. Occurrences: 47. Action Sequence: [3.,0.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
50. Occurrences: 47. Action Sequence: [2.,2.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 4
14678 different trajectories
1. Occurrences: 33398. Action Sequence: [0.]
2. Occurrences: 23856. Action Sequence: [1.]
3. Occurrences: 2771. Action Sequence: [1.,0.]
4. Occurrences: 1949. Action Sequence: [0.,0.]
5. Occurrences: 1566. Action Sequence: [1.,1.]
6. Occurrences: 1260. Action Sequence: [1.,0.,0.]
7. Occurrences: 883. Action Sequence: [1.,1.,0.]
8. Occurrences: 713. Action Sequence: [0.,0.,0.]
9. Occurrences: 557. Action Sequence: [1.,1.,0.,0.]
10. Occurrences: 521. Action Sequence: [0.,1.,0.]
11. Occurrences: 447. Action Sequence: [1.,0.,0.,0.]
12. Occurrences: 371. Action Sequence: [1.,1.,1.,0.]
13. Occurrences: 370. Action Sequence: [1.,1.,1.]
14. Occurrences: 366. Action Sequence: [0.,1.,0.,0.]
15. Occurrences: 343. Action Sequence: [0.,1.]
16. Occurrences: 333. Action Sequence: [1.,0.,1.]
17. Occurrences: 325. Action Sequence: [1.,0.,1.,0.]
18. Occurrences: 282. Action Sequence: [1.,1.,1.,0.,0.]
19. Occurrences: 280. Action Sequence: [0.,0.,0.,0.]
20. Occurrences: 279. Action Sequence: [0.,1.,1.,0.]
21. Occurrences: 251. Action Sequence: [1.,1.,0.,0.,0.]
22. Occurrences: 202. Action Sequence: [1.,0.,1.,0.,0.]
23. Occurrences: 190. Action Sequence: [0.,0.,1.,0.]
24. Occurrences: 174. Action Sequence: [0.,1.,1.,0.,0.]
25. Occurrences: 173. Action Sequence: [1.,1.,1.,1.,0.]
26. Occurrences: 168. Action Sequence: [1.,0.,1.,1.,0.]
27. Occurrences: 164. Action Sequence: [1.,1.,0.,1.,0.]
28. Occurrences: 156. Action Sequence: [1.,0.,0.,0.,0.]
29. Occurrences: 150. Action Sequence: [0.,1.,0.,0.,0.]
30. Occurrences: 145. Action Sequence: [0.,0.,1.,0.,0.]
31. Occurrences: 143. Action Sequence: [0.,1.,1.]
32. Occurrences: 141. Action Sequence: [0.,0.,1.]
33. Occurrences: 140. Action Sequence: [1.,0.,0.,1.,0.]
34. Occurrences: 132. Action Sequence: [1.,0.,0.,1.]
35. Occurrences: 125. Action Sequence: [1.,1.,1.,1.,0.,0.]
36. Occurrences: 121. Action Sequence: [1.,0.,1.,1.]
37. Occurrences: 118. Action Sequence: [1.,1.,0.,1.]
38. Occurrences: 117. Action Sequence: [0.,1.,0.,1.,0.]
39. Occurrences: 109. Action Sequence: [1.,1.,1.,1.]
40. Occurrences: 109. Action Sequence: [1.,0.,1.,1.,0.,0.]
41. Occurrences: 105. Action Sequence: [0.,1.,1.,1.,0.]
42. Occurrences: 103. Action Sequence: [1.,1.,1.,0.,0.,0.]
43. Occurrences: 100. Action Sequence: [0.,0.,1.,1.,0.]
44. Occurrences: 100. Action Sequence: [0.,0.,0.,0.,0.]
45. Occurrences: 95. Action Sequence: [1.,1.,0.,1.,0.,0.]
46. Occurrences: 92. Action Sequence: [1.,0.,1.,0.,0.,0.]
47. Occurrences: 91. Action Sequence: [1.,0.,0.,1.,0.,0.]
48. Occurrences: 80. Action Sequence: [1.,1.,1.,1.,1.,0.]
49. Occurrences: 78. Action Sequence: [1.,1.,1.,0.,1.,0.]
50. Occurrences: 76. Action Sequence: [1.,0.,1.,1.,1.,0.]

FACTORED SPACE 1, factorisation 4
12416 different trajectories
1. Occurrences: 43638. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 16800. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 3826. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 1990. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1655. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1516. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 748. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 729. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 672. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 629. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 455. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 377. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 354. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 348. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 336. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 312. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 312. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 304. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 303. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 272. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 173. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 162. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 162. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 159. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 154. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 153. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 148. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 144. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 143. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 138. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 137. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 135. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 133. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 129. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 126. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 122. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 121. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 106. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 104. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 90. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 89. Action Sequence: [1.,0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 84. Action Sequence: [1.,1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 76. Action Sequence: [1.,1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 76. Action Sequence: [1.,1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 75. Action Sequence: [1.,1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 74. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 73. Action Sequence: [1.,1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 72. Action Sequence: [1.,1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 70. Action Sequence: [1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 70. Action Sequence: [1.,1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 2, factorisation 4
12380 different trajectories
1. Occurrences: 34855. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 29042. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 2754. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 1713. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1191. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 884. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 751. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 708. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 583. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 455. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 431. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 409. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 370. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 333. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 311. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 296. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 248. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 208. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 198. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 196. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 194. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 181. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 177. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 158. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 157. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 152. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 148. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 144. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 142. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 130. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 123. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 121. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 121. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 98. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 92. Action Sequence: [0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 92. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 91. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 90. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 87. Action Sequence: [0.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 80. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 79. Action Sequence: [0.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 78. Action Sequence: [0.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 78. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 78. Action Sequence: [0.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 74. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 72. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 71. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 71. Action Sequence: [0.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 70. Action Sequence: [0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 70. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

--------------------------------------------------------
'''

### State 439 No Diabetes

In [ ]:
'''
NON FACTORED SPACE
32659 different trajectories
1. Occurrences: 12603. Action Sequence: [1.]
2. Occurrences: 11376. Action Sequence: [0.]
3. Occurrences: 10339. Action Sequence: [5.]
4. Occurrences: 6153. Action Sequence: [4.]
5. Occurrences: 4600. Action Sequence: [3.]
6. Occurrences: 4276. Action Sequence: [2.]
7. Occurrences: 3179. Action Sequence: [7.]
8. Occurrences: 1835. Action Sequence: [6.]
9. Occurrences: 549. Action Sequence: [4.,1.]
10. Occurrences: 472. Action Sequence: [4.,5.]
11. Occurrences: 384. Action Sequence: [6.,1.]
12. Occurrences: 300. Action Sequence: [2.,1.]
13. Occurrences: 292. Action Sequence: [3.,1.]
14. Occurrences: 281. Action Sequence: [6.,5.]
15. Occurrences: 255. Action Sequence: [7.,1.]
16. Occurrences: 241. Action Sequence: [3.,0.]
17. Occurrences: 235. Action Sequence: [2.,3.]
18. Occurrences: 233. Action Sequence: [4.,3.]
19. Occurrences: 222. Action Sequence: [2.,0.]
20. Occurrences: 212. Action Sequence: [3.,3.]
21. Occurrences: 207. Action Sequence: [7.,0.]
22. Occurrences: 183. Action Sequence: [3.,2.]
23. Occurrences: 176. Action Sequence: [6.,3.]
24. Occurrences: 164. Action Sequence: [4.,7.]
25. Occurrences: 160. Action Sequence: [5.,1.]
26. Occurrences: 156. Action Sequence: [6.,0.]
27. Occurrences: 149. Action Sequence: [2.,2.]
28. Occurrences: 141. Action Sequence: [5.,5.]
29. Occurrences: 139. Action Sequence: [7.,3.]
30. Occurrences: 138. Action Sequence: [4.,0.]
31. Occurrences: 125. Action Sequence: [7.,5.]
32. Occurrences: 121. Action Sequence: [2.,5.]
33. Occurrences: 94. Action Sequence: [0.,1.]
34. Occurrences: 91. Action Sequence: [0.,5.]
35. Occurrences: 79. Action Sequence: [7.,2.]
36. Occurrences: 76. Action Sequence: [3.,5.]
37. Occurrences: 73. Action Sequence: [5.,3.]
38. Occurrences: 71. Action Sequence: [7.,4.]
39. Occurrences: 71. Action Sequence: [6.,2.]
40. Occurrences: 64. Action Sequence: [5.,0.]
41. Occurrences: 62. Action Sequence: [6.,7.]
42. Occurrences: 54. Action Sequence: [4.,4.,1.]
43. Occurrences: 51. Action Sequence: [3.,4.]
44. Occurrences: 48. Action Sequence: [4.,0.,1.]
45. Occurrences: 48. Action Sequence: [6.,0.,1.]
46. Occurrences: 47. Action Sequence: [5.,7.]
47. Occurrences: 46. Action Sequence: [6.,4.,5.]
48. Occurrences: 45. Action Sequence: [4.,2.]
49. Occurrences: 45. Action Sequence: [4.,4.,5.]
50. Occurrences: 44. Action Sequence: [4.,0.,5.]

FACTORED SPACE 0, factorisation 1
14448 different trajectories
1. Occurrences: 41433. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 16219. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 3863. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2054. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1581. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1464. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 711. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 686. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 679. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 633. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 530. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 424. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 340. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 335. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 291. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 290. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 279. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 273. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 271. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 251. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 203. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 191. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 169. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 164. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 158. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 151. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 147. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 145. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 142. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 141. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 138. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 136. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 133. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 128. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 127. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 127. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 124. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 123. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 119. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 111. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 90. Action Sequence: [1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 84. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 84. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 80. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 76. Action Sequence: [1.,1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 76. Action Sequence: [1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 75. Action Sequence: [1.,1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 74. Action Sequence: [1.,0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 72. Action Sequence: [1.,1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 72. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 1
27524 different trajectories
1. Occurrences: 17203. Action Sequence: [1.]
2. Occurrences: 15652. Action Sequence: [0.]
3. Occurrences: 13518. Action Sequence: [3.]
4. Occurrences: 7988. Action Sequence: [2.]
5. Occurrences: 1342. Action Sequence: [2.,1.]
6. Occurrences: 979. Action Sequence: [2.,3.]
7. Occurrences: 673. Action Sequence: [0.,1.]
8. Occurrences: 627. Action Sequence: [3.,1.]
9. Occurrences: 504. Action Sequence: [1.,1.]
10. Occurrences: 428. Action Sequence: [0.,0.]
11. Occurrences: 424. Action Sequence: [1.,0.]
12. Occurrences: 410. Action Sequence: [2.,0.]
13. Occurrences: 370. Action Sequence: [3.,0.]
14. Occurrences: 333. Action Sequence: [3.,3.]
15. Occurrences: 268. Action Sequence: [2.,0.,1.]
16. Occurrences: 260. Action Sequence: [0.,3.]
17. Occurrences: 215. Action Sequence: [2.,2.,1.]
18. Occurrences: 190. Action Sequence: [2.,1.,1.]
19. Occurrences: 172. Action Sequence: [3.,0.,1.]
20. Occurrences: 161. Action Sequence: [2.,0.,0.]
21. Occurrences: 154. Action Sequence: [2.,2.,3.]
22. Occurrences: 150. Action Sequence: [2.,3.,1.]
23. Occurrences: 150. Action Sequence: [2.,0.,3.]
24. Occurrences: 148. Action Sequence: [3.,1.,1.]
25. Occurrences: 140. Action Sequence: [0.,0.,1.]
26. Occurrences: 136. Action Sequence: [3.,2.,1.]
27. Occurrences: 118. Action Sequence: [2.,1.,0.]
28. Occurrences: 111. Action Sequence: [0.,1.,1.]
29. Occurrences: 107. Action Sequence: [1.,0.,1.]
30. Occurrences: 105. Action Sequence: [3.,1.,0.]
31. Occurrences: 102. Action Sequence: [3.,2.]
32. Occurrences: 102. Action Sequence: [3.,3.,1.]
33. Occurrences: 93. Action Sequence: [3.,0.,0.]
34. Occurrences: 92. Action Sequence: [0.,2.,1.]
35. Occurrences: 84. Action Sequence: [0.,3.,1.]
36. Occurrences: 84. Action Sequence: [0.,0.,0.]
37. Occurrences: 82. Action Sequence: [1.,1.,1.]
38. Occurrences: 80. Action Sequence: [2.,3.,0.]
39. Occurrences: 77. Action Sequence: [1.,1.,0.]
40. Occurrences: 76. Action Sequence: [2.,3.,3.]
41. Occurrences: 76. Action Sequence: [1.,3.]
42. Occurrences: 75. Action Sequence: [3.,3.,0.]
43. Occurrences: 71. Action Sequence: [3.,2.,3.]
44. Occurrences: 70. Action Sequence: [1.,0.,0.]
45. Occurrences: 69. Action Sequence: [2.,2.,0.]
46. Occurrences: 68. Action Sequence: [1.,3.,1.]
47. Occurrences: 66. Action Sequence: [1.,3.,0.]
48. Occurrences: 66. Action Sequence: [2.,1.,3.]
49. Occurrences: 64. Action Sequence: [2.,0.,0.,1.]
50. Occurrences: 63. Action Sequence: [0.,1.,0.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 2
14394 different trajectories
1. Occurrences: 33173. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 28123. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 2562. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2015. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1131. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 875. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 748. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 718. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 507. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 430. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 418. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 408. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 378. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 347. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 329. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 326. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 244. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 217. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 214. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 213. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 184. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 181. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 168. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 161. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 159. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 155. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 152. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 148. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 145. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 134. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 130. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 128. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 124. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 124. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 102. Action Sequence: [0.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 101. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 97. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 96. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 89. Action Sequence: [0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 87. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 86. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 86. Action Sequence: [0.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 85. Action Sequence: [0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 81. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 80. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 78. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 76. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 74. Action Sequence: [0.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 73. Action Sequence: [0.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 70. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 1, factorisation 2
26987 different trajectories
1. Occurrences: 23979. Action Sequence: [0.]
2. Occurrences: 16492. Action Sequence: [2.]
3. Occurrences: 8876. Action Sequence: [1.]
4. Occurrences: 5014. Action Sequence: [3.]
5. Occurrences: 1055. Action Sequence: [1.,0.]
6. Occurrences: 1002. Action Sequence: [3.,0.]
7. Occurrences: 911. Action Sequence: [2.,0.]
8. Occurrences: 779. Action Sequence: [1.,1.]
9. Occurrences: 636. Action Sequence: [2.,2.]
10. Occurrences: 503. Action Sequence: [3.,2.]
11. Occurrences: 465. Action Sequence: [3.,1.]
12. Occurrences: 371. Action Sequence: [2.,1.]
13. Occurrences: 282. Action Sequence: [1.,2.]
14. Occurrences: 255. Action Sequence: [3.,1.,0.]
15. Occurrences: 247. Action Sequence: [3.,0.,0.]
16. Occurrences: 218. Action Sequence: [1.,1.,0.]
17. Occurrences: 217. Action Sequence: [3.,2.,0.]
18. Occurrences: 216. Action Sequence: [2.,3.]
19. Occurrences: 207. Action Sequence: [3.,0.,1.]
20. Occurrences: 192. Action Sequence: [3.,1.,1.]
21. Occurrences: 170. Action Sequence: [1.,0.,0.]
22. Occurrences: 168. Action Sequence: [1.,2.,0.]
23. Occurrences: 162. Action Sequence: [3.,3.,0.]
24. Occurrences: 150. Action Sequence: [1.,1.,1.]
25. Occurrences: 149. Action Sequence: [1.,0.,1.]
26. Occurrences: 138. Action Sequence: [1.,3.,0.]
27. Occurrences: 137. Action Sequence: [0.,0.]
28. Occurrences: 122. Action Sequence: [2.,0.,0.]
29. Occurrences: 119. Action Sequence: [3.,2.,2.]
30. Occurrences: 118. Action Sequence: [2.,2.,0.]
31. Occurrences: 101. Action Sequence: [2.,1.,0.]
32. Occurrences: 96. Action Sequence: [3.,2.,1.]
33. Occurrences: 93. Action Sequence: [3.,3.,1.]
34. Occurrences: 91. Action Sequence: [0.,2.]
35. Occurrences: 87. Action Sequence: [2.,3.,0.]
36. Occurrences: 85. Action Sequence: [3.,3.]
37. Occurrences: 83. Action Sequence: [2.,1.,1.]
38. Occurrences: 80. Action Sequence: [2.,2.,2.]
39. Occurrences: 79. Action Sequence: [3.,0.,2.]
40. Occurrences: 77. Action Sequence: [1.,2.,1.]
41. Occurrences: 76. Action Sequence: [1.,2.,2.]
42. Occurrences: 76. Action Sequence: [2.,0.,2.]
43. Occurrences: 74. Action Sequence: [3.,1.,2.]
44. Occurrences: 70. Action Sequence: [1.,3.,1.]
45. Occurrences: 67. Action Sequence: [1.,0.,2.]
46. Occurrences: 63. Action Sequence: [3.,3.,2.]
47. Occurrences: 62. Action Sequence: [3.,2.,1.,0.]
48. Occurrences: 61. Action Sequence: [3.,3.,0.,0.]
49. Occurrences: 60. Action Sequence: [3.,0.,0.,0.]
50. Occurrences: 59. Action Sequence: [3.,3.,1.,0.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 3
16784 different trajectories
1. Occurrences: 32855. Action Sequence: [0.]
2. Occurrences: 21506. Action Sequence: [1.]
3. Occurrences: 2749. Action Sequence: [1.,0.]
4. Occurrences: 2029. Action Sequence: [0.,0.]
5. Occurrences: 1440. Action Sequence: [1.,1.]
6. Occurrences: 1255. Action Sequence: [1.,0.,0.]
7. Occurrences: 873. Action Sequence: [1.,1.,0.]
8. Occurrences: 734. Action Sequence: [0.,0.,0.]
9. Occurrences: 554. Action Sequence: [1.,1.,0.,0.]
10. Occurrences: 502. Action Sequence: [0.,1.,0.]
11. Occurrences: 472. Action Sequence: [1.,0.,0.,0.]
12. Occurrences: 421. Action Sequence: [0.,1.]
13. Occurrences: 392. Action Sequence: [1.,1.,1.,0.]
14. Occurrences: 372. Action Sequence: [1.,1.,1.]
15. Occurrences: 359. Action Sequence: [1.,0.,1.,0.]
16. Occurrences: 347. Action Sequence: [0.,1.,0.,0.]
17. Occurrences: 339. Action Sequence: [1.,0.,1.]
18. Occurrences: 292. Action Sequence: [0.,0.,0.,0.]
19. Occurrences: 254. Action Sequence: [1.,1.,1.,0.,0.]
20. Occurrences: 240. Action Sequence: [1.,1.,0.,0.,0.]
21. Occurrences: 237. Action Sequence: [0.,1.,1.,0.]
22. Occurrences: 230. Action Sequence: [1.,0.,1.,0.,0.]
23. Occurrences: 212. Action Sequence: [0.,0.,1.,0.]
24. Occurrences: 200. Action Sequence: [1.,0.,0.,0.,0.]
25. Occurrences: 178. Action Sequence: [1.,1.,1.,1.,0.]
26. Occurrences: 177. Action Sequence: [1.,0.,1.,1.,0.]
27. Occurrences: 166. Action Sequence: [0.,1.,1.]
28. Occurrences: 160. Action Sequence: [1.,1.,0.,1.,0.]
29. Occurrences: 158. Action Sequence: [0.,0.,1.]
30. Occurrences: 149. Action Sequence: [0.,1.,1.,0.,0.]
31. Occurrences: 149. Action Sequence: [0.,0.,1.,0.,0.]
32. Occurrences: 144. Action Sequence: [1.,0.,0.,1.,0.]
33. Occurrences: 143. Action Sequence: [1.,1.,0.,1.]
34. Occurrences: 139. Action Sequence: [1.,0.,0.,1.]
35. Occurrences: 137. Action Sequence: [1.,1.,1.,1.,0.,0.]
36. Occurrences: 134. Action Sequence: [0.,1.,0.,0.,0.]
37. Occurrences: 127. Action Sequence: [1.,0.,0.,1.,0.,0.]
38. Occurrences: 126. Action Sequence: [0.,0.,0.,0.,0.]
39. Occurrences: 126. Action Sequence: [1.,1.,1.,1.]
40. Occurrences: 115. Action Sequence: [1.,0.,1.,1.]
41. Occurrences: 109. Action Sequence: [1.,0.,1.,1.,0.,0.]
42. Occurrences: 103. Action Sequence: [0.,1.,1.,1.,0.]
43. Occurrences: 102. Action Sequence: [0.,1.,0.,1.,0.]
44. Occurrences: 102. Action Sequence: [1.,1.,0.,0.,0.,0.]
45. Occurrences: 96. Action Sequence: [1.,1.,0.,1.,0.,0.]
46. Occurrences: 95. Action Sequence: [1.,1.,1.,0.,0.,0.]
47. Occurrences: 94. Action Sequence: [0.,1.,0.,1.]
48. Occurrences: 91. Action Sequence: [0.,1.,1.,1.,0.,0.]
49. Occurrences: 90. Action Sequence: [1.,0.,1.,0.,0.,0.]
50. Occurrences: 83. Action Sequence: [1.,0.,0.,0.,0.,0.]

FACTORED SPACE 1, factorisation 3
26222 different trajectories
1. Occurrences: 23071. Action Sequence: [1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
2. Occurrences: 18034. Action Sequence: [0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
3. Occurrences: 8241. Action Sequence: [3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
4. Occurrences: 6703. Action Sequence: [2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
5. Occurrences: 1246. Action Sequence: [0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
6. Occurrences: 1194. Action Sequence: [2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
7. Occurrences: 843. Action Sequence: [3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
8. Occurrences: 684. Action Sequence: [3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
9. Occurrences: 577. Action Sequence: [2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
10. Occurrences: 359. Action Sequence: [3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
11. Occurrences: 319. Action Sequence: [2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
12. Occurrences: 310. Action Sequence: [1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
13. Occurrences: 283. Action Sequence: [0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
14. Occurrences: 282. Action Sequence: [2.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
15. Occurrences: 237. Action Sequence: [0.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
16. Occurrences: 222. Action Sequence: [3.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
17. Occurrences: 202. Action Sequence: [2.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
18. Occurrences: 195. Action Sequence: [3.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
19. Occurrences: 193. Action Sequence: [2.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
20. Occurrences: 188. Action Sequence: [2.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
21. Occurrences: 173. Action Sequence: [3.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
22. Occurrences: 173. Action Sequence: [3.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
23. Occurrences: 136. Action Sequence: [0.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
24. Occurrences: 132. Action Sequence: [0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
25. Occurrences: 130. Action Sequence: [3.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
26. Occurrences: 122. Action Sequence: [1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
27. Occurrences: 120. Action Sequence: [3.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
28. Occurrences: 116. Action Sequence: [2.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
29. Occurrences: 113. Action Sequence: [2.,3.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
30. Occurrences: 113. Action Sequence: [2.,1.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
31. Occurrences: 107. Action Sequence: [3.,2.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
32. Occurrences: 104. Action Sequence: [3.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
33. Occurrences: 92. Action Sequence: [2.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
34. Occurrences: 90. Action Sequence: [3.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
35. Occurrences: 90. Action Sequence: [2.,0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
36. Occurrences: 85. Action Sequence: [0.,1.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
37. Occurrences: 84. Action Sequence: [0.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
38. Occurrences: 84. Action Sequence: [2.,3.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
39. Occurrences: 83. Action Sequence: [3.,1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
40. Occurrences: 78. Action Sequence: [0.,0.,0.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
41. Occurrences: 73. Action Sequence: [2.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
42. Occurrences: 69. Action Sequence: [2.,0.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
43. Occurrences: 68. Action Sequence: [2.,1.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
44. Occurrences: 65. Action Sequence: [3.,0.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
45. Occurrences: 65. Action Sequence: [3.,2.,2.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
46. Occurrences: 61. Action Sequence: [3.,0.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
47. Occurrences: 58. Action Sequence: [3.,1.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
48. Occurrences: 52. Action Sequence: [0.,0.,0.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
49. Occurrences: 51. Action Sequence: [3.,2.,3.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]
50. Occurrences: 51. Action Sequence: [2.,0.,2.,1.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.,3.]

--------------------------------------------------------
FACTORED SPACE 0, factorisation 4
16784 different trajectories
1. Occurrences: 32855. Action Sequence: [0.]
2. Occurrences: 21506. Action Sequence: [1.]
3. Occurrences: 2749. Action Sequence: [1.,0.]
4. Occurrences: 2029. Action Sequence: [0.,0.]
5. Occurrences: 1440. Action Sequence: [1.,1.]
6. Occurrences: 1255. Action Sequence: [1.,0.,0.]
7. Occurrences: 873. Action Sequence: [1.,1.,0.]
8. Occurrences: 734. Action Sequence: [0.,0.,0.]
9. Occurrences: 554. Action Sequence: [1.,1.,0.,0.]
10. Occurrences: 502. Action Sequence: [0.,1.,0.]
11. Occurrences: 472. Action Sequence: [1.,0.,0.,0.]
12. Occurrences: 421. Action Sequence: [0.,1.]
13. Occurrences: 392. Action Sequence: [1.,1.,1.,0.]
14. Occurrences: 372. Action Sequence: [1.,1.,1.]
15. Occurrences: 359. Action Sequence: [1.,0.,1.,0.]
16. Occurrences: 347. Action Sequence: [0.,1.,0.,0.]
17. Occurrences: 339. Action Sequence: [1.,0.,1.]
18. Occurrences: 292. Action Sequence: [0.,0.,0.,0.]
19. Occurrences: 254. Action Sequence: [1.,1.,1.,0.,0.]
20. Occurrences: 240. Action Sequence: [1.,1.,0.,0.,0.]
21. Occurrences: 237. Action Sequence: [0.,1.,1.,0.]
22. Occurrences: 230. Action Sequence: [1.,0.,1.,0.,0.]
23. Occurrences: 212. Action Sequence: [0.,0.,1.,0.]
24. Occurrences: 200. Action Sequence: [1.,0.,0.,0.,0.]
25. Occurrences: 178. Action Sequence: [1.,1.,1.,1.,0.]
26. Occurrences: 177. Action Sequence: [1.,0.,1.,1.,0.]
27. Occurrences: 166. Action Sequence: [0.,1.,1.]
28. Occurrences: 160. Action Sequence: [1.,1.,0.,1.,0.]
29. Occurrences: 158. Action Sequence: [0.,0.,1.]
30. Occurrences: 149. Action Sequence: [0.,1.,1.,0.,0.]
31. Occurrences: 149. Action Sequence: [0.,0.,1.,0.,0.]
32. Occurrences: 144. Action Sequence: [1.,0.,0.,1.,0.]
33. Occurrences: 143. Action Sequence: [1.,1.,0.,1.]
34. Occurrences: 139. Action Sequence: [1.,0.,0.,1.]
35. Occurrences: 137. Action Sequence: [1.,1.,1.,1.,0.,0.]
36. Occurrences: 134. Action Sequence: [0.,1.,0.,0.,0.]
37. Occurrences: 127. Action Sequence: [1.,0.,0.,1.,0.,0.]
38. Occurrences: 126. Action Sequence: [0.,0.,0.,0.,0.]
39. Occurrences: 126. Action Sequence: [1.,1.,1.,1.]
40. Occurrences: 115. Action Sequence: [1.,0.,1.,1.]
41. Occurrences: 109. Action Sequence: [1.,0.,1.,1.,0.,0.]
42. Occurrences: 103. Action Sequence: [0.,1.,1.,1.,0.]
43. Occurrences: 102. Action Sequence: [0.,1.,0.,1.,0.]
44. Occurrences: 102. Action Sequence: [1.,1.,0.,0.,0.,0.]
45. Occurrences: 96. Action Sequence: [1.,1.,0.,1.,0.,0.]
46. Occurrences: 95. Action Sequence: [1.,1.,1.,0.,0.,0.]
47. Occurrences: 94. Action Sequence: [0.,1.,0.,1.]
48. Occurrences: 91. Action Sequence: [0.,1.,1.,1.,0.,0.]
49. Occurrences: 90. Action Sequence: [1.,0.,1.,0.,0.,0.]
50. Occurrences: 83. Action Sequence: [1.,0.,0.,0.,0.,0.]

FACTORED SPACE 1, factorisation 4
14448 different trajectories
1. Occurrences: 41433. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 16219. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 3863. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2054. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1581. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 1464. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 711. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 686. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 679. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 633. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 530. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 424. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 340. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 335. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 291. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 290. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 279. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 273. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 271. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 251. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 203. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 191. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 169. Action Sequence: [1.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 164. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 158. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 151. Action Sequence: [1.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 147. Action Sequence: [1.,1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 145. Action Sequence: [1.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 142. Action Sequence: [1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 141. Action Sequence: [1.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 138. Action Sequence: [1.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 136. Action Sequence: [1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 133. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 128. Action Sequence: [1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 127. Action Sequence: [1.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 127. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 124. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 123. Action Sequence: [1.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 119. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 111. Action Sequence: [1.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 90. Action Sequence: [1.,1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 84. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 84. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 80. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 76. Action Sequence: [1.,1.,1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 76. Action Sequence: [1.,1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 75. Action Sequence: [1.,1.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 74. Action Sequence: [1.,0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 72. Action Sequence: [1.,1.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 72. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

FACTORED SPACE 2, factorisation 4
14394 different trajectories
1. Occurrences: 33173. Action Sequence: [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
2. Occurrences: 28123. Action Sequence: [0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
3. Occurrences: 2562. Action Sequence: [0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
4. Occurrences: 2015. Action Sequence: [1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
5. Occurrences: 1131. Action Sequence: [0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
6. Occurrences: 875. Action Sequence: [0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
7. Occurrences: 748. Action Sequence: [1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
8. Occurrences: 718. Action Sequence: [1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
9. Occurrences: 507. Action Sequence: [0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
10. Occurrences: 430. Action Sequence: [0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
11. Occurrences: 418. Action Sequence: [0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
12. Occurrences: 408. Action Sequence: [1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
13. Occurrences: 378. Action Sequence: [0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
14. Occurrences: 347. Action Sequence: [1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
15. Occurrences: 329. Action Sequence: [1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
16. Occurrences: 326. Action Sequence: [1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
17. Occurrences: 244. Action Sequence: [0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
18. Occurrences: 217. Action Sequence: [0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
19. Occurrences: 214. Action Sequence: [0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
20. Occurrences: 213. Action Sequence: [0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
21. Occurrences: 184. Action Sequence: [0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
22. Occurrences: 181. Action Sequence: [0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
23. Occurrences: 168. Action Sequence: [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
24. Occurrences: 161. Action Sequence: [0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
25. Occurrences: 159. Action Sequence: [1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
26. Occurrences: 155. Action Sequence: [1.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
27. Occurrences: 152. Action Sequence: [1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
28. Occurrences: 148. Action Sequence: [1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
29. Occurrences: 145. Action Sequence: [1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
30. Occurrences: 134. Action Sequence: [1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
31. Occurrences: 130. Action Sequence: [1.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
32. Occurrences: 128. Action Sequence: [1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
33. Occurrences: 124. Action Sequence: [0.,0.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
34. Occurrences: 124. Action Sequence: [0.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
35. Occurrences: 102. Action Sequence: [0.,0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
36. Occurrences: 101. Action Sequence: [0.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
37. Occurrences: 97. Action Sequence: [0.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
38. Occurrences: 96. Action Sequence: [0.,1.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
39. Occurrences: 89. Action Sequence: [0.,0.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
40. Occurrences: 87. Action Sequence: [1.,0.,0.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
41. Occurrences: 86. Action Sequence: [0.,1.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
42. Occurrences: 86. Action Sequence: [0.,0.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
43. Occurrences: 85. Action Sequence: [0.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
44. Occurrences: 81. Action Sequence: [0.,0.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
45. Occurrences: 80. Action Sequence: [0.,1.,1.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
46. Occurrences: 78. Action Sequence: [1.,1.,0.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
47. Occurrences: 76. Action Sequence: [1.,0.,1.,1.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
48. Occurrences: 74. Action Sequence: [0.,1.,0.,1.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
49. Occurrences: 73. Action Sequence: [0.,1.,0.,0.,1.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
50. Occurrences: 70. Action Sequence: [1.,0.,1.,0.,0.,0.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]

--------------------------------------------------------
'''